In [2]:
from spiketurnpike_postanalysis.Extract_ephys_from_struct import ExtractEphysData
import pandas as pd
import seaborn as sns 
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.stats import mannwhitneyu, kruskal, shapiro, ttest_ind
from scipy import stats
from scipy.stats import ttest_rel, permutation_test
from scipy.stats import pearsonr


import matplotlib.pyplot as plt
plt.rcParams['svg.fonttype'] = 'none'


# class and methods set up 

In [3]:


class DataFrameManager:
    def __init__(self, eed_object):
        self.eed = eed_object
        self.dataframes = {}  # Dictionary to hold multiple DataFrames
        self.detailed_dataframes = {}
        
    def create_dataframe(self, columns, df_name):
        data = []
        for groupname, recordings in self.eed.all_data.items():
            for recordingname, cells in recordings.items():
                for cid, metrics in cells.items():
                    
                    row = {'groupname': groupname, 'recordingname': recordingname, 'cid': cid}
                    for column in columns:
                        value = metrics.get(column, None)
                        
                        # Reassign 'IsSingleUnit' based on 'ISI_violations_percent' if necessary
                        if column == 'ISI_violations_percent' and value is not None and value < 0.01:
                            print(f"Changing IsSingleUnit to 1.0 for group: {groupname}, recording: {recordingname}, cid: {cid}")
                            row['IsSingleUnit'] = 1.0
                            continue  # Skip to the next column
                        
                        # Reassign 'Cell_Type' based on 'TroughToPeak_duration' if necessary
                        if column == 'TroughToPeak_duration' and value is not None and value <= 0.5:
                            print(f"Changing Cell_Type to FS for group: {groupname}, recording: {recordingname}, cid: {cid}")
                            row['Cell_Type'] = 'FS'
                            # Now add the 'TroughToPeak_duration' value to the row
                            row['TroughToPeak_duration'] = value
                            continue  # Skip to the next column
                        
                        # Reassign 'ModulationIndex' based on its value if necessary
                        if column == 'ModulationIndex' and value is not None:
                            if value >= 0.3:
                                print(f"Changing ModulationIndex {value} to positive for group: {groupname}, recording: {recordingname}, cid: {cid}")
                                row['ModulationIndex'] = 'positive'
                            elif value <= -0.3:
                                print(f"Changing ModulationIndex {value} to negative for group: {groupname}, recording: {recordingname}, cid: {cid}")
                                row['ModulationIndex'] = 'negative'
                            else:
                                print(f"Changing ModulationIndex {value} to none for group: {groupname}, recording: {recordingname}, cid: {cid}")
                                row['ModulationIndex'] = 'none'
                            continue  # Skip to the next column

                        # Add the original value if no reassignment happened
                        row[column] = value
                    
                    data.append(row)
        
        new_df = pd.DataFrame(data)
        if df_name in self.dataframes:
            self.dataframes[df_name] = pd.concat([self.dataframes[df_name], new_df], ignore_index=True)
        else:
            self.dataframes[df_name] = new_df

    def revert_modulation_index_to_numeric(self):
        """
        Revert the 'ModulationIndex' labels ('positive', 'negative', 'none') 
        back to the original numeric values and store them in a new column 
        named 'ModulationIndex_Numeric'.
        """
        for df_name, df in self.dataframes.items():
            if 'ModulationIndex' in df.columns:
                for index, row in df.iterrows():
                    groupname = row['groupname']
                    recordingname = row['recordingname']
                    cid = row['cid']
                    
                    # Retrieve the original ModulationIndex value from the self.eed.all_data structure
                    original_value = self.eed.all_data[groupname][recordingname][cid].get('ModulationIndex', None)
                    
                    if original_value is not None:
                        print(f"Reverting ModulationIndex from {row['ModulationIndex']} to {original_value} for group: {groupname}, recording: {recordingname}, cid: {cid}")
                        df.at[index, 'ModulationIndex_Numeric'] = original_value  # Store the numeric value in a new column

                # Ensure the DataFrame is updated in the self.dataframes dictionary
                self.dataframes[df_name] = df


    def append_data(self, df_name, new_data):
        if df_name in self.dataframes:
            self.dataframes[df_name] = pd.concat([self.dataframes[df_name], new_data], ignore_index=True)
        else:
            self.dataframes[df_name] = new_data    
    
                
    def get_filtered_data(self, df_name, is_single_unit=None, cell_type=None, stim_responsivity=None, groupname=None, modulation_label=None):
        """
        Retrieve filtered data from the specified DataFrame based on IsSingleUnit, Cell_Type, 
        StimResponsivity, groupname, and ModulationIndex with the option to not filter on any of these by passing None.

        Parameters:
        df_name (str): Name of the DataFrame to filter.
        is_single_unit (float or None): If 1.0, filter for single units, if 0.0, filter for non-single units, 
                                        if None, do not filter by this criterion.
        cell_type (str or None): Filter for 'FS' or 'RS' cell types. If None, no filtering by cell type.
        stim_responsivity (float or None): Value to filter by in the StimResponsivity column. Can be 1.0, 0.0, or -1.0.
                                        If None, no filtering by StimResponsivity.
        groupname (str or None): Filter by groupname. If None, no filtering by groupname.
        modulation_label (str or None): Filter for 'positive', 'negative', or 'none' in the ModulationIndex column.
                                        If None, no filtering by ModulationIndex.

        Returns:
        pandas.DataFrame: The filtered DataFrame.
        """
        
        if df_name not in self.dataframes:
            print(f"No DataFrame found with the name '{df_name}'.")
            return None

        # Start with the full DataFrame
        filtered_df = self.dataframes[df_name]

        # Filter by cell type if provided
        if cell_type:
            filtered_df = filtered_df[filtered_df['Cell_Type'] == cell_type]

        # Filter by IsSingleUnit if not None
        if is_single_unit is not None:
            filtered_df = filtered_df[filtered_df['IsSingleUnit'] == is_single_unit]

        # Filter by StimResponsivity if not None
        if stim_responsivity is not None:
            filtered_df = filtered_df[filtered_df['StimResponsivity'] == stim_responsivity]

        # Filter by groupname if provided
        if groupname is not None:
            filtered_df = filtered_df[filtered_df['groupname'] == groupname]

        # Filter by ModulationIndex if provided
        if modulation_label is not None:
            filtered_df = filtered_df[filtered_df['ModulationIndex'] == modulation_label]

        return filtered_df
   
   
    def create_psth_dataframe(self):
        """
        Creates and stores a DataFrame for each stimulation type using the 'SpikeTrains_for_PSTHs' and 'PSTHs_conv' columns from the base PSTH DataFrame. 
        Each DataFrame is stored as an attribute of the DataFrameManager under a name that corresponds to the stimulation type.

        Details:
            'SpikeTrains_for_PSTHs' is expected to be a pandas Series where each entry is a list of arrays.
            Each array in the list corresponds to spike train data for one of the four distinct stimulations, with dimensions (n_trials, n_time_points), 
            where 'n_trials' varies per stimulation and 'n_time_points' is consistent (usually the length of the trial in ms).

            'PSTHs_conv' is expected to be a pandas Series where each entry is a numpy ndarray with dimensions (4, n_time_points), 
            where the first dimension corresponds to the four stimulation types and 'n_time_points' matches the second dimension of the arrays in 'SpikeTrains_for_PSTHs'.

        Processes:
            - A base DataFrame is created with necessary columns.
            - For each stimulation label (e.g., 'Zero', 'Low', 'Mid', 'Max'), a new DataFrame is created.
            - Each new DataFrame includes adjusted 'SpikeTrains_for_PSTHs' and 'PSTHs_conv' columns to isolate the data corresponding to the respective stimulation type.
            - Each DataFrame is stored in the class dictionary, keyed by the name 'psth_dataframe_' followed by the stimulation label.
        """
        # Create the base dataframe for PSTH analysis
        self.create_dataframe(['Cell_Type', 'LaminarLabel','IsSingleUnit', 'StimResponsivity', 'SpikeTrains_for_PSTHs', 'PSTHs_conv', 'ModulationIndex'], 'psth_dataframe')

        # Extracting trial tags
        stim_labels = self.eed.trialTagsLabels['trialTagsLabels']  # ['Zero', 'Low', 'Mid', 'Max']

        # Process each label
        for index, label in enumerate(stim_labels):
            # Retrieve spike trains and PSTHs for each label and create a new DataFrame
            df_name = f'psth_dataframe_{label}'
            self.dataframes[df_name] = self.dataframes['psth_dataframe'].copy()
            self.dataframes[df_name]['SpikeTrains_for_PSTHs'] = self.dataframes['psth_dataframe']['SpikeTrains_for_PSTHs'].apply(lambda x: x[index])
            self.dataframes[df_name]['PSTHs_conv'] = self.dataframes['psth_dataframe']['PSTHs_conv'].apply(lambda x: x[index])

    def create_psth_dataframe_opto(self, modulation_type=None):
        """
        Creates and stores a DataFrame for optogenetic stimulation using the 'SpikeTrains_for_PSTHs' and 'PSTHs_conv' columns from the base PSTH DataFrame. 
        The DataFrame is stored as an attribute of the DataFrameManager under a name that corresponds to the optogenetic stimulation type.

        Details:
            'SpikeTrains_for_PSTHs' is expected to be a pandas Series where each entry is an array of spike train data for the optogenetic stimulation,
            with dimensions (n_trials, n_time_points), where 'n_trials' is the number of trials and 'n_time_points' is consistent (usually the length of the trial in ms).

            'PSTHs_conv' is expected to be a pandas Series where each entry is a numpy ndarray with dimensions (n_time_points),
            where 'n_time_points' is consistent and should be 1500, corresponding to the time points of the optogenetic stimulation.

        Processes:
            - A base DataFrame is created with necessary columns.
            - Retrieve trial tags specific for optogenetic stimulation.
            - For each label (expected to be one for optogenetic stimulation), a new DataFrame is created.
            - Adjusted 'SpikeTrains_for_PSTHs' and 'PSTHs_conv' columns are created to isolate the data corresponding to the optogenetic stimulation.
            - The DataFrame is stored in the class dictionary, keyed by the name 'psth_dataframe_' followed by the stimulation label.
        """
        # Create the base dataframe for PSTH analysis
        
          # Create the base dataframe for PSTH analysis with modulation filtering
        self.create_dataframe(['Cell_Type', 
            'LaminarLabel', 
            'IsSingleUnit', 
            'StimResponsivity', 
            'SpikeTrains_for_PSTHs', 
            'PSTHs_conv', 
            'ModulationIndex',
            'FirstSpikeLatency', 
            'FirstSpikeLatency_Reliability', 
            'TroughToPeak_duration', 
            'SpikeHalfWidth', 
            'PeakToPeak_ratio', 
            'peak1_normalized_amplitude', 
            'Peak1ToTrough_ratio', 
            'Peak2ToTrough_ratio', 
            'Template_Channel', 
            'MeanFR_baseline', 
            'Normalized_Template_Waveform', 
            'UnNormalized_Template_Waveform',
            'SpikeTimes_all', 
            'ISI_baseline_CV', 
            'ISI_baseline_vec', 
            'ISI_pdf_peak_xy', 
            'ISI_pdf_x', 
            'ISI_pdf_y',
            'StimProb', 
            'MeanFR_stim', 
            'PeakEvokedFR', 
            'PeakEvokedFR_Latency', 
            'FanoFactor_baseline', 
            'FanoFactor_stim', 
            'MeanFR_inst_stim', 
            'ISI_violations_percent', 
            'Recording_Duration', 
            'Sampling_Frequency'], 'psth_dataframe')
        
       
        # Extracting trial tags
        stim_labels = self.eed.trialTagsLabels['trialTagsLabels']  # This should only contain the optogenetic stimulation label

        # Process each label (typically only one for optogenetic stimulation)
        for index, label in enumerate(stim_labels):
            # Create a new DataFrame for optogenetic stimulation
            df_name = f'psth_dataframe_{label}'
            self.dataframes[df_name] = self.dataframes['psth_dataframe'].copy()
            self.dataframes[df_name]['SpikeTrains_for_PSTHs'] = self.dataframes['psth_dataframe']['SpikeTrains_for_PSTHs'].apply(lambda x: x[index])
            # Keep the same number of time points (1500) for the PSTH data
            self.dataframes[df_name]['PSTHs_conv'] = self.dataframes['psth_dataframe']['PSTHs_conv'].apply(lambda x: x[:, index] if x.ndim > 1 else x)
            
    def plot_psth(self, stim_label, cell_type=None, is_single_unit=None, stim_responsivity=None, groupname=None):
        """
        Plots the PSTH for a given stimulation type, with optional filtering on cell type, single unit status, and stimulus responsivity.
        Uses the 'relative_time_ms' from the ExtractEphysData object to correctly label the time axis.

        Args:
            stim_label (str): The label of the stimulation type to plot (e.g., 'Zero', 'Low', 'Mid', 'Max').
            cell_type (str, optional): Filter for specific cell types (e.g., 'FS', 'RS'). Default is None, which means no filtering by cell type.
            is_single_unit (float, optional): Filter for single units (1.0) or multi-units (0.0). None means no filtering.
            stim_responsivity (float, optional): Filter by stimulus responsivity (1.0, 0.0, -1.0). None means no filtering.

        Processes:
            - Retrieves the corresponding DataFrame for the specified stimulation.
            - Applies additional filtering based on the provided arguments.
            - Averages the PSTH data across all remaining units and plots the result using the relative time axis.
        """
        # Retrieve the DataFrame for the specified stimulation
        df_name = f'psth_dataframe_{stim_label}'
        if df_name not in self.dataframes:
            print(f"No data available for the stimulation: {stim_label}")
            return

        df = self.dataframes[df_name]

        # Apply filtering based on the optional parameters
        if cell_type is not None:
            df = df[df['Cell_Type'] == cell_type]
        if is_single_unit is not None:
            df = df[df['IsSingleUnit'] == is_single_unit]
        if stim_responsivity is not None:
            df = df[df['StimResponsivity'] == stim_responsivity]
        if groupname is not None:
            df = df[df['groupname'] == groupname]

        # Check if there is any data left after filtering
        if df.empty:
            print("No data matches the specified filters.")
            return

        # Get relative time array for x-axis
        time_array = self.eed.relative_time_ms['relative_time_ms']

        # Aggregate the PSTH data
        aggregated_psth = df['PSTHs_conv'].apply(pd.Series).mean(axis=0)

        # Plotting the aggregated PSTH
        plt.figure(figsize=(10, 5))
        plt.plot(time_array, aggregated_psth, label=f'PSTH for {stim_label}')
        plt.title(f'PSTH for {stim_label} - {cell_type or "All Types"}, SingleUnit: {is_single_unit}, Responsivity: {stim_responsivity}')
        plt.xlabel('Time (ms)')
        plt.ylabel('Average Spike Rate')
        plt.legend()
        plt.show()
    
    def create_time_mask(self, time_array, time_range):
        """
        Creates a mask for the time array based on the specified time range.
        
        Args:
            time_array (np.array): The array of time points.
            time_range (tuple): The start and end time for the mask (in ms).
            
        Returns:
            time_mask (np.array): Boolean array where True indicates the time points within the specified range.
        """
        if time_range:
            time_mask = (time_array >= time_range[0]) & (time_array <= time_range[1])
        else:
            time_mask = slice(None)
        return time_mask

    def filter_data(self, stim_label, cell_type=None, is_single_unit=None, stim_responsivity=None, groupname=None, modulation_label=None, recordingname=None):
        df_name = f'psth_dataframe_{stim_label}'
        if df_name not in self.dataframes:
            print(f"No data available for the stimulation: {stim_label}")
            return pd.DataFrame()

        df = self.dataframes[df_name]
        
        # Apply filtering based on the optional parameters
        if cell_type is not None:
            df = df[df['Cell_Type'] == cell_type]
        if is_single_unit is not None:
            df = df[df['IsSingleUnit'] == is_single_unit]
        if stim_responsivity is not None:
            df = df[df['StimResponsivity'] == stim_responsivity]
        if groupname is not None:
            df = df[df['groupname'] == groupname]
        if recordingname is not None:
            df = df[df['recordingname'] == recordingname]
        
        if modulation_label is not None:
            # Validate the modulation label input
            if modulation_label not in ['positive', 'negative', 'none']:
                raise ValueError("Modulation label must be one of 'positive', 'negative', or 'none'.")
            
            # Apply modulation label filtering
            if modulation_label == 'positive' or modulation_label == 'negative':
                df = df[df['ModulationIndex'] == modulation_label]
            elif modulation_label == 'none':
                df = df[df['ModulationIndex'].isnull()]
        
        return df

    def plot_individual_psths(self, stim_label, cell_type=None, is_single_unit=None, stim_responsivity=None, groupname=None, modulation_label=None, time_range=None):
        df = self.filter_data(stim_label, cell_type, is_single_unit, stim_responsivity, groupname, modulation_label)
        
        # Print the number of units that match the filter
        print(f"Number of units that match the filter: {df.shape[0]}")

        if df.empty:
            print("No data matches the specified filters.")
            return

        # Get relative time array for x-axis
        time_array = self.eed.relative_time_ms['relative_time_ms']
        
        # Create time mask and adjust time array
        time_mask = self.create_time_mask(time_array, time_range)
        time_array = time_array[time_mask]

        plt.figure(figsize=(10, 5))

        for index, row in df.iterrows():
            individual_psth = row['PSTHs_conv']
            # Apply time mask to individual PSTH
            individual_psth = np.array(individual_psth)[time_mask]
            # Apply a smoothing window
            window = np.ones(3) / 3  # 3ms window of smoothing


            
            individual_psth = np.convolve(individual_psth, window, mode='same')
            plt.plot(time_array, individual_psth, label=f'Unit {index}')

        plt.title(f'Individual PSTHs for {stim_label} - {cell_type or "All Types"}, SingleUnit: {is_single_unit}, Responsivity: {stim_responsivity}, Modulation: {modulation_label}')
        plt.xlabel('Time (ms)')
        plt.ylabel('Spike Rate')
        plt.legend().set_visible(False)
        plt.show()
        
    def compare_groups(self, group1, group2, stim_label, cell_type=None, is_single_unit=None, stim_responsivity=None, modulation_label=None):
        """
        Compares PSTH data between two specified groups for a given stimulation.

        Args:
            group1 (str): The first group name for comparison.
            group2 (str): The second group name for comparison.
            stim_label (str): The label of the stimulation type (e.g., 'Zero', 'Low', 'Mid', 'Max').
            cell_type (str, optional): Specific cell type to filter by.
            is_single_unit (float, optional): Filter for single units (1.0) or multi-units (0.0).
            stim_responsivity (float, optional): Filter by stimulus responsivity.
            modulation_label (str, optional): Filter by modulation label ('positive', 'negative', 'none').

        Returns:
            A tuple containing two pandas DataFrames representing the filtered data for each group.
        """
        df_name = f'psth_dataframe_{stim_label}'
        if df_name not in self.dataframes:
            print(f"No data available for the stimulation: {stim_label}")
            return None, None

        base_df = self.dataframes[df_name]
        
        # Print the shape of the pre-filtered DataFrame
        print(f"Pre-filtered DataFrame shape: {base_df.shape}")

        # Define a helper function to create a mask for a given group
        def create_mask(group):
            mask = (base_df['groupname'] == group)
            if cell_type is not None:
                mask &= (base_df['Cell_Type'] == cell_type)
            if is_single_unit is not None:
                mask &= (base_df['IsSingleUnit'] == is_single_unit)
            if stim_responsivity is not None:
                mask &= (base_df['StimResponsivity'] == stim_responsivity)
            if modulation_label is not None:
                if modulation_label not in ['positive', 'negative', 'none']:
                    raise ValueError("Modulation label must be one of 'positive', 'negative', or 'none'.")
                mask &= (base_df['ModulationIndex'] == modulation_label)
            return mask

        # Create masks for each group
        mask1 = create_mask(group1)
        mask2 = create_mask(group2)

        # Filter the DataFrame for each group
        channel1 = base_df[mask1]
        channel2 = base_df[mask2]

        # Print the shape of the filtered DataFrames
        print(f"Filtered df1 (group {group1}) shape: {channel1.shape}")
        print(f"Filtered df2 (group {group2}) shape: {channel2.shape}")

        return channel1, channel2

        
    def plot_all_stimulations(self, group1, group2, cell_type=None, is_single_unit=None, stim_responsivity=None, time_range=None, plot_mode='mean', smoothing_window=None):
        """
        Plots a 1x4 subplot of PSTH comparisons for all stimulation types.
        1. Grabs the trial tags labels from the ExtractEphysData object to determine the stimulation types
        and create labels for the subplots.
        2. Calls the plot_psth_comparison method for each stimulation type and plots them in a single figure.
        
        Args:
            group1 (str): First group name.
            group2 (str): Second group name.
            cell_type (str): 'FS' or 'RS'
            is_single_unit (float, optional): Single unit filter.
            stim_responsivity (float, optional): Stimulus responsivity filter.
            time_range (tuple, optional): Tuple specifying the start and end of the time range (e.g., (-100, 200)).
            plot_mode (str, optional): Plotting mode ('mean', 'traces', 'sem').
        """
        
        
        stim_labels = self.eed.trialTagsLabels['trialTagsLabels']  # ['Zero', 'Low', 'Mid', 'Max']
        fig, axs = plt.subplots(1, 4, figsize=(20, 5), sharey=True)

        for i, stim_label in enumerate(stim_labels):
            self.plot_psth_comparison(group1, group2, stim_label, axs[i], cell_type=cell_type, is_single_unit=is_single_unit, stim_responsivity=stim_responsivity, time_range=time_range, plot_mode=plot_mode, smoothing_window=smoothing_window)
            axs[i].set_title(stim_label)

        plt.tight_layout()
        plt.show()

    def plot_psth_comparison(self, group1, group2, stim_label, 
                             ax=None, cell_type=None, is_single_unit=None, stim_responsivity=None, modulation_label=None,
                             time_range=None, plot_mode='mean', smoothing_window=None):
        """
        called within plot_all_stimulations which is called within the main function plot_all_stimulations
        
        uses the compare_groups method to get the data for the two groups and the specified stimulation type 
        and fetches the two DataFrames for the groups
        
        Plots a PSTH comparison on the provided axes object or creates a new figure if not provided.
        Optionally overlays the stimulator signal as a thin black line.

        Args:
            group1 (str): First group name.
            group2 (str): Second group name.
            stim_label (str): Stimulation label.
            ax (matplotlib.axes.Axes, optional): Axes object to plot on.
            cell_type (str, optional): Cell type to filter.
            is_single_unit (float, optional): Single unit filter.
            stim_responsivity (float, optional): Stimulus responsivity filter.
            time_range (tuple, optional): Tuple specifying the start and end of the time range (e.g., (-100, 200)).
            plot_mode (str, optional): Plotting mode ('mean', 'traces', 'sem').
            smoothing_window (int, optional): Size of the smoothing window; if None, no smoothing is applied.
            groups_to_plot (str, optional): Specifies which group(s) to plot ('both', 'group1', 'group2').
        """
        # Define colors
        group_colors = {
            'No_CTZ': '#797979',
            'CTZ': '#5a00c2'}
        # Fetch data
        df1, df2 = self.compare_groups(group1, group2, stim_label, cell_type, is_single_unit, stim_responsivity, modulation_label)
        if df1.empty or df2.empty:
            print("One of the groups has no data after filtering.")
            return
        if df1.empty or df2.empty:
            print("One of the groups has no data after filtering.")
            return
        

        # Get the time array and adjust for the specified time range
        time_array = self.eed.relative_time_ms['relative_time_ms']
        if time_range:
            time_mask = (time_array >= time_range[0]) & (time_array <= time_range[1])
            time_array = time_array[time_mask]
        else:
            time_mask = slice(None)

        # Check if we need to create a new figure
        if ax is None:
            fig, ax = plt.subplots(figsize=(10, 5))
        
        # DataFrame to store all traces ###new addition to the code
        all_traces_df = pd.DataFrame()
        
        # Process and plot data
        for df, group in zip([df1, df2], [group1, group2]):
            data = df['PSTHs_conv'].apply(lambda x: np.array(x)[time_mask])
            if smoothing_window:
                window = np.ones(smoothing_window) / smoothing_window
                data = data.apply(lambda x: np.convolve(x, window, mode='same'))
                
            mean_psth = data.apply(pd.Series).mean(axis=0)
            
            # Store each trace in the DataFrame ## new addition to the code 
            # Store each trace in the DataFrame with additional metadata
            group_traces_df = pd.DataFrame(data.tolist(), columns=time_array)
            group_traces_df['Group'] = group
            group_traces_df['Stimulation'] = stim_label
            group_traces_df['Cell_Type'] = cell_type
            group_traces_df['IsSingleUnit'] = is_single_unit
            group_traces_df['StimResponsivity'] = stim_responsivity
            all_traces_df = pd.concat([all_traces_df, group_traces_df])

            
            if plot_mode == 'sem':
                sem = data.apply(pd.Series).sem(axis=0)

            # Plot individual traces or mean with SEM
            if plot_mode == 'traces':
                for trace in data:
                    ax.plot(time_array, trace, color=group_colors[group]+'33', alpha=0.2)  # Lighter traces
            
            elif plot_mode == 'sem':
                ax.fill_between(time_array, mean_psth - sem, mean_psth + sem, color=group_colors[group], alpha=0.2)  # SEM shading

            ax.plot(time_array, mean_psth, label=f'{group}', color=group_colors[group])  # Mean trace

        # Set plot attributes
        ax.set_title(f'PSTH Comparison of {stim_label} between {group1} and {group2}')
        ax.set_xlabel('Time (ms)')
        ax.set_ylabel('Average Spike Rate')
        ax.legend()

        # Only show the plot if an axes object was not provided
        if ax is None:
            plt.tight_layout()
            plt.show()
        
        # Return the DataFrame containing all traces
        return all_traces_df
    
    def plot_psth_comparison_grouprecordings(self, group1, group2, stim_label, 
                                            ax=None, cell_type=None, is_single_unit=None, stim_responsivity=None, 
                                            time_range=None, plot_mode='mean', smoothing_window=None, group_by_recordings=False):
        """
        Plots a PSTH comparison on the provided axes object or creates a new figure if not provided.
        Optionally overlays the stimulator signal as a thin black line.
        Optionally groups data by recordings before calculating mean PSTHs.

        Args:
            group1 (str): First group name.
            group2 (str): Second group name.
            stim_label (str): Stimulation label.
            ax (matplotlib.axes.Axes, optional): Axes object to plot on.
            cell_type (str, optional): Cell type to filter.
            is_single_unit (float, optional): Single unit filter.
            stim_responsivity (float, optional): Stimulus responsivity filter.
            time_range (tuple, optional): Tuple specifying the start and end of the time range (e.g., (-100, 200)).
            plot_mode (str, optional): Plotting mode ('mean', 'traces', 'sem').
            smoothing_window (int, optional): Size of the smoothing window; if None, no smoothing is applied.
            group_by_recordings (bool, optional): If True, calculates the mean PSTHs at the recording level.
        """
        # Define colors
        group_colors = {
            'No_CTZ': '#797979',
            'CTZ': '#5a00c2'
        }
        
        # Fetch data
        df1, df2 = self.compare_groups(group1, group2, stim_label, cell_type, is_single_unit, stim_responsivity)
        if df1.empty or df2.empty:
            print("One of the groups has no data after filtering.")
            return

        # Get the time array and adjust for the specified time range
        time_array = self.eed.relative_time_ms['relative_time_ms']
        if time_range:
            time_mask = (time_array >= time_range[0]) & (time_array <= time_range[1])
            time_array = time_array[time_mask]
        else:
            time_mask = slice(None)

        # Check if we need to create a new figure
        if ax is None:
            fig, ax = plt.subplots(figsize=(10, 5))

        # Helper function to process data
        def process_data(df, group):
            if group_by_recordings:
                grouped = df.groupby('recordingname')
                grouped_data = grouped['PSTHs_conv'].apply(lambda x: np.mean([np.array(i)[time_mask] for i in x], axis=0))
            else:
                grouped_data = df['PSTHs_conv'].apply(lambda x: np.array(x)[time_mask])

            if smoothing_window:
                window = np.ones(smoothing_window) / smoothing_window
                grouped_data = grouped_data.apply(lambda x: np.convolve(x, window, mode='same'))
            
            return grouped_data

        # Process and plot data
        for df, group in zip([df1, df2], [group1, group2]):
            data = process_data(df, group)
            mean_psth = data.apply(pd.Series).mean(axis=0)
            
            if plot_mode == 'sem':
                sem = data.apply(pd.Series).sem(axis=0)

            # Plot individual traces or mean with SEM
            if plot_mode == 'traces':
                for trace in data:
                    ax.plot(time_array, trace, color=group_colors[group]+'33', alpha=0.2)  # Lighter traces
            
            elif plot_mode == 'sem':
                ax.fill_between(time_array, mean_psth - sem, mean_psth + sem, color=group_colors[group], alpha=0.2)  # SEM shading

            ax.plot(time_array, mean_psth, label=f'{group}', color=group_colors[group])  # Mean trace

        # Set plot attributes
        ax.set_title(f'PSTH Comparison of {stim_label} between {group1} and {group2}')
        ax.set_xlabel('Time (ms)')
        ax.set_ylabel('Average Spike Rate')
        ax.legend()

        # Only show the plot if an axes object was not provided
        if ax is None:
            plt.tight_layout()
            plt.show()
        return data 

    def extract_stim_signals(self):
        """
        Extracts and formats the stimulation signals for each relevant stimulation type and synthesizes a flat line for the 'Zero' stimulation.
        """
        stim_voltages = self.eed.StimVoltageTraces_ms['StimVoltageTraces_ms']
        stim_labels = self.eed.trialTagsLabels['trialTagsLabels']  # ['Zero', 'Low', 'Mid', 'Max']

        aligned_signals = {}
        full_length = len(self.eed.relative_time_ms['relative_time_ms'])
        stim_start_index = 500
        stim_end_index = 1000

        # Determine the global maximum voltage to set a unified y-axis for the stimulation signals
        max_voltage = np.max(stim_voltages)

        # Extract and pad signals
        for index, label in enumerate(stim_labels):
            if label in ['Low', 'Mid', 'Max']:
                signal_column_index = ['Low', 'Mid', 'Max'].index(label)
                signal = stim_voltages[:, signal_column_index]

                pre_padding = np.zeros(stim_start_index)
                post_padding = np.zeros(full_length - stim_end_index)
                padded_signal = np.concatenate((pre_padding, signal, post_padding))

                aligned_signals[label] = padded_signal[:1500]

            elif label == 'Zero':
                # Create a flat line using the minimum voltage from 'Low'
                min_voltage = np.min(stim_voltages[:, 0])
                flat_signal = np.full(full_length, min_voltage)
                aligned_signals[label] = flat_signal[:1500]

        return aligned_signals, max_voltage
    
    def extract_stim_signals_opto(self):
        """
        Extracts and formats the stimulation signals for the optogenetic stimulation type. 
        Since there is only one stimulation type and the array is 1D, the method simplifies handling of the array.
        """
        stim_voltages = self.eed.StimVoltageTraces_ms['StimVoltageTrace_ms'] #Trace not Traces for opto
        stim_labels = self.eed.trialTagsLabels['trialTagsLabels']  # This should contain only the optogenetic stimulation label.

        aligned_signals = {}
        full_length = len(self.eed.relative_time_ms['relative_time_ms'])
        stim_start_index = 500
        stim_end_index = 1000

        # Determine the global maximum voltage to set a unified y-axis for the stimulation signals
        max_voltage = np.max(stim_voltages)

        # Extract and pad the signal for the optogenetic stimulation
        for label in stim_labels:
            signal = stim_voltages  # Directly use the 1D array as there's only one type of stimulation

            pre_padding = np.zeros(stim_start_index)
            post_padding = np.zeros(full_length - stim_end_index)
            padded_signal = np.concatenate((pre_padding, signal, post_padding))

            aligned_signals[label] = padded_signal[:1500]

        return aligned_signals, max_voltage

    def plot_psth_with_stim(self, group1, group2, stim_label, ax=None, max_voltage=None, cell_type=None, is_single_unit=None, stim_responsivity=None, 
                            time_range=None, plot_mode='mean', smoothing_window=None):
        """
        Plots a PSTH comparison on the provided axes object or creates a new figure if not provided,
        and overlays the stimulation signal on a secondary y-axis.

        The other parameters function as in the original plot_psth_comparison method.
        """

        # Add debug statement to check input parameters and data
        print(f"Plotting PSTH for groups: {group1}, {group2} with stimulation: {stim_label}")
        
        # Get the time array and adjust for the specified time range
        time_array = self.eed.relative_time_ms['relative_time_ms']
        if time_range:
            time_mask = (time_array >= time_range[0]) & (time_array <= time_range[1])
            time_array = time_array[time_mask]
        else:
            time_mask = slice(None)

        # Check if we need to create a new figure
        if ax is None:
            fig, ax = plt.subplots(figsize=(10, 5))
            created_fig = True
        else:
            created_fig = False

        # Plot PSTH data as before
        traces_df = self.plot_psth_comparison(group1, group2, stim_label, ax=ax, cell_type=cell_type, is_single_unit=is_single_unit, 
                                              stim_responsivity=stim_responsivity, time_range=time_range, plot_mode=plot_mode, 
                                              smoothing_window=smoothing_window)
        
        ax2 = ax.twinx()
        stim_signals, _ = self.extract_stim_signals()
        if stim_label in stim_signals:
            stim_signal = stim_signals[stim_label]  # Assuming full signal is handled correctly
            stim_signal = stim_signal[:1500]
            if time_range:
                stim_signal = stim_signal[time_mask]  # Apply time mask if time range is specified
            ax2.plot(time_array, stim_signal, 'r-', label='Stim Signal', alpha=0.5)
            ax2.set_ylabel('Stimulation Voltage (uV)', color='r')
            ax2.legend(loc='upper right')
            ax2.set_ylim(0, max_voltage)  # Set consistent y-axis scale

        # Only show the plot if an axes object was not provided
        if created_fig:
            plt.tight_layout()
            plt.show()
        
        return traces_df
    
    def plot_psth_with_stim_opto(self, group1, group2, stim_label, ax=None, max_voltage=None, cell_type=None, is_single_unit=None, stim_responsivity=None, time_range=None, plot_mode='mean', smoothing_window=None):
        """
        Plots a PSTH comparison on the provided axes object or creates a new figure if not provided,
        and overlays the stimulation signal on a secondary y-axis.

        The other parameters function as in the original plot_psth_comparison method.
        """

        # Get the time array and adjust for the specified time range
        time_array = self.eed.relative_time_ms['relative_time_ms']
        if time_range:
            time_mask = (time_array >= time_range[0]) & (time_array <= time_range[1])
            time_array = time_array[time_mask]
        else:
            time_mask = slice(None)

        # Check if we need to create a new figure
        if ax is None:
            fig, ax = plt.subplots(figsize=(10, 5))
            created_fig = True
        else:
            created_fig = False

        # Plot PSTH data as before
        self.plot_psth_comparison(group1, group2, stim_label, ax=ax, 
                                  cell_type=cell_type, is_single_unit=is_single_unit, stim_responsivity=stim_responsivity, 
                                  time_range=time_range, plot_mode=plot_mode, smoothing_window=smoothing_window)
        ax2 = ax.twinx()
        stim_signals, _ = self.extract_stim_signals_opto()
        if stim_label in stim_signals:
            stim_signal = stim_signals[stim_label]  # Assuming full signal is handled correctly
            stim_signal = stim_signal[:1500]
            if time_range:
                stim_signal = stim_signal[time_mask]  # Apply time mask if time range is specified
            ax2.plot(time_array, stim_signal, 'r-', label='Stim Signal', alpha=0.5)
            ax2.set_ylabel('LED Stimulation Voltage (uV)', color='r')
            ax2.legend(loc='upper right')
            ax2.set_ylim(0, max_voltage)  # Set consistent y-axis scale

        # Only show the plot if an axes object was not provided
        if created_fig:
            plt.tight_layout()
            plt.show()

    def plot_all_stimulations_with_stim(self, group1, group2, cell_type=None, is_single_unit=None, stim_responsivity=None, time_range=None, plot_mode='mean', smoothing_window=None, directory=None, file_name=None, firstspike_latency=None):
        
        stim_labels = self.eed.trialTagsLabels['trialTagsLabels']  # ['Zero', 'Low', 'Mid', 'Max']
        fig, axs = plt.subplots(1, len(stim_labels), figsize=(20, 5), sharey=True)

        # Retrieve stimulation signals to determine the maximum voltage
        stim_signals, max_voltage = self.extract_stim_signals()

        all_traces_df = pd.DataFrame()
        
        for i, stim_label in enumerate(stim_labels):
            traces_df = self.plot_psth_with_stim(group1, group2, stim_label, axs[i], max_voltage=max_voltage, 
                                                 cell_type=cell_type, is_single_unit=is_single_unit, stim_responsivity=stim_responsivity, 
                                                 time_range=time_range, plot_mode=plot_mode, 
                                                 smoothing_window=smoothing_window, firstspike_latency=firstspike_latency)
            
            traces_df['Stimulation'] = stim_label  # Ensure the correct stim label is assigned
            
            all_traces_df = pd.concat([all_traces_df, traces_df], ignore_index=True)
            axs[i].set_title(stim_label)
            
        # Create directory if it does not exist
        os.makedirs(directory, exist_ok=True)
        
        # Save the figure as an SVG file in the specified directory
        file_path = os.path.join(directory, f'{file_name}.svg')
        fig.savefig(file_path, format='svg', transparent=True)

        plt.tight_layout()
        plt.show()
        
        return all_traces_df
    
    def calculate_basic_stats(self, group1, group2, stim_label=None, baseline_range=(-100, 1), stim_range=(0, 50), cell_type=None, is_single_unit=None, stim_responsivity=None, smoothing_window=None, modulation_label=None):
        """
        Calculates basic statistics for baseline and stimulation windows for two groups across all or a specific stimulation type and stores the detailed data used for these calculations.

        Args:
            group1 (str): First group name.
            group2 (str): Second group name.
            stim_label (str or None): Specific stimulation label to analyze, or None to analyze all.
            baseline_range (tuple): Time range for baseline window.
            stim_range (tuple): Time range for stimulation window.
            cell_type (str, optional): Filter for specific cell types.
            is_single_unit (float, optional): Filter for single units.
            stim_responsivity (float, optional): Filter by stimulus responsivity.
            smoothing_window (int, optional): Size of the smoothing window.
            modulation_label (str, optional): Filter by modulation label ('positive', 'negative', 'none').

        Returns:
            pandas.DataFrame: A DataFrame containing means and standard deviations for the specified windows.
        """
        stim_labels = self.eed.trialTagsLabels['trialTagsLabels']  # ['Zero', 'Low', 'Mid', 'Max']
        if stim_label:
            stim_labels = [stim_label]  # If specific stim_label is provided, use that

        results = []

        for stim in stim_labels:
            df1, df2 = self.compare_groups(group1, group2, stim, cell_type, is_single_unit, 
                                           stim_responsivity, modulation_label)

            # Function to calculate stats and add to DataFrame
            def calculate_and_store_stats(df, time_range, window_label):
                if df is None or df.empty:
                    return df

                time_array = self.eed.relative_time_ms['relative_time_ms']
                time_mask = (time_array >= time_range[0]) & (time_array <= time_range[1])
                
                # Apply the time mask and store the result in a new column safely
                df.loc[:, 'masked_data'] = df['PSTHs_conv'].apply(lambda x: np.array(x)[time_mask])

                if smoothing_window:
                    window = np.ones(smoothing_window) / smoothing_window
                    df.loc[:, 'smoothed_data'] = df['masked_data'].apply(lambda x: np.convolve(x, window, mode='same'))
                else:
                    df.loc[:, 'smoothed_data'] = df['masked_data']

                # Calculate mean and standard deviation and store them in new columns
                df.loc[:, f'mean_{window_label}'] = df['smoothed_data'].apply(np.max)
                df.loc[:, f'std_{window_label}'] = df['smoothed_data'].apply(np.std)

                return df


            # Calculate and store stats in the DataFrames
            df1 = calculate_and_store_stats(df1, baseline_range, 'baseline')
            df1 = calculate_and_store_stats(df1, stim_range, 'stimulation')
            df2 = calculate_and_store_stats(df2, baseline_range, 'baseline')
            df2 = calculate_and_store_stats(df2, stim_range, 'stimulation')

            self.detailed_dataframes[(group1, stim)] = df1
            self.detailed_dataframes[(group2, stim)] = df2

            # Extract group-level results to return
            baseline_stats_group1 = {'mean': df1['mean_baseline'].max(), 'std': df1['std_baseline'].max()}
            stim_stats_group1 = {'mean': df1['mean_stimulation'].max(), 'std': df1['std_stimulation'].max()}
            baseline_stats_group2 = {'mean': df2['mean_baseline'].max(), 'std': df2['std_baseline'].max()}
            stim_stats_group2 = {'mean': df2['mean_stimulation'].max(), 'std': df2['std_stimulation'].max()}

            results.extend([
                {'Group': group1, 'Stimulation': stim, 'Window': 'Baseline', 'Mean': baseline_stats_group1['mean'], 'Std': baseline_stats_group1['std']},
                {'Group': group1, 'Stimulation': stim, 'Window': 'Stimulation', 'Mean': stim_stats_group1['mean'], 'Std': stim_stats_group1['std']},
                {'Group': group2, 'Stimulation': stim, 'Window': 'Baseline', 'Mean': baseline_stats_group2['mean'], 'Std': baseline_stats_group2['std']},
                {'Group': group2, 'Stimulation': stim, 'Window': 'Stimulation', 'Mean': stim_stats_group2['mean'], 'Std': stim_stats_group2['std']}
            ])

        return pd.DataFrame(results)

    def calculate_basic_stats_laminar(self, group1, group2, stim_label=None, baseline_range=(-100, 1), stim_range=(0, 50), cell_type=None, is_single_unit=None, stim_responsivity=None, smoothing_window=None, modulation_label=None):
            """
            Calculates basic statistics for baseline and stimulation windows for two groups across all or a specific stimulation type and stores the detailed data used for these calculations.

            Args:
                group1 (str): First group name.
                group2 (str): Second group name.
                stim_label (str or None): Specific stimulation label to analyze, or None to analyze all.
                baseline_range (tuple): Time range for baseline window.
                stim_range (tuple): Time range for stimulation window.
                cell_type (str, optional): Filter for specific cell types.
                is_single_unit (float, optional): Filter for single units.
                stim_responsivity (float, optional): Filter by stimulus responsivity.
                smoothing_window (int, optional): Size of the smoothing window.
                modulation_label (str, optional): Filter by modulation label ('positive', 'negative', 'none').

            Returns:
                pandas.DataFrame: A DataFrame containing means and standard deviations for the specified windows.
            """
            stim_labels = self.eed.trialTagsLabels['trialTagsLabels']  # ['Zero', 'Low', 'Mid', 'Max']
            if stim_label:
                stim_labels = [stim_label]  # If specific stim_label is provided, use that

            results = []

            for stim in stim_labels:
                df1, df2 = self.compare_groups(group1, group2, stim, cell_type, is_single_unit, 
                                            stim_responsivity, modulation_label)

                # Function to calculate stats and add to DataFrame
                def calculate_and_store_stats(df, time_range, window_label):
                    if df is None or df.empty:
                        return df

                    time_array = self.eed.relative_time_ms['relative_time_ms']
                    time_mask = (time_array >= time_range[0]) & (time_array <= time_range[1])
                    
                    # Apply the time mask and store the result in a new column safely
                    df.loc[:, 'masked_data'] = df['PSTHs_conv'].apply(lambda x: np.array(x)[time_mask])

                    if smoothing_window:
                        window = np.ones(smoothing_window) / smoothing_window
                        df.loc[:, 'smoothed_data'] = df['masked_data'].apply(lambda x: np.convolve(x, window, mode='same'))
                    else:
                        df.loc[:, 'smoothed_data'] = df['masked_data']

                    # Calculate mean and standard deviation and store them in new columns
                    df.loc[:, f'mean_{window_label}'] = df['smoothed_data'].apply(np.max)
                    df.loc[:, f'std_{window_label}'] = df['smoothed_data'].apply(np.std)

                    return df


                # Calculate and store stats in the DataFrames
                df1 = calculate_and_store_stats(df1, baseline_range, 'baseline')
                df1 = calculate_and_store_stats(df1, stim_range, 'stimulation')
                df2 = calculate_and_store_stats(df2, baseline_range, 'baseline')
                df2 = calculate_and_store_stats(df2, stim_range, 'stimulation')

                self.detailed_dataframes[(group1, stim)] = df1
                self.detailed_dataframes[(group2, stim)] = df2

                # Extract group-level results to return
                baseline_stats_group1 = {'mean': df1['mean_baseline'].max(), 'std': df1['std_baseline'].max()}
                stim_stats_group1 = {'mean': df1['mean_stimulation'].max(), 'std': df1['std_stimulation'].max()}
                baseline_stats_group2 = {'mean': df2['mean_baseline'].max(), 'std': df2['std_baseline'].max()}
                stim_stats_group2 = {'mean': df2['mean_stimulation'].max(), 'std': df2['std_stimulation'].max()}

                results.extend([
                    {'Group': group1, 'Stimulation': stim, 'Window': 'Baseline', 'Mean': baseline_stats_group1['mean'], 'Std': baseline_stats_group1['std']},
                    {'Group': group1, 'Stimulation': stim, 'Window': 'Stimulation', 'Mean': stim_stats_group1['mean'], 'Std': stim_stats_group1['std']},
                    {'Group': group2, 'Stimulation': stim, 'Window': 'Baseline', 'Mean': baseline_stats_group2['mean'], 'Std': baseline_stats_group2['std']},
                    {'Group': group2, 'Stimulation': stim, 'Window': 'Stimulation', 'Mean': stim_stats_group2['mean'], 'Std': stim_stats_group2['std']}
                ])

            return pd.DataFrame(results)    
    
    
    def prepare_for_boxplot(self):
        """
        Organizes data into a DataFrame suitable for plotting boxplots. It extracts the 'mean_stimulation'
        values from detailed DataFrames, including labels for stimulation type and group.

        Returns:
            pandas.DataFrame: A DataFrame with columns for 'mean_stimulation', 'Stimulation', and 'Group'.
        """
        boxplot_data = []

        # Iterate over each stored DataFrame key (group and stimulation)
        for (group, stim), df in self.detailed_dataframes.items():
            if not df.empty:  # Corrected check for an empty DataFrame
                # Extract 'mean_stimulation' and corresponding labels
                for index, row in df.iterrows():
                    boxplot_data.append({
                        'mean_stimulation': row['mean_stimulation'],
                        'Stimulation': stim,
                        'Group': group, 
                        'recordingname': row['recordingname'],
                        'ModulationIndex': row['ModulationIndex'], 
                        'cid': row['cid'], 
                        'Cell_Type': row['Cell_Type'], 
                        'Template_Channel': row['Template_Channel'],
                        'LaminarLabel': row['LaminarLabel'],
                        'IsSingleUnit': row['IsSingleUnit'],
                        'TroughToPeak_duration': row['TroughToPeak_duration'],
                        'PeakToPeak_ratio': row['PeakToPeak_ratio'],
                        'peak1_normalized_amplitude': row['peak1_normalized_amplitude'],
                        'Peak1ToTrough_ratio': row['Peak1ToTrough_ratio'],
                        'Peak2ToTrough_ratio': row['Peak2ToTrough_ratio'],
                        'SpikeHalfWidth': row['SpikeHalfWidth'],
                        'MeanFR_baseline': row['MeanFR_baseline'],
                        'StimResponsivity': row['StimResponsivity'],
                        'UnNormalized_Template_Waveform': row['UnNormalized_Template_Waveform'],
                        'Normalized_Template_Waveform': row['Normalized_Template_Waveform'],
                        'FirstSpikeLatency': row['FirstSpikeLatency'],
                        'FirstSpikeLatency_Reliability': row['FirstSpikeLatency_Reliability'],
                        'FanoFactor_baseline': row['FanoFactor_baseline'],
                        'FanoFactor_stim': row['FanoFactor_stim']
                    })

        # Convert list of data to DataFrame
        boxplot_df = pd.DataFrame(boxplot_data)

        return boxplot_df
    
    def plot_box_and_strip(self, df, groups=None, stimulations=None, show_outliers=True, hue_order=None, directory=None, file_name=None, ylim=None, modulation_label=None):
        """
        Plots boxplots and stripplots for specified groups and stimulations, with color adjustments made directly in the plotting calls.

        Args:
            df (pandas.DataFrame): DataFrame containing the data to plot made with prepare_for_boxplot.
            groups (list of str, optional): List of groups to include in the plot.
            stimulations (list of str, optional): List of stimulations to include in the plot.
            show_outliers (bool, optional): Whether to show outliers.
            hue_order (list, optional): Order of the hue levels.
        """
        # Define color mapping for groups
        group_colors = {
            'No_CTZ': '#797979',
            'CTZ': '#5a00c2'
        }

        # Generate lighter versions for the box face color
        lightened_colors = {k: v + '33' for k, v in group_colors.items()}

        # Boxplot customization
        boxprops = {'edgecolor': 'k', 'linewidth': 2}
        whiskerprops = {'color': 'k', 'linewidth': 2}
        boxplot_kwargs = {
            'boxprops': boxprops,
            'medianprops': whiskerprops,
            'whiskerprops': whiskerprops,
            'capprops': {'linewidth': 0},  # Hide the caps
            'showfliers': show_outliers,
            'palette': group_colors,
            'hue_order': hue_order,
            'width': 0.75
        }

        # Stripplot customization
        stripplot_kwargs = {
            'linewidth': 0.6,
            'size': 6,
            'alpha': 0.7,
            'jitter': True,
            'dodge': True,
            'marker': 'o' if show_outliers else 'd',
            'palette': lightened_colors,
            'hue_order': hue_order
        }

     
        # Filter by specified groups and stimulations
        boxplot_df = df
        # Filter by specified groups and stimulations
        if groups:
            boxplot_df = boxplot_df[boxplot_df['Group'].isin(groups)]
        if stimulations:
            boxplot_df = boxplot_df[boxplot_df['Stimulation'].isin(stimulations)]
        if modulation_label:
            boxplot_df = boxplot_df[boxplot_df['ModulationIndex'] == modulation_label]

        # Plotting
        plt.figure(figsize=(6, 6))
        ax = sns.boxplot(data=boxplot_df, x='Stimulation', y='mean_stimulation', hue='Group', **boxplot_kwargs)

        # Manually set the facecolor for boxplot
        for i, artist in enumerate(ax.artists):
            col = lightened_colors[ax.get_legend_handles_labels()[1][i // len(stimulations)]]
            artist.set_facecolor(col)

        # Add stripplot on top of boxplot for raw data visualization
        sns.stripplot(data=boxplot_df, x='Stimulation', y='mean_stimulation', hue='Group', **stripplot_kwargs)
        
        #control the upper and lower limits of the y-axis
        if ylim is not None:
            plt.ylim(ylim)

        # Enhance the plot
        plt.title('Comparison of Mean Stimulation Across Groups and Stimulations')
        plt.ylabel('Mean Stimulation')
        plt.xlabel('Stimulation Type')
        ax.legend(title='Group')
        
        # Prompt user for directory and file name if not provided
        if directory is None:
            directory = input("Please enter the directory to save the plot: ")
        if file_name is None:
            file_name = input("Please enter the file name to save the plot: ")

        # Create directory if it does not exist
        os.makedirs(directory, exist_ok=True)

        # Save the figure as an SVG file in the specified directory
        file_path = os.path.join(directory, f'{file_name}.svg')
        plt.savefig(file_path, format='svg', transparent=True)

        plt.show()
        
        return boxplot_df # Return the DataFrame for further analysis

    def plot_box_and_strip_with_controls(self, df, groups=None, stimulations=None, show_outliers=True, show_scatter=True, hue_order=None, directory=None, file_name=None, ylim=None, modulation_label=None):
        """
        Plots boxplots and optional stripplots for specified groups and stimulations, with control over outlier and scatter display.
        
        Args:
        df (pandas.DataFrame): DataFrame containing the data to plot made with prepare_for_boxplot.
        groups (list of str, optional): List of groups to include in the plot.
        stimulations (list of str, optional): List of stimulations to include in the plot.
        show_outliers (bool, optional): Whether to show outliers in both boxplot and stripplot. Default is True.
        show_scatter (bool, optional): Whether to show the scatter plot (stripplot). Default is True.
        hue_order (list, optional): Order of the hue levels.
        directory (str, optional): Directory to save the plot.
        file_name (str, optional): File name to save the plot.
        ylim (tuple, optional): Y-axis limits.
        modulation_label (str, optional): Label for modulation index filtering.
        """
        # Define color mapping for groups
        group_colors = {
            'No_CTZ': '#797979',
            'CTZ': '#5a00c2'
        }
        # Generate lighter versions for the box face color
        lightened_colors = {k: v + '33' for k, v in group_colors.items()}

        # Filter by specified groups and stimulations
        boxplot_df = df
        if groups:
            boxplot_df = boxplot_df[boxplot_df['Group'].isin(groups)]
        if stimulations:
            boxplot_df = boxplot_df[boxplot_df['Stimulation'].isin(stimulations)]
        if modulation_label:
            boxplot_df = boxplot_df[boxplot_df['ModulationIndex'] == modulation_label]

        # Calculate outliers
        Q1 = boxplot_df.groupby(['Stimulation', 'Group'])['mean_stimulation'].transform('quantile', 0.25)
        Q3 = boxplot_df.groupby(['Stimulation', 'Group'])['mean_stimulation'].transform('quantile', 0.75)
        IQR = Q3 - Q1
        outlier_mask = (boxplot_df['mean_stimulation'] < Q1 - 1.5 * IQR) | (boxplot_df['mean_stimulation'] > Q3 + 1.5 * IQR)
        
        # Separate outliers and non-outliers
        outliers = boxplot_df[outlier_mask]
        non_outliers = boxplot_df[~outlier_mask]

        # Boxplot customization
        boxprops = {'edgecolor': 'k', 'linewidth': 2}
        whiskerprops = {'color': 'k', 'linewidth': 2}
        boxplot_kwargs = {
            'boxprops': boxprops,
            'medianprops': whiskerprops,
            'whiskerprops': whiskerprops,
            'capprops': {'linewidth': 0},  # Hide the caps
            'showfliers': show_outliers,
            'palette': group_colors,
            'hue_order': hue_order,
            'width': 0.75
        }

        # Stripplot customization
        stripplot_kwargs = {
            'linewidth': 0.6,
            'size': 6,
            'alpha': 0.7,
            'jitter': True,
            'dodge': True,
            'marker': 'o',
            'palette': lightened_colors,
            'hue_order': hue_order
        }

        # Plotting
        plt.figure(figsize=(6, 6))
        ax = sns.boxplot(data=boxplot_df, x='Stimulation', y='mean_stimulation', hue='Group', **boxplot_kwargs)

        # Manually set the facecolor for boxplot
        for i, artist in enumerate(ax.artists):
            col = lightened_colors[ax.get_legend_handles_labels()[1][i // len(stimulations)]]
            artist.set_facecolor(col)

        # Add stripplot if show_scatter is True
        if show_scatter:
            # Add stripplot for non-outliers
            sns.stripplot(data=non_outliers, x='Stimulation', y='mean_stimulation', hue='Group', **stripplot_kwargs)

            # Add stripplot for outliers if show_outliers is True
            if show_outliers:
                sns.stripplot(data=outliers, x='Stimulation', y='mean_stimulation', hue='Group', 
                            **{**stripplot_kwargs, 'marker': 'D', 'size': 8})

        # Control the upper and lower limits of the y-axis
        if ylim is not None:
            plt.ylim(ylim)

        # Enhance the plot
        plt.title('Comparison of Mean Stimulation Across Groups and Stimulations')
        plt.ylabel('Mean Stimulation')
        plt.xlabel('Stimulation Type')
        ax.legend(title='Group')

        # Prompt user for directory and file name if not provided
        if directory is None:
            directory = input("Please enter the directory to save the plot: ")
        if file_name is None:
            file_name = input("Please enter the file name to save the plot: ")

        # Create directory if it does not exist
        os.makedirs(directory, exist_ok=True)

        # Save the figure as an SVG file in the specified directory
        file_path = os.path.join(directory, f'{file_name}.svg')
        plt.savefig(file_path, format='svg', transparent=True)
        plt.show()

        return boxplot_df  # Return the DataFrame for further analysis

    def plot_mean_and_sem_lineplot(self, df, groups=None, stimulations=None, show_outliers=True, hue_order=None, directory=None, file_name=None, ylim=None, modulation_label=None, file_format='svg'):
        """
        Plots mean and SEM bars for specified groups and stimulations, with color adjustments.

        Args:
            df (pandas.DataFrame): DataFrame containing the data to plot made with prepare_for_boxplot 
            groups (list of str, optional): List of groups to include in the plot.
            stimulations (list of str, optional): List of stimulations to include in the plot.
            show_outliers (bool, optional): Whether to show outliers.
            hue_order (list, optional): Order of the hue levels.
            directory (str, optional): Directory to save the plot.
            file_name (str, optional): File name to save the plot.
            ylim (tuple, optional): Y-axis limits for the plot.
            modulation_label (str, optional): Label to filter modulation.
            firstspike_latency (bool, optional): Whether to filter by first spike latency.
            file_format (str, optional): Format to save the plot ('svg' or 'png'). Default is 'svg'.
        """
        # Validate the file format
        if file_format not in ['svg', 'png']:
            raise ValueError("file_format must be either 'svg' or 'png'")

        # Define color mapping for groups
        group_colors = {
            'No_CTZ': '#797979',
            'CTZ': '#5a00c2'
        }

        # Define the order of stimulations
        stim_order = ['Zero', 'Low', 'Mid', 'Max']

        # Prepare data for plotting
        boxplot_df = df

        # Filter by specified groups and stimulations
        #if groups:
        #    boxplot_df = boxplot_df[boxplot_df['Group'].isin(groups)]
        #if stimulations:
        #    boxplot_df = boxplot_df[boxplot_df['Stimulation'].isin(stimulations)]
        #if modulation_label:
        #    boxplot_df = boxplot_df[boxplot_df['ModulationIndex'] == modulation_label]

        # Calculate mean and SEM
        summary_df = boxplot_df.groupby(['Stimulation', 'Group']).agg(
            mean_stimulation=('mean_stimulation', 'mean'),
            sem_stimulation=('mean_stimulation', 'sem')
        ).reset_index()

        # Ensure the stimulations are ordered correctly
        summary_df['Stimulation'] = pd.Categorical(summary_df['Stimulation'], categories=stim_order, ordered=True)

        # Plotting
        plt.figure(figsize=(6, 6))
        ax = sns.lineplot(data=summary_df, x='Stimulation', y='mean_stimulation', hue='Group', palette=group_colors, hue_order=hue_order, markers=True, style='Group', markersize=10, linewidth=2.5)

        # Add error bars
        for group in summary_df['Group'].unique():
            group_data = summary_df[summary_df['Group'] == group]
            plt.errorbar(group_data['Stimulation'], group_data['mean_stimulation'], yerr=group_data['sem_stimulation'], fmt='o', c=group_colors[group], capsize=5, elinewidth=2, markersize=10)

        # Control the upper and lower limits of the y-axis
        if ylim is not None:
            plt.ylim(ylim)

        # Enhance the plot
        plt.title('Comparison of Mean Stimulation Across Groups and Stimulations')
        plt.ylabel('Mean Stimulation')
        plt.xlabel('Stimulation Type')
        ax.legend(title='Group')

        # Prompt user for directory and file name if not provided
        if directory is None:
            directory = input("Please enter the directory to save the plot: ")
        if file_name is None:
            file_name = input("Please enter the file name to save the plot: ")

        # Create directory if it does not exist
        os.makedirs(directory, exist_ok=True)

        # Save the figure as an SVG or PNG file in the specified directory
        file_path = os.path.join(directory, f'{file_name}.{file_format}')
        plt.savefig(file_path, format=file_format, transparent=True)

        plt.show()

        
    def plot_rasters_for_cid(self, groupname, recordingname, cid, time_window=None):
        """
        Plots raster plots for a specific cid within a specific group and recording across all stimulations,
        with an optional custom time window.

        Args:
            groupname (str): The name of the group.
            recordingname (str): The name of the recording.
            cid (str): The cell ID for which to plot the raster plots.
            time_window (tuple, optional): The window of time to plot, within the range -500 to 999 ms. 
                                           Default is None, which uses the full range.
        """
        stim_labels = self.eed.trialTagsLabels['trialTagsLabels']  # ['Zero', 'Low', 'Mid', 'Max']
        time_array = self.eed.relative_time_ms['relative_time_ms']  # Full time array from -500 to 999 ms

        # Determine the indices for slicing time_array based on the specified or default time window
        if time_window is not None:
            start_index = np.searchsorted(time_array, time_window[0])
            end_index = np.searchsorted(time_array, time_window[1], side='right')
        else:
            start_index, end_index = 0, len(time_array)  # Use full range if no window is specified
        
        # Prepare the figure
        fig, axes = plt.subplots(1, 4, figsize=(20, 5), sharey=True)
        fig.suptitle(f'Raster plots for CID: {cid}, Group: {groupname}, Recording: {recordingname}')

        # Iterate through each stimulation type
        for i, stim in enumerate(stim_labels):
            ax = axes[i]
            df_name = f'psth_dataframe_{stim}'
            if df_name in self.dataframes:
                df = self.dataframes[df_name]
                # Filter data for specific cid
                condition = (df['groupname'] == groupname) & (df['recordingname'] == recordingname) & (df['cid'] == cid)
                spike_trains = df.loc[condition, 'SpikeTrains_for_PSTHs'].values #a numpy.ndarry of shape(1,) and size 1 which contains a list of arrays
                
                # Check if there is any data to plot
                if spike_trains.size > 0:
                    spike_trains = spike_trains[0]  # extract the binary spike trains with numpy.ndarray of shape (n_trials, n_time_points)
                    # Plot each trial's spikes within the time window
                    for trial_index, trial in enumerate(spike_trains): #enumerate over the trials which are the rows of the spike_trains. trial is a numpy.ndarray of shape (n_time_points,) and trial_index is the index of the trial
                        spikes = np.where(trial == 1)[0]  # Get indices where spikes occur
                        spikes = spikes[(spikes >= start_index) & (spikes < end_index)]  # Filter spikes by time window
                        spike_times = time_array[spikes]  # Convert indices to times
                        ax.eventplot(spike_times, lineoffsets=trial_index + 1, linelengths=1, colors='black')

                    ax.set_xlim(time_window[0], time_window[1]) if time_window else ax.set_xlim(time_array[start_index], time_array[end_index-1])
                    ax.set_title(f'Stimulation: {stim}')
                    ax.set_xlabel('Time (ms)')
                    if i == 0:
                        ax.set_ylabel('Trial')

        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.show()

    def plot_rasters_for_cid_2msbins(self, groupname, recordingname, cid, time_window=None):
        """
        Plots raster plots for a specific cid within a specific group and recording across all stimulations,
        with an optional custom time window.

        Args:
            groupname (str): The name of the group.
            recordingname (str): The name of the recording.
            cid (str): The cell ID for which to plot the raster plots.
            time_window (tuple, optional): The window of time to plot, within the range -500 to 999 ms. 
                                        Default is None, which uses the full range.
        """
        stim_labels = self.eed.trialTagsLabels['trialTagsLabels']  # ['Zero', 'Low', 'Mid', 'Max']
        time_array = self.eed.relative_time_ms['relative_time_ms']  # Full time array from -500 to 999 ms

        # Create a new time array with 2ms bins
        new_time_array = np.arange(time_array[0], time_array[-1] + 1, 2)

        # Determine the indices for slicing time_array based on the specified or default time window
        if time_window is not None:
            start_index = np.searchsorted(new_time_array, time_window[0])
            end_index = np.searchsorted(new_time_array, time_window[1], side='right')
        else:
            start_index, end_index = 0, len(new_time_array)  # Use full range if no window is specified
        
        # Prepare the figure
        fig, axes = plt.subplots(1, 4, figsize=(20, 5), sharey=True)
        fig.suptitle(f'Raster plots for CID: {cid}, Group: {groupname}, Recording: {recordingname}')

        # Iterate through each stimulation type
        for i, stim in enumerate(stim_labels):
            ax = axes[i]
            df_name = f'psth_dataframe_{stim}'
            if df_name in self.dataframes:
                df = self.dataframes[df_name]
                # Filter data for specific cid
                condition = (df['groupname'] == groupname) & (df['recordingname'] == recordingname) & (df['cid'] == cid)
                spike_trains = df.loc[condition, 'SpikeTrains_for_PSTHs'].values #a numpy.ndarry of shape(1,) and size 1 which contains a list of arrays
                
                # Check if there is any data to plot
                if spike_trains.size > 0:
                    spike_trains = spike_trains[0]  # extract the binary spike trains with numpy.ndarray of shape (n_trials, n_time_points)
                    
                    # Bin the spike trains into 2ms intervals
                    binned_spike_trains = np.add.reduceat(spike_trains, np.arange(0, spike_trains.shape[1], 2), axis=1)
                    
                    # Plot each trial's spikes within the time window
                    for trial_index, trial in enumerate(binned_spike_trains): #enumerate over the trials which are the rows of the spike_trains. trial is a numpy.ndarray of shape (n_time_points,) and trial_index is the index of the trial
                        spikes = np.where(trial > 0)[0]  # Get indices where spikes occur (considering 2ms bins)
                        spikes = spikes[(spikes >= start_index) & (spikes < end_index)]  # Filter spikes by time window
                        spike_times = new_time_array[spikes]  # Convert indices to times
                        ax.eventplot(spike_times, lineoffsets=trial_index + 1, linelengths=1, colors='black')

                    ax.set_xlim(time_window[0], time_window[1]) if time_window else ax.set_xlim(new_time_array[start_index], new_time_array[end_index-1])
                    ax.set_title(f'Stimulation: {stim}')
                    ax.set_xlabel('Time (ms)')
                    if i == 0:
                        ax.set_ylabel('Trial')

        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.show()


    def plot_rasters_for_cid_changebinsize(self, groupname, recordingname, cid, time_window=None, bin_size=1, filter_empty_trials=False, recording_dir=None):
        """
        Plots raster plots for a specific cid within a specific group and recording across all stimulations,
        with an optional custom time window and bin size.

        Args:
            groupname (str): The name of the group.
            recordingname (str): The name of the recording.
            cid (str): The cell ID for which to plot the raster plots.
            time_window (tuple, optional): The window of time to plot, within the range -500 to 999 ms. 
                                        Default is None, which uses the full range.
            bin_size (int, optional): The size of the bins in milliseconds. Default is 1ms.
            filter_empty_trials (bool, optional): If True, only plots trials with at least one spike. Default is False.
        """
        stim_labels = self.eed.trialTagsLabels['trialTagsLabels']  # ['Zero', 'Low', 'Mid', 'Max']
        time_array = self.eed.relative_time_ms['relative_time_ms']  # Full time array from -500 to 999 ms

        # Create a new time array with the specified bin size
        new_time_array = np.arange(time_array[0], time_array[-1] + 1, bin_size)

        # Determine the indices for slicing time_array based on the specified or default time window
        if time_window is not None:
            start_index = np.searchsorted(new_time_array, time_window[0])
            end_index = np.searchsorted(new_time_array, time_window[1], side='right')
        else:
            start_index, end_index = 0, len(new_time_array)  # Use full range if no window is specified
        
        # Prepare the figure
        fig, axes = plt.subplots(1, 4, figsize=(20, 5), sharey=True)
        fig.suptitle(f'Raster plots for CID: {cid}, Group: {groupname}, Recording: {recordingname}')

        # Iterate through each stimulation type
        for i, stim in enumerate(stim_labels):
            ax = axes[i]
            df_name = f'psth_dataframe_{stim}'
            if df_name in self.dataframes:
                df = self.dataframes[df_name]
                # Filter data for specific cid
                condition = (df['groupname'] == groupname) & (df['recordingname'] == recordingname) & (df['cid'] == cid)
                spike_trains = df.loc[condition, 'SpikeTrains_for_PSTHs'].values
                
                # Check if there is any data to plot
                if spike_trains.size > 0:
                    spike_trains = spike_trains[0]  # extract the binary spike trains
                    
                    # Bin the spike trains into the specified bin size
                    binned_spike_trains = np.add.reduceat(spike_trains, np.arange(0, spike_trains.shape[1], bin_size), axis=1)
                    
                    # Optionally filter out trials without any spikes
                    if filter_empty_trials:
                        trial_spike_counts = binned_spike_trains.sum(axis=1)
                        binned_spike_trains = binned_spike_trains[trial_spike_counts > 0]
                    
                    # Plot each trial's spikes within the time window
                    for trial_index, trial in enumerate(binned_spike_trains):
                        spikes = np.where(trial > 0)[0]  # Get indices where spikes occur (considering bin size)
                        spikes = spikes[(spikes >= start_index) & (spikes < end_index)]  # Filter spikes by time window
                        spike_times = new_time_array[spikes]  # Convert indices to times
                        ax.eventplot(spike_times, lineoffsets=trial_index + 1, linelengths=1, colors='black')

                    ax.set_xlim(time_window[0], time_window[1]) if time_window else ax.set_xlim(new_time_array[start_index], new_time_array[end_index-1])
                    ax.set_title(f'Stimulation: {stim}')
                    ax.set_xlabel('Time (ms)')
                    if i == 0:
                        ax.set_ylabel('Trial')

        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        save_path = os.path.join(recording_dir, f'raster_{groupname}_{recordingname}_{cid}.svg')
        fig.savefig(save_path, format='svg', transparent=True)
        plt.close(fig)



        print("Raster plots have been saved successfully.")


    def smooth_data(self, data, window_size=5):
        """Smooths data using a moving average filter with a specified window size."""
        window = np.ones(int(window_size)) / float(window_size)
        return np.convolve(data, window, 'same')
    
    def plot_combined_psth_and_raster(self, groupname, recordingname, cid, time_window=None, smoothing_window=5, show=False):
        stim_labels = self.eed.trialTagsLabels['trialTagsLabels']
        time_array = self.eed.relative_time_ms['relative_time_ms']

        fig, axes = plt.subplots(3, 4, figsize=(24, 15))  # Adjusted for 3 rows of plots
        fig.suptitle(f'Comprehensive Neural Response Analysis for CID: {cid}, Group: {groupname}, Recording: {recordingname}')

        for i, stim in enumerate(stim_labels):
            smoothed_psth_ax = axes[0, i]
            raw_psth_ax = axes[1, i]
            raster_ax = axes[2, i]
            df_name = f'psth_dataframe_{stim}'

            if df_name in self.dataframes:
                df = self.dataframes[df_name]
                condition = (df['groupname'] == groupname) & (df['recordingname'] == recordingname) & (df['cid'] == cid)
                spike_trains = df.loc[condition, 'SpikeTrains_for_PSTHs'].values

                if spike_trains.size > 0:
                    spike_trains = spike_trains[0]
                    # Calculate PSTH
                    all_spikes = np.concatenate([np.where(trial == 1)[0] for trial in spike_trains])
                    counts, _ = np.histogram(all_spikes, bins=len(time_array), range=(0, len(time_array)))

                    # Apply time window if specified
                    if time_window:
                        start_idx = np.searchsorted(time_array, time_window[0])
                        end_idx = np.searchsorted(time_array, time_window[1], side='right')
                        displayed_time_array = time_array[start_idx:end_idx]
                        displayed_counts = counts[start_idx:end_idx]
                    else:
                        displayed_time_array = time_array
                        displayed_counts = counts

                    # Smooth the data for the smoothed PSTH
                    smoothed_counts = self.smooth_data(displayed_counts, smoothing_window)

                    # Plot Smoothed PSTH
                    smoothed_psth_ax.bar(displayed_time_array, smoothed_counts, width=1, align='edge', color='skyblue')
                    smoothed_psth_ax.set_title(f'{stim} Smoothed PSTH')
                    smoothed_psth_ax.set_xlim(displayed_time_array[0], displayed_time_array[-1])

                    # Plot Raw PSTH
                    raw_psth_ax.bar(displayed_time_array, displayed_counts, width=1, align='edge', color='gray')
                    raw_psth_ax.set_title(f'{stim} Raw PSTH')
                    raw_psth_ax.set_xlim(displayed_time_array[0], displayed_time_array[-1])

                    # Plot Raster
                    for trial_index, trial in enumerate(spike_trains):
                        spike_times = time_array[np.where(trial == 1)]
                        raster_ax.eventplot(spike_times, lineoffsets=trial_index + 1, linelengths=1)
                    raster_ax.set_title(f'{stim} Raster')
                    raster_ax.set_xlim(displayed_time_array[0], displayed_time_array[-1])

        plt.tight_layout()
        if show:
            plt.show()
        return fig

    def plot_combined_psth_and_raster_changebinsize(self, groupname, recordingname, cid, time_window=None, smoothing_window=5, bin_size=1, show=False):
        """
        Plots combined PSTH and raster plots for a specific cid within a specific group and recording across all stimulations,
        with an optional custom time window, smoothing window, and bin size.

        Args:
            groupname (str): The name of the group.
            recordingname (str): The name of the recording.
            cid (str): The cell ID for which to plot the PSTH and raster plots.
            time_window (tuple, optional): The window of time to plot, within the range -500 to 999 ms. 
                                        Default is None, which uses the full range.
            smoothing_window (int, optional): The size of the smoothing window for the smoothed PSTH. Default is 5.
            bin_size (int, optional): The size of the bins in milliseconds for the PSTH and raster. Default is 1ms.
            show (bool, optional): Whether to display the plot. Default is False.
        """
        stim_labels = self.eed.trialTagsLabels['trialTagsLabels']
        time_array = self.eed.relative_time_ms['relative_time_ms']

        # Create a new time array with the specified bin size
        new_time_array = np.arange(time_array[0], time_array[-1] + 1, bin_size)

        fig, axes = plt.subplots(3, 4, figsize=(24, 15))  # Adjusted for 3 rows of plots
        fig.suptitle(f'Comprehensive Neural Response Analysis for CID: {cid}, Group: {groupname}, Recording: {recordingname}')

        max_smoothed_counts = 0
        max_raw_counts = 0

        all_smoothed_counts = []
        all_raw_counts = []

        for i, stim in enumerate(stim_labels):
            df_name = f'psth_dataframe_{stim}'

            if df_name in self.dataframes:
                df = self.dataframes[df_name]
                condition = (df['groupname'] == groupname) & (df['recordingname'] == recordingname) & (df['cid'] == cid)
                spike_trains = df.loc[condition, 'SpikeTrains_for_PSTHs'].values

                if spike_trains.size > 0:
                    spike_trains = spike_trains[0]

                    # Bin the spike trains into the specified bin size
                    binned_spike_trains = np.add.reduceat(spike_trains, np.arange(0, spike_trains.shape[1], bin_size), axis=1)

                    # Calculate PSTH
                    all_spikes = np.concatenate([np.where(trial > 0)[0] for trial in binned_spike_trains])
                    counts, _ = np.histogram(all_spikes, bins=len(new_time_array), range=(0, len(new_time_array)))

                    # Apply time window if specified
                    if time_window:
                        start_idx = np.searchsorted(new_time_array, time_window[0])
                        end_idx = np.searchsorted(new_time_array, time_window[1], side='right')
                        displayed_time_array = new_time_array[start_idx:end_idx]
                        displayed_counts = counts[start_idx:end_idx]
                    else:
                        displayed_time_array = new_time_array
                        displayed_counts = counts

                    # Smooth the data for the smoothed PSTH
                    smoothed_counts = self.smooth_data(displayed_counts, smoothing_window)

                    all_smoothed_counts.append(smoothed_counts)
                    all_raw_counts.append(displayed_counts)

                    max_smoothed_counts = max(max_smoothed_counts, np.max(smoothed_counts))
                    max_raw_counts = max(max_raw_counts, np.max(displayed_counts))

        # Set y-limits based on the maximum counts across all stimulations
        ylimit_smoothed = 1.1 * max_smoothed_counts
        ylimit_raw = 1.1 * max_raw_counts

        for i, stim in enumerate(stim_labels):
            smoothed_psth_ax = axes[0, i]
            raw_psth_ax = axes[1, i]
            raster_ax = axes[2, i]
            df_name = f'psth_dataframe_{stim}'

            if df_name in self.dataframes:
                df = self.dataframes[df_name]
                condition = (df['groupname'] == groupname) & (df['recordingname'] == recordingname) & (df['cid'] == cid)
                spike_trains = df.loc[condition, 'SpikeTrains_for_PSTHs'].values

                if spike_trains.size > 0:
                    spike_trains = spike_trains[0]

                    # Bin the spike trains into the specified bin size
                    binned_spike_trains = np.add.reduceat(spike_trains, np.arange(0, spike_trains.shape[1], bin_size), axis=1)

                    # Calculate PSTH
                    all_spikes = np.concatenate([np.where(trial > 0)[0] for trial in binned_spike_trains])
                    counts, _ = np.histogram(all_spikes, bins=len(new_time_array), range=(0, len(new_time_array)))

                    # Apply time window if specified
                    if time_window:
                        start_idx = np.searchsorted(new_time_array, time_window[0])
                        end_idx = np.searchsorted(new_time_array, time_window[1], side='right')
                        displayed_time_array = new_time_array[start_idx:end_idx]
                        displayed_counts = counts[start_idx:end_idx]
                    else:
                        displayed_time_array = new_time_array
                        displayed_counts = counts

                    # Smooth the data for the smoothed PSTH
                    smoothed_counts = self.smooth_data(displayed_counts, smoothing_window)

                    # Plot Smoothed PSTH
                    smoothed_psth_ax.bar(displayed_time_array, smoothed_counts, width=bin_size, align='edge', color='skyblue')
                    smoothed_psth_ax.set_title(f'{stim} Smoothed PSTH')
                    smoothed_psth_ax.set_xlim(displayed_time_array[0], displayed_time_array[-1])
                    smoothed_psth_ax.set_ylim(0, ylimit_smoothed)

                    # Plot Raw PSTH
                    raw_psth_ax.bar(displayed_time_array, displayed_counts, width=bin_size, align='edge', color='gray')
                    raw_psth_ax.set_title(f'{stim} Raw PSTH')
                    raw_psth_ax.set_xlim(displayed_time_array[0], displayed_time_array[-1])
                    raw_psth_ax.set_ylim(0, ylimit_raw)

                    # Plot Raster
                    for trial_index, trial in enumerate(binned_spike_trains):
                        spike_times = new_time_array[np.where(trial > 0)]
                        spike_times = spike_times[(spike_times >= displayed_time_array[0]) & (spike_times < displayed_time_array[-1])]
                        raster_ax.eventplot(spike_times, lineoffsets=trial_index + 1, linelengths=1, color='black')
                    raster_ax.set_title(f'{stim} Raster')
                    raster_ax.set_xlim(displayed_time_array[0], displayed_time_array[-1])

        plt.tight_layout()


    def extract_psth_and_raster_data(self, groupname, recordingname, cid, time_window=None, bin_size=1, smoothing_window=5):
        """
        Extracts the PSTH and raster data for a specific cid within a specific group and recording across all stimulations.

        Args:
            groupname (str): The name of the group.
            recordingname (str): The name of the recording.
            cid (str): The cell ID for which to extract the PSTH and raster data.
            time_window (tuple, optional): The window of time to extract, within the range -500 to 999 ms. 
                                        Default is None, which uses the full range.
            bin_size (int, optional): The size of the bins in milliseconds for the PSTH and raster. Default is 1ms.
            smoothing_window (int, optional): The size of the smoothing window for the smoothed PSTH. Default is 5.

        Returns:
            dict: A dictionary containing the extracted data. The structure of the dictionary is as follows:
            
            {
                'groupname': str,  # The name of the group
                'recordingname': str,  # The name of the recording
                'cid': str,  # The cell ID
                'stim_labels': list of str,  # List of stimulation labels
                'time_array': numpy.ndarray,  # Array of time points based on the specified bin size (1D array, length depends on bin size)
                'data': {  # Nested dictionary containing data for each stimulation
                    'stim_label_1': {  # Replace 'stim_label_1' with actual stimulation labels
                        'spike_trains': numpy.ndarray,  # Binned spike trains (2D array, shape: [number of trials, length of new_time_array])
                        'raw_counts': numpy.ndarray,  # Raw spike counts for each time bin (1D array, length depends on time window and bin size)
                        'smoothed_counts': numpy.ndarray,  # Smoothed spike counts for each time bin (1D array, same length as raw_counts)
                        'time_array': numpy.ndarray  # Array of time points for the specified time window (1D array, length depends on time window and bin size)
                    },
                    'stim_label_2': { ... },  # Repeat for each stimulation label
                    ...
                }
            }
        """
        stim_labels = self.eed.trialTagsLabels['trialTagsLabels']
        time_array = self.eed.relative_time_ms['relative_time_ms']

        # Create a new time array with the specified bin size
        new_time_array = np.arange(time_array[0], time_array[-1] + 1, bin_size)

        extracted_data = {
            'groupname': groupname,
            'recordingname': recordingname,
            'cid': cid,
            'stim_labels': stim_labels,
            'time_array': new_time_array,
            'data': {}
        }

        for stim in stim_labels:
            df_name = f'psth_dataframe_{stim}'

            if df_name in self.dataframes:
                df = self.dataframes[df_name]
                condition = (df['groupname'] == groupname) & (df['recordingname'] == recordingname) & (df['cid'] == cid)
                spike_trains = df.loc[condition, 'SpikeTrains_for_PSTHs'].values

                if spike_trains.size > 0:
                    spike_trains = spike_trains[0]

                    # Bin the spike trains into the specified bin size
                    binned_spike_trains = np.add.reduceat(spike_trains, np.arange(0, spike_trains.shape[1], bin_size), axis=1)

                    # Calculate PSTH
                    all_spikes = np.concatenate([np.where(trial > 0)[0] for trial in binned_spike_trains])
                    counts, _ = np.histogram(all_spikes, bins=len(new_time_array), range=(0, len(new_time_array)))
                    
                    # Apply time window if specified
                    if time_window:
                        start_idx = np.searchsorted(new_time_array, time_window[0])
                        end_idx = np.searchsorted(new_time_array, time_window[1], side='right')
                        displayed_time_array = new_time_array[start_idx:end_idx]
                        displayed_counts = counts[start_idx:end_idx]
                    else:
                        displayed_time_array = new_time_array
                        displayed_counts = counts

                    # Smooth the data for the smoothed PSTH
                    smoothed_counts = self.smooth_data(displayed_counts, smoothing_window)

                    extracted_data['data'][stim] = {
                        'spike_trains': binned_spike_trains,
                        'raw_counts': displayed_counts,
                        'smoothed_counts': smoothed_counts,
                        'time_array': displayed_time_array
                    }

        return extracted_data

    def plot_psth_and_raster(self, extracted_data, show=False):
        """
        Plots combined PSTH and raster plots using the extracted data.

        Args:
            extracted_data (dict): The data extracted by the extract_psth_and_raster_data method. 
                                The structure of the dictionary should be:
                                {
                                    'groupname': str,
                                    'recordingname': str,
                                    'cid': str,
                                    'stim_labels': list of str,
                                    'time_array': numpy.ndarray,
                                    'data': {
                                        'stim_label_1': {
                                            'spike_trains': numpy.ndarray,
                                            'raw_counts': numpy.ndarray,
                                            'smoothed_counts': numpy.ndarray,
                                            'time_array': numpy.ndarray
                                        },
                                        ...
                                    }
                                }
            show (bool, optional): Whether to display the plot. Default is False.
        """
        fig, axes = plt.subplots(3, 4, figsize=(24, 15))  # Adjusted for 3 rows of plots
        fig.suptitle(f'Comprehensive Neural Response Analysis for CID: {extracted_data["cid"]}, '
                    f'Group: {extracted_data["groupname"]}, Recording: {extracted_data["recordingname"]}')

        max_smoothed_counts = 0
        max_raw_counts = 0

        for stim, data in extracted_data['data'].items():
            max_smoothed_counts = max(max_smoothed_counts, np.max(data['smoothed_counts']))
            max_raw_counts = max(max_raw_counts, np.max(data['raw_counts']))

        ylimit_smoothed = 1.1 * max_smoothed_counts
        ylimit_raw = 1.1 * max_raw_counts

        for i, stim in enumerate(extracted_data['stim_labels']):
            if stim in extracted_data['data']:
                smoothed_psth_ax = axes[0, i]
                raw_psth_ax = axes[1, i]
                raster_ax = axes[2, i]
                data = extracted_data['data'][stim]

                # Plot Smoothed PSTH
                smoothed_psth_ax.bar(data['time_array'], data['smoothed_counts'], width=1, align='edge', color='skyblue')
                smoothed_psth_ax.set_title(f'{stim} Smoothed PSTH')
                smoothed_psth_ax.set_xlim(data['time_array'][0], data['time_array'][-1])
                smoothed_psth_ax.set_ylim(0, ylimit_smoothed)

                # Plot Raw PSTH
                raw_psth_ax.bar(data['time_array'], data['raw_counts'], width=1, align='edge', color='gray')
                raw_psth_ax.set_title(f'{stim} Raw PSTH')
                raw_psth_ax.set_xlim(data['time_array'][0], data['time_array'][-1])
                raw_psth_ax.set_ylim(0, ylimit_raw)

                # Plot Raster
                for trial_index, trial in enumerate(data['spike_trains']):
                    spike_times = extracted_data['time_array'][np.where(trial > 0)]
                    spike_times = spike_times[(spike_times >= data['time_array'][0]) & (spike_times < data['time_array'][-1])]
                    raster_ax.eventplot(spike_times, lineoffsets=trial_index + 1, linelengths=1, color='black')
                raster_ax.set_title(f'{stim} Raster')
                raster_ax.set_xlim(data['time_array'][0], data['time_array'][-1])

        plt.tight_layout()

        # Show the plot if requested
        if show:
            plt.show()



    def process_and_store_psth_raster_data(self, is_single_unit=None, cell_type=None, stim_responsivity=None, 
                                        time_window=None, bin_size=1, smoothing_window=5):
        """
        Iterates over the dataset grouped by 'groupname', 'recordingname', and 'cid', extracts the PSTH and raster data,
        and stores the results in a structured format.

        Args:
            is_single_unit (bool, optional): If specified, filters cells based on whether they are considered single units.
            cell_type (str, optional): If specified, filters cells based on their type.
            stim_responsivity (bool, optional): If specified, filters cells based on their responsiveness to stimuli.
            time_window (tuple, optional): The window of time to extract, within the range -500 to 999 ms. 
                                        Default is None, which uses the full range.
            bin_size (int, optional): The size of the bins in milliseconds for the PSTH and raster. Default is 1ms.
            smoothing_window (int, optional): The size of the smoothing window for the smoothed PSTH. Default is 5.

        Returns:
            dict: A nested dictionary containing the extracted data for each group, recording, and cid.
                The structure of the dictionary is as follows:
                {
                    'groupname_1': {
                        'recordingname_1': {
                            'cid_1': extracted_data,
                            'cid_2': extracted_data,
                            ...
                        },
                        'recordingname_2': { ... },
                        ...
                    },
                    'groupname_2': { ... },
                    ...
                }
        """
        stored_data = {}

        for (groupname, recordingname, cid), group_df in self.iterate_by_group_recording_cid(
                is_single_unit=is_single_unit, cell_type=cell_type, stim_responsivity=stim_responsivity):

            print(f"Processing Group: {groupname}, Recording: {recordingname}, CID: {cid}")

            # Extract the PSTH and raster data
            extracted_data = self.extract_psth_and_raster_data(groupname=groupname, recordingname=recordingname, cid=cid, 
                                                            time_window=time_window, bin_size=bin_size, smoothing_window=smoothing_window)

            # Add the parameters used to the extracted data
            extracted_data['bin_size'] = bin_size
            extracted_data['smoothing_window'] = smoothing_window
            extracted_data['time_window'] = time_window

            # Ensure the nested dictionary structure is created
            if groupname not in stored_data:
                stored_data[groupname] = {}
            if recordingname not in stored_data[groupname]:
                stored_data[groupname][recordingname] = {}
            
            stored_data[groupname][recordingname][cid] = extracted_data

        return stored_data

    def plot_all_psth_and_raster(self, stored_data, output_dir='/Volumes/MannySSD/figures', folder_name='PSTH_and_Rasters_RS_SUA',file_format='svg', show=False):
        """
        Plots combined PSTH and raster plots using the stored data for each unique cid per recording per group.

        Args:
            stored_data (dict): The nested dictionary containing the extracted data.
                                The structure of the dictionary should be:
                                {
                                    'groupname_1': {
                                        'recordingname_1': {
                                            'cid_1': {
                                                'data': extracted_data,
                                                'bin_size': int,
                                                'smoothing_window': int,
                                                'time_window': tuple,
                                            },
                                            'cid_2': { ... },
                                            ...
                                        },
                                        'recordingname_2': { ... },
                                        ...
                                    },
                                    'groupname_2': { ... },
                                    ...
                                }
            output_dir (str): The root directory where the figures will be saved.
            folder_name (str): The folder name to be created within the output directory.
            file_format (str, optional): The format in which to save the figures. Default is 'svg'.
            show (bool, optional): Whether to display the plot. Default is False.
        """
        # Validate the file format
        if file_format not in ['svg', 'png']:
            raise ValueError("file_format must be either 'svg' or 'png'")

        # Create the output directory if it doesn't exist
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        # Create the folder within the output directory
        save_dir = os.path.join(output_dir, folder_name)
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        for groupname, recordings in stored_data.items():
            group_dir = os.path.join(save_dir, groupname)
            if not os.path.exists(group_dir):
                os.makedirs(group_dir)

            for recordingname, cids in recordings.items():
                recording_dir = os.path.join(group_dir, recordingname)
                if not os.path.exists(recording_dir):
                    os.makedirs(recording_dir)

                for cid, extracted_data in cids.items():
                    fig, axes = plt.subplots(3, 4, figsize=(24, 15))  # Adjusted for 3 rows of plots
                    fig.suptitle(f'Comprehensive Neural Response Analysis for CID: {cid}, '
                                f'Group: {groupname}, Recording: {recordingname}')

                    max_smoothed_counts = 0
                    max_raw_counts = 0

                    for stim, data in extracted_data['data'].items():
                        max_smoothed_counts = max(max_smoothed_counts, np.max(data['smoothed_counts']))
                        max_raw_counts = max(max_raw_counts, np.max(data['raw_counts']))

                    ylimit_smoothed = 1.1 * max_smoothed_counts
                    ylimit_raw = 1.1 * max_raw_counts
                    bin_size = extracted_data['bin_size']

                    for i, stim in enumerate(extracted_data['stim_labels']):
                        if stim in extracted_data['data']:
                            smoothed_psth_ax = axes[0, i]
                            raw_psth_ax = axes[1, i]
                            raster_ax = axes[2, i]
                            data = extracted_data['data'][stim]

                            # Plot Smoothed PSTH
                            smoothed_psth_ax.bar(data['time_array'], data['smoothed_counts'], width=bin_size, align='edge', color='skyblue')
                            smoothed_psth_ax.set_title(f'{stim} Smoothed PSTH')
                            smoothed_psth_ax.set_xlim(data['time_array'][0], data['time_array'][-1])
                            smoothed_psth_ax.set_ylim(0, ylimit_smoothed)

                            # Plot Raw PSTH
                            raw_psth_ax.bar(data['time_array'], data['raw_counts'], width=bin_size, align='edge', color='gray')
                            raw_psth_ax.set_title(f'{stim} Raw PSTH')
                            raw_psth_ax.set_xlim(data['time_array'][0], data['time_array'][-1])
                            raw_psth_ax.set_ylim(0, ylimit_raw)

                            # Plot Raster
                            for trial_index, trial in enumerate(data['spike_trains']):
                                spike_times = extracted_data['time_array'][np.where(trial > 0)]
                                spike_times = spike_times[(spike_times >= data['time_array'][0]) & (spike_times < data['time_array'][-1])]
                                raster_ax.eventplot(spike_times, lineoffsets=trial_index + 1, linelengths=1, color='black')
                            raster_ax.set_title(f'{stim} Raster')
                            raster_ax.set_xlim(data['time_array'][0], data['time_array'][-1])

                    plt.tight_layout()

                    if show:
                        plt.show()
                    else:
                        save_path = os.path.join(recording_dir, f'psth_raster_{cid}.{file_format}')
                        fig.savefig(save_path, transparent=True)





    def calculate_baseline_and_response(self, groupname, recordingname, cid, baseline_window=(-500, 0), response_window=(0, 50)):
        """
        Calculates the mean baseline firing rate, mean response magnitude, and z-scored magnitude for each stimulation.

        Args:
            groupname (str): The name of the group.
            recordingname (str): The name of the recording.
            cid (str): The cell ID.
            baseline_window (tuple): The time window for calculating the baseline firing rate. Default is (-500, 0).
            response_window (tuple): The time window for calculating the response magnitude. Default is (0, 50).

        Returns:
            dict: A dictionary containing the calculated values for each stimulation.
                {
                    'stim1': {
                        'mean_baseline_firing_rate': float,
                        'mean_response_magnitude': float,
                        'z_scored_magnitude': float
                    },
                    'stim2': { ... },
                    ...
                }
        """
        # Get the full time array from the eed object
        time_array = self.eed.relative_time_ms['relative_time_ms']
        
        # Set a fixed bin size of 1 ms for accurate baseline and response calculation
        bin_size = 1
        
        # Create a new time array with the specified bin size
        new_time_array = np.arange(time_array[0], time_array[-1] + 1, bin_size)

        # Create masks for baseline and response windows based on the new time array
        baseline_indices = (new_time_array >= baseline_window[0]) & (new_time_array < baseline_window[1])
        response_indices = (new_time_array >= response_window[0]) & (new_time_array < response_window[1])

        # Dictionary to store results for each stimulation
        stim_results = {}

        # Extract PSTH and raster data with the fixed bin size and no smoothing
        extracted_data = self.extract_psth_and_raster_data(groupname=groupname, recordingname=recordingname, cid=cid, 
                                                        time_window=None, bin_size=bin_size, smoothing_window=1)

        # Iterate over each stimulation label in the extracted data
        for stim, data in extracted_data['data'].items():
            # Get the spike trains for the current stimulation
            spike_trains = data['spike_trains']
            
            # Calculate the total number of spikes in the baseline and response windows for each trial
            baseline_spikes = spike_trains[:, baseline_indices].sum(axis=1)
            response_spikes = spike_trains[:, response_indices].sum(axis=1)
            
            # Calculate the mean number of spikes during the baseline and response windows
            mean_baseline_firing_rate = baseline_spikes.mean()
            mean_response_magnitude = response_spikes.mean() - mean_baseline_firing_rate
            
            # Calculate the standard deviation of the baseline firing rates across trials
            baseline_sd = np.std(baseline_spikes)
            
            # Calculate the z-scored magnitude by dividing the mean response magnitude by the baseline standard deviation
            z_scored_magnitude = mean_response_magnitude / baseline_sd if baseline_sd > 0 else 0

            # Store the results for the current stimulation in the dictionary
            stim_results[stim] = {
                'mean_baseline_firing_rate': mean_baseline_firing_rate,
                'mean_response_magnitude': mean_response_magnitude,
                'z_scored_magnitude': z_scored_magnitude
            }

        # Return the dictionary containing results for all stimulations
        return stim_results

    def calculate_population_psth(self, groupname, recordingname, cid, baseline_window=(-500, 0), bin_size=1):
        """
        Calculates the population PSTH by subtracting the overall prestimulus baseline spike rate from every bin value.

        Args:
            groupname (str): The name of the group.
            recordingname (str): The name of the recording.
            cid (str): The cell ID.
            baseline_window (tuple): The time window for calculating the baseline firing rate. Default is (-500, 0).
            bin_size (int): The size of the bins in milliseconds for the PSTH. Default is 1ms.

        Returns:
            dict: A dictionary containing the adjusted PSTH values for each stimulation.
                {
                    'stim1': adjusted_psth_array,
                    'stim2': { ... },
                    ...
                }
        """
        # Get the full time array from the eed object
        time_array = self.eed.relative_time_ms['relative_time_ms']
        
        # Create a new time array with the specified bin size
        new_time_array = np.arange(time_array[0], time_array[-1] + 1, bin_size)

        # Create a mask for the baseline window based on the new time array
        baseline_indices = (new_time_array >= baseline_window[0]) & (new_time_array < baseline_window[1])

        # Dictionary to store adjusted PSTH results for each stimulation
        stim_results = {}

        # Extract PSTH and raster data with the specified bin size and no smoothing
        extracted_data = self.extract_psth_and_raster_data(groupname=groupname, recordingname=recordingname, cid=cid, 
                                                        time_window=None, bin_size=bin_size, smoothing_window=1)

        # Iterate over each stimulation label in the extracted data
        for stim, data in extracted_data['data'].items():
            # Get the spike trains for the current stimulation
            spike_trains = data['spike_trains']
            
            # Calculate the total number of spikes in the baseline window for each trial
            baseline_spikes = spike_trains[:, baseline_indices].sum(axis=1)
            
            # Calculate the mean baseline firing rate
            mean_baseline_firing_rate = baseline_spikes.mean()

            # Calculate the PSTH by summing spike counts across trials for each time bin
            psth_counts = np.sum(spike_trains, axis=0)
            
            # Subtract the mean baseline firing rate from each bin value in the PSTH
            adjusted_psth = psth_counts - mean_baseline_firing_rate
            
            # Store the adjusted PSTH values for the current stimulation
            stim_results[stim] = adjusted_psth

        return stim_results

    def process_and_store_psth_raster_data_comprehensive(self, is_single_unit=None, cell_type=None, stim_responsivity=None, 
                                                        time_window=None, bin_size=1, smoothing_window=5):
        """
        Iterates over the dataset grouped by 'groupname', 'recordingname', and 'cid', extracts the PSTH and raster data,
        calculates the baseline and response magnitudes, and stores the results in a structured format.

        Args:
            is_single_unit (bool, optional): If specified, filters cells based on whether they are considered single units.
            cell_type (str, optional): If specified, filters cells based on their type.
            stim_responsivity (bool, optional): If specified, filters cells based on their responsiveness to stimuli.
            time_window (tuple, optional): The window of time to extract, within the range -500 to 999 ms. 
                                        Default is None, which uses the full range.
            bin_size (int, optional): The size of the bins in milliseconds for the PSTH and raster. Default is 1ms.
            smoothing_window (int, optional): The size of the smoothing window for the smoothed PSTH. Default is 5.

        Returns:
            dict: A nested dictionary containing the extracted data for each group, recording, and cid.
                The structure of the dictionary is as follows:
                {
                    'groupname_1': {
                        'recordingname_1': {
                            'cid_1': {
                                'data': extracted_data,
                                'bin_size': int,
                                'smoothing_window': int,
                                'time_window': tuple,
                                'stimulation_results': dict,
                                'population_psth': dict
                            },
                            'cid_2': { ... },
                            ...
                        },
                        'recordingname_2': { ... },
                        ...
                    },
                    'groupname_2': { ... },
                    ...
                }
        """
        stored_data = {}

        for (groupname, recordingname, cid), group_df in self.iterate_by_group_recording_cid(
                is_single_unit=is_single_unit, 
                cell_type=cell_type, 
                stim_responsivity=stim_responsivity):

            print(f"Processing Group: {groupname}, Recording: {recordingname}, CID: {cid}")

            # Extract the PSTH and raster data
            extracted_data = self.extract_psth_and_raster_data(groupname=groupname, recordingname=recordingname, cid=cid, 
                                                            time_window=time_window, bin_size=bin_size, smoothing_window=smoothing_window)

            
            # Calculate baseline and response magnitudes using consistent binning and smoothing parameters store the 
            # mean_baseline_firing_rate, mean_response_magnitude, z_scored_magnitude per stimulation recoreded at each cid 
            # for 1ms bins and smoothing window of 1 
            # Calculate baseline and response magnitudes for each stimulation
            stim_results = self.calculate_baseline_and_response(groupname=groupname, recordingname=recordingname, cid=cid)

            # Calculate population PSTH for each stimulation
            population_psth = self.calculate_population_psth(groupname=groupname, recordingname=recordingname, cid=cid)

            # Add the parameters and calculated values to the extracted data
            extracted_data['bin_size'] = bin_size
            extracted_data['smoothing_window'] = smoothing_window
            extracted_data['time_window'] = time_window
            extracted_data['stimulation_results'] = stim_results
            extracted_data['population_psth'] = population_psth

            # Ensure the nested dictionary structure is created
            if groupname not in stored_data:
                stored_data[groupname] = {}
            if recordingname not in stored_data[groupname]:
                stored_data[groupname][recordingname] = {}
            
            stored_data[groupname][recordingname][cid] = extracted_data

        return stored_data

    def plot_population_psth(self, stored_data, groupname, time_window=(-500, 999), bin_size=1):
        """
        Plots the mean and SEM population PSTHs using the formatted data for a specified group.

        Args:
            stored_data (dict): The nested dictionary containing the extracted data.
            groupname (str): The name of the group.
            time_window (tuple): The window of time to plot, within the range -500 to 999 ms. Default is (-500, 999).
            bin_size (int): The size of the bins in milliseconds for the PSTH. Default is 1ms.
        """
        # Initialize a dictionary to store population PSTHs for each stimulation
        population_psths = {}

        # Iterate through recordings and cell IDs within the specified group
        for recordingname, cids in stored_data[groupname].items():
            for cid, extracted_data in cids.items():
                for stim, psth in extracted_data['population_psth'].items():
                    if stim not in population_psths:
                        population_psths[stim] = []
                    population_psths[stim].append(psth)

        # Create a new time array with the specified bin size
        time_array = self.eed.relative_time_ms['relative_time_ms']
        new_time_array = np.arange(time_array[0], time_array[-1] + 1, bin_size)

        # Create a mask for the time window to plot
        time_mask = (new_time_array >= time_window[0]) & (new_time_array <= time_window[1])
        plot_time_array = new_time_array[time_mask]

        # Initialize variables to track the global y-axis limits
        global_min, global_max = np.inf, -np.inf

        # Calculate the mean and SEM for each stimulation to find global y-axis limits
        mean_sems = {}
        for stim, psth_list in population_psths.items():
            if len(psth_list) > 0:
                psth_array = np.array(psth_list)
                mean_psth = np.mean(psth_array, axis=0)
                sem_psth = np.std(psth_array, axis=0) / np.sqrt(psth_array.shape[0])
                mean_sems[stim] = (mean_psth, sem_psth)
                global_min = min(global_min, (mean_psth - sem_psth).min())
                global_max = max(global_max, (mean_psth + sem_psth).max())

        # Plot the mean and SEM for each stimulation in a 1x4 grid
        fig, axes = plt.subplots(1, len(population_psths), figsize=(20, 5), sharey=True)
        if len(population_psths) == 1:
            axes = [axes]  # Ensure axes is always iterable

        for i, (stim, (mean_psth, sem_psth)) in enumerate(mean_sems.items()):
            axes[i].plot(plot_time_array, mean_psth[time_mask], label=f'{stim} Mean PSTH')
            axes[i].fill_between(plot_time_array, mean_psth[time_mask] - sem_psth[time_mask], 
                                mean_psth[time_mask] + sem_psth[time_mask], alpha=0.5, label=f'{stim} SEM')
            axes[i].set_title(f'Mean and SEM Population PSTH for {stim}')
            axes[i].set_xlabel('Time (ms)')
            axes[i].set_ylabel('Firing Rate (spikes/bin)')
            axes[i].set_ylim(global_min, global_max)
            axes[i].legend()

        plt.tight_layout()
        plt.show()





    def compute_psth_and_raster_data(self, groupname, recordingname, cid, time_window=None, smoothing_window=5, bin_size=1):
        """
        Computes PSTH and raster data for a specific cid within a specific group and recording across all stimulations,
        with an optional custom time window, smoothing window, and bin size.

        Args:
            groupname (str): The name of the group.
            recordingname (str): The name of the recording.
            cid (str): The cell ID for which to compute the PSTH and raster data.
            time_window (tuple, optional): The window of time to consider, within the range -500 to 999 ms. 
                                        Default is None, which uses the full range.
            smoothing_window (int, optional): The size of the smoothing window for the smoothed PSTH. Default is 5.
            bin_size (int, optional): The size of the bins in milliseconds for the PSTH and raster. Default is 1ms.

        Returns:
            dict: A dictionary containing the group name, recording name, cid, and the smoothed PSTH counts, raw PSTH counts, 
                raster spike times, and time arrays for each stimulation.
        """
        stim_labels = self.eed.trialTagsLabels['trialTagsLabels']
        time_array = self.eed.relative_time_ms['relative_time_ms']

        # Create a new time array with the specified bin size
        new_time_array = np.arange(time_array[0], time_array[-1] + 1, bin_size)

        results = {
            'groupname': groupname,
            'recordingname': recordingname,
            'cid': cid,
            'data': {
                stim: {
                    'smoothed_counts': None,
                    'raw_counts': None,
                    'raster_spike_times': [],
                    'time_array': None
                }
                for stim in stim_labels
            }
        }

        for i, stim in enumerate(stim_labels):
            df_name = f'psth_dataframe_{stim}'

            if df_name in self.dataframes:
                df = self.dataframes[df_name]
                condition = (df['groupname'] == groupname) & (df['recordingname'] == recordingname) & (df['cid'] == cid)
                spike_trains = df.loc[condition, 'SpikeTrains_for_PSTHs'].values

                if spike_trains.size > 0:
                    spike_trains = spike_trains[0]

                    # Bin the spike trains into the specified bin size
                    binned_spike_trains = np.add.reduceat(spike_trains, np.arange(0, spike_trains.shape[1], bin_size), axis=1)

                    # Calculate PSTH
                    all_spikes = np.concatenate([np.where(trial > 0)[0] for trial in binned_spike_trains])
                    counts, _ = np.histogram(all_spikes, bins=len(new_time_array), range=(0, len(new_time_array)))

                    # Apply time window if specified
                    if time_window:
                        start_idx = np.searchsorted(new_time_array, time_window[0])
                        end_idx = np.searchsorted(new_time_array, time_window[1], side='right')
                        displayed_time_array = new_time_array[start_idx:end_idx]
                        displayed_counts = counts[start_idx:end_idx]
                    else:
                        displayed_time_array = new_time_array
                        displayed_counts = counts

                    # Smooth the data for the smoothed PSTH
                    smoothed_counts = self.smooth_data(displayed_counts, smoothing_window)

                    # Collect raster spike times
                    raster_spike_times = []
                    for trial in binned_spike_trains:
                        spike_times = new_time_array[np.where(trial > 0)]
                        spike_times = spike_times[(spike_times >= displayed_time_array[0]) & (spike_times < displayed_time_array[-1])]
                        raster_spike_times.append(spike_times)

                    # Store results
                    results['data'][stim]['smoothed_counts'] = smoothed_counts
                    results['data'][stim]['raw_counts'] = displayed_counts
                    results['data'][stim]['raster_spike_times'] = raster_spike_times
                    results['data'][stim]['time_array'] = displayed_time_array

        return results

    def plot_from_computed_data(self, computed_data, bin_size=1, show=False):
        """
        Plots combined PSTH and raster plots using precomputed data.

        Args:
            computed_data (dict): The precomputed data containing the group name, recording name, cid, 
                                smoothed PSTH counts, raw PSTH counts, raster spike times, and time arrays for each stimulation.
            bin_size (int, optional): The size of the bins in milliseconds for the PSTH and raster. Default is 1ms.
            show (bool, optional): Whether to display the plot. Default is False.
        """
        groupname = computed_data['groupname']
        recordingname = computed_data['recordingname']
        cid = computed_data['cid']
        data = computed_data['data']

        stim_labels = list(data.keys())

        fig, axes = plt.subplots(3, 4, figsize=(24, 15))  # Adjusted for 3 rows of plots
        fig.suptitle(f'Comprehensive Neural Response Analysis for CID: {cid}, Group: {groupname}, Recording: {recordingname}')

        max_smoothed_counts = max(np.max(data[stim]['smoothed_counts']) for stim in stim_labels)
        max_raw_counts = max(np.max(data[stim]['raw_counts']) for stim in stim_labels)

        ylimit_smoothed = 1.1 * max_smoothed_counts
        ylimit_raw = 1.1 * max_raw_counts

        for i, stim in enumerate(stim_labels):
            smoothed_psth_ax = axes[0, i]
            raw_psth_ax = axes[1, i]
            raster_ax = axes[2, i]

            smoothed_counts = data[stim]['smoothed_counts']
            raw_counts = data[stim]['raw_counts']
            raster_spike_times = data[stim]['raster_spike_times']
            displayed_time_array = data[stim]['time_array']

            # Plot Smoothed PSTH
            smoothed_psth_ax.bar(displayed_time_array, smoothed_counts, width=bin_size, align='edge', color='skyblue')
            smoothed_psth_ax.set_title(f'{stim} Smoothed PSTH')
            smoothed_psth_ax.set_xlim(displayed_time_array[0], displayed_time_array[-1])
            smoothed_psth_ax.set_ylim(0, ylimit_smoothed)

            # Plot Raw PSTH
            raw_psth_ax.bar(displayed_time_array, raw_counts, width=bin_size, align='edge', color='gray')
            raw_psth_ax.set_title(f'{stim} Raw PSTH')
            raw_psth_ax.set_xlim(displayed_time_array[0], displayed_time_array[-1])
            raw_psth_ax.set_ylim(0, ylimit_raw)

            # Plot Raster
            for trial_index, spike_times in enumerate(raster_spike_times):
                raster_ax.eventplot(spike_times, lineoffsets=trial_index + 1, linelengths=1, color='black')
            raster_ax.set_title(f'{stim} Raster')
            raster_ax.set_xlim(displayed_time_array[0], displayed_time_array[-1])

        plt.tight_layout()

    def compute_population_psth_and_raster_data(self, groupname, cell_type, is_single_unit=1.0, time_window=None, smoothing_window=5, bin_size=1):
        """
        Computes population-level PSTH data for a specific cell type within a specific group across all recordings,
        with an optional custom time window, smoothing window, and bin size.

        Args:
            groupname (str): The name of the group.
            cell_type (str): The cell type for which to compute the population PSTH data.
            is_single_unit (float, optional): Filter for single units. Default is 1.0.
            time_window (tuple, optional): The window of time to consider, within the range -500 to 999 ms. 
                                        Default is None, which uses the full range.
            smoothing_window (int, optional): The size of the smoothing window for the smoothed PSTH. Default is 5.
            bin_size (int, optional): The size of the bins in milliseconds for the PSTH. Default is 1ms.

        Returns:
            dict: A dictionary containing the group name, cell type, and the smoothed PSTH counts, raw PSTH counts, 
                and time arrays for each stimulation.
        """
        # Filter data to get the relevant cells
        channel = self.get_filtered_data('basic_metrics', is_single_unit=is_single_unit, cell_type=cell_type, groupname=groupname)

        stim_labels = self.eed.trialTagsLabels['trialTagsLabels']
        time_array = self.eed.relative_time_ms['relative_time_ms']

        # Create a new time array with the specified bin size
        new_time_array = np.arange(time_array[0], time_array[-1] + 1, bin_size)

        results = {
            'groupname': groupname,
            'cell_type': cell_type,
            'data': {
                stim: {
                    'smoothed_counts': np.zeros(len(new_time_array)),
                    'raw_counts': np.zeros(len(new_time_array)),
                    'time_array': new_time_array
                }
                for stim in stim_labels
            }
        }

        num_cells = 0

        # Loop through each cell and recording
        for _, row in channel.iterrows():
            recordingname = row['recordingname']
            cid = row['cid']
            cell_data = self.compute_psth_and_raster_data(groupname, recordingname, cid, time_window, smoothing_window, bin_size)
            cell_data = cell_data['data']

            for stim in stim_labels:
                if cell_data[stim]['smoothed_counts'] is not None:
                    results['data'][stim]['smoothed_counts'] += cell_data[stim]['smoothed_counts']
                if cell_data[stim]['raw_counts'] is not None:
                    results['data'][stim]['raw_counts'] += cell_data[stim]['raw_counts']

            num_cells += 1

        # Compute the mean by dividing by the number of cells
        if num_cells > 0:
            for stim in stim_labels:
                results['data'][stim]['smoothed_counts'] /= num_cells
                results['data'][stim]['raw_counts'] /= num_cells

        return results



 
    def plot_combined_psth_and_raster_normalized(self, groupname, recordingname, cid, time_window=None, smoothing_window=5, normalize=True, show=False):
        stim_labels = self.eed.trialTagsLabels['trialTagsLabels']
        time_array = self.eed.relative_time_ms['relative_time_ms']

        fig, axes = plt.subplots(3, 4, figsize=(24, 15))
        fig.suptitle(f'Normalized Neural Response Analysis for CID: {cid}, Group: {groupname}, Recording: {recordingname}')

        for i, stim in enumerate(stim_labels):
            smoothed_psth_ax = axes[0, i]
            raw_psth_ax = axes[1, i]
            raster_ax = axes[2, i]
            df_name = f'psth_dataframe_{stim}'

            if df_name in self.dataframes:
                df = self.dataframes[df_name]
                condition = (df['groupname'] == groupname) & (df['recordingname'] == recordingname) & (df['cid'] == cid)
                spike_trains = df.loc[condition, 'SpikeTrains_for_PSTHs'].values

                if spike_trains.size > 0:
                    spike_trains = [train for train in spike_trains[0] if np.any(train == 1)]  # Filter to include only trials with spikes

                    if not spike_trains:
                        for ax in axes[:, i]:  # Iterate over each subplot in the column
                            ax.text(0.5, 0.5, 'No spikes detected', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
                            ax.set_axis_off()  # Optionally turn off the axis if no spikes are detected

                    else:
                        all_spikes = np.concatenate([np.where(train == 1)[0] for train in spike_trains])
                        counts, _ = np.histogram(all_spikes, bins=len(time_array), range=(0, len(time_array)))
                        number_of_trials = len(spike_trains)

                        # Apply normalization if enabled
                        normalized_counts = counts / number_of_trials if normalize else counts

                        if time_window:
                            start_idx = np.searchsorted(time_array, time_window[0])
                            end_idx = np.searchsorted(time_array, time_window[1], side='right')
                            displayed_time_array = time_array[start_idx:end_idx]
                            displayed_counts = normalized_counts[start_idx:end_idx]
                        else:
                            displayed_time_array = time_array
                            displayed_counts = normalized_counts

                        # Smoothed PSTH
                        smoothed_counts = self.smooth_data(displayed_counts, smoothing_window)
                        smoothed_psth_ax.bar(displayed_time_array, smoothed_counts, width=1, align='edge', color='skyblue')
                        smoothed_psth_ax.set_xlim(displayed_time_array[0], displayed_time_array[-1])

                        # Raw PSTH
                        raw_psth_ax.bar(displayed_time_array, displayed_counts, width=1, align='edge', color='gray')
                        raw_psth_ax.set_xlim(displayed_time_array[0], displayed_time_array[-1])

                        # Raster Plot
                        for trial_index, trial in enumerate(spike_trains):
                            spike_times = time_array[np.where(trial == 1)]
                            raster_ax.eventplot(spike_times, lineoffsets=trial_index + 0.5, linelengths=1)
                        raster_ax.set_xlim(displayed_time_array[0], displayed_time_array[-1])

        plt.tight_layout()
        if show: 
            plt.show()
        return fig 



    def save_plots_for_all_units(self):
        df = self.get_filtered_data('basic_metrics', is_single_unit=None, cell_type=None, stim_responsivity=None)
        base_dir = '/Volumes/MannySSD/output_data_for_emx/saved_rasterplots'
        os.makedirs(base_dir, exist_ok=True)

        for (groupname, recordingname, cid), group_df in df.groupby(['groupname', 'recordingname', 'cid']):
            cid_dir = os.path.join(base_dir, groupname, recordingname, str(cid))
            os.makedirs(cid_dir, exist_ok=True)

            # Generate and save the standard plot
            fig_standard = self.plot_combined_psth_and_raster(groupname, recordingname, cid, time_window=None, smoothing_window=8)
            if fig_standard:  # Check if a figure was returned
                fig_standard.savefig(os.path.join(cid_dir, 'standard_plot.png'), dpi=300, format='png', bbox_inches='tight')
                plt.close(fig_standard)

            # Generate and save the normalized plot
            fig_normalized = self.plot_combined_psth_and_raster_normalized(groupname, recordingname, cid, time_window=None, smoothing_window=8, normalize=True)
            if fig_normalized:  # Check if a figure was returned
                fig_normalized.savefig(os.path.join(cid_dir, 'normalized_plot.png'), dpi=300, format='png', bbox_inches='tight')
                plt.close(fig_normalized)

    def calculate_psth_data(self, groupname, recordingname, cid, time_window=None, normalize=True, filter_empty_trials=True):
        """
        Overview

        The calculate_psth_data method is part of the DataFrameManager class.
        It calculates peristimulus time histograms (PSTH) for specified neural recordings and cells. 
        The method handles multiple stimuli and can be configured to include all trials or only those with spikes, based on user preference. 
        This method does not plot the data but returns a structured dictionary containing counts and time arrays for further analysis or visualization.

        Parameters

        groupname (str): The name of the experimental group. This parameter specifies which group the data should be retrieved from.
        recordingname (str): The name of the recording session. It determines from which recording to fetch the spike data.
        cid (str): Cell identifier. This specifies the neuron for which the PSTH will be calculated.
        time_window (tuple of int, optional): A tuple representing the start and end of the time window in milliseconds within which to calculate the PSTH. If None, the entire span of time_array will be used. Default is None.
        normalize (bool): Determines whether the spike counts should be normalized. If True, the counts are divided by the number of trials used in the calculation. The exact denominator depends on the filter_empty_trials parameter. Default is True.
        filter_empty_trials (bool): Controls whether trials without any spikes should be included in the calculation. If True, only trials with spikes are considered. This affects both the counts and the normalization process. Default is True.
        
        Returns

        psth_results (dict): A dictionary where keys are stimulus labels and values are dictionaries containing:
        'counts' (numpy.array): An array of spike counts per time bin. This array is normalized if normalize is set to True.
        'time_array' (numpy.array): The time points corresponding to the bins in 'counts'.
        
        Detailed Description

        The method begins by accessing stimulus labels and relative time arrays from the trialTagsLabels and relative_time_ms attributes of the eed (Electrophysiology Extraction Data) instance, respectively. 
        It then iterates over each stimulus, fetching spike train data from the appropriate DataFrame. The spike trains are filtered based on the filter_empty_trials setting.

        Spike trains are processed into histograms using numpy.histogram, with bin edges aligned to time_array. 
        If a time_window is provided, the method restricts the calculation to the specified window, adjusting both the counts and time arrays accordingly.

        Normalization, when enabled, divides the counts by the number of trials that were used in the histogram calculation. 
        The choice of denominator is influenced by the filter_empty_trials flag: if True, it uses the count of trials with spikes; if False, it uses the total number of trials.
        """
        stim_labels = self.eed.trialTagsLabels['trialTagsLabels']
        time_array = self.eed.relative_time_ms['relative_time_ms']
        psth_results = {}

        for stim in stim_labels:
            df_name = f'psth_dataframe_{stim}'
            if df_name in self.dataframes:
                df = self.dataframes[df_name]
                condition = (df['groupname'] == groupname) & (df['recordingname'] == recordingname) & (df['cid'] == cid) 
                spike_trains = df.loc[condition, 'SpikeTrains_for_PSTHs'].values
                
                # Get the laminar label for this cid
                if condition.sum() == 0:
                    print(f"No data found for Group: {groupname}, Recording: {recordingname}, CID: {cid}")
                    return None
                
                #get the laminar label for this cid
                label = df.loc[condition, 'LaminarLabel'].values[0]
                
                
                
                if spike_trains.size > 0:
                    spike_trains = spike_trains[0]  # Get the array of spike trains
                    if filter_empty_trials:
                        spike_trains = [train for train in spike_trains if np.any(train == 1)]  # Only include trials with spikes

                    if len(spike_trains) == 0:  # Checking if the list is empty
                        psth_results[stim] = {'counts': [], 'time_array': [], 'LaminarLabel': label}
                    else:
                        all_spikes = np.concatenate([np.where(train == 1)[0] for train in spike_trains])
                        counts, _ = np.histogram(all_spikes, bins=len(time_array), range=(0, len(time_array)))
                        
                        if normalize:
                            number_of_trials = len(spike_trains) if filter_empty_trials else len(spike_trains[0])
                            normalized_counts = counts / number_of_trials
                        else:
                            normalized_counts = counts

                        if time_window:
                            start_idx = np.searchsorted(time_array, time_window[0])
                            end_idx = np.searchsorted(time_array, time_window[1], side='right')
                            displayed_time_array = time_array[start_idx:end_idx]
                            displayed_counts = normalized_counts[start_idx:end_idx]
                        else:
                            displayed_time_array = time_array
                            displayed_counts = normalized_counts

                        psth_results[stim] = {'counts': displayed_counts, 'time_array': displayed_time_array, 'LaminarLabel': label}

        return psth_results

    def iterate_by_group_recording_cid(self, is_single_unit=None, cell_type=None, stim_responsivity=None):
        """
        Iterates over the dataset grouped by 'groupname', 'recordingname', and 'cid' after applying specified filters.
        
        This generator function fetches data using predefined criteria, applies additional filters, and yields
        each subset of the data grouped by 'groupname', 'recordingname', and 'cid'. It is designed to facilitate
        the processing of large datasets by providing chunks of data one group at a time, which can be particularly
        useful for processing steps that do not require the complete dataset to be held in memory.

        Parameters:
            is_single_unit (bool, optional): If specified, filters cells based on whether they are considered single units.
            cell_type (str, optional): If specified, filters cells based on their type.
            stim_responsivity (bool, optional): If specified, filters cells based on their responsiveness to stimuli.
        
        Yields:
            tuple: A tuple containing (groupname, recordingname, cid) as a tuple and the corresponding group DataFrame. 
                This allows for further processing of the data specific to each group.

        Example:
            for (groupname, recordingname, cid), group_df in self.iterate_by_group_recording_cid(cell_type='pyramidal'):
                # Process each group DataFrame here
                print(groupname, recordingname, cid, len(group_df))
        """
        # Fetch the data with potential filters applied
        df = self.get_filtered_data('basic_metrics', is_single_unit=is_single_unit, 
                                    cell_type=cell_type, stim_responsivity=stim_responsivity)
        
        # Group the data by 'groupname', 'recordingname', and 'cid'
        for (groupname, recordingname, cid), group_df in df.groupby(['groupname', 'recordingname', 'cid']):
            yield (groupname, recordingname, cid), group_df

    def calculate_all_psths(self, is_single_unit=None, cell_type=None, stim_responsivity=None, time_window=None, normalize=True, filter_empty_trials=True):
        """
        Calculates PSTH for all cells across specified groups and recordings, aggregates the results into two DataFrames.
        
        One DataFrame contains the means of 'counts' over the entire duration for each stimulation. The other DataFrame contains detailed PSTH data for 
        each stimulation including both 'counts' and 'time_array', which can be selectively extracted based on a 'time_window'.

        Parameters:
            is_single_unit (bool, optional): Filter based on whether cells are considered single units.
            cell_type (str, optional): Filter based on cell type.
            stim_responsivity (bool, optional): Filter based on cell responsiveness to stimuli.
            time_window (tuple of int, optional): Specific time window to calculate PSTH data over. If None, the full range is used.
            normalize (bool): Whether to normalize the spike counts.
            filter_empty_trials (bool): Whether to include only trials with spikes.

        Returns:
            tuple of DataFrames: (mean_df, detailed_df)
                mean_df: DataFrame with columns 'mean_stimulation', 'Stimulation', 'Group', 'cid', 'recordingname'.
                detailed_df: DataFrame with columns 'Stimulation', 'cid', 'groupname', 'recordingname', 'counts', 'time_array'.
        """
        mean_results = []
        detailed_results = []
        stim_labels = self.eed.trialTagsLabels['trialTagsLabels']  # Get stimulation labels from the eed object

        for (groupname, recordingname, cid), group_df in self.iterate_by_group_recording_cid(
            is_single_unit=is_single_unit, cell_type=cell_type, stim_responsivity=stim_responsivity):

            print(f"Processing Group: {groupname}, Recording: {recordingname}, CID: {cid}")
            
            psth_results = self.calculate_psth_data(groupname, recordingname, cid, time_window=time_window, 
                                                    normalize=normalize, filter_empty_trials=filter_empty_trials)
            if psth_results is None:
                continue
            
            # Process each stimulation's PSTH results
            for stim_label in stim_labels:
                
                if stim_label in psth_results:
                    counts = psth_results[stim_label]['counts']
                    time_array = psth_results[stim_label]['time_array']
                    
                     # Get 'LaminarLabel' with a default value if it's not found
                    laminar_label = psth_results[stim_label].get('LaminarLabel', 'DefaultLabel') #added 'LaminarLabel' if it's not found, it will be assigned 'DefaultLabel'
                    #print(stim_label, laminar_label) #use print to check if the laminar label is being correctly assigned to the stim_label

                    # Ensure counts is an array and has elements before calculating mean
                    if isinstance(counts, np.ndarray) and counts.size > 0:
                        mean_stimulation = np.mean(counts)
                        #instead find the max value of the counts array and store it in mean_stimulation
                        #mean_stimulation = np.max(counts)
                    else:
                        mean_stimulation = np.nan

                    # Append to results for mean DataFrame
                    mean_results.append({
                        'Group': groupname,
                        'Stimulation': stim_label,
                        'mean_stimulation': mean_stimulation,
                        'cid': cid,
                        'LaminarLabel': laminar_label, #added 'LaminarLabel
                        'recordingname': recordingname,
                 
                    })

                    # Append to results for detailed DataFrame
                    detailed_results.append({
                        'Stimulation': stim_label,
                        'cid': cid,
                        'LaminarLabel': laminar_label, #added 'LaminarLabel
                        'groupname': groupname,
                        'recordingname': recordingname,
                        'counts': counts,
                        'time_array': time_array, 
                    })

        # Create DataFrames from the aggregated results
        mean_df = pd.DataFrame(mean_results)
        detailed_df = pd.DataFrame(detailed_results)
        return mean_df, detailed_df, psth_results
    
    def plot_mean_stimulation_box_and_strip(self, mean_df, groups=None, stimulations=None, show_outliers=True, hue_order=None):
        """
        Plots boxplots and stripplots for specified groups and stimulations using the mean stimulation data.

        Args:
            mean_df (DataFrame): The DataFrame containing 'mean_stimulation' along with 'Group' and 'Stimulation' columns.
            groups (list of str, optional): List of groups to include in the plot.
            stimulations (list of str, optional): List of stimulations to include in the plot.
            show_outliers (bool, optional): Whether to show outliers in the stripplot.
            hue_order (list, optional): Order of the hue levels, specifying how groups should be ordered in the plot.

        Usage:
            mean_df, _ = whisker_df_manager.calculate_all_psths(...)
            whisker_df_manager.plot_mean_stimulation_box_and_strip(mean_df, ...)
        """
        # Define color mapping for groups
        group_colors = {
            'No_CTZ': '#797979',
            'CTZ': '#5a00c2'
        }

        # Generate lighter versions for the box face color
        lightened_colors = {k: v + '33' for k, v in group_colors.items()}

        # Boxplot customization
        boxprops = {'edgecolor': 'k', 'linewidth': 2}
        whiskerprops = {'color': 'k', 'linewidth': 2}
        boxplot_kwargs = {
            'boxprops': boxprops,
            'medianprops': whiskerprops,
            'whiskerprops': whiskerprops,
            'capprops': {'linewidth': 0},  # Hide the caps
            'showfliers': show_outliers,
            'palette': group_colors,
            'hue_order': hue_order,
            'width': 0.75
        }

        # Stripplot customization
        stripplot_kwargs = {
            'linewidth': 0.6,
            'size': 6,
            'alpha': 0.7,
            'jitter': True,
            'dodge': True,
            'marker': 'o' if show_outliers else 'd',
            'palette': lightened_colors,
            'hue_order': hue_order
        }

        # Filter by specified groups and stimulations if provided
        if groups:
            mean_df = mean_df[mean_df['Group'].isin(groups)]
        if stimulations:
            mean_df = mean_df[mean_df['Stimulation'].isin(stimulations)]

        # Plotting
        plt.figure(figsize=(12, 6))
        ax = sns.boxplot(data=mean_df, x='Stimulation', y='mean_stimulation', hue='Group', **boxplot_kwargs)

        # Manually set the facecolor for boxplot
        for i, artist in enumerate(ax.artists):
            col = lightened_colors[ax.get_legend_handles_labels()[1][i // len(stimulations)]]
            artist.set_facecolor(col)

        # Add stripplot on top of boxplot for raw data visualization
        sns.stripplot(data=mean_df, x='Stimulation', y='mean_stimulation', hue='Group', **stripplot_kwargs)

        # Enhance the plot
        plt.title('Comparison of Mean Stimulation Across Groups and Stimulations')
        plt.ylabel('Mean Stimulation')
        plt.xlabel('Stimulation Type')
        ax.legend(title='Group')
        plt.show()
               
    def remove_outliers_by_stimulation(self, df, value_column='mean_stimulation'):
        """
        Removes outliers within each stimulation group based on the interquartile range (IQR).
        
        Args:
            df (DataFrame): DataFrame containing the data, expected to have a 'Stimulation' column.
            value_column (str): The name of the column from which outliers will be removed.
        
        Returns:
            DataFrame: DataFrame with outliers removed within each stimulation group.
        """
        # Create an empty DataFrame to store results after removing outliers
        channel = pd.DataFrame()
        
        # Process each stimulation group separately
        for stim in df['Stimulation'].unique():
            sub_df = df[df['Stimulation'] == stim]
            Q1 = sub_df[value_column].quantile(0.25)
            Q3 = sub_df[value_column].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            # Filter out outliers and append the results to the filtered DataFrame
            result_df = sub_df[(sub_df[value_column] >= lower_bound) & (sub_df[value_column] <= upper_bound)]
            filtered_df = pd.concat([filtered_df, result_df], ignore_index=True)
        
        return filtered_df
    
    def plot_mean_stimulation_box_and_strip2(self, mean_df, groups=None, stimulations=None, hue_order=None, remove_outliers_option=False, ylim=None, laminar_labels=None, directory=None, file_name=None):
        """
        Plots boxplots and stripplots for specified groups and stimulations using the mean stimulation data, with an option to exclude outliers.
        Args:
            mean_df (DataFrame): The DataFrame containing 'mean_stimulation' along with 'Group' and 'Stimulation' columns.
            groups (list of str, optional): List of groups to include in the plot.
            stimulations (list of str, optional): List of stimulations to include in the plot.
            hue_order (list, optional): Order of the hue levels, specifying how groups should be ordered in the plot.
            remove_outliers_option (bool, optional): If True, outliers will be removed before plotting.
            laminar_labels (list of str, optional): List of laminar labels to include in the plot. '['IG, 'SG','L4']
        """
        # Filter the DataFrame by groups and stimulations if provided
        print("Initial mean_df shape:", mean_df.shape)
        print("mean_df columns:", mean_df.columns)
        
        if groups:
            mean_df = mean_df[mean_df['Group'].isin(groups)]
            print("After filtering by groups, mean_df shape:", mean_df.shape)
        if stimulations:
            mean_df = mean_df[mean_df['Stimulation'].isin(stimulations)]
            print("After filtering by stimulations, mean_df shape:", mean_df.shape)
        if laminar_labels:
            mean_df = mean_df[mean_df['LaminarLabel'].isin(laminar_labels)]
            print("After filtering by laminar labels, mean_df shape:", mean_df.shape)
        
        # Remove outliers if the option is set to True
        if remove_outliers_option:
            mean_df = self.remove_outliers_by_stimulation(mean_df, 'mean_stimulation')
            print("After removing outliers, mean_df shape:", mean_df.shape)
        
        # Check if DataFrame is empty
        if mean_df.empty:
            raise ValueError("The DataFrame is empty after filtering. Check the input parameters and data.")
        
        stats_df = self.run_group_comparisons(mean_df, group_column='Group', value_column='mean_stimulation', stim_column='Stimulation')
        
        # Convert the mean_stimulation column to Hz by multiplying by 1000
        mean_df['mean_stimulation'] = mean_df['mean_stimulation']*1000
        
        # Plotting
        plt.figure(figsize=(12, 6))
        ax = sns.boxplot(data=mean_df, x='Stimulation', y='mean_stimulation', hue='Group', 
                        boxprops={'edgecolor': 'k', 'linewidth': 2},
                        medianprops={'color': 'k', 'linewidth': 2},
                        whiskerprops={'color': 'k', 'linewidth': 2},
                        capprops={'linewidth': 0},  # Hide the caps
                        showfliers=not remove_outliers_option,
                        palette={'No_CTZ': '#797979', 'CTZ': '#5a00c2'},
                        hue_order=hue_order,
                        width=0.75)

        # Add stripplot on top of boxplot for raw data visualization
        sns.stripplot(data=mean_df, x='Stimulation', y='mean_stimulation', hue='Group', 
                    linewidth=0.6, size=6, alpha=0.7, jitter=True, dodge=True, marker='o',
                    palette={'No_CTZ': '#79797933', 'CTZ': '#5a00c233'},  # Lighter colors
                    hue_order=hue_order)
        
        # Adjust opacity of the box fill
        for patch in ax.artists:
            r, g, b, a = patch.get_facecolor()
            patch.set_facecolor((r, g, b, .1))  # Set the fill alpha to 10%
        
        # Removing other components of the boxplot
        for line in ax.lines:
            # Only keep the median and the whiskers visible:
            # The boxplot consists of 6 lines per group: 0 and 1 are the box, 2 is the median,
            # 3 and 4 are the whiskers, 5 is the cap. This might slightly vary depending on Seaborn's implementation.
            if line.get_linestyle() != '-':  # Only preserve the median line
                line.set_linewidth(0)
        
        # Set the y-axis limits if specified
        if ylim:
            plt.ylim(ylim) #ylim is a tuple (min, max)
        
        # Enhance the plot
        plt.title('Comparison of Mean Stimulation Across Groups and Stimulations')
        plt.ylabel('Mean Stimulation')
        plt.xlabel('Stimulation Type')
        ax.legend(title='Group')
        
        # Prompt user for directory and file name if not provided
        if directory is None:
            directory = input("Please enter the directory to save the plot: ")
        if file_name is None:
            file_name = input("Please enter the file name to save the plot: ")

        # Create directory if it does not exist
        os.makedirs(directory, exist_ok=True)

        # Save the figure as an SVG file in the specified directory
        file_path = os.path.join(directory, f'{file_name}.svg')
        plt.savefig(file_path, format='svg')
        
        plt.show()
        
        return stats_df, mean_df

    def plot_mean_sem_line2(self, mean_df, groups=None, stimulations=None, hue_order=None, remove_outliers_option=False, ylim=None, laminar_labels=None, directory=None, file_name=None):
        """
        Plots mean and SEM line plots for specified groups and stimulations using the mean stimulation data, with an option to exclude outliers.

        Args:
            mean_df (DataFrame): The DataFrame containing 'mean_stimulation' along with 'Group' and 'Stimulation' columns.
            groups (list of str, optional): List of groups to include in the plot.
            stimulations (list of str, optional): List of stimulations to include in the plot.
            hue_order (list, optional): Order of the hue levels, specifying how groups should be ordered in the plot.
            remove_outliers_option (bool, optional): If True, outliers will be removed before plotting.
            ylim (tuple, optional): Tuple specifying the y-axis limits (min, max).
            laminar_labels (list of str, optional): List of laminar labels to include in the plot. '['IG, 'SG','L4']
        """
        # Filter the DataFrame by groups and stimulations if provided
        print("Initial mean_df shape:", mean_df.shape)
        print("mean_df columns:", mean_df.columns)
        
        if groups:
            mean_df = mean_df[mean_df['Group'].isin(groups)]
            print("After filtering by groups, mean_df shape:", mean_df.shape)
        if stimulations:
            mean_df = mean_df[mean_df['Stimulation'].isin(stimulations)]
            print("After filtering by stimulations, mean_df shape:", mean_df.shape)
        if laminar_labels:
            mean_df = mean_df[mean_df['LaminarLabel'].isin(laminar_labels)]
            print("After filtering by laminar labels, mean_df shape:", mean_df.shape)
        
        # Remove outliers if the option is set to True
        if remove_outliers_option:
            mean_df = self.remove_outliers_by_stimulation(mean_df, 'mean_stimulation')
            print("After removing outliers, mean_df shape:", mean_df.shape)
        
        # Check if DataFrame is empty
        if mean_df.empty:
            raise ValueError("The DataFrame is empty after filtering. Check the input parameters and data.")
        
        # Compute means and SEMs
        summary_df = mean_df.groupby(['Group', 'Stimulation']).agg(
            mean_value=('mean_stimulation', 'mean'),
            sem_value=('mean_stimulation', 'sem')
        ).reset_index()
        
        # Convert the mean_stimulation column to Hz by multiplying by 1000
        summary_df['mean_value'] = summary_df['mean_value'] * 1000
        summary_df['sem_value'] = summary_df['sem_value'] * 1000
        
        # Plotting
        plt.figure(figsize=(12, 6))
        
        # Define color mapping for groups
        group_colors = {
            'No_CTZ': '#797979',
            'CTZ': '#5a00c2'
        }

        for group in groups:
            group_data = summary_df[summary_df['Group'] == group]
            plt.errorbar(group_data['Stimulation'], group_data['mean_value'], yerr=group_data['sem_value'],
                        label=group, color=group_colors[group], capsize=5, marker='o', linestyle='-')
        
        # Set the y-axis limits if specified
        if ylim:
            plt.ylim(ylim)
        
        # Enhance the plot
        plt.title('Comparison of Mean Stimulation Across Groups and Stimulations')
        plt.ylabel('Mean Stimulation (Hz)')
        plt.xlabel('Stimulation Type')
        plt.legend(title='Group')
        
        # Prompt user for directory and file name if not provided
        if directory is None:
            directory = input("Please enter the directory to save the plot: ")
        if file_name is None:
            file_name = input("Please enter the file name to save the plot: ")

        # Create directory if it does not exist
        os.makedirs(directory, exist_ok=True)

        # Save the figure as an SVG file in the specified directory
        file_path = os.path.join(directory, f'{file_name}.svg')
        plt.savefig(file_path, format='svg')
        
        plt.show()
        
        return summary_df, mean_df


    def plot_group_cell_distribution(self):
        # Dictionary to store the counts for each group and recording
        group_counts = {}
        recording_count_per_group = {}  # To store the number of recordings per group
        stim_responsivity_counts = {}   # To store StimResponsivity counts for SUA and MUA separately
        combined_stim_responsivity_counts = {}  # To store combined StimResponsivity counts

        # Iterate over each cell grouping
        for (groupname, recordingname, cid), group_df in self.iterate_by_group_recording_cid():
            if groupname not in group_counts:
                group_counts[groupname] = {'SUA': {'FS': 0, 'RS': 0}, 'MUA': {'FS': 0, 'RS': 0}}
                stim_responsivity_counts[groupname] = {'SUA': {'-1.0': 0, '0.0': 0, '1.0': 0}, 'MUA': {'-1.0': 0, '0.0': 0, '1.0': 0}}
                combined_stim_responsivity_counts[groupname] = {'-1.0': 0, '0.0': 0, '1.0': 0}
                recording_count_per_group[groupname] = set()

            recording_count_per_group[groupname].add(recordingname)

            # Summarize the data across recordings
            for index, row in group_df.iterrows():
                unit_type = 'SUA' if row['IsSingleUnit'] == 1.0 else 'MUA'
                cell_type = row['Cell_Type']
                stim_responsivity = str(row['StimResponsivity'])
                if cell_type in ['FS', 'RS']:
                    group_counts[groupname][unit_type][cell_type] += 1
                if stim_responsivity in ['-1.0', '0.0', '1.0']:
                    stim_responsivity_counts[groupname][unit_type][stim_responsivity] += 1
                    combined_stim_responsivity_counts[groupname][stim_responsivity] += 1

        # Plotting
        plt.rcParams.update({'font.size': 14})  # Increase the base font size
        for groupname in group_counts:
            fig, axs = plt.subplots(1, 5, figsize=(24, 5))  # Adjust subplot for 5 charts
            total_recordings = len(recording_count_per_group[groupname])
            total_units = sum(sum(sub_counts.values()) for sub_counts in group_counts[groupname].values())
            sua_total = sum(group_counts[groupname]['SUA'].values())
            mua_total = sum(group_counts[groupname]['MUA'].values())

            # Titles and SUA/MUA Distribution
            fig.suptitle(f'Group: {groupname} | Mice: {total_recordings} | Total Units: {total_units} (SUA: {sua_total}, MUA: {mua_total})',
                        fontsize=16)

            for i, (unit_type, sub_counts) in enumerate(group_counts[groupname].items()):
                labels = [f'{ct} ({n})' for ct, n in sub_counts.items()]
                sizes = sub_counts.values()
                if any(sizes):
                    axs[i].pie(sizes, labels=labels, autopct=lambda p: '{:.1f}%'.format(p) if p > 0 else '',
                            startangle=90)
                    axs[i].set_title(f'{unit_type} Distribution', fontsize=14)

            # StimResponsivity Distribution for SUA and MUA
            for i, unit_type in enumerate(['SUA', 'MUA']):
                labels = [f'{resp} ({count})' for resp, count in stim_responsivity_counts[groupname][unit_type].items()]
                sizes = stim_responsivity_counts[groupname][unit_type].values()
                if any(sizes):
                    axs[i+2].pie(sizes, labels=labels, autopct=lambda p: '{:.1f}%'.format(p) if p > 0 else '',
                                startangle=90)
                    axs[i+2].set_title(f'{unit_type} Stim Responsivity', fontsize=14)

            # Combined StimResponsivity Distribution
            labels = [f'{resp} ({count})' for resp, count in combined_stim_responsivity_counts[groupname].items()]
            sizes = combined_stim_responsivity_counts[groupname].values()
            if any(sizes):
                axs[4].pie(sizes, labels=labels, autopct=lambda p: '{:.1f}%'.format(p) if p > 0 else '',
                        startangle=90)
                axs[4].set_title('Combined Stim Responsivity', fontsize=14)

            plt.tight_layout()  # Adjust layout
            plt.show()

    def run_group_comparisons(self, df, group_column='Group', value_column='mean_stimulation', stim_column='Stimulation'):
        """
        Performs statistical comparisons between groups for each type of stimulation, checks for normality,
        uses the appropriate non-parametric tests, and includes detailed descriptive statistics.
        """
        results = []

        for stim in df[stim_column].unique():
            sub_df = df[df[stim_column] == stim]
            groups = sub_df[group_column].unique()

            if len(groups) < 2:
                continue  # Skip if not enough groups for comparison

            group_data = [sub_df[sub_df[group_column] == g][value_column].dropna() for g in groups]

            # Skip normality test if any group has less than 3 data points
            normality_results = []
            normality_p_values = []
            for data in group_data:
                if len(data) < 3:
                    normality_results.append(None)
                    normality_p_values.append(None)
                else:
                    result = shapiro(data)
                    normality_results.append(result)
                    normality_p_values.append(result.pvalue)

            # Choose the appropriate statistical test based on group count
            if len(groups) == 2:
                stat, p_value = mannwhitneyu(*group_data)
                test_used = 'Mann-Whitney U'
            elif len(groups) > 2:
                stat, p_value = kruskal(*group_data)
                test_used = 'Kruskal-Wallis'
                # Note: If using Kruskal-Wallis, consider post-hoc tests for detailed group comparisons
            else:
                continue

            descriptive_stats = [{
                'mean': data.mean(),
                'SD': data.std(),
                'median': data.median(),
                'range_min': data.min(),
                'range_max': data.max(),
            } for data in group_data]

            result_entry = {
                'Stimulation': stim,
                'Test Used': test_used,
                'Test Statistic': stat,
                'p-value': p_value,
                **{f'N Group{i+1}': len(data) for i, data in enumerate(group_data)},
                **{f'Normality p-value Group{i+1}': p for i, p in enumerate(normality_p_values) if p is not None},
                **{f'{stat_key} Group{i+1}': stat_val for i, stats in enumerate(descriptive_stats) for stat_key, stat_val in stats.items()}
            }

            results.append(result_entry)

        return pd.DataFrame(results)

    def prepare_for_boxplot_default(self, dataframe_key, value_column, group_label='groupname'):
        """
        Organizes data into a DataFrame suitable for plotting boxplots by extracting values
        from a specified column in detailed DataFrames, including labels for group.

        Parameters:
            dataframe_key (str): Key to access the specific DataFrame.
            value_column (str): Column name from which to extract the value for plotting.
            group_label (str): Column name to use as label for grouping in the plot. For example, 'groupname'

        Returns:
            pandas.DataFrame: A DataFrame with columns for the specified 'value_column', and 'Group'.
        """
        # Access the specified DataFrame
        df = self.dataframes[dataframe_key]

        boxplot_data = []

        # Iterate over each stored DataFrame key (group)
        for group in df[group_label].unique():
            group_df = df[df[group_label] == group]

            # Check for empty DataFrame
            if not group_df.empty:
                # Extract specified value column and corresponding group label
                for index, row in group_df.iterrows():
                    boxplot_data.append({
                        value_column: row[value_column],
                        'Group': group
                    })

        # Convert list of data to DataFrame
        boxplot_df = pd.DataFrame(boxplot_data)

        return boxplot_df
    
    def unpack_based_on_stim(self, dataframe_key, list_column, cell_type_filter=None, is_single_unit=True, 
                             stim_responsivity_filter=None, modulation_label=None):
        """
        Unpacks values from a specified list column in the DataFrame if they are four elements long,
        and labels each value according to provided trial tags stored in eed. Includes group and additional filtering options.

        Parameters:
            dataframe_key (str): Key to access the specific DataFrame.
            list_column (str): Column name from which to extract numpy.ndarray for unpacking.
            cell_type_filter (str, optional): Filters rows based on 'Cell_Type' (e.g., 'FS' or 'RS').
            is_single_unit (bool, optional): Filters rows where 'IsSingleUnit' is 1.0 (True) or 0.0 (False).
            stim_responsivity_filter (int, optional): Filters rows based on 'StimResponsivity' (-1, 0, 1).
            modulation_label (stf, optional): Column name to use, its a string: 'positive', 'negative' or 'none'

        Returns:
            pandas.DataFrame: A DataFrame with unpacked values labeled by stimulation, including additional columns.
        """
        df = self.dataframes[dataframe_key]
        
        trial_tags_labels = self.eed.trialTagsLabels['trialTagsLabels']

        # Convert and filter data as per user requirements
        df['IsSingleUnit'] = df['IsSingleUnit'].astype(bool)
        df['StimResponsivity'] = df['StimResponsivity'].astype(int)

        if cell_type_filter is not None:
            df = df[df['Cell_Type'] == cell_type_filter]
        if is_single_unit is not None:
            df = df[df['IsSingleUnit'] == is_single_unit]
        if stim_responsivity_filter is not None:
            df = df[df['StimResponsivity'] == stim_responsivity_filter]
        if modulation_label is not None:
            df = df[df['ModulationIndex'] == modulation_label]
        
        unpacked_data = []

        for index, row in df.iterrows():
            values = row[list_column]
            if isinstance(values, np.ndarray) and len(values) == 4:
                for i, value in enumerate(values):
                    unpacked_data.append({
                        'Group': row['groupname'],
                        'Stimulation': trial_tags_labels[i],
                        list_column: value,
                        'Cell_Type': row['Cell_Type'],
                        'IsSingleUnit': row['IsSingleUnit'],
                        'StimResponsivity': row['StimResponsivity'],
                        'recordingname': row['recordingname'],
                        'cid': row['cid'],
                        'ModulationIndex': row['ModulationIndex']
                    })

        unpacked_df = pd.DataFrame(unpacked_data)

        return unpacked_df
    
    def plot_box_and_strip_default(self, groups=None, stimulations=None, show_outliers=True, hue_order=None, dataframe_key=None, list_column=None, 
                                   cell_type_filter=None, is_single_unit=None, stim_responsivity_filter=None, 
                                   remove_outliers_option=False, directory=None, file_name=None, 
                                   modulation_label=None, firstspike_latency=True):
        """
        Plots boxplots and stripplots for specified groups and stimulations from the unpacked DataFrame,
        with color adjustments made directly in the plotting calls. Designed to work with unpacked DataFrame structure.

        Args:
            groups (list of str, optional): List of groups to include in the plot.
            stimulations (list of str, optional): List of stimulations to include in the plot.
            show_outliers (bool, optional): Whether to show outliers.
            hue_order (list, optional): Order of the hue levels.
            modululation_label (str, optional): Filter for 'positive', 'negative' or 'none' modulation
        """
        # Define color mapping for groups
        group_colors = {
            'No_CTZ': '#797979',
            'CTZ': '#5a00c2'
        }

        # Generate lighter versions for the box face color
        lightened_colors = {k: v + '33' for k, v in group_colors.items()}

        # Boxplot customization
        boxprops = {'edgecolor': 'k', 'linewidth': 2}
        whiskerprops = {'color': 'k', 'linewidth': 2}
        boxplot_kwargs = {
            'boxprops': boxprops,
            'medianprops': whiskerprops,
            'whiskerprops': whiskerprops,
            'capprops': {'linewidth': 0},  # Hide the caps
            'showfliers': show_outliers,
            'palette': group_colors,
            'hue_order': hue_order,
            'width': 0.75
        }

        # Stripplot customization
        stripplot_kwargs = {
            'linewidth': 0.6,
            'size': 6,
            'alpha': 0.7,
            'jitter': True,
            'dodge': True,
            'marker': 'o' if show_outliers else 'd',
            'palette': lightened_colors,
            'hue_order': hue_order
        }

        # Prepare data for boxplot (already prepared by a previous method)
        boxplot_df = self.unpack_based_on_stim(dataframe_key=dataframe_key, list_column=list_column, cell_type_filter=cell_type_filter, 
                                               is_single_unit=is_single_unit, stim_responsivity_filter=stim_responsivity_filter, 
                                               modulation_label=modulation_label, firstspike_latency=firstspike_latency)
        # Filter by specified groups and stimulations
        if groups:
            boxplot_df = boxplot_df[boxplot_df['Group'].isin(groups)]
        if stimulations:
            boxplot_df = boxplot_df[boxplot_df['Stimulation'].isin(stimulations)]

        # Remove outliers if the option is set to True
        if remove_outliers_option:
            boxplot_df = self.remove_outliers_by_stimulation_default(boxplot_df, value_column=list_column)
        
        # Plotting
        plt.figure(figsize=(12, 6))
        ax = sns.boxplot(data=boxplot_df, x='Stimulation', y=list_column, hue='Group', **boxplot_kwargs)

        # Manually set the facecolor for boxplot
        for i, artist in enumerate(ax.artists):
            col = lightened_colors[ax.get_legend_handles_labels()[1][i // len(stimulations)]]
            artist.set_facecolor(col)

        # Add stripplot on top of boxplot for raw data visualization
        sns.stripplot(data=boxplot_df, x='Stimulation', y=list_column, hue='Group', **stripplot_kwargs)

        # Enhance the plot
        plt.title('Comparison of Mean Stimulation Across Groups and Stimulations')
        plt.ylabel(list_column)
        plt.xlabel('Stimulation Type')
        ax.legend(title='Group')
        
        # Prompt user for directory and file name if not provided
        if directory is None:
            directory = input("Please enter the directory to save the plot: ")
        if file_name is None:
            file_name = input("Please enter the file name to save the plot: ")

        # Create directory if it does not exist
        os.makedirs(directory, exist_ok=True)

        # Save the figure as an SVG file in the specified directory
        file_path = os.path.join(directory, f'{file_name}.svg')
        plt.savefig(file_path, format='svg', transparent=True)

        
        plt.show()
        
    def plot_mean_sem_line_default(self, groups=None, stimulations=None, show_outliers=True, hue_order=None, dataframe_key=None, list_column=None, cell_type_filter=None, is_single_unit=None, stim_responsivity_filter=None, remove_outliers_option=False, directory=None, file_name=None):
        """
        Plots mean and SEM line plots for specified groups and stimulations from the unpacked DataFrame.

        Args:
            groups (list of str, optional): List of groups to include in the plot.
            stimulations (list of str, optional): List of stimulations to include in the plot.
            show_outliers (bool, optional): Whether to show outliers.
            hue_order (list, optional): Order of the hue levels.
            dataframe_key (str, optional): Key for accessing the correct DataFrame.
            list_column (str, optional): Column name for the values to be plotted.
            cell_type_filter (str, optional): Filter for cell type.
            is_single_unit (bool, optional): Filter for single units.
            stim_responsivity_filter (str, optional): Filter for stimulus responsivity.
            remove_outliers_option (bool, optional): Whether to remove outliers.
            directory (str, optional): Directory to save the plot.
            file_name (str, optional): File name to save the plot.
        """
        # Define color mapping for groups
        group_colors = {
            'No_CTZ': '#797979',
            'CTZ': '#5a00c2'
        }

        # Prepare data for plotting (already prepared by a previous method)
        plot_df = self.unpack_based_on_stim(dataframe_key=dataframe_key, list_column=list_column, cell_type_filter=cell_type_filter, is_single_unit=is_single_unit, stim_responsivity_filter=stim_responsivity_filter)

        # Filter by specified groups and stimulations
        if groups:
            plot_df = plot_df[plot_df['Group'].isin(groups)]
        if stimulations:
            plot_df = plot_df[plot_df['Stimulation'].isin(stimulations)]

        # Remove outliers if the option is set to True
        if remove_outliers_option:
            plot_df = self.remove_outliers_by_stimulation_default(plot_df, value_column=list_column)

        # Compute means and SEMs
        summary_df = plot_df.groupby(['Group', 'Stimulation']).agg(
            mean_value=(list_column, 'mean'),
            sem_value=(list_column, 'sem')
        ).reset_index()

        # Plotting
        plt.figure(figsize=(12, 6))
        
        for group in groups:
            group_data = summary_df[summary_df['Group'] == group]
            plt.errorbar(group_data['Stimulation'], group_data['mean_value'], yerr=group_data['sem_value'],
                        label=group, color=group_colors[group], capsize=5, marker='o', linestyle='-')

        # Enhance the plot
        plt.title('Mean and SEM of Stimulation Responses Across Groups')
        plt.ylabel(list_column)
        plt.xlabel('Stimulation Type')
        plt.legend(title='Group')
        
        # Prompt user for directory and file name if not provided
        if directory is None:
            directory = input("Please enter the directory to save the plot: ")
        if file_name is None:
            file_name = input("Please enter the file name to save the plot: ")

        # Create directory if it does not exist
        os.makedirs(directory, exist_ok=True)

        # Save the figure as an SVG file in the specified directory
        file_path = os.path.join(directory, f'{file_name}.svg')
        plt.savefig(file_path, format='svg', transparent=True)

        plt.show() 
        
        
        
    def remove_outliers_by_stimulation_default(self, df, value_column=None):
        """
        Removes outliers within each stimulation group based on the interquartile range (IQR).
        
        Args:
            df (DataFrame): DataFrame containing the data, expected to have a 'Stimulation' column.
            value_column (str): The name of the column from which outliers will be removed.
        
        Returns:
            DataFrame: DataFrame with outliers removed within each stimulation group.
        """
        # Create an empty DataFrame to store results after removing outliers
        filtered_df = pd.DataFrame()
        
        # Process each stimulation group separately
        for stim in df['Stimulation'].unique():
            sub_df = df[df['Stimulation'] == stim]
            Q1 = sub_df[value_column].quantile(0.25)
            Q3 = sub_df[value_column].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            # Filter out outliers and append the results to the filtered DataFrame
            result_df = sub_df[(sub_df[value_column] >= lower_bound) & (sub_df[value_column] <= upper_bound)]
            filtered_df = pd.concat([filtered_df, result_df], ignore_index=True)
        
        return filtered_df
    
    def extract_spike_trains(self, recording_name):
        # Filter the data for a specific recording and only single units
        filtered_data = self.data[(self.data['recordingname'] == recording_name) & (self.data['IsSingleUnit'] == 1.0)]
        
        # Extract the SpikeTrains_for_PSTHs for these filtered entries
        spike_trains = filtered_data['SpikeTrains_for_PSTHs'].tolist()
        
        # Optional: Convert any additional processing on spike trains here
        # For example, converting to numpy array, etc.
        
        return spike_trains
    
    def simplify_and_filter_dataframe(self, df_name, group_name=None, is_single_unit=None, cell_type=None, stim_responsivity=None, modulation_type=None):
        """
        Simplify and filter the DataFrame based on IsSingleUnit, Cell_Type, and StimResponsivity.

        Parameters:
        df_name (str): Name of the DataFrame to filter.
        is_single_unit (float or None): If 1.0, filter for single units, if 0.0, filter for non-single units, 
                                        if None, do not filter by this criterion.
        cell_type (str or None): Filter for 'FS' or 'RS' cell types. If None, no filtering by cell type.
        stim_responsivity (float or None): Value to filter by in the StimResponsivity column. Can be 1.0, 0.0, or -1.0.
                                           If None, no filtering by StimResponsivity.
        modulation_type (str or None): Filter for 'positive', 'negative', or 'none'. 
                                            If None, no filtering by modulation type.

        Returns:
        pandas.DataFrame: The simplified and filtered DataFrame.
        """
        if df_name not in self.dataframes:
            print(f"No DataFrame found with the name '{df_name}'.")
            return None
        
        # Ensure ModulationIndex column is present
        self.label_modulation_index(df_name)

        # Start with the full DataFrame
        df = self.dataframes[df_name]

        # Print the first few entries and data types for debugging
        #print("Original DataFrame:")
        #print(df.head())
        #print("\nData types of the columns:")
        #print(df.dtypes)
        

        # Convert numpy arrays with single values to scalar values
        def extract_single_value(x):
            if isinstance(x, np.ndarray) and x.ndim > 0:
                return x[0]
            return x

        for col in ['IsSingleUnit', 'StimResponsivity', 'ModulationIndex', 'Template_Channel']:
            if col in df.columns:
                df[col] = df[col].apply(extract_single_value)

        # Print the simplified DataFrame for debugging
        #print("\nSimplified DataFrame:")
        #print(df.head())
        #print("\nData types of the columns after simplification:")
        #print(df.dtypes)

        # Filter by cell type if provided
        if cell_type:
            df = df[df['Cell_Type'] == cell_type]
            #print(f"\nDataFrame after filtering by Cell_Type='{cell_type}':")
            #print(df)

        # Filter by IsSingleUnit if not None
        if is_single_unit is not None:
            df = df[df['IsSingleUnit'] == is_single_unit]
            #print(f"\nDataFrame after filtering by IsSingleUnit={is_single_unit}:")
            #print(df)

        # Filter by StimResponsivity if not None
        if stim_responsivity is not None:
            df = df[df['StimResponsivity'] == stim_responsivity]
            #print(f"\nDataFrame after filtering by StimResponsivity={stim_responsivity}:")
            #print(df)
        
        # Filter by group name if provided
        if group_name:
            df = df[df['groupname'] == group_name]
            #print(f"\nDataFrame after filtering by groupname='{group_name}':")
            #print(df)
        
        # Filter by modulation type if provided
        if modulation_type:
            df = df[df['ModulationIndex'] == modulation_type]
            print(f"Filtered by modulation type='{modulation_type}': {df.shape}")
        
        return df
    
    def prepare_plotting_data(self, df_name, group_name=None, is_single_unit=None, cell_type=None, stim_responsivity=None, modulation_type=None):
        """
        Prepare data for plotting the modulation index as a function of electrode location.

        Parameters:
        df_name (str): Name of the DataFrame to filter and prepare.
        group_name (str or None): Name of the group to filter.
        is_single_unit (float or None): If 1.0, filter for single units, if 0.0, filter for non-single units, 
                                        if None, do not filter by this criterion.
        cell_type (str or None): Filter for 'FS' or 'RS' cell types. If None, no filtering by cell type.
        stim_responsivity (float or None): Value to filter by in the StimResponsivity column. Can be 1.0, 0.0, or -1.0.
                                        If None, no filtering by StimResponsivity.
        modulation_type (str or None): Filter for 'positive', 'negative', or 'none'. If None, no filtering by modulation type.

        Returns:
        pandas.DataFrame: The DataFrame ready for plotting.
        """
        # Ensure the original numeric ModulationIndex column is present and labeled correctly
        df = self.simplify_and_filter_dataframe(df_name, group_name, is_single_unit, cell_type, stim_responsivity, modulation_type)
        if df is None or df.empty:
            print("No data to prepare for plotting.")
            return None

        # Define the electrode order
        electrodes_order = [14, 20, 16, 18, 1, 31, 3, 29, 5, 27, 7, 25, 9, 23, 11, 21, 13, 19, 15, 17, 12, 22, 10, 24, 8, 26, 6, 28, 4, 30, 2, 32]

        # Create a mapping from Template_Channel to electrode order index
        channel_mapping = {electrode: idx for idx, electrode in enumerate(electrodes_order)}

        # Map Template_Channel to electrode order index
        df['ElectrodeOrder'] = df['Template_Channel'].map(channel_mapping)

        # Drop rows where the mapping resulted in NaN values
        df = df.dropna(subset=['ElectrodeOrder'])

        # Print the prepared DataFrame for debugging
        print("\nPrepared DataFrame for plotting:")
        #print(df[['ElectrodeOrder', 'ModulationIndex_Numeric', 'MeanFR_baseline', 'groupname', 'StimProb', 'MeanFR_stim'  ]].head())  # Use the new numeric column

        return df[['ElectrodeOrder', 'ModulationIndex_Numeric', 'groupname', 'MeanFR_baseline', 'StimProb', 'MeanFR_stim' ]].sort_values('ElectrodeOrder')


    def plot_modulation_index(self, df_name, group_name=None, is_single_unit=None, cell_type=None, stim_responsivity=None):
        """
        Plot the modulation index as a function of electrode location.

        Parameters:
        df_name (str): Name of the DataFrame to filter and plot.
        group_name (str or None): Name of the group to filter.
        is_single_unit (float or None): If 1.0, filter for single units, if 0.0, filter for non-single units, 
                                        if None, do not filter by this criterion.
        cell_type (str or None): Filter for 'FS' or 'RS' cell types. If None, no filtering by cell type.
        stim_responsivity (float or None): Value to filter by in the StimResponsivity column. Can be 1.0, 0.0, or -1.0.
                                        If None, no filtering by StimResponsivity.
        """
        # Prepare the data for plotting
        plotting_data = self.prepare_plotting_data(df_name, group_name, is_single_unit, cell_type, stim_responsivity)
        if plotting_data is None or plotting_data.empty:
            print("No data available for plotting.")
            return

        # Drop rows with NaN in relevant columns
        plotting_data = plotting_data.dropna(subset=['MeanFR_baseline', 'ModulationIndex_Numeric', 'ElectrodeOrder'])

        # Convert 'MeanFR_baseline' from array-like to scalar values
        plotting_data['MeanFR_baseline'] = plotting_data['MeanFR_baseline'].apply(lambda x: x.item() if isinstance(x, np.ndarray) else x)

        # Convert to numeric and ensure correct length
        plotting_data['MeanFR_baseline'] = pd.to_numeric(plotting_data['MeanFR_baseline'], errors='coerce')
        plotting_data['ModulationIndex_Numeric'] = pd.to_numeric(plotting_data['ModulationIndex_Numeric'], errors='coerce')
        plotting_data['ElectrodeOrder'] = pd.to_numeric(plotting_data['ElectrodeOrder'], errors='coerce')

        # Drop rows where any of these columns have NaNs after conversion
        plotting_data = plotting_data.dropna(subset=['MeanFR_baseline', 'ModulationIndex_Numeric', 'ElectrodeOrder'])

        # Ensure all columns have the same length
        if not (len(plotting_data['MeanFR_baseline']) == len(plotting_data['ModulationIndex_Numeric']) == len(plotting_data['ElectrodeOrder'])):
            print("Mismatch in data sizes after cleaning.")
            return

        # Normalize size values if necessary (optional step)
        size_values = plotting_data['MeanFR_baseline'].values
        size_values = size_values / np.max(size_values) * 200  # Normalize to range suitable for plotting

        # Plot the modulation index
        plt.figure(figsize=(2, 6))
        scatter = plt.scatter(plotting_data['ModulationIndex_Numeric'], plotting_data['ElectrodeOrder'], 
                            s=size_values,  # Use normalized sizes
                            c=plotting_data['ModulationIndex_Numeric'], 
                            cmap='bwr', alpha=0.6)
        
        
        # Enforce the color bar is always from -1 to 1 
        plt.clim(-1, 1)
        plt.colorbar(scatter, label='Modulation Index')
        plt.xlabel('Spontaneous MI')
        plt.ylabel('Electrode Order')
        plt.title(f'{cell_type} multi-units from {group_name}')
        plt.gca().invert_yaxis()  # To match the order from the provided list

        
        
        # Ensure the y-axis has all the electrodes listed 
        plt.xlim(-1.5, 1.5)
        plt.grid(False)
        plt.yticks(np.arange(0, 32, 1))

        # Save the plot as a SVG file
        directory = '/Volumes/MannySSD/figures/laminar_plots'
        try:
            os.makedirs(directory, exist_ok=True)
        except OSError as e:
            print(f"Error creating directory: {e}")
            return

        # Make the file name more descriptive based on the user input, if single unit or multi unit or if stim responsive is provided in the file name 
        if is_single_unit == 1.0:
            file_name = f'{group_name}_single_units'
        elif is_single_unit == 0.0:
            file_name = f'{group_name}_multi_units'
        else:
            file_name = f'{group_name}_all_units'
        
        if cell_type:
            file_name += f'_{cell_type}'
        if stim_responsivity:
            file_name += f'_stim_{stim_responsivity}'
        
        file_path = os.path.join(directory, f'{file_name}.svg')
        
        try:
            plt.savefig(file_path, format='svg')
            print(f"Plot saved to {file_path}")
            plt.show()
        except Exception as e:
            print(f"Error saving plot: {e}")





    def label_modulation_index(self, df_name):
        """
        Label the ModulationIndex as 'positive', 'negative', or 'none'.

        Parameters:
        df_name (str): Name of the DataFrame to label.

        Returns:
        pandas.DataFrame: The DataFrame with a new column 'ModulationIndex'.
        """
        if df_name not in self.dataframes:
            print(f"No DataFrame found with the name '{df_name}'.")
            return None

        df = self.dataframes[df_name]
        if 'ModulationIndex' not in df.columns:
            df['ModulationIndex'] = df['ModulationIndex'].apply(lambda x: 'positive' if x > 0.3 else ('negative' if x < -0.3 else 'none'))
        return df
    
    def label_modulation_index2(self, df):
        """
        Label the ModulationIndex as 'positive', 'negative', or 'none'.

        Parameters:
        df (pandas.DataFrame): The DataFrame to label.

        Returns:
        pandas.DataFrame: The DataFrame with a new column 'ModulationIndex'.
        """
        df['ModulationIndex'] = df['ModulationIndex'].apply(lambda x: 'positive' if x > 0 else ('negative' if x < 0 else 'none'))
        return df

    def plot_modulation_index_density(self, df_name, is_single_unit=None, cell_type=None, stim_responsivity=None, modulation_type=None):
        """
        Compare the density of cells across electrode numbers between two groups.

        Parameters:
        df_name (str): Name of the DataFrame to filter and plot.
        is_single_unit (float or None): If 1.0, filter for single units, if 0.0, filter for non-single units, 
                                        if None, do not filter by this criterion.
        cell_type (str or None): Filter for 'FS' or 'RS' cell types. If None, no filtering by cell type.
        stim_responsivity (float or None): Value to filter by in the StimResponsivity column. Can be 1.0, 0.0, or -1.0.
                                           If None, no filtering by StimResponsivity.
        modulation_type (str or None): Filter for 'positive', 'negative', or 'none'. If None, no filtering by modulation type.
        """
        # Prepare the data for both groups
        df_ctz = self.prepare_plotting_data(df_name, group_name=self.eed.group_names[0], is_single_unit=is_single_unit, cell_type=cell_type, stim_responsivity=stim_responsivity, modulation_type=modulation_type)
        df_no_ctz = self.prepare_plotting_data(df_name, group_name=self.eed.group_names[1], is_single_unit=is_single_unit, cell_type=cell_type, stim_responsivity=stim_responsivity, modulation_type=modulation_type)
        
        if df_ctz is None or df_ctz.empty or df_no_ctz is None or df_no_ctz.empty:
            print("No data available for plotting.")
            return

        # Ensure the bins are consistent for both groups
        bins = np.linspace(df_ctz['ElectrodeOrder'].min(), df_ctz['ElectrodeOrder'].max(), len(df_ctz['ElectrodeOrder'].unique()))

        # Plot the density of cells across electrode numbers comparison
        plt.figure(figsize=(10, 6))

        sns.histplot(df_ctz, y='ElectrodeOrder', bins=bins, color='blue', alpha=0.5, label='CTZ', kde=True, common_norm=False)
        sns.histplot(df_no_ctz, y='ElectrodeOrder', bins=bins, color='red', alpha=0.5, label='No CTZ', kde=False, common_norm=False)

        plt.xlabel('Density')
        plt.ylabel('Electrode Order')
        plt.title(f'{cell_type} multi-units: CTZ vs No CTZ')
        plt.legend()
        plt.show()
        
    def add_modulation_label_column(self, df_name):
        """
        Add the ModulationIndex column to the specified DataFrame in self.dataframes.

        Parameters:
        df_name (str): Name of the DataFrame to which the column should be added.
        """
        if df_name not in self.dataframes:
            print(f"No DataFrame found with the name '{df_name}'.")
            return None

        df = self.dataframes[df_name]
        df = self.label_modulation_index2(df)
        self.dataframes[df_name] = df
        print(f"Added ModulationIndex column to '{df_name}' DataFrame.")
        return df
     
    def filter_dataframe(self, df_name, modulation_type):
        """
        Filter the DataFrame to retain only specific entries based on ModulationIndex.

        Parameters:
        df_name (str): Name of the DataFrame to filter.
        modulation_type (str): The type of modulation to retain ('positive', 'negative', 'none').

        Returns:
        pandas.DataFrame: The filtered DataFrame.
        """
        if df_name not in self.dataframes:
            print(f"No DataFrame found with the name '{df_name}'.")
            return None

        df = self.dataframes[df_name]

        if 'ModulationIndex' not in df.columns:
            print("ModulationIndex column not found in the DataFrame.")
            return None

        # Filter the DataFrame based on ModulationIndex
        filtered_df = df[df['ModulationIndex'] == modulation_type]
        
        return filtered_df
 
    def save_filtered_dataframe(self, df_name, modulation_type):
        """
        Save the filtered DataFrame to self.dataframes and reset the index.

        Parameters:
        df_name (str): Name of the DataFrame to filter and save.
        modulation_type (str): The type of modulation to retain ('positive', 'negative', 'none').
        """
        filtered_df = self.filter_dataframe(df_name, modulation_type)
        if filtered_df is not None:
            filtered_df = filtered_df.reset_index(drop=True)  # Reset the index
            self.dataframes[df_name] = filtered_df
            print(f"Filtered DataFrame with {modulation_type} modulation saved to '{df_name}'.")
        else:
            print("Filtered DataFrame was not saved.")

    def create_psth_dataframe_2(self):
        """
        Creates and stores a DataFrame for each stimulation type using the 'SpikeTrains_for_PSTHs' and 'PSTHs_conv' columns from the base PSTH DataFrame. 
        Each DataFrame is stored as an attribute of the DataFrameManager under a name that corresponds to the stimulation type.

        """
        # Create the base dataframe for PSTH analysis with modulation filtering
        self.create_dataframe(['Cell_Type', 
            'LaminarLabel', 
            'IsSingleUnit', 
            'StimResponsivity', 
            'SpikeTrains_for_PSTHs', 
            'PSTHs_conv', 
            'ModulationIndex',
            'FirstSpikeLatency', 
            'FirstSpikeLatency_Reliability', 
            'TroughToPeak_duration', 
            'SpikeHalfWidth', 
            'PeakToPeak_ratio', 
            'peak1_normalized_amplitude', 
            'Peak1ToTrough_ratio', 
            'Peak2ToTrough_ratio', 
            'Template_Channel', 
            'MeanFR_baseline', 
            'Normalized_Template_Waveform', 
            'UnNormalized_Template_Waveform',
            'SpikeTimes_all', 
            'ISI_baseline_CV', 
            'ISI_baseline_vec', 
            'ISI_pdf_peak_xy', 
            'ISI_pdf_x', 
            'ISI_pdf_y',
            'StimProb', 
            'MeanFR_stim', 
            'PeakEvokedFR', 
            'PeakEvokedFR_Latency', 
            'FanoFactor_baseline', 
            'FanoFactor_stim', 
            'MeanFR_inst_stim', 
            'ISI_violations_percent', 
            'Recording_Duration', 
            'Sampling_Frequency'], 'psth_dataframe')
        # Extracting trial tags
        stim_labels = self.eed.trialTagsLabels['trialTagsLabels']  # ['Zero', 'Low', 'Mid', 'Max']

        # Process each label
        for index, label in enumerate(stim_labels):
            # Retrieve spike trains and PSTHs for each label and create a new DataFrame
            df_name = f'psth_dataframe_{label}'
            self.dataframes[df_name] = self.dataframes['psth_dataframe'].copy()
            self.dataframes[df_name]['SpikeTrains_for_PSTHs'] = self.dataframes['psth_dataframe']['SpikeTrains_for_PSTHs'].apply(lambda x: x[index] if isinstance(x, (list, np.ndarray)) else None)
            self.dataframes[df_name]['PSTHs_conv'] = self.dataframes['psth_dataframe']['PSTHs_conv'].apply(lambda x: x[index] if isinstance(x, (list, np.ndarray)) else None)
        

    def plot_group_comparison(self, stim_labels, cell_type=None, is_single_unit=None, stim_responsivity=None, groupnames=None, modulation_label=None, time_range=None, smoothing_window=10, ylim=None):
        fig, axes = plt.subplots(2, 4, figsize=(20, 10), sharex=True, sharey=True)
        axes = axes.flatten()

        # Define color mapping for groups
        group_colors = {
            'No_CTZ': '#797979',
            'CTZ': '#5a00c2'
        }

        # Generate lighter versions for the shaded area
        lightened_colors = {k: v + '33' for k, v in group_colors.items()}

        plot_idx = 0
        for groupname in groupnames:
            for stim_label in stim_labels:
                df = self.filter_data(stim_label, cell_type, is_single_unit, stim_responsivity, groupname, modulation_label)
                
                # Print the number of units that match the filter
                print(f"Number of units that match the filter for group {groupname} and stimulation {stim_label}: {df.shape[0]}")

                if df.empty:
                    print(f"No data matches the specified filters for group {groupname} and stimulation {stim_label}.")
                    plot_idx += 1
                    continue

                # Get relative time array for x-axis
                time_array = self.eed.relative_time_ms['relative_time_ms']
                
                # Create time mask and adjust time array
                time_mask = self.create_time_mask(time_array, time_range)
                time_array = time_array[time_mask]

                all_psths = []

                for index, row in df.iterrows():
                    individual_psth = row['PSTHs_conv']
                    # Apply time mask to individual PSTH
                    individual_psth = np.array(individual_psth)[time_mask]
                    
                    # Apply a smoothing window
                    window = np.ones(smoothing_window) / smoothing_window  # 10ms window of smoothing
                    individual_psth = np.convolve(individual_psth, window, mode='same')
                    
                    all_psths.append(individual_psth)
                    # Plot the individual PSTH with lighter color
                    color = lightened_colors.get(groupname, '#CCCCCC')
                    axes[plot_idx].plot(time_array, individual_psth, color=color, linewidth=1, alpha=0.5)

                # Calculate the mean PSTH
                all_psths = np.array(all_psths)
                mean_psth = np.mean(all_psths, axis=0)
                # Apply a smoothing window to the mean PSTH
                mean_psth = np.convolve(mean_psth, window, mode='same')
                # Plot the mean PSTH with a thicker line
                color = group_colors.get(groupname, 'black')
                axes[plot_idx].plot(time_array, mean_psth, label='Mean PSTH', color=color, linewidth=2)

                axes[plot_idx].set_title(f'Group: {groupname}, Stimulation: {stim_label}')
                axes[plot_idx].set_xlabel('Time (ms)')
                axes[plot_idx].set_ylabel('Spike Rate')
                axes[plot_idx].legend().set_visible(False)
                
                plot_idx += 1
                #specify the ylimit for all the subplots
        for ax in axes:
            ax.set_ylim(ylim)
        plt.tight_layout()
        plt.show()

    def plot_group_comparison_meansem(self, stim_labels, cell_type=None, is_single_unit=None, stim_responsivity=None, groupnames=None, modulation_label=None, time_range=None, smoothing_window=10, ylim=None, directory=None, file_name=None):
        fig, axes = plt.subplots(2, 4, figsize=(20, 10), sharex=True, sharey=True)
        axes = axes.flatten()

        # Define color mapping for groups
        group_colors = {
            'No_CTZ': '#797979',
            'CTZ': '#5a00c2'
        }

        # Generate lighter versions for the shaded area
        lightened_colors = {k: v + '33' for k, v in group_colors.items()}

        plot_idx = 0
        for groupname in groupnames:
            for stim_label in stim_labels:
                df = self.filter_data(stim_label, cell_type, is_single_unit, stim_responsivity, groupname, modulation_label)
                
                # Print the number of units that match the filter
                print(f"Number of units that match the filter for group {groupname} and stimulation {stim_label}: {df.shape[0]}")

                if df.empty:
                    print(f"No data matches the specified filters for group {groupname} and stimulation {stim_label}.")
                    plot_idx += 1
                    continue

                # Get relative time array for x-axis
                time_array = self.eed.relative_time_ms['relative_time_ms']
                
                # Create time mask and adjust time array
                time_mask = self.create_time_mask(time_array, time_range)
                time_array = time_array[time_mask]

                all_psths = []

                for index, row in df.iterrows():
                    individual_psth = row['PSTHs_conv']
                    # Apply time mask to individual PSTH
                    individual_psth = np.array(individual_psth)[time_mask]
                    
                    # Apply a smoothing window
                    window = np.ones(smoothing_window) / smoothing_window  # 10ms window of smoothing
                    individual_psth = np.convolve(individual_psth, window, mode='same')
                    
                    
                    all_psths.append(individual_psth)

                # Calculate the mean and SEM PSTH
                all_psths = np.array(all_psths)
                mean_psth = np.mean(all_psths, axis=0)
                sem_psth = np.std(all_psths, axis=0) / np.sqrt(all_psths.shape[0])

                # Plot the mean PSTH with SEM as shaded error bars
                color = group_colors.get(groupname, 'black')
                shaded_color = lightened_colors.get(groupname, 'gray')
                axes[plot_idx].plot(time_array, mean_psth, label='Mean PSTH', color=color, linewidth=2)
                axes[plot_idx].fill_between(time_array, mean_psth - sem_psth, mean_psth + sem_psth, color=shaded_color, alpha=0.3)

                axes[plot_idx].set_title(f'Group: {groupname}, Stimulation: {stim_label}')
                axes[plot_idx].set_xlabel('Time (ms)')
                axes[plot_idx].set_ylabel('Spike Rate')
                axes[plot_idx].legend().set_visible(False)
                
                plot_idx += 1
        #specify the ylimit for all the subplots
        for ax in axes:
            ax.set_ylim(ylim)
        plt.tight_layout()
        plt.show()
        
        # Create directory if it does not exist
        os.makedirs(directory, exist_ok=True)
        
        # Save the figure as an SVG file in the specified directory
        file_path = os.path.join(directory, f'{file_name}.svg')
        fig.savefig(file_path, format='svg', transparent=True)
    
    def plot_group_comparison_1v1(self, stim_labels, cell_type=None, is_single_unit=None, stim_responsivity=None, groupnames=None, modulation_label=None, time_range=None, smoothing_window=10, ylim=None, recordingnames=None):
        fig, axes = plt.subplots(2, 4, figsize=(20, 10), sharex=True, sharey=True)
        axes = axes.flatten()

        # Define color mapping for groups
        group_colors = {
            'No_CTZ': '#797979',
            'CTZ': '#5a00c2'
        }

        # Generate lighter versions for the shaded area
        lightened_colors = {k: v + '33' for k, v in group_colors.items()}

        plot_idx = 0
        for group_idx, groupname in enumerate(groupnames):
            recordingname = recordingnames[group_idx]
            for stim_label in stim_labels:
                df = self.filter_data(stim_label, cell_type, is_single_unit, stim_responsivity, groupname, modulation_label, recordingname)
                
                # Print the number of units that match the filter
                print(f"Number of units that match the filter for group {groupname}, recording {recordingname}, and stimulation {stim_label}: {df.shape[0]}")

                if df.empty:
                    print(f"No data matches the specified filters for group {groupname}, recording {recordingname}, and stimulation {stim_label}.")
                    plot_idx += 1
                    continue

                # Get relative time array for x-axis
                time_array = self.eed.relative_time_ms['relative_time_ms']
                
                # Create time mask and adjust time array
                time_mask = self.create_time_mask(time_array, time_range)
                time_array = time_array[time_mask]

                all_psths = []

                for index, row in df.iterrows():
                    individual_psth = row['PSTHs_conv']
                    # Apply time mask to individual PSTH
                    individual_psth = np.array(individual_psth)[time_mask]
                    
                    # Apply a smoothing window
                    window = np.ones(smoothing_window) / smoothing_window  # 10ms window of smoothing
                    individual_psth = np.convolve(individual_psth, window, mode='same')
                    
                    all_psths.append(individual_psth)
                    # Plot the individual PSTH with lighter color
                    color = lightened_colors.get(groupname, '#CCCCCC')
                    axes[plot_idx].plot(time_array, individual_psth, color=color, linewidth=1, alpha=0.5)

                # Calculate the mean PSTH
                all_psths = np.array(all_psths)
                mean_psth = np.mean(all_psths, axis=0)
                # Apply a smoothing window to the mean PSTH
                mean_psth = np.convolve(mean_psth, window, mode='same')
                # Plot the mean PSTH with a thicker line
                color = group_colors.get(groupname, 'black')
                axes[plot_idx].plot(time_array, mean_psth, label='Mean PSTH', color=color, linewidth=2)

                axes[plot_idx].set_title(f'Group: {groupname}, Recording: {recordingname}, Stimulation: {stim_label}')
                axes[plot_idx].set_xlabel('Time (ms)')
                axes[plot_idx].set_ylabel('Spike Rate')
                axes[plot_idx].legend().set_visible(False)
                
                plot_idx += 1
        
        # Set y-limit for all subplots
        for ax in axes:
            ax.set_ylim(ylim)
        plt.tight_layout()
        plt.show()



    def process_waveforms(self, cell_type, groupname=None, stim_responsivity=None, modulation_label=None, is_single_unit=None):
        """
        Processes waveforms for the specified cell type and optional filters.

        Parameters:
        cell_type (str): 'FS' or 'RS'.
        groupname (str or None): Filter by groupname. If None, no filtering by groupname.
        stim_responsivity (float or None): Filter by StimResponsivity (1.0, 0.0, or -1.0). If None, no filtering by this criterion.
        modulation_label (str or None): Filter by ModulationIndex ('positive', 'negative', 'none'). If None, no filtering by this criterion.
        is_single_unit (float or None): If 1.0, filter for single units, if 0.0, filter for non-single units, if None, do not filter by this criterion.

        Returns:
        tuple: (waveforms, mean_waveform, sem_waveform)
        """
        df = self.get_filtered_data(
            df_name='basic_metrics', 
            is_single_unit=is_single_unit,
            cell_type=cell_type, 
            stim_responsivity=stim_responsivity, 
            groupname=groupname, 
            modulation_label=modulation_label, 
            spike_cutoff=spike_cutoff)
        
        if df.empty:
            print(f"No data available for cell type: {cell_type}")
            return None, None, None
        
        waveforms = df['UnNormalized_Template_Waveform'].values
        mean_waveform = np.mean(np.stack(waveforms), axis=0)
        sem_waveform = np.std(np.stack(waveforms), axis=0) / np.sqrt(len(waveforms))
        
        #convet data to microvolts conversion_factor = 0.25  # µV per unit
        waveforms = waveforms * 0.25
        mean_waveform = mean_waveform * 0.25
        sem_waveform = sem_waveform * 0.25
        
        
        return waveforms, mean_waveform, sem_waveform
    
    def align_waveforms_on_negative_peak(self, waveforms):
        """
        Aligns waveforms based on the most negative deflection (peak). 

        Parameters:
        waveforms (list of numpy.ndarray): A list of 1D numpy arrays where each array is a waveform.

        Returns:
        numpy.ndarray: A 2D array of aligned waveforms with NaNs added to handle alignment.
        """
        # Find the index of the most negative value in each waveform
        peak_indices = [np.argmin(waveform) for waveform in waveforms]

        # Determine the maximum shift needed to align the peaks
        max_shift = max(peak_indices)

        # Determine the length of the waveforms (assuming they are all the same length)
        waveform_length = waveforms[0].shape[0]

        # Initialize an array to hold the aligned waveforms with NaNs
        aligned_waveforms = np.full((len(waveforms), waveform_length + max_shift), np.nan)

        # Align each waveform
        for i, waveform in enumerate(waveforms):
            shift = max_shift - peak_indices[i]
            aligned_waveforms[i, shift:shift + waveform_length] = waveform

        return aligned_waveforms
    
    def process_waveforms(self, cell_type, groupname=None, stim_responsivity=None, modulation_label=None,  is_single_unit= None, align_on_negative_peak=True):
        """
        Processes waveforms for the specified cell type and optional filters.

        Parameters:
        cell_type (str): 'FS' or 'RS'.
        groupname (str or None): Filter by groupname. If None, no filtering by groupname.
        stim_responsivity (float or None): Filter by StimResponsivity (1.0, 0.0, or -1.0). If None, no filtering by this criterion.
        modulation_label (str or None): Filter by ModulationIndex ('positive', 'negative', 'none'). If None, no filtering by this criterion.
        align_on_negative_peak (bool): If True, align waveforms on the most negative deflection before returning.

        Returns:
        tuple: (waveforms, mean_waveform, sem_waveform)
        """
        df = self.get_filtered_data(
            df_name= 'basic_metrics', 
            is_single_unit=is_single_unit, 
            cell_type=cell_type, 
            stim_responsivity=stim_responsivity, 
            groupname=groupname, 
            modulation_label=modulation_label
        )
        
        if df.empty:
            print(f"No data available for cell type: {cell_type}")
            return None, None, None
        
        waveforms = df['UnNormalized_Template_Waveform'].values

        if align_on_negative_peak:
            waveforms = self.align_waveforms_on_negative_peak(waveforms)
        
        mean_waveform = np.nanmean(waveforms, axis=0)
        sem_waveform = np.nanstd(waveforms, axis=0) / np.sqrt(np.sum(~np.isnan(waveforms), axis=0))
        
        #convert data to microvolts conversion_factor = 0.25  # µV per unit, speficic for blackrock data
        waveforms = waveforms * 0.25
        mean_waveform = mean_waveform * 0.25
        sem_waveform = sem_waveform * 0.25
        
        return waveforms, mean_waveform, sem_waveform
    
    
    def plot_waveforms(self, cell_type, color, label, groupname=None, stim_responsivity=None, modulation_label=None, is_single_unit=None, ylim=None):
        """
        Plots individual and mean waveforms for the specified cell type with optional filters.

        Parameters:
        cell_type (str): 'FS' or 'RS'.
        color (str): Color for the plot.
        label (str): Label for the mean waveform plot.
        groupname (str or None): Filter by groupname. If None, no filtering by groupname.
        stim_responsivity (float or None): Filter by StimResponsivity (1.0, 0.0, or -1.0). If None, no filtering by this criterion.
        modulation_label (str or None): Filter by ModulationIndex ('positive', 'negative', 'none'). If None, no filtering by this criterion.
        ylim (tuple or None): Set y-axis limits. If None, no limit is set.
        is_single_units (float or None): If 1.0, filter for single units, if 0.0, filter for non-single units, if None, do not filter by this criterion.

        Returns:
        None
        """
        waveforms, mean_waveform, _ = self.process_waveforms(
            cell_type, 
            groupname=groupname, 
            stim_responsivity=stim_responsivity, 
            modulation_label=modulation_label, 
            is_single_unit=is_single_unit
        )
        
        if waveforms is None:
            return
        
        # Plot individual waveforms
        for waveform in waveforms:
            plt.plot(waveform, color=color, alpha=0.1)
        
        # Plot mean waveform
        plt.plot(mean_waveform, color=color, label=label, linewidth=2)
        
        # Add ylim range if provided
        if ylim:
            plt.ylim(ylim)
            
        # Save the figure
        plt.savefig('/Volumes/MannySSD/figures/waveforms.svg', transparent=True)

    def plot_trough_to_peak_histogram(self, df_name='basic_metrics', groupname=None, is_single_unit=None, stim_responsivity=None, modulation_label=None):
        """
        Plots histograms of Trough-to-Peak duration for FS and RS cells, with options to filter data.

        Parameters:
        df_name (str): The name of the DataFrame to use (default is 'basic_metrics').
        groupname (str or None): Filter by groupname. If None, no filtering by groupname.
        stim_responsivity (float or None): Filter by StimResponsivity (1.0, 0.0, or -1.0). If None, no filtering by this criterion.
        modulation_label (str or None): Filter by ModulationIndex ('positive', 'negative', 'none'). If None, no filtering by this criterion.
        is_single_unit (float or None): If 1.0, filter for single units, if 0.0, filter for non-single units, if None, do not filter by this criterion.

        Returns:
        None: The function plots and saves the histogram.
        """

        # Retrieve filtered data for FS and RS cells
        fs_df = self.get_filtered_data(df_name, is_single_unit=is_single_unit, cell_type='FS', stim_responsivity=stim_responsivity, groupname=groupname, modulation_label=modulation_label)
        rs_df = self.get_filtered_data(df_name, is_single_unit=is_single_unit, cell_type='RS', stim_responsivity=stim_responsivity, groupname=groupname, modulation_label=modulation_label)
        
        #print the shape of the dataframes 
        print(f'For FS cells: {fs_df.shape} after filtering')
        print(f'For RS cells: {rs_df.shape} after filtering')
        
        # Extract the Trough-to-Peak durations, dropping NaNs
        fs_durations = fs_df['TroughToPeak_duration'].dropna()
        rs_durations = rs_df['TroughToPeak_duration'].dropna()
        
        print(f'For FS cells: {fs_durations.shape} after dropping NaNs')
        print(f'For RS cells: {rs_durations.shape} after dropping NaNs')
        
        # Determine the bins based on the combined range of both FS and RS durations
        all_durations = np.concatenate((fs_durations, rs_durations))
        bins = np.histogram_bin_edges(all_durations, bins=30)
        
        plt.figure(figsize=(10, 5))
        
        # Plot histograms
        plt.hist(fs_durations, bins=bins, alpha=0.7, color='darkgoldenrod', label='FS')
        plt.hist(rs_durations, bins=bins, alpha=0.7, color='sienna', label='RS')
        
        # Customize plot
        plt.title('Trough-to-Peak Duration Histogram')
        plt.xlabel('Trough-to-Peak Duration (ms)')
        plt.ylabel('Counts')
        plt.legend()
    
    def plot_combined_waveforms(self, groupname=None, stim_responsivity=None, modulation_label=None,  is_single_unit=None, ylim=None):
        """
        Plots combined waveforms for FS and RS cells with optional filters.

        Parameters:
        groupname (str or None): Filter by groupname. If None, no filtering by groupname.
        stim_responsivity (float or None): Filter by StimResponsivity (1.0, 0.0, or -1.0). If None, no filtering by this criterion.
        modulation_label (str or None): Filter by ModulationIndex ('positive', 'negative', 'none'). If None, no filtering by this criterion.
        ylim (tuple or None): Set y-axis limits. If None, no limit is set.
        is_single_unit (float or None): If 1.0, filter for single units, if 0.0, filter for non-single units, if None, do not filter by this criterion.

        Returns:
        None
        """
        plt.figure(figsize=(10, 5))
        
        # Plot FS waveforms
        self.plot_waveforms('FS', color='darkgoldenrod', label='FS Mean', groupname=groupname, stim_responsivity=stim_responsivity, modulation_label=modulation_label, is_single_unit=is_single_unit, ylim=ylim)
        
        # Plot RS waveforms
        self.plot_waveforms('RS', color='sienna', label='RS Mean', groupname=groupname, stim_responsivity=stim_responsivity, modulation_label=modulation_label,  is_single_unit=is_single_unit, ylim=ylim)
        
        plt.title('Combined Waveforms of FS and RS Units')
        plt.xlabel('Time (ms)')
        plt.ylabel('Normalized Amplitude')
        plt.legend()
        
        # CONTROL THE X LIMITS
        plt.xlim(30, 120)
        plt.ylim(-2800, 1000)
        
        # Save the figure
        plt.savefig('/Volumes/MannySSD/figures/combined_waveforms.svg', transparent=True)

    def plot_combined_waveforms_meansem(self, groupname=None, stim_responsivity=None, modulation_label=None, is_single_unit=None, ylim=None):
        """
        Plots combined mean and SEM waveforms for FS and RS cells with optional filters.

        Parameters:
        groupname (str or None): Filter by groupname. If None, no filtering by groupname.
        stim_responsivity (float or None): Filter by StimResponsivity (1.0, 0.0, or -1.0). If None, no filtering by this criterion.
        modulation_label (str or None): Filter by ModulationIndex ('positive', 'negative', 'none'). If None, no filtering by this criterion.
        is_single_unit (float or None): If 1.0, filter for single units, if 0.0, filter for non-single units, if None, do not filter by this criterion.
        ylim (tuple or None): Set y-axis limits. If None, no limit is set.

        Returns:
        None
        """
        fs_waveforms, fs_mean_waveform, fs_sem_waveform = self.process_waveforms(
            'FS', groupname=groupname, stim_responsivity=stim_responsivity, modulation_label=modulation_label, is_single_unit=is_single_unit
        )
        rs_waveforms, rs_mean_waveform, rs_sem_waveform = self.process_waveforms(
            'RS', groupname=groupname, stim_responsivity=stim_responsivity, modulation_label=modulation_label, is_single_unit=is_single_unit
        )
        
        if fs_waveforms is None and rs_waveforms is None:
            print("No data available for both FS and RS units.")
            return
        
        plt.figure(figsize=(10, 5))
        
        # Plot FS mean and SEM
        if fs_waveforms is not None:
            plt.plot(fs_mean_waveform, color='darkgoldenrod', label='FS Mean', linewidth=2)
            plt.fill_between(np.arange(len(fs_mean_waveform)), fs_mean_waveform - fs_sem_waveform, fs_mean_waveform + fs_sem_waveform, color='darkgoldenrod', alpha=0.3)
        
        # Plot RS mean and SEM
        if rs_waveforms is not None:
            plt.plot(rs_mean_waveform, color='sienna', label='RS Mean', linewidth=2)
            plt.fill_between(np.arange(len(rs_mean_waveform)), rs_mean_waveform - rs_sem_waveform, rs_mean_waveform + rs_sem_waveform, color='sienna', alpha=0.3)
        
        plt.title('Mean and SEM of FS and RS Units')
        plt.xlabel('Time (ms)')
        plt.ylabel('Normalized Amplitude')
        plt.legend()
        
        # CONTROL THE X LIMITS
        plt.xlim(30, 120)
        plt.ylim(-2000, 500)
        
        # Save the figure
        plt.savefig('/Volumes/MannySSD/figures/mean_sem_waveforms.svg', transparent=True)
 
        
        
    def process_and_store_analysis_data(self, is_single_unit=None, cell_type=None, stim_responsivity=None, 
                                        time_window=None, bin_size=1, smoothing_window=5, analysis_functions=None):
        """
        Iterates over the dataset grouped by 'groupname', 'recordingname', and 'cid', extracts the PSTH data,
        performs specified analyses, and stores the results in a structured format.

        Args:
            is_single_unit (bool, optional): If specified, filters cells based on whether they are considered single units.
            cell_type (str, optional): If specified, filters cells based on their type.
            stim_responsivity (bool, optional): If specified, filters cells based on their responsiveness to stimuli.
            time_window (tuple, optional): The window of time to extract, within the range -500 to 999 ms. 
                                        Default is None, which uses the full range.
            bin_size (int, optional): The size of the bins in milliseconds for the PSTH and raster. Default is 1ms.
            smoothing_window (int, optional): The size of the smoothing window for the smoothed PSTH. Default is 5.
            analysis_functions (list, optional): List of functions to apply to the extracted data. Each function should
                                                 take the extracted data as input and return a dictionary with results.

        Returns:
            dict: A nested dictionary containing the extracted data and analysis results for each group, recording, and cid.
        """
        stored_data = {}

        for (groupname, recordingname, cid), group_df in self.iterate_by_group_recording_cid(
                is_single_unit=is_single_unit, cell_type=cell_type, stim_responsivity=stim_responsivity):

            print(f"Processing Group: {groupname}, Recording: {recordingname}, CID: {cid}")

            # Extract the PSTH data
            extracted_data = self.extract_psth_and_raster_data(groupname=groupname, recordingname=recordingname, cid=cid, 
                                                            time_window=time_window, bin_size=bin_size, smoothing_window=smoothing_window)

            # Perform additional analyses
            if analysis_functions:
                for func in analysis_functions:
                    analysis_results = func(extracted_data)
                    extracted_data.update(analysis_results)

            # Add the parameters used to the extracted data
            extracted_data['bin_size'] = bin_size
            extracted_data['smoothing_window'] = smoothing_window
            extracted_data['time_window'] = time_window

            # Ensure the nested dictionary structure is created
            if groupname not in stored_data:
                stored_data[groupname] = {}
            if recordingname not in stored_data[groupname]:
                stored_data[groupname][recordingname] = {}
            
            stored_data[groupname][recordingname][cid] = extracted_data

        return stored_data

### this will be now the new methods for plotting distributuions
    def plot_stim_prob_distribution(self, df_name, groupname=None, is_single_unit=None, cell_type=None, xlim_range=None, include_no_change=True, plot_stacked_bar=True):
        """
        Plot the step plots for the StimResponsivity unique labels (-1.0, 0.0, 1.0)
        showing the distribution of StimProb AUROC values, and print the total counts for each category.
        Also, plot a stacked bar chart visualizing the proportions of the three groups.
        
        Parameters:
        df_name (str): Name of the DataFrame to filter.
        groupname (str or None): Filter by group name. If None, no filtering by group name.
        is_single_unit (float or None): If 1.0, filter for single units, if 0.0, filter for non-single units,
                                        if None, do not filter by this criterion.
        cell_type (str or None): Filter for 'FS' or 'RS' cell types. If None, no filtering by cell type.
        xlim_range (tuple or None): Tuple specifying the lower and upper bounds of the x-axis. 
                                    If None, the full range of [0, 1] is used.
        include_no_change (bool): If True, include the "No Change" group in the plot. Default is True.
        plot_stacked_bar (bool): If True, plot a stacked bar chart showing the proportions of the three groups. Default is True.
        """
        # Define StimResponsivity labels and their corresponding colors
        stim_responsivity_labels = [-1.0, 0.0, 1.0]
        colors = ['red', 'grey', 'green']
        labels = ['Decrease', 'No Change', 'Increase']
        
        # Prepare bin edges for the histogram
        bin_edges = np.concatenate([np.arange(0, 0.5, 0.01), np.arange(0.501, 1.01, 0.01)])

        plt.figure(figsize=(10, 6))

        # Dictionary to store total counts
        total_counts = {'Decrease': 0, 'No Change': 0, 'Increase': 0}

        # Step 1: Get the filtered and categorized DataFrame
        filtered_df = self.prepare_StimProb_distribution(df_name, is_single_unit, cell_type, groupname=groupname)
        
        # Step 2: Iterate over each category and count and plot the AUROC values
        for label, stim_label in zip(labels, stim_responsivity_labels):
            # Count the number of entries in the current category
            total_counts[label] = filtered_df[filtered_df['StimResponsivity'] == stim_label].shape[0]

            # Plot only if it's either "Decrease" or "Increase" or if "No Change" is included
            if label == 'No Change' and not include_no_change:
                continue  # Skip plotting for "No Change" if not included

            # Extract AUROC values for the current category without additional filtering
            auroc_values = filtered_df[filtered_df['StimResponsivity'] == stim_label]['StimProb'].apply(lambda x: x[0]).tolist()

            # Calculate the histogram
            hist, _ = np.histogram(auroc_values, bins=bin_edges)
            
            # Create the step plot
            plt.step(bin_edges[:-1], hist, where='post', label=f'{label} (StimResponsivity={stim_label})', color=colors[labels.index(label)])
            
            # Add light fill for "Decrease"
            if stim_label == -1.0:
                plt.fill_between(bin_edges[:-1], hist, step='post', color=colors[labels.index(label)], alpha=0.1)
        
        # Add legend and title to the plot
        plt.legend()
        plt.title('StimProb AUROC Distribution by StimResponsivity')
        plt.xlabel('AUROC Value')
        plt.ylabel('Count')
        
        # Print the total counts for each category
        print(f"Total counts - Decrease: {total_counts['Decrease']}, No Change: {total_counts['No Change']}, Increase: {total_counts['Increase']}")

        # Set x-axis limits based on the user input
        if xlim_range is not None:
            plt.xlim(xlim_range)
        else:
            plt.xlim([0, 1])  # Default full range
        
        plt.show()

        # Step 3: Plot the stacked bar chart if requested
        if plot_stacked_bar:
            sizes = [total_counts['Decrease'], total_counts['No Change'], total_counts['Increase']]
            colors_bar = ['red', 'grey', 'green']

            plt.figure(figsize=(8, 6))
            plt.bar(['StimResponsivity'], sizes[0], color=colors_bar[0], label='Decrease')
            plt.bar(['StimResponsivity'], sizes[1], bottom=sizes[0], color=colors_bar[1], label='No Change')
            plt.bar(['StimResponsivity'], sizes[2], bottom=sizes[0] + sizes[1], color=colors_bar[2], label='Increase')
            
            plt.title('Proportions of StimResponsivity Categories')
            plt.ylabel('Count')
            plt.legend(loc='upper right')
            plt.show()



    def prepare_StimProb_distribution(self, df_name, is_single_unit=None, cell_type=None, groupname=None):
        """
        Prepare a distribution of AUROC values from the 'StimProb' column of the filtered DataFrame.
        
        Parameters:
        df_name (str): Name of the DataFrame to filter.
        is_single_unit (float or None): If 1.0, filter for single units, if 0.0, filter for non-single units,
                                        if None, do not filter by this criterion.
        cell_type (str or None): Filter for 'FS' or 'RS' cell types. If None, no filtering by cell type.
        groupname (str or None): Filter by group name. If None, no filtering by group name.
        
        Returns:
        DataFrame: Filtered DataFrame with StimProb AUROC values.
        """
        # Retrieve the filtered DataFrame
        filtered_df = self.get_filtered_data(df_name, is_single_unit, cell_type, None, groupname)
        
        # Apply the StimResponsivity logic
        filtered_df['StimResponsivity'] = filtered_df.apply(determine_stim_responsivity, axis=1)
        
        return filtered_df
    
    def plot_cumulative_probability(self, df_name, group1, group2, is_single_unit=None, cell_type=None):
        """
        Plot the cumulative probability of baseline firing rates between two groups.
        
        Parameters:
        df_name (str): Name of the DataFrame to filter.
        group1 (str): Name of the first group.
        group2 (str): Name of the second group.
        is_single_unit (float or None): If 1.0, filter for single units, if 0.0, filter for non-single units,
                                        if None, do not filter by this criterion.
        cell_type (str or None): Filter for 'FS' or 'RS' cell types. If None, no filtering by cell type.
        """
        plt.figure(figsize=(10, 6))

        # Define groups and colors
        groups = [group1, group2]
        colors = ['blue', 'orange']
        
        for group, color in zip(groups, colors):
            # Get filtered data for the current group
            filtered_df = self.get_filtered_data(df_name, is_single_unit, cell_type, groupname=group)
            
            # Extract baseline firing rates
            baseline_firing_rates = filtered_df['MeanFR_baseline'].dropna()
            
            # Calculate cumulative distribution
            sorted_fr = np.sort(baseline_firing_rates)
            cum_prob = np.arange(1, len(sorted_fr) + 1) / len(sorted_fr)
            
            # Plot cumulative probability
            plt.plot(sorted_fr, cum_prob, label=f'{group} (n={len(sorted_fr)})', color=color)
        
        # Add legend and title
        plt.legend(loc='lower right')
        plt.title('Cumulative Probability of Baseline Firing Rates')
        plt.xlabel('Mean Baseline Firing Rate')
        plt.ylabel('Cumulative Probability')
        plt.grid(False)
        plt.show()

    def generate_summary_statistics(self, column_name, df_name='basic_metrics', handle_arrays=True, array_element=0):
        """
        Generates summary statistics for a specified column in the DataFrame.
        
        Parameters:
        column_name (str): The name of the column to generate statistics for.
        df_name (str): The name of the DataFrame to use (default is 'basic_metrics').
        handle_arrays (bool): Whether to handle numpy arrays by extracting a specific element.
        array_element (int): The index of the element to extract from numpy arrays (default is 0).
        
        Returns:
        summary (dict): A dictionary with counts of unique values in the specified column.
        """
        # Ensure the DataFrame exists
        if df_name not in self.dataframes:
            raise ValueError(f"DataFrame '{df_name}' does not exist in self.dataframes.")
        
        df = self.dataframes[df_name]
        
        # Ensure the column exists in the DataFrame
        if column_name not in df.columns:
            raise ValueError(f"Column '{column_name}' does not exist in DataFrame '{df_name}'.")
        
        # Access the column data
        column_data = df[column_name]
        
        # If the column might contain numpy arrays or lists, and handle_arrays is True,
        # process the column to extract a specific element from these arrays/lists.
        if handle_arrays:
            # Apply the extract_array_element function to each element in the column.
            # This replaces numpy arrays or lists with the specific element (defined by array_element)
            # or leaves other data types unchanged.
            column_data = column_data.apply(extract_array_element, args=(array_element,))
        
        # Calculate the value counts for the processed column data.
        # This will count how often each unique value appears in the column.
        summary = column_data.value_counts().to_dict()
        
        # Return the summary as a dictionary, where the keys are unique values in the column
        # and the values are the counts of how often these unique values appear.
        return summary
    
    def inspect_raw_data(self, df_name, is_single_unit=None, cell_type=None, groupname=None):
        """
        Inspect the raw data to understand the distribution of cells before any transformations.
        
        Parameters:
        df_name (str): Name of the DataFrame to inspect.
        is_single_unit (float or None): If 1.0, filter for single units, if 0.0, filter for non-single units,
                                        if None, do not filter by this criterion.
        cell_type (str or None): Filter for 'FS' or 'RS' cell types. If None, no filtering by cell type.
        groupname (str or None): Filter by group name. If None, no filtering by group name.
        
        Returns:
        None: Prints out the information about the data.
        """
        # Retrieve the filtered DataFrame
        filtered_df = self.get_filtered_data(df_name, is_single_unit, cell_type, None, groupname)
        
        # Print out the basic information
        print("Total number of cells:", len(filtered_df))
        print(filtered_df.head())  # Show the first few rows of the DataFrame

        # Display the distribution of AUROC values
        print("AUROC values distribution:")
        print(filtered_df['StimProb'].apply(lambda x: x[0]).describe())

        # Display the initial distribution of ModulationIndex
        print("Initial ModulationIndex distribution:")
        print(filtered_df['ModulationIndex'].value_counts())
        
    def inspect_categorized_data(self, df_name, is_single_unit=None, cell_type=None, groupname=None):
        """
        Categorize the data using determine_stim_responsivity and inspect the resulting distribution.
        
        Parameters:
        df_name (str): Name of the DataFrame to inspect.
        is_single_unit (float or None): If 1.0, filter for single units, if 0.0, filter for non-single units,
                                        if None, do not filter by this criterion.
        cell_type (str or None): Filter for 'FS' or 'RS' cell types. If None, no filtering by cell type.
        groupname (str or None): Filter by group name. If None, no filtering by group name.
        
        Returns:
        None: Prints out the categorized information.
        """
        # Retrieve the filtered DataFrame
        filtered_df = self.get_filtered_data(df_name, is_single_unit, cell_type, None, groupname)
        
        # Apply the StimResponsivity logic
        filtered_df['StimResponsivity'] = filtered_df.apply(determine_stim_responsivity, axis=1)
        
        # Print out the basic information after categorization
        print("Total number of cells after categorization:", len(filtered_df))
        print(filtered_df.head())  # Show the first few rows of the categorized DataFrame

        # Display the distribution of StimResponsivity labels
        print("Distribution of StimResponsivity labels:")
        print(filtered_df['StimResponsivity'].value_counts())

    def check_stim_responsivity_distribution_before_after(self, df_name, is_single_unit=None, cell_type=None, groupname=None):
        """
        Check the distribution of the 'StimResponsivity' column before and after applying the categorization logic.
        
        Parameters:
        df_name (str): Name of the DataFrame to inspect.
        is_single_unit (float or None): If 1.0, filter for single units, if 0.0, filter for non-single units,
                                        if None, do not filter by this criterion.
        cell_type (str or None): Filter for 'FS' or 'RS' cell types. If None, no filtering by cell type.
        groupname (str or None): Filter by group name. If None, no filtering by group name.
        
        Returns:
        None: Prints out the distributions before and after the categorization.
        """
        # Retrieve the filtered DataFrame
        filtered_df = self.get_filtered_data(df_name, is_single_unit, cell_type, None, groupname)
        
        # Check initial distribution of StimResponsivity
        print("Initial StimResponsivity distribution:")
        print(filtered_df['StimResponsivity'].value_counts())

        # Check how many cells have an AUROC lower CI > 0.5
        auroc_lower_ci = filtered_df['StimProb'].apply(lambda x: x[1] if len(x) > 1 else None)
        responsive_cells = auroc_lower_ci > 0.5
        print(f"Number of cells with AUROC lower CI > 0.5: {responsive_cells.sum()} / {len(filtered_df)}")
        
        # Apply the StimResponsivity logic
        filtered_df['StimResponsivity_New'] = filtered_df.apply(determine_stim_responsivity, axis=1)
        
        # Check the distribution of StimResponsivity after categorization
        print("StimResponsivity distribution after categorization:")
        print(filtered_df['StimResponsivity_New'].value_counts())
        
        
        #### plot distrubution without beign redundant

    def plot_raw_auroc_distribution(self, df_name, is_single_unit=None, cell_type=None, groupname=None, xlim_range=None):
        """
        Plot the distribution of AUROC values, color-coded based on the 'StimResponsivity' labels.
        
        Parameters:
        df_name (str): Name of the DataFrame to plot.
        is_single_unit (float or None): If 1.0, filter for single units, if 0.0, filter for non-single units,
                                        if None, do not filter by this criterion.
        cell_type (str or None): Filter for 'FS' or 'RS' cell types. If None, no filtering by cell type.
        groupname (str or None): Filter by group name. If None, no filtering by group name.
        xlim_range (tuple or None): Tuple specifying the lower and upper bounds of the x-axis. 
                                    If None, the full range of [0, 1] is used.
        
        Returns:
        None: Displays the plot.
        """
        # Retrieve the filtered DataFrame
        filtered_df = self.get_filtered_data(df_name, is_single_unit, cell_type, None, groupname)
        
        # Define StimResponsivity labels and their corresponding colors
        stim_responsivity_labels = [-1.0, 0.0, 1.0]
        colors = ['red', 'grey', 'green']
        labels = ['Decrease', 'No Change', 'Increase']
        
        # Prepare bin edges for the histogram
        bin_edges = np.linspace(0, 1, 51)  # 50 bins between 0 and 1

        plt.figure(figsize=(10, 6))

        # Dictionary to store total counts
        total_counts = {'Decrease': 0, 'No Change': 0, 'Increase': 0}

        # Iterate over each category and plot the AUROC values
        for label, stim_label in zip(labels, stim_responsivity_labels):
            # Extract AUROC values for the current category
            auroc_values = filtered_df[filtered_df['StimResponsivity'] == stim_label]['StimProb'].apply(lambda x: x[0]).tolist()
            total_counts[label] = len(auroc_values)
            
            # Calculate the histogram
            hist, _ = np.histogram(auroc_values, bins=bin_edges)
            
            # Create the step plot
            plt.step(bin_edges[:-1], hist, where='post', label=f'{label} (StimResponsivity={stim_label})', color=colors[labels.index(label)])
            
            # Add light fill
            plt.fill_between(bin_edges[:-1], hist, step='post', color=colors[labels.index(label)], alpha=0.1)
        
        # Add legend and title to the plot
        plt.legend()
        plt.title('AUROC Distribution by StimResponsivity')
        plt.xlabel('AUROC Value')
        plt.ylabel('Count')
        
        # Set x-axis limits based on the user input
        if xlim_range is not None:
            plt.xlim(xlim_range)
        else:
            plt.xlim([0, 1])  # Default full range
        
        plt.show()

        # Print the total counts for each category
        print(f"Total counts - Decrease: {total_counts['Decrease']}, No Change: {total_counts['No Change']}, Increase: {total_counts['Increase']}")

        
    def plot_modulation_index_distribution(self, df_name, is_single_unit=None, cell_type=None, groupname=None):
        """
        Plot the distribution of ModulationIndex values directly from the original DataFrame.
        
        Parameters:
        df_name (str): Name of the DataFrame to plot.
        is_single_unit (float or None): If 1.0, filter for single units, if 0.0, filter for non-single units,
                                        if None, do not filter by this criterion.
        cell_type (str or None): Filter for 'FS' or 'RS' cell types. If None, no filtering by cell type.
        groupname (str or None): Filter by group name. If None, no filtering by group name.
        
        Returns:
        None: Displays the plot.
        """
        # Retrieve the filtered DataFrame
        filtered_df = self.get_filtered_data(df_name, is_single_unit, cell_type, None, groupname)
        
        # Count the occurrences of each ModulationIndex category
        modulation_index_counts = filtered_df['ModulationIndex'].value_counts()

        # Plot the bar chart of ModulationIndex counts
        plt.figure(figsize=(8, 6))
        modulation_index_counts.plot(kind='bar', color=['green', 'red', 'grey'])
        plt.title('Distribution of ModulationIndex Categories')
        plt.xlabel('ModulationIndex')
        plt.ylabel('Count')
        plt.show()

    def plot_modulation_index_with_firing_rate_color_and_size(self, df_name, group_name=None, is_single_unit=None, cell_type=None, stim_responsivity=None, jitter=0.1):
        """
        Plot the modulation index as a function of electrode location, with firing rates represented by both color and size of circles.
        Circles have black outlines and are slightly offset horizontally to reduce overlap. Horizontal lines connect each point to x=0.

        Parameters:
        df_name (str): Name of the DataFrame to filter and plot.
        group_name (str or None): Name of the group to filter.
        is_single_unit (float or None): If 1.0, filter for single units, if 0.0, filter for non-single units, 
                                        if None, do not filter by this criterion.
        cell_type (str or None): Filter for 'FS' or 'RS' cell types. If None, no filtering by cell type.
        stim_responsivity (float or None): Value to filter by in the StimResponsivity column. Can be 1.0, 0.0, or -1.0.
                                        If None, no filtering by StimResponsivity.
        jitter (float): Amount of horizontal jitter to apply to points. Default is 0.1.
        """
        # Prepare the data for plotting
        plotting_data = self.prepare_plotting_data(df_name, group_name, is_single_unit, cell_type, stim_responsivity)
        if plotting_data is None or plotting_data.empty:
            print("No data available for plotting.")
            return

        # Data preparation (same as before)
        plotting_data = plotting_data.dropna(subset=['MeanFR_baseline', 'ModulationIndex_Numeric', 'ElectrodeOrder'])
        plotting_data['MeanFR_baseline'] = plotting_data['MeanFR_baseline'].apply(lambda x: x.item() if isinstance(x, np.ndarray) else x)
        plotting_data['MeanFR_baseline'] = pd.to_numeric(plotting_data['MeanFR_baseline'], errors='coerce')
        plotting_data['ModulationIndex_Numeric'] = pd.to_numeric(plotting_data['ModulationIndex_Numeric'], errors='coerce')
        plotting_data['ElectrodeOrder'] = pd.to_numeric(plotting_data['ElectrodeOrder'], errors='coerce')
        plotting_data = plotting_data.dropna(subset=['MeanFR_baseline', 'ModulationIndex_Numeric', 'ElectrodeOrder'])

        # Calculate sizes for scatter points
        min_size = 10
        max_size = 200
        sizes = min_size + (plotting_data['MeanFR_baseline'] / plotting_data['MeanFR_baseline'].max()) * (max_size - min_size)

        # Add horizontal jitter
        plotting_data['Jittered_ElectrodeOrder'] = plotting_data['ElectrodeOrder'] + np.random.uniform(-jitter, jitter, len(plotting_data))

        # Create the plot
        fig, ax = plt.subplots(figsize=(4, 8))

        # Plot horizontal lines from x=0 to each point
        for _, row in plotting_data.iterrows():
            ax.plot([0, row['ModulationIndex_Numeric']], [row['Jittered_ElectrodeOrder'], row['Jittered_ElectrodeOrder']], 
                    color='gray', alpha=0.3, linewidth=0.5)

        # Plot the modulation index points
        scatter = ax.scatter(plotting_data['ModulationIndex_Numeric'], plotting_data['Jittered_ElectrodeOrder'], 
                            c=plotting_data['MeanFR_baseline'], cmap='hot', 
                            s=sizes, alpha=0.8, vmin=0, edgecolors='black', linewidths=0.5)
        
        # Add colorbar for firing rates
        cbar = plt.colorbar(scatter, label='Firing Rate (Hz)')
        max_fr = plotting_data['MeanFR_baseline'].max()
        cbar.set_ticks([0, max_fr])
        cbar.set_ticklabels(['0', f'{max_fr:.2f}'])

        # Add a vertical line at x=0
        ax.axvline(x=0, color='black', linestyle='--', linewidth=0.5)

        ax.set_xlabel('Spontaneous MI')
        ax.set_ylabel('Electrode Order')
        ax.set_title(f'{cell_type} {"single" if is_single_unit == 1.0 else "multi"}-units from {group_name}')
        ax.invert_yaxis()  # To match the order from the provided list

        ax.set_xlim(-1.5, 1.5)
        ax.grid(False)
        ax.set_yticks(np.arange(0, 32, 1))

        # Adjust layout
        plt.tight_layout()

        # Save the plot
        directory = '/Volumes/MannySSD/figures/laminar_plots'
        try:
            os.makedirs(directory, exist_ok=True)
        except OSError as e:
            print(f"Error creating directory: {e}")
            return

        file_name = f'{group_name}_{"single" if is_single_unit == 1.0 else "multi"}_units_firing_rate_color_size_jittered_hot'
        if cell_type:
            file_name += f'_{cell_type}'
        if stim_responsivity is not None:
            file_name += f'_stim_{stim_responsivity}'
        
        file_path = os.path.join(directory, f'{file_name}.svg')
        
        try:
            plt.savefig(file_path, format='svg', dpi=300, bbox_inches='tight')
            print(f"Plot saved to {file_path}")
            plt.show()
        except Exception as e:
            print(f"Error saving plot: {e}")

    def plot_modulation_index_with_firing_rate_color_and_size_groupcomparison(self, df_name, is_single_unit=None, cell_type=None, stim_responsivity=None, jitter=0.1, size_multiplier=1, min_size=20, min_fr_threshold=0.001):
        """
        Plot the modulation index as a function of electrode location for CTZ and No_CTZ groups side by side.
        Firing rates are represented by both color and size of circles, with customizable size controls.
        Circles have black outlines and are slightly offset horizontally to reduce overlap. 
        Horizontal lines connect each point to x=0.

        Parameters:
        df_name (str): Name of the DataFrame to filter and plot.
        is_single_unit (float or None): If 1.0, filter for single units, if 0.0, filter for non-single units, 
                                        if None, do not filter by this criterion.
        cell_type (str or None): Filter for 'FS' or 'RS' cell types. If None, no filtering by cell type.
        stim_responsivity (float or None): Value to filter by in the StimResponsivity column. Can be 1.0, 0.0, or -1.0.
                                        If None, no filtering by StimResponsivity.
        jitter (float): Amount of horizontal jitter to apply to points. Default is 0.1.
        size_multiplier (float): Multiplier for circle sizes. Default is 1.
        min_size (float): Minimum size for circles with firing rates below the threshold. Default is 20.
        min_fr_threshold (float): Firing rate threshold below which circles will have the minimum size. Default is 0.001.
        """
        groups = ['CTZ', 'No_CTZ']
        fig, axes = plt.subplots(1, 2, figsize=(6, 8), sharey=True)

        all_data = []
        for group in groups:
            data = self.prepare_plotting_data(df_name, group, is_single_unit, cell_type, stim_responsivity)
            if data is not None and not data.empty:
                all_data.append(data)

        if not all_data:
            print("No data available for plotting.")
            return

        # Combine all data for global color scaling
        combined_data = pd.concat(all_data)
        
        # Convert MeanFR_baseline to scalar values
        combined_data['MeanFR_baseline'] = combined_data['MeanFR_baseline'].apply(lambda x: x.item() if isinstance(x, np.ndarray) else x)
        combined_data['MeanFR_baseline'] = pd.to_numeric(combined_data['MeanFR_baseline'], errors='coerce')
        
        max_fr = combined_data['MeanFR_baseline'].max()

        # Define size calculation function
        def calculate_size(fr):
            if fr <= min_fr_threshold:
                return min_size
            else:
                return min_size + (fr - min_fr_threshold) * size_multiplier

        for ax, group in zip(axes, groups):
            plotting_data = combined_data[combined_data['groupname'] == group]

            if plotting_data.empty:
                ax.text(0.5, 0.5, f'No data for {group}', ha='center', va='center', transform=ax.transAxes)
                continue

            # Add horizontal jitter
            plotting_data['Jittered_ElectrodeOrder'] = plotting_data['ElectrodeOrder'] + np.random.uniform(-jitter, jitter, len(plotting_data))

            # Calculate sizes
            plotting_data['sizes'] = plotting_data['MeanFR_baseline'].apply(calculate_size)

            # Plot horizontal lines from x=0 to each point
            for _, row in plotting_data.iterrows():
                ax.plot([0, row['ModulationIndex_Numeric']], [row['Jittered_ElectrodeOrder'], row['Jittered_ElectrodeOrder']], 
                        color='gray', alpha=0.3, linewidth=0.5)

            # Plot the modulation index points
            scatter = ax.scatter(plotting_data['ModulationIndex_Numeric'], plotting_data['Jittered_ElectrodeOrder'], 
                                c=plotting_data['MeanFR_baseline'], cmap='hot', 
                                s=plotting_data['sizes'], alpha=0.8, vmin=0, vmax=max_fr, 
                                edgecolors='black', linewidths=0.5)

            # Add a vertical line at x=0
            ax.axvline(x=0, color='black', linestyle='--', linewidth=0.5)

            ax.set_xlabel('Spontaneous MI')
            ax.set_title(f'{group}\n{cell_type} {"single" if is_single_unit == 1.0 else "multi"}-units')
            ax.invert_yaxis()  # To match the order from the provided list
            ax.set_xlim(-1.0, 1.0)
            ax.grid(False)
            ax.set_yticks(np.arange(0, 32, 1))

        # Add a common y-label
        fig.text(0.04, 0.5, 'Electrode Order', va='center', rotation='vertical')

        # Add colorbar for firing rates
        cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])  # [left, bottom, width, height]
        cbar = fig.colorbar(scatter, cax=cbar_ax)
        cbar.set_label('Firing Rate (Hz)')
        cbar.set_ticks([0, max_fr])
        cbar.set_ticklabels(['0', f'{max_fr:.2f}'])

        # Adjust layout
        plt.tight_layout()
        plt.subplots_adjust(right=0.9)  # Make room for the colorbar

        # Save the plot
        directory = '/Volumes/MannySSD/figures/laminar_plots'
        try:
            os.makedirs(directory, exist_ok=True)
        except OSError as e:
            print(f"Error creating directory: {e}")
            return

        file_name = f'CTZ_No_CTZ_comparison_{"single" if is_single_unit == 1.0 else "multi"}_units_firing_rate_color_size_jittered_hot'
        if cell_type:
            file_name += f'_{cell_type}'
        if stim_responsivity is not None:
            file_name += f'_stim_{stim_responsivity}'
        
        file_path = os.path.join(directory, f'{file_name}.svg')
        
        try:
            plt.savefig(file_path, format='svg', dpi=300, bbox_inches='tight')
            print(f"Plot saved to {file_path}")
            plt.show()
        except Exception as e:
            print(f"Error saving plot: {e}")

    def plot_modulation_index_with_firing_rate_color_and_size_groupcomparison_evoked(self, df_name, stim_index=3, is_single_unit=None, cell_type=None, stim_responsivity=None, jitter=0.1, size_multiplier=1, min_size=20, min_fr_threshold=0.001):
        """
        Plot the modulation index as a function of electrode location for CTZ and No_CTZ groups side by side,
        using evoked firing rates from MeanFR_stim.
        Firing rates are represented by both color and size of circles, with a minimum size for rates below a threshold.
        Circles have black outlines and are slightly offset horizontally to reduce overlap. 
        Horizontal lines connect each point to x=0.

        Parameters:
        df_name (str): Name of the DataFrame to filter and plot.
        stim_index (int): Index of the stimulus intensity to use (0-3, where 3 is max stimulation). Default is 3.
        is_single_unit (float or None): If 1.0, filter for single units, if 0.0, filter for non-single units, 
                                        if None, do not filter by this criterion.
        cell_type (str or None): Filter for 'FS' or 'RS' cell types. If None, no filtering by cell type.
        stim_responsivity (float or None): Value to filter by in the StimResponsivity column. Can be 1.0, 0.0, or -1.0.
                                        If None, no filtering by StimResponsivity.
        jitter (float): Amount of horizontal jitter to apply to points. Default is 0.1.
        size_multiplier (float): Multiplier for circle sizes. Default is 1.
        min_size (float): Minimum size for circles with firing rates below the threshold. Default is 20.
        min_fr_threshold (float): Firing rate threshold below which circles will have the minimum size. Default is 0.001.
        """
        groups = ['CTZ', 'No_CTZ']
        fig, axes = plt.subplots(1, 2, figsize=(12, 8), sharey=True)

        all_data = []
        for group in groups:
            data = self.prepare_plotting_data(df_name, group, is_single_unit, cell_type, stim_responsivity)
            if data is not None and not data.empty:
                all_data.append(data)

        if not all_data:
            print("No data available for plotting.")
            return

        # Combine all data for global color scaling
        combined_data = pd.concat(all_data)
        
        # Extract the specified stim_index from MeanFR_stim
        combined_data['EvokeFR'] = combined_data['MeanFR_stim'].apply(lambda x: x[stim_index] if isinstance(x, (list, np.ndarray)) and len(x) > stim_index else np.nan)
        combined_data['EvokeFR'] = pd.to_numeric(combined_data['EvokeFR'], errors='coerce')
        
        max_fr = combined_data['EvokeFR'].max()

        # Define size calculation function
        def calculate_size(fr):
            if fr <= min_fr_threshold:
                return min_size
            else:
                return min_size + (fr - min_fr_threshold) * size_multiplier

        for ax, group in zip(axes, groups):
            plotting_data = combined_data[combined_data['groupname'] == group]

            if plotting_data.empty:
                ax.text(0.5, 0.5, f'No data for {group}', ha='center', va='center', transform=ax.transAxes)
                continue

            # Add horizontal jitter
            plotting_data['Jittered_ElectrodeOrder'] = plotting_data['ElectrodeOrder'] + np.random.uniform(-jitter, jitter, len(plotting_data))

            # Calculate sizes
            plotting_data['sizes'] = plotting_data['EvokeFR'].apply(calculate_size)

            # Plot horizontal lines from x=0 to each point
            for _, row in plotting_data.iterrows():
                ax.plot([0, row['ModulationIndex_Numeric']], [row['Jittered_ElectrodeOrder'], row['Jittered_ElectrodeOrder']], 
                        color='gray', alpha=0.3, linewidth=0.5)

            # Plot the modulation index points
            scatter = ax.scatter(plotting_data['ModulationIndex_Numeric'], plotting_data['Jittered_ElectrodeOrder'], 
                                c=plotting_data['EvokeFR'], cmap='hot', 
                                s=plotting_data['sizes'], alpha=0.8, vmin=0, vmax=max_fr, 
                                edgecolors='black', linewidths=0.5)

            # Add a vertical line at x=0
            ax.axvline(x=0, color='black', linestyle='--', linewidth=0.5)

            ax.set_xlabel('Spontaneous MI')
            ax.set_title(f'{group}\n{cell_type} {"single" if is_single_unit == 1.0 else "multi"}-units\nStim Index: {stim_index}')
            ax.invert_yaxis()  # To match the order from the provided list
            ax.set_xlim(-1.0, 1.0)
            ax.grid(False)
            ax.set_yticks(np.arange(0, 32, 1))

        # Add a common y-label
        fig.text(0.04, 0.5, 'Electrode Order', va='center', rotation='vertical')

        # Add colorbar for firing rates
        cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])  # [left, bottom, width, height]
        cbar = fig.colorbar(scatter, cax=cbar_ax)
        cbar.set_label('Evoked Firing Rate (Hz)')
        cbar.set_ticks([0, max_fr])
        cbar.set_ticklabels(['0', f'{max_fr:.2f}'])

        # Adjust layout
        plt.tight_layout()
        plt.subplots_adjust(right=0.9)  # Make room for the colorbar

        # Save the plot
        directory = '/Volumes/MannySSD/figures/laminar_plots'
        try:
            os.makedirs(directory, exist_ok=True)
        except OSError as e:
            print(f"Error creating directory: {e}")
            return

        file_name = f'CTZ_No_CTZ_comparison_{"single" if is_single_unit == 1.0 else "multi"}_units_evoked_FR_stim{stim_index}_color_size_jittered_hot'
        if cell_type:
            file_name += f'_{cell_type}'
        if stim_responsivity is not None:
            file_name += f'_stim_{stim_responsivity}'
        
        file_path = os.path.join(directory, f'{file_name}.svg')
        
        try:
            plt.savefig(file_path, format='svg', dpi=300, bbox_inches='tight')
            print(f"Plot saved to {file_path}")
            plt.show()
        except Exception as e:
            print(f"Error saving plot: {e}")
    
    def plot_optogenetic_psth_comparison(self, group1, group2, opto_freq, 
                                        ax=None, cell_type=None, is_single_unit=None, 
                                        stim_responsivity=None, modulation_label=None,
                                        time_range=None, plot_mode='mean', smoothing_window=None):
        """
        Plots a PSTH comparison for optogenetic experiments on the provided axes object 
        or creates a new figure if not provided.

        Args:
            group1 (str): First group name.
            group2 (str): Second group name.
            opto_freq (str): Optogenetic stimulation frequency (e.g., '8 Hz LED').
            ax (matplotlib.axes.Axes, optional): Axes object to plot on.
            cell_type (str, optional): Cell type to filter.
            is_single_unit (float, optional): Single unit filter.
            stim_responsivity (float, optional): Stimulus responsivity filter.
            modulation_label (str, optional): Modulation label filter.
            time_range (tuple, optional): Tuple specifying the start and end of the time range (e.g., (-100, 200)).
            plot_mode (str, optional): Plotting mode ('mean', 'traces', 'sem').
            smoothing_window (int, optional): Size of the smoothing window; if None, no smoothing is applied.
        """
        # Define colors
        group_colors = {
            'No_CTZ': '#797979',
            'CTZ': '#5a00c2'
        }

        # Fetch data
        df1, df2 = self.compare_optogenetic_groups(group1, group2, opto_freq, cell_type, is_single_unit, stim_responsivity, modulation_label)
        if df1.empty or df2.empty:
            print("One of the groups has no data after filtering.")
            return

        # Get the time array and adjust for the specified time range
        time_array = self.eed.relative_time_ms['relative_time_ms']
        if time_range:
            time_mask = (time_array >= time_range[0]) & (time_array <= time_range[1])
            time_array = time_array[time_mask]
        else:
            time_mask = slice(None)

        # Check if we need to create a new figure
        if ax is None:
            fig, ax = plt.subplots(figsize=(10, 5))

        # DataFrame to store all traces
        all_traces_df = pd.DataFrame()

        # Process and plot data
        for df, group in zip([df1, df2], [group1, group2]):
            data = df['PSTHs_conv'].apply(lambda x: np.array(x)[time_mask])
            if smoothing_window:
                window = np.ones(smoothing_window) / smoothing_window
                data = data.apply(lambda x: np.convolve(x, window, mode='same'))

            mean_psth = data.apply(pd.Series).mean(axis=0)

            # Store each trace in the DataFrame with additional metadata
            group_traces_df = pd.DataFrame(data.tolist(), columns=time_array)
            group_traces_df['Group'] = group
            group_traces_df['Stimulation'] = opto_freq
            group_traces_df['Cell_Type'] = cell_type
            group_traces_df['IsSingleUnit'] = is_single_unit
            group_traces_df['StimResponsivity'] = stim_responsivity
            all_traces_df = pd.concat([all_traces_df, group_traces_df])

            if plot_mode == 'sem':
                sem = data.apply(pd.Series).sem(axis=0)

            # Plot individual traces or mean with SEM
            if plot_mode == 'traces':
                for trace in data:
                    ax.plot(time_array, trace, color=group_colors[group]+'33', alpha=0.2)  # Lighter traces
            elif plot_mode == 'sem':
                ax.fill_between(time_array, mean_psth - sem, mean_psth + sem, color=group_colors[group], alpha=0.2)  # SEM shading

            ax.plot(time_array, mean_psth, label=f'{group}', color=group_colors[group])  # Mean trace

        # Set plot attributes
        ax.set_title(f'PSTH Comparison of {opto_freq} between {group1} and {group2}')
        ax.set_xlabel('Time (ms)')
        ax.set_ylabel('Average Spike Rate')
        ax.legend()

        # Only show the plot if an axes object was not provided
        if ax is None:
            plt.tight_layout()
            plt.show()

        # Return the DataFrame containing all traces
        return all_traces_df

    def compare_optogenetic_groups(self, group1, group2, opto_freq, cell_type=None, is_single_unit=None, stim_responsivity=None, modulation_label=None):
        """
        Compares PSTH data between two specified groups for a given optogenetic stimulation frequency.

        Args:
            group1 (str): The first group name for comparison.
            group2 (str): The second group name for comparison.
            opto_freq (str): The optogenetic stimulation frequency (e.g., '8 Hz LED').
            cell_type (str, optional): Specific cell type to filter by.
            is_single_unit (float, optional): Filter for single units (1.0) or multi-units (0.0).
            stim_responsivity (float, optional): Filter by stimulus responsivity.
            modulation_label (str, optional): Filter by modulation label ('positive', 'negative', 'none').

        Returns:
            A tuple containing two pandas DataFrames representing the filtered data for each group.
        """
        df_name = f'psth_dataframe_{opto_freq}'
        if df_name not in self.dataframes:
            print(f"No data available for the optogenetic stimulation: {opto_freq}")
            return None, None

        base_df = self.dataframes[df_name]

        # Print the shape of the pre-filtered DataFrame
        print(f"Pre-filtered DataFrame shape: {base_df.shape}")

        # Define a helper function to create a mask for a given group
        def create_mask(group):
            mask = (base_df['groupname'] == group)
            if cell_type is not None:
                mask &= (base_df['Cell_Type'] == cell_type)
            if is_single_unit is not None:
                mask &= (base_df['IsSingleUnit'] == is_single_unit)
            if stim_responsivity is not None:
                mask &= (base_df['StimResponsivity'] == stim_responsivity)
            if modulation_label is not None:
                if modulation_label not in ['positive', 'negative', 'none']:
                    raise ValueError("Modulation label must be one of 'positive', 'negative', or 'none'.")
                mask &= (base_df['ModulationIndex'] == modulation_label)
            return mask

        # Create masks for each group
        mask1 = create_mask(group1)
        mask2 = create_mask(group2)

        # Filter the DataFrame for each group
        channel1 = base_df[mask1]
        channel2 = base_df[mask2]

        # Print the shape of the filtered DataFrames
        print(f"Filtered df1 (group {group1}) shape: {channel1.shape}")
        print(f"Filtered df2 (group {group2}) shape: {channel2.shape}")

        return channel1, channel2
    
    def plot_optogenetic_psth_comparison_subplot(self, group1, group2, opto_freq, 
                                                cell_type=None, is_single_unit=None, 
                                                stim_responsivity=None, modulation_label=None,
                                                time_range=None, plot_mode='mean', smoothing_window=None):
        """
        Plots a PSTH comparison for optogenetic experiments in a 1x2 subplot,
        with each group plotted separately.

        Args:
            group1 (str): First group name.
            group2 (str): Second group name.
            opto_freq (str): Optogenetic stimulation frequency (e.g., '8 Hz LED').
            cell_type (str, optional): Cell type to filter.
            is_single_unit (float, optional): Single unit filter.
            stim_responsivity (float, optional): Stimulus responsivity filter.
            modulation_label (str, optional): Modulation label filter.
            time_range (tuple, optional): Tuple specifying the start and end of the time range (e.g., (-100, 200)).
            plot_mode (str, optional): Plotting mode ('mean', 'traces', 'sem').
            smoothing_window (int, optional): Size of the smoothing window; if None, no smoothing is applied.
        """
        # Define colors
        group_colors = {
            'No_CTZ': '#797979',
            'CTZ': '#5a00c2'
        }

        # Fetch data
        df1, df2 = self.compare_optogenetic_groups(group1, group2, opto_freq, cell_type, is_single_unit, stim_responsivity, modulation_label)
        if df1.empty or df2.empty:
            print("One of the groups has no data after filtering.")
            return

        # Get the time array and adjust for the specified time range
        time_array = self.eed.relative_time_ms['relative_time_ms']
        if time_range:
            time_mask = (time_array >= time_range[0]) & (time_range[1] >= time_array)
            time_array = time_array[time_mask]
        else:
            time_mask = slice(None)

        # Create a new figure with 1x2 subplots
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 5), sharey=True)
        axes = [ax1, ax2]

        # DataFrame to store all traces
        all_traces_df = pd.DataFrame()

        # Process and plot data
        for df, group, ax in zip([df1, df2], [group1, group2], axes):
            data = df['PSTHs_conv'].apply(lambda x: np.array(x)[time_mask])
            if smoothing_window:
                window = np.ones(smoothing_window) / smoothing_window
                data = data.apply(lambda x: np.convolve(x, window, mode='same'))

            mean_psth = data.apply(pd.Series).mean(axis=0)

            # Store each trace in the DataFrame with additional metadata
            group_traces_df = pd.DataFrame(data.tolist(), columns=time_array)
            group_traces_df['Group'] = group
            group_traces_df['Stimulation'] = opto_freq
            group_traces_df['Cell_Type'] = cell_type
            group_traces_df['IsSingleUnit'] = is_single_unit
            group_traces_df['StimResponsivity'] = stim_responsivity
            all_traces_df = pd.concat([all_traces_df, group_traces_df])

            if plot_mode == 'sem':
                sem = data.apply(pd.Series).sem(axis=0)

            # Plot individual traces or mean with SEM
            if plot_mode == 'traces':
                for trace in data:
                    ax.plot(time_array, trace, color=group_colors[group]+'33', alpha=0.2)  # Lighter traces
            elif plot_mode == 'sem':
                ax.fill_between(time_array, mean_psth - sem, mean_psth + sem, color=group_colors[group], alpha=0.2)  # SEM shading

            ax.plot(time_array, mean_psth, label=f'{group}', color=group_colors[group])  # Mean trace

            # Set plot attributes for each subplot
            ax.set_title(f'{group} - {opto_freq}')
            ax.set_xlabel('Time (ms)')
            ax.set_ylabel('Average Spike Rate')
            ax.legend()

        # Set overall title
        fig.suptitle(f'PSTH Comparison of {opto_freq} between {group1} and {group2}', fontsize=16)

        # Adjust layout and display
        plt.tight_layout()
        plt.show()

        # Return the DataFrame containing all traces
        return all_traces_df

    def plot_optogenetic_psth_comparison_subplot_with_led(self, group1, group2, opto_freq, 
                                                        cell_type=None, is_single_unit=None, 
                                                        stim_responsivity=None, modulation_label=None,
                                                        time_range=None, plot_mode='mean', smoothing_window=None):
        """
        Plots a PSTH comparison for optogenetic experiments in a 1x2 subplot,
        with each group plotted separately and the LED signal on a shared third axis.

        Args:
            group1 (str): First group name.
            group2 (str): Second group name.
            opto_freq (str): Optogenetic stimulation frequency (e.g., '8 Hz LED').
            cell_type (str, optional): Cell type to filter.
            is_single_unit (float, optional): Single unit filter.
            stim_responsivity (float, optional): Stimulus responsivity filter.
            modulation_label (str, optional): Modulation label filter.
            time_range (tuple, optional): Tuple specifying the start and end of the time range (e.g., (-100, 200)).
            plot_mode (str, optional): Plotting mode ('mean', 'traces', 'sem').
            smoothing_window (int, optional): Size of the smoothing window; if None, no smoothing is applied.
        """
        # Define colors
        group_colors = {
            'No_CTZ': '#797979',
            'CTZ': '#5a00c2'
        }

        # Fetch data
        df1, df2 = self.compare_optogenetic_groups(group1, group2, opto_freq, cell_type, is_single_unit, stim_responsivity, modulation_label)
        if df1.empty or df2.empty:
            print("One of the groups has no data after filtering.")
            return

        # Get the time array and adjust for the specified time range
        time_array = self.eed.relative_time_ms['relative_time_ms']
        if time_range:
            time_mask = (time_array >= time_range[0]) & (time_range[1] >= time_array)
            time_array = time_array[time_mask]
        else:
            time_mask = slice(None)

        # Extract LED signal
        extracted_led_signal = self.extract_stim_signals_opto()
        led_array = extracted_led_signal[0][opto_freq]
        led_array = led_array[time_mask]  # Apply the same time mask to LED signal

        # Create a new figure with 1x2 subplots
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 5), sharey=True)
        axes = [ax1, ax2]

        # Create a third axis for LED signal, shared between the two subplots
        ax_led = ax1.twinx()
        ax2.twinx().set_ylabel('')  # Hide y-axis label for the second LED axis

        # DataFrame to store all traces
        all_traces_df = pd.DataFrame()

        # Process and plot data
        for df, group, ax in zip([df1, df2], [group1, group2], axes):
            data = df['PSTHs_conv'].apply(lambda x: np.array(x)[time_mask])
            if smoothing_window:
                window = np.ones(smoothing_window) / smoothing_window
                data = data.apply(lambda x: np.convolve(x, window, mode='same'))

            mean_psth = data.apply(pd.Series).mean(axis=0)

            # Store each trace in the DataFrame with additional metadata
            group_traces_df = pd.DataFrame(data.tolist(), columns=time_array)
            group_traces_df['Group'] = group
            group_traces_df['Stimulation'] = opto_freq
            group_traces_df['Cell_Type'] = cell_type
            group_traces_df['IsSingleUnit'] = is_single_unit
            group_traces_df['StimResponsivity'] = stim_responsivity
            all_traces_df = pd.concat([all_traces_df, group_traces_df])

            if plot_mode == 'sem':
                sem = data.apply(pd.Series).sem(axis=0)

            # Plot individual traces or mean with SEM
            if plot_mode == 'traces':
                for trace in data:
                    ax.plot(time_array, trace, color=group_colors[group]+'33', alpha=0.2)  # Lighter traces
            elif plot_mode == 'sem':
                ax.fill_between(time_array, mean_psth - sem, mean_psth + sem, color=group_colors[group], alpha=0.2)  # SEM shading

            ax.plot(time_array, mean_psth, label=f'{group}', color=group_colors[group])  # Mean trace

            # Set plot attributes for each subplot
            ax.set_title(f'{group} - {opto_freq}')
            ax.set_xlabel('Time (ms)')
            ax.set_ylabel('Average Spike Rate')
            ax.legend(loc='upper left')

        # Plot LED signal on both subplots
        ax_led.plot(time_array, led_array, color='blue', linestyle='-', linewidth=1.5, alpha=0.7, label='LED Signal')
        ax2.twinx().plot(time_array, led_array, color='blue', linestyle='-', linewidth=1.5, alpha=0.7)

        # Set y-axis label for LED signal
        ax_led.set_ylabel('LED Signal (a.u.)', color='blue')
        ax_led.tick_params(axis='y', labelcolor='blue')

        # Add legend for LED signal
        ax_led.legend(loc='upper right')

        # Set overall title
        fig.suptitle(f'PSTH Comparison of {opto_freq} between {group1} and {group2}', fontsize=16)

        # Adjust layout and display
        plt.tight_layout()
        plt.show()

        # Return the DataFrame containing all traces
        return all_traces_df

    def plot_optogenetic_psth_per_recording(self, group1, group2, opto_freq, 
                                            cell_type=None, is_single_unit=None, 
                                            stim_responsivity=None, modulation_label=None,
                                            time_range=None, plot_mode='mean', smoothing_window=None):
        """
        Plots PSTHs for optogenetic experiments, filtered at the recording level for each group,
        saves the results in a specified directory structure, and ensures comparable y-axes between groups.

        Args:
            group1 (str): First group name.
            group2 (str): Second group name.
            opto_freq (str): Optogenetic stimulation frequency (e.g., '8 Hz LED').
            cell_type (str, optional): Cell type to filter.
            is_single_unit (float, optional): Single unit filter.
            stim_responsivity (float, optional): Stimulus responsivity filter.
            modulation_label (str, optional): Modulation label filter.
            time_range (tuple, optional): Tuple specifying the start and end of the time range (e.g., (-100, 200)).
            plot_mode (str, optional): Plotting mode ('mean', 'traces', 'sem').
            smoothing_window (int, optional): Size of the smoothing window; if None, no smoothing is applied.
        """
        # Define colors
        group_colors = {
            'No_CTZ': '#797979',
            'CTZ': '#5a00c2'
        }

        # Fetch data
        df1, df2 = self.compare_optogenetic_groups(group1, group2, opto_freq, cell_type, is_single_unit, stim_responsivity, modulation_label)
        if df1.empty and df2.empty:
            print("Both groups have no data after filtering.")
            return
        
        # Get the time array and adjust for the specified time range
        time_array = self.eed.relative_time_ms['relative_time_ms']
        if time_range:
            time_mask = (time_array >= time_range[0]) & (time_range[1] >= time_array)
            time_array = time_array[time_mask]
        else:
            time_mask = slice(None)

        # Extract LED signal
        extracted_led_signal = self.extract_stim_signals_opto()
        led_array = extracted_led_signal[0][opto_freq]
        led_array = led_array[time_mask]  # Apply the same time mask to LED signal

        # Base directory for saving figures
        base_dir = "/Volumes/MannySSD/figures/opto_psths_perrecording"

        # Find global y-axis limits
        all_psth_data = pd.concat([df1['PSTHs_conv'], df2['PSTHs_conv']])
        all_psth_data = all_psth_data.apply(lambda x: np.array(x)[time_mask])
        global_ymin = all_psth_data.apply(np.min).min()
        global_ymax = all_psth_data.apply(np.max).max()

        # Process each group
        for group, df in zip([group1, group2], [df1, df2]):
            if df.empty:
                print(f"No data for group {group} after filtering.")
                continue

            # Create group directory
            group_dir = os.path.join(base_dir, group)
            os.makedirs(group_dir, exist_ok=True)

            # Process each unique recording
            for recording in df['recordingname'].unique():
                recording_df = df[df['recordingname'] == recording]
                
                # Create recording directory
                recording_dir = os.path.join(group_dir, recording)
                os.makedirs(recording_dir, exist_ok=True)

                # Create a new figure for this recording
                fig, ax = plt.subplots(figsize=(10, 5))

                # Plot PSTH data
                data = recording_df['PSTHs_conv'].apply(lambda x: np.array(x)[time_mask])
                if smoothing_window:
                    window = np.ones(smoothing_window) / smoothing_window
                    data = data.apply(lambda x: np.convolve(x, window, mode='same'))

                mean_psth = data.apply(pd.Series).mean(axis=0)

                if plot_mode == 'sem':
                    sem = data.apply(pd.Series).sem(axis=0)
                    ax.fill_between(time_array, mean_psth - sem, mean_psth + sem, color=group_colors[group], alpha=0.2)

                elif plot_mode == 'traces':
                    for trace in data:
                        ax.plot(time_array, trace, color=group_colors[group]+'33', alpha=0.2)

                ax.plot(time_array, mean_psth, label=f'{group}', color=group_colors[group])

                # Plot LED signal
                ax_led = ax.twinx()
                ax_led.plot(time_array, led_array, color='blue', linestyle='-', linewidth=1.5, alpha=0.7, label='LED Signal')

                # Set plot attributes
                ax.set_title(f'{group} - {opto_freq} - Recording: {recording}')
                ax.set_xlabel('Time (ms)')
                ax.set_ylabel('Average Spike Rate')
                ax_led.set_ylabel('LED Signal (a.u.)', color='blue')
                ax_led.tick_params(axis='y', labelcolor='blue')

                # Set y-axis limits to be the same for all plots
                ax.set_ylim(global_ymin, global_ymax)

                # Add legends
                ax.legend(loc='upper left')
                ax_led.legend(loc='upper right')

                # Save the figure
                plt.tight_layout()
                
                # Create filename with user input parameters
                filename = f'PSTH_{opto_freq}'
                if cell_type:
                    filename += f'_celltype-{cell_type}'
                if is_single_unit is not None:
                    filename += f'_singleunit-{is_single_unit}'
                if stim_responsivity is not None:
                    filename += f'_stimresp-{stim_responsivity}'
                if time_range:
                    filename += f'_timerange-{time_range[0]}to{time_range[1]}'
                filename += '.png'
                
                fig_path = os.path.join(recording_dir, filename)
                plt.savefig(fig_path)
                plt.close(fig)

                print(f"Saved figure for {group} - {recording} at {fig_path}")

        print("Finished processing all recordings.")

    def plot_optogenetic_psth_comparison_grid(self, group1, group2, opto_freq, 
                                            cell_types=['RS', 'FS'], is_single_unit=None, 
                                            stim_responsivity=None, modulation_label=None,
                                            time_range=None, plot_mode='mean', smoothing_window=None):
        """
        Plots a 2x2 grid PSTH comparison for optogenetic experiments,
        with each subplot showing a specific group and cell type combination.

        Args:
            group1 (str): First group name (e.g., 'CTZ').
            group2 (str): Second group name (e.g., 'No_CTZ').
            opto_freq (str): Optogenetic stimulation frequency (e.g., '8 Hz LED').
            cell_types (list): List of cell types to plot (default: ['RS', 'FS']).
            is_single_unit (float, optional): Single unit filter.
            stim_responsivity (float, optional): Stimulus responsivity filter.
            modulation_label (str, optional): Modulation label filter.
            time_range (tuple, optional): Tuple specifying the start and end of the time range (e.g., (-100, 200)).
            plot_mode (str, optional): Plotting mode ('mean', 'traces', 'sem').
            smoothing_window (int, optional): Size of the smoothing window; if None, no smoothing is applied.
        """
        # Define colors
        group_colors = {
            group1: '#5a00c2',  # CTZ
            group2: '#797979'   # No_CTZ
        }

        # Create a new figure with 2x2 subplots
        fig, axes = plt.subplots(2, 2, figsize=(20, 16), sharex=True, sharey=True)

        # Define the mapping of group and cell type to subplot position
        plot_mapping = {
            (group1, 'RS'): (0, 0),  # Top left
            (group2, 'RS'): (0, 1),  # Top right
            (group1, 'FS'): (1, 0),  # Bottom left
            (group2, 'FS'): (1, 1)   # Bottom right
        }

        # Get the time array and adjust for the specified time range
        time_array = self.eed.relative_time_ms['relative_time_ms']
        if time_range:
            time_mask = (time_array >= time_range[0]) & (time_range[1] >= time_array)
            time_array = time_array[time_mask]
        else:
            time_mask = slice(None)

        # Extract LED signal
        extracted_led_signal = self.extract_stim_signals_opto()
        led_array = extracted_led_signal[0][opto_freq][time_mask]

        # Process and plot data for each combination of group and cell type
        for (group, cell_type), (row, col) in plot_mapping.items():
            ax = axes[row, col]

            # Fetch data for the current group and cell type
            df = self.compare_optogenetic_groups(group, group, opto_freq, cell_type, is_single_unit, stim_responsivity, modulation_label)[0]

            if df.empty:
                print(f"No data after filtering for {group} - {cell_type}.")
                continue

            # Create a third axis for LED signal
            ax_led = ax.twinx()

            # Process data
            data = df['PSTHs_conv'].apply(lambda x: np.array(x)[time_mask])
            if smoothing_window:
                window = np.ones(smoothing_window) / smoothing_window
                data = data.apply(lambda x: np.convolve(x, window, mode='same'))

            mean_psth = data.apply(pd.Series).mean(axis=0)

            if plot_mode == 'sem':
                sem = data.apply(pd.Series).sem(axis=0)

            # Plot individual traces or mean with SEM
            if plot_mode == 'traces':
                for trace in data:
                    ax.plot(time_array, trace, color=group_colors[group]+'33', alpha=0.2)  # Lighter traces
            elif plot_mode == 'sem':
                ax.fill_between(time_array, mean_psth - sem, mean_psth + sem, color=group_colors[group], alpha=0.2)  # SEM shading

            ax.plot(time_array, mean_psth, label=f'{group}', color=group_colors[group])  # Mean trace

            # Plot LED signal
            ax_led.plot(time_array, led_array, color='blue', linestyle='-', linewidth=1.5, alpha=0.7, label='LED Signal')

            # Set plot attributes
            ax.set_title(f'{group} - {cell_type}')
            if row == 1:  # Bottom row
                ax.set_xlabel('Time (ms)')
            if col == 0:  # Left column
                ax.set_ylabel('Average Spike Rate')
            ax.legend(loc='upper left')

            # Set y-axis label for LED signal only for the rightmost subplots
            if col == 1:
                ax_led.set_ylabel('LED Signal (a.u.)', color='blue')
                ax_led.tick_params(axis='y', labelcolor='blue')
            else:
                ax_led.set_ylabel('')

        # Set overall title
        fig.suptitle(f'PSTH Comparison of {opto_freq} between {group1} and {group2} for {" and ".join(cell_types)} cells', fontsize=16)

        # Adjust layout and display
        plt.tight_layout()
        
        #save the plot in the normal directory
        plt.savefig(f'/Volumes/MannySSD/figures/opto_psths_comparison/CTZ_No_CTZ_comparison_{"single" if is_single_unit == 1.0 else "multi"}_units_evoked_FR.svg', format='svg', dpi=300, bbox_inches='tight', transparent=True)
        
        plt.show()

    def plot_interactive_psth_comparison_grid(self, group1, group2, opto_freq, 
                                            cell_types=['RS', 'FS'], is_single_unit=None, 
                                            stim_responsivity=None, modulation_label=None,
                                            time_range=None, smoothing_window=None):
        """
        Plots an interactive 2x2 grid PSTH comparison for optogenetic experiments using Plotly,
        with each subplot showing a specific group and cell type combination, and LED signal on a separate axis.

        Args:
            group1 (str): First group name (e.g., 'CTZ').
            group2 (str): Second group name (e.g., 'No_CTZ').
            opto_freq (str): Optogenetic stimulation frequency (e.g., '8 Hz LED').
            cell_types (list): List of cell types to plot (default: ['RS', 'FS']).
            is_single_unit (float, optional): Single unit filter.
            stim_responsivity (float, optional): Stimulus responsivity filter.
            modulation_label (str, optional): Modulation label filter.
            time_range (tuple, optional): Tuple specifying the start and end of the time range (e.g., (-200, 800)).
            smoothing_window (int, optional): Size of the smoothing window; if None, no smoothing is applied.
        """
        # Define colors
        group_colors = {
            group1: 'rgba(90, 0, 194, 0.7)',  # CTZ (semi-transparent purple)
            group2: 'rgba(121, 121, 121, 0.7)'  # No_CTZ (semi-transparent gray)
        }

        # Create subplot titles
        subplot_titles = [
            f'{group1} - RS', f'{group2} - RS',
            f'{group1} - FS', f'{group2} - FS'
        ]

        # Create a 2x2 subplot layout with secondary y-axes
        fig = make_subplots(rows=2, cols=2, shared_xaxes=True, shared_yaxes=True,
                            subplot_titles=subplot_titles,
                            specs=[[{"secondary_y": True}, {"secondary_y": True}],
                                [{"secondary_y": True}, {"secondary_y": True}]])

        # Define the mapping of group and cell type to subplot position
        plot_mapping = {
            (group1, 'RS'): (1, 1),  # Top left
            (group2, 'RS'): (1, 2),  # Top right
            (group1, 'FS'): (2, 1),  # Bottom left
            (group2, 'FS'): (2, 2)   # Bottom right
        }

        # Get the time array and adjust for the specified time range
        time_array = self.eed.relative_time_ms['relative_time_ms']
        if time_range:
            time_mask = (time_array >= time_range[0]) & (time_range[1] >= time_array)
            time_array = time_array[time_mask]
        else:
            time_mask = slice(None)

        # Extract LED signal
        extracted_led_signal = self.extract_stim_signals_opto()
        led_array = extracted_led_signal[0][opto_freq][time_mask]

        # Process and plot data for each combination of group and cell type
        for (group, cell_type), (row, col) in plot_mapping.items():
            # Fetch data for the current group and cell type
            df = self.compare_optogenetic_groups(group, group, opto_freq, cell_type, is_single_unit, stim_responsivity, modulation_label)[0]

            if df.empty:
                print(f"No data after filtering for {group} - {cell_type}.")
                continue

            # Process data
            data = df['PSTHs_conv'].apply(lambda x: np.array(x)[time_mask])
            if smoothing_window:
                window = np.ones(smoothing_window) / smoothing_window
                data = data.apply(lambda x: np.convolve(x, window, mode='same'))

            # Plot individual traces
            for idx, (trace, groupname, recordingname, cid) in enumerate(zip(data, df['groupname'], df['recordingname'], df['cid'])):
                fig.add_trace(
                    go.Scatter(
                        x=time_array,
                        y=trace,
                        mode='lines',
                        line=dict(color=group_colors[group]),
                        name=f'{group} - {cell_type}',
                        legendgroup=f'{group} - {cell_type}',
                        showlegend=idx == 0,  # Show legend only for the first trace of each group
                        hoverinfo='text',
                        hovertext=[f'Group: {groupname}<br>Recording: {recordingname}<br>CID: {cid}<br>Time: {t:.2f} ms<br>Rate: {r:.2f}' for t, r in zip(time_array, trace)],
                        customdata=np.column_stack((np.full(len(time_array), groupname), 
                                                    np.full(len(time_array), recordingname), 
                                                    np.full(len(time_array), cid)))
                    ),
                    row=row, col=col, secondary_y=False
                )

            # Plot mean PSTH
            mean_psth = data.apply(pd.Series).mean(axis=0)
            fig.add_trace(
                go.Scatter(
                    x=time_array,
                    y=mean_psth,
                    mode='lines',
                    line=dict(color='black', width=2),
                    name=f'Mean {group} - {cell_type}',
                    legendgroup=f'Mean {group} - {cell_type}',
                ),
                row=row, col=col, secondary_y=False
            )

            # Plot LED signal on secondary y-axis
            fig.add_trace(
                go.Scatter(
                    x=time_array,
                    y=led_array,
                    mode='lines',
                    line=dict(color='blue', width=1.5),
                    name='LED Signal',
                    legendgroup='LED Signal',
                    showlegend=row == 1 and col == 1,  # Show legend only once for LED signal
                ),
                row=row, col=col, secondary_y=True
            )

            # Update axes
            fig.update_xaxes(title_text="Time (ms)", row=row, col=col)
            fig.update_yaxes(title_text="Average Spike Rate" if col == 1 else "", row=row, col=col, secondary_y=False)
            fig.update_yaxes(title_text="LED Signal (a.u.)" if col == 2 else "", row=row, col=col, secondary_y=True)

        # Update layout
        fig.update_layout(
            title=f'PSTH Comparison of {opto_freq} between {group1} and {group2} for {" and ".join(cell_types)} cells',
            height=800,
            width=1200,
            hovermode='closest'
        )

        # Add custom JavaScript for trace highlighting
        fig.update_layout(
            updatemenus=[
                dict(
                    type="buttons",
                    direction="left",
                    buttons=[
                        dict(
                            args=[{"visible": [True] * len(fig.data)}],
                            label="Show All",
                            method="restyle"
                        ),
                        dict(
                            args=[{"visible": "legendonly"}],
                            label="Hide All",
                            method="restyle"
                        )
                    ],
                    pad={"r": 10, "t": 10},
                    showactive=False,
                    x=0.11,
                    xanchor="left",
                    y=1.1,
                    yanchor="top"
                ),
            ]
        )

        fig.show()
 
 
 

def extract_array_element(value, array_element=0):
    """
    Extracts the scalar value from a numpy array or list.
    
    Parameters:
    value: The value from which to extract the element. This can be of any type, 
           but we are particularly interested in cases where it's a numpy array or list.
    array_element (int): The index of the element to extract from the array or list (default is 0).
    
    Returns:
    The extracted element if the value is a numpy array or list. 
    If the value is a scalar within a numpy array, it extracts and returns the scalar.
    Otherwise, it returns the original value.
    """
    
    # If the value is a numpy array with a single element, extract that element.
    if isinstance(value, np.ndarray) and value.size == 1:
        return value.item()
    
    # If the value is a numpy array or list with multiple elements, extract the specified element.
    elif isinstance(value, (np.ndarray, list)) and len(value) > array_element:
        return value[array_element]
    
    # If the value is not a numpy array or list, or if it's not possible to extract a valid element, 
    # return the value as-is.
    return value



def determine_stim_responsivity(row):
    """
    Determine the StimResponsivity based on the lower bound of the AUROC confidence interval and ModulationIndex.
    
    Parameters:
    row (Series): A row of the DataFrame.
    
    Returns:
    int: StimResponsivity value (1 for increase/positive modulation, -1 for decrease/negative modulation, 0 for no significant change).
    """
    # Access the lower bound of the AUROC confidence interval
    auroc_lower_bound = row['StimProb'][1] if len(row['StimProb']) > 1 else None
    modulation_index = row['ModulationIndex']
    
    print(f"AUROC lower bound: {auroc_lower_bound}, Modulation Index: {modulation_index}")

    # Step 1: Determine if the neuron is sensory responsive
    if auroc_lower_bound is not None and auroc_lower_bound > 0.5:
        # Step 2: Categorize modulation if the neuron is sensory responsive
        if modulation_index == 'positive':
            return 1  # Sensory responsive and positively modulated (Increase)
        elif modulation_index == 'negative':
            return -1  # Sensory responsive and negatively modulated (Decrease)
    return 0  # No significant change or non-responsive

def example_analysis_function(extracted_data):
    """
    Example analysis function that calculates the mean spike count for each stimulus.

    Args:
        extracted_data (dict): The extracted data from the PSTH and raster data.

    Returns:
        dict: A dictionary containing the analysis results.
    """
    analysis_results = {'mean_spike_counts': {}}
    for stim, data in extracted_data['data'].items():
        mean_spike_count = np.mean(data['raw_counts'])
        analysis_results['mean_spike_counts'][stim] = mean_spike_count
    return analysis_results

def analyze_sensory_responsiveness(extracted_data, baseline_window=(-500, 0), response_window=(0, 50), analysis_window=(0, 200)):
    """
    Analyze the sensory responsiveness of neurons by calculating the sensory response magnitude, z-scoring it,
    and performing a nonparametric permutation test to determine if a neuron is modulated by the stimulus.

    Args:
        extracted_data (dict): The extracted data from the PSTH and raster data.
        baseline_window (tuple): The window (in ms) to calculate the baseline firing rate. Default is (-500, 0).
        response_window (tuple): The window (in ms) to calculate the response magnitude. Default is (0, 50).
        analysis_window (tuple): The window (in ms) to analyze modulation. Default is (0, 200).

    Returns:
        dict: A dictionary containing the sensory response analysis results.
    """
    results = {}

    for stim, data in extracted_data['data'].items():
        spike_trains = data['spike_trains']
        time_array = data['time_array']

        # Calculate baseline firing rate
        baseline_start_idx = np.searchsorted(time_array, baseline_window[0])
        baseline_end_idx = np.searchsorted(time_array, baseline_window[1])
        baseline_rates = spike_trains[:, baseline_start_idx:baseline_end_idx].mean(axis=1)

        # Calculate response magnitude
        response_start_idx = np.searchsorted(time_array, response_window[0])
        response_end_idx = np.searchsorted(time_array, response_window[1])
        response_rates = spike_trains[:, response_start_idx:response_end_idx].mean(axis=1)
        response_magnitude = response_rates.mean() - baseline_rates.mean()

        # Calculate z-scored magnitude
        baseline_sd = baseline_rates.std()
        z_scored_magnitude = response_magnitude / baseline_sd if baseline_sd != 0 else np.nan

        # Permutation test for modulation analysis
        analysis_start_idx = np.searchsorted(time_array, analysis_window[0])
        analysis_end_idx = np.searchsorted(time_array, analysis_window[1])
        pre_stimulus_rates = spike_trains[:, baseline_start_idx:baseline_end_idx].mean(axis=1)
        post_stimulus_rates = spike_trains[:, analysis_start_idx:analysis_end_idx].mean(axis=1)

        def permutation_test_func(x, y):
            return x.mean() - y.mean()

        permutation_result = permutation_test((pre_stimulus_rates, post_stimulus_rates),
                                              statistic=permutation_test_func,
                                              n_resamples=10000,
                                              alternative='two-sided')

        results[stim] = {
            'response_magnitude': response_magnitude,
            'z_scored_magnitude': z_scored_magnitude,
            'permutation_p_value': permutation_result.pvalue
        }

    return results

In [ ]:
import matplotlib.colors as mcolors
import mat73
class CCGHandler:
    def __init__(self, eed_object, ccg_directory):
        """
        Initializes the CCGHandler class with an object of ExtractEphysData and a directory for CCG files.

        Parameters:
            eed_object (ExtractEphysData): An instance of ExtractEphysData which provides
                                           access to electrophysiological data methods and attributes.
            
            ccg_directory (str): The directory path where the CCG files are stored.
        """
        self.eed = eed_object
        self.ccg_directory = ccg_directory
        self.ccg = {}
        self.load_ccg_files()
        self.ccg['MonoConnectionsTable'] = self.unpack_mono_connections()
        self.integrate_ccgs() # Integrate CCG data into the MonoConnectionsTable DataFrame and delete the MonoConnCCGs data attribute
    
    def load_ccg_files(self):
        """
        Loads all required .mat files from the specified CCG directory and sets them as attributes under the `ccg` dictionary.

        Required Files:
            MonoConnCCGs.mat - Contains CCG data for monosynaptic connections.
            
            MonoConnectionsTable.mat - Contains a struct MonoConnectionsTable converted into a struct by group,
            and recording all connection data; accessed using the key 'nested_struct'.
            
            PointwiseABs.mat - Contains 95% pointwise acceptacne bands for CCG.
            SimultaneousABs.mat - Contains 95% pointwise acceptacne bands for CCG.
            
            t.mat - Contains the time array for the x axis of the CCG.

        The data from each file is accessed by stripping the '.mat' and accessing the corresponding key in the loaded dictionary, except for specific exceptions noted.
        """
        required_files = [
            'MonoConnCCGs.mat',
            'MonoConnectionsTable.mat',
            'PointwiseABs.mat',
            'SimultaneousABs.mat',
            't.mat'
        ]

        # Check if the directory exists
        if not os.path.exists(self.ccg_directory):
            print(f"CCG directory not found: {self.ccg_directory}")
            return

        # Load each required .mat file from the directory
        for file_name in required_files:
            file_path = os.path.join(self.ccg_directory, file_name)
            print(f"Loading CCG data from: {file_path}")
            try:
                # Load the file and extract the data using the base filename as the key
                data = mat73.loadmat(file_path)
                base_key = file_name.replace('.mat', '')
                # Handle special case for 'MonoConnectionsTable'
                if base_key == 'MonoConnectionsTable':
                    self.ccg[base_key] = data['nested_struct']
                elif base_key in data:
                    self.ccg[base_key] = data[base_key]
                else:
                    print(f"Expected key '{base_key}' not found in {file_name}")
                    self.ccg[base_key] = None
            except FileNotFoundError:
                print(f"File not found: {file_path}")
                self.ccg[base_key] = None
            except Exception as e:
                print(f"Error loading file {file_path}: {e}")
                self.ccg[base_key] = None
    
    def unpack_mono_connections(self):
        """
        Unpacks the MonoConnectionsTable and assembles the data into a comprehensive DataFrame.
        This method iterates over each group and recording in the MonoConnectionsTable, which
        is structured by group and recording name. Each entry represents connectivity data between
        neuron pairs, including various metrics and identifiers.

        The resulting DataFrame compiles all the connectivity data across different groups and recordings,
        with scalar values extracted from lists for each metric, providing a holistic view of the monosynaptic connections analyzed.

        Adjustments:
        - Converts 'Significance' from [1.0, 0.0] to True/False.
        - Maps 'StimResp_A' and 'StimResp_B' from 'nr', '-', '+' to 0, -1, 1 respectively for easier filtering.

        Returns:
            DataFrame: A DataFrame containing all the unpacked connectivity data with scalar values.
        """
        # Initialize an empty list to collect data
        all_data = []

        # Mapping for stimulus response
        stim_resp_mapping = {'nr': 0, '-': -1, '+': 1}

        # Navigate through each group and recording
        for groupname, recordings in self.ccg['MonoConnectionsTable'].items():
            for recording, data in recordings.items():
                for item in data:
                    # Flatten the dictionary and add groupname and recording info
                    item_data = {}
                    for key, val in item.items():
                        if isinstance(val, list) and len(val) == 1:
                            if key == 'Significance':
                                # Convert Significance to boolean
                                item_data[key] = bool(val[0])
                            elif key in ['StimResp_A', 'StimResp_B']:
                                # Map StimResp_A and StimResp_B
                                item_data[key] = stim_resp_mapping[val[0]]
                            else:
                                item_data[key] = val[0]
                        else:
                            item_data[key] = val
                    item_data['groupname'] = groupname
                    item_data['recording'] = recording
                    all_data.append(item_data)
        return pd.DataFrame(all_data)
      
    def integrate_ccgs(self):
        """
        Integrates the CCG data from 'MonoConnCCGs' into the 'MonoConnectionsTable' DataFrame.
        Each row in 'MonoConnectionsTable' corresponds to a column in 'MonoConnCCGs', which contains
        the time points for the CCGs in 1ms bins. This method adds these CCGs as a new column 'ccgs' in
        the DataFrame, ensuring each pair's CCGs are correctly aligned with the DataFrame rows.

        Post-process:
        - Deletes the original 'MonoConnCCGs' data from the 'ccg' attribute to save memory and avoid redundancy.
        """
        # Retrieve the DataFrame and the CCG data
        df = self.ccg['MonoConnectionsTable']
        ccgs_data = self.ccg['MonoConnCCGs']

        # Ensure that the rows of the DataFrame match the columns in the CCGs data
        if df.shape[0] != ccgs_data.shape[1]:
            raise ValueError("Mismatch between DataFrame rows and CCG data columns.")

        # Add the CCGs as a new column to the DataFrame
        df['ccgs'] = [ccgs_data[:, i] for i in range(ccgs_data.shape[1])]

        # Update the 'MonoConnectionsTable' with the new DataFrame
        self.ccg['MonoConnectionsTable'] = df

        # Remove the 'MonoConnCCGs' data
        del self.ccg['MonoConnCCGs']
        
    def access_ccgs(self, groupname=None, significance=True, sd_sig=None, EorI=None, connectiontype=None, response=None, layers=None, min_spike_pairs=150, limit=None):
        """
        Accesses CCGs from 'MonoConnectionsTable' based on specified criteria.
        Returns a filtered DataFrame containing CCGs that meet the specified criteria.

        Parameters:
            groupname (str or None): The group name to filter by; defaults to None, which includes all groups.
            significance (bool): Filter connections by significance; defaults to True, meaning only significant connections are included.
            sd_sig (bool): Filter connection based on 7*SD based on peak signal relative to noise of CCGs
            EorI (list of str or None): Filter connections by a list of excitatory/inhibitory statuses; defaults to None, which includes all statuses (e.g 'E', 'I', 'NS', 'S').
            connectiontype (str, list of str, or None): Filter connections by type (e.g., 'FS->RS', 'RS->FS' or ['FS->RS', 'RS->FS']); defaults to None, which includes all types.
            response (str or None): Filters connections based on responsiveness; 'responsive' for either StimResp_A being 1 or 'nonresponsive' for both being 0.
            layers (list of str or None): Filter connections by layer (e.g., 'IG->L4', 'L4->L4'); defaults to None, which includes all layers.
            min_spike_pairs (int): Minimum number of spike pairs to include; defaults to 150.
            threshold (bool): Filter connections by threshold; defaults to None.

        Returns:
            DataFrame: A filtered DataFrame containing CCGs that meet the specified criteria.
        """
        df = self.ccg['MonoConnectionsTable']

        # Filter by group name if specified
        if groupname is not None:
            df = df[df['groupname'] == groupname]

        # Filter by significance
        if significance:
            df = df[df['Significance'] == True]

        # Filter by it being sig based on 7*SD of peak relative to noise 
        if sd_sig: 
            df = df[df['sd_sig'] == True]

        # Filter by EorI status if specified
        if EorI is not None:
            df = df[df['EorI'].isin(EorI)]

        # Filter by connection type if specified
        if connectiontype is not None:
            if isinstance(connectiontype, list):
                df = df[df['CellTypes'].isin(connectiontype)]
            else:
                df = df[df['CellTypes'] == connectiontype]

        # Filter by response if specified
        if response:
            if response == 'responsive':
                df = df[(df['StimResp_A'] == 1)]
            elif response == 'nonresponsive':
                df = df[(df['StimResp_A'] == 0)]

        # Filter by layers if specified
        if layers is not None:
            df = df[df['layers'].isin(layers)]

        # Filter by threshold if specified
        if limit:
            df = df[df['threshold'] == limit]

        # Filter by minimum number of spike pairs
        df = df[df['N_SpikePairs'] >= min_spike_pairs]

        return df
    
    def plot_ccgs(self, groupname=None, significance=True, EorI=None, connectiontype=None):
        """
        Assembles the CCGs for connections based on specified criteria and plots
        individual CCG traces. Returns the DataFrame containing the filtered connections.
        
        Parameters:
            groupname (str or None): The group name to filter by; defaults to None.
            significance (bool): Filter connections by significance; defaults to True.
            EorI (str or None): Filter connections by excitatory/inhibitory status; defaults to None, which includes all statuses (e.g 'E', 'I', 'NS', 'S').
            connectiontype (str or None): Filter connections by type (e.g., 'FS->RS', 'RS->FS'); defaults to None, which includes all types.
        """
        # Access the filtered data
        df = self.access_ccgs(groupname=groupname, significance=significance, EorI=EorI, connectiontype=connectiontype)

        # Retrieve the 'ccgs' column which contains the CCG data arrays
        ccgs_data = df['ccgs'].tolist() # Convert the column to a list of arrays which represent the CCGs of each connection
        #calculate the mean of the ccgs
        time_points = self.ccg['t']  # assuming this is the correct key for time points array

        # Plotting each CCG
        fig, ax = plt.subplots(figsize=(10, 6))
        for ccg in ccgs_data:
            ax.plot(time_points, ccg)
        ax.set_title('CCGs Plot')
        ax.set_xlabel('Time (ms)')
        ax.set_ylabel('Counts')
        plt.show()
        
    def plot_mean_ccgs(self, groupname=None, significance=True, EorI=None, connectiontype=None):
        """
        Plots mean CCGs for connections based on specified criteria.
        
        Parameters:
            groupname (str or None): The group name to filter by; defaults to None.
            significance (bool): Filter connections by significance; defaults to True.
            EorI (str or None): Filter connections by excitatory/inhibitory status; defaults to None, which includes all statuses (e.g 'E', 'I', 'NS', 'S').
            connectiontype (str or None): Filter connections by type (e.g., 'FS->RS', 'RS->FS'); defaults to None, which includes all types.
        """
        # Access the filtered data
        df = self.access_ccgs(groupname=groupname, significance=significance, EorI=EorI, connectiontype=connectiontype)

        # Retrieve the 'ccgs' column which contains the CCG data arrays
        ccgs_data = df['ccgs'].tolist() # Convert the column to a list of arrays which represent the CCGs of each connection
        
        #calculate the mean of the ccgs
        mean_ccg = np.mean(ccgs_data, axis=0)
        #calculte the SEM of the ccgs
        sem_ccg = np.std(ccgs_data, axis=0) / np.sqrt(len(ccgs_data))

        time_points = self.ccg['t']  # assuming this is the correct key for time points array
        

        # Plotting the mean CCG
        fig, ax = plt.subplots(figsize=(10, 6)) #plot the mean CCG and shade the SEM region
        ax.plot(time_points, mean_ccg, color='b', label='Mean CCG')
        ax.fill_between(time_points, mean_ccg-sem_ccg, mean_ccg+sem_ccg, color='b', alpha=0.2, label='SEM')
        ax.set_title('Mean CCGs Plot')
        ax.set_xlabel('Time (ms)')
        ax.set_ylabel('Counts')
        plt.show()

    def plot_group_ccgs(self, significance=True, sd_sig=None, EorI=None, connectiontype=None, response=None, layers=None, min_spike_pairs=150, xlim=None, front_group='CTZ',limit=None, directory=None, file_name=None):
        
        
        """
        Plots overlaid mean CCGs for the 'No_CTZ' and 'CTZ' groups with their SEM as shaded regions.
        The SEM shading uses a lighter version of the group colors.

        Parameters:
            significance (bool): Filter connections by significance; defaults to True.
            sd_sig (bool): Filter connection based on 7*SD based on peak signal relative to noise of CCGs
            EorI (str or None): Filter connections by excitatory/inhibitory status; defaults to None, which includes all statuses.
            connectiontype (str or None): Filter connections by type (e.g., 'FS->RS', 'RS->FS'); defaults to None, which includes all types.
            response (str or None): Filters connections based on responsiveness; 'responsive' for either StimResp_A being 1 or 'nonresponsive' for both being 0.
            layers (list of str or None): Filter connections by layer (e.g., 'IG->L4', 'L4->L4'); defaults to None, which includes all layers.
            min_spike_pairs (int): Minimum number of spike pairs to include; defaults to 150.
            xlim (tuple of int or None): The x-axis limits for the plot; defaults to None.
            
        """
        group_colors = {
            'No_CTZ': '#797979',  # Grey color
            'CTZ': '#5a00c2'  # Purple color
        }

        # Define lighter versions of the colors for SEM shading
        sem_colors = {
            'No_CTZ': mcolors.to_rgba(group_colors['No_CTZ'], alpha=0.3),
            'CTZ': mcolors.to_rgba(group_colors['CTZ'], alpha=0.3)
        }

        fig, ax = plt.subplots(figsize=(5, 5)) #figsize is (width, height)
        time_points = self.ccg['t']  # assuming this is the correct key for time points array
        groups = ['No_CTZ', 'CTZ']
        # Ensure the front group is plotted last by sorting the list based on the front_group parameter
        groups = sorted(groups, key=lambda x: x == front_group)

        for group in groups:
            df = self.access_ccgs(groupname=group, significance=significance, sd_sig=sd_sig, EorI=EorI, connectiontype=connectiontype, response=response, layers=layers, min_spike_pairs=min_spike_pairs, limit=limit)
            ccgs_data = df['ccgs'].tolist()

            if len(ccgs_data) == 0:
                continue  # Skip if no data to plot
            
            mean_ccg = np.mean(ccgs_data, axis=0)
            sem_ccg = np.std(ccgs_data, axis=0) / np.sqrt(len(ccgs_data))

            # Plotting the mean CCG
            ax.plot(time_points, mean_ccg, color=group_colors[group], label=f'Mean CCG {group}', linewidth=2)
            ax.fill_between(time_points, mean_ccg - sem_ccg, mean_ccg + sem_ccg, color=sem_colors[group], label=f'SEM {group}')
            

        ax.set_title('Comparison of Mean CCGs by Group')
        ax.set_xlabel('Time (ms)')
        ax.set_ylabel('Counts')
        
        #change the x-axis limits if specified
        if xlim:
            ax.set_xlim(xlim)
        
        
        
        #hide legend
        ax.legend().set_visible(False)
        
        # Prompt user for directory and file name if not provided
        if directory is None:
            directory = input("Please enter the directory to save the plot: ")
        if file_name is None:
            file_name = input("Please enter the file name to save the plot: ")

        # Create directory if it does not exist
        os.makedirs(directory, exist_ok=True)

        # Save the figure as an SVG file in the specified directory
        file_path = os.path.join(directory, f'{file_name}.svg')
        plt.savefig(file_path, format='svg', transparent=True)
        
        plt.show()
    
    def prepare_for_boxplot_ccg(self, significance=True, EorI=None, connectiontype=None, response=None, layers=None, min_spike_pairs=150, xlim=None):
        """
        Organizes data into a DataFrame suitable for plotting boxplots by unique 'CellTypes' and group.
        Uses filtered data from 'access_ccgs' method based on provided parameters.
        
        Parameters:
            significance (bool): Filter connections by significance; defaults to True.
            EorI (str or None): Filter connections by excitatory/inhibitory status; defaults to None, which includes all statuses.
            connectiontype (str or None): Filter connections by type; defaults to None, which includes all types.
        
        Returns:
            pandas.DataFrame: A DataFrame with columns for the value to plot, 'CellTypes', and 'Group'.
        """
        boxplot_data = []

        # Access filtered data using the 'access_ccgs' method
        df = self.access_ccgs(significance=significance, EorI=EorI, connectiontype=connectiontype)

        if not df.empty:
            unique_cell_types = df['CellTypes'].unique()  # Get unique 'CellTypes'
            for cell_type in unique_cell_types:
                # Filter DataFrame for each cell type
                cell_type_df = df[df['CellTypes'] == cell_type]
                
                # Collect data for each cell type entry
                for index, row in cell_type_df.iterrows():
                    boxplot_data.append({
                        'Value': row['ccgs'],  # Adjust this key as needed
                        'CellTypes': cell_type,
                        'Group': row['group']  # Assuming 'group' column exists
                    })

        # Convert list of data to DataFrame
        boxplot_df = pd.DataFrame(boxplot_data)
        return boxplot_df

    def plot_group_ccgs_traces(self, significance=True, sd_sig=None, EorI=None, connectiontype=None, response=None, layers=None, min_spike_pairs=150, xlim=None, front_group='CTZ', limit=None):
        """
        Plots overlaid mean CCGs for the 'No_CTZ' and 'CTZ' groups along with individual CCG traces.

        Parameters:
            significance (bool): Filter connections by significance; defaults to True.
            sd_sig (bool): Filter connection based on 7*SD based on peak signal relative to noise of CCGs
            EorI (str or None): Filter connections by excitatory/inhibitory status; defaults to None, which includes all statuses.
            connectiontype (str or None): Filter connections by type (e.g., 'FS->RS', 'RS->FS'); defaults to None, which includes all types.
            response (str or None): Filters connections based on responsiveness; 'responsive' for either StimResp_A being 1 or 'nonresponsive' for both being 0.
            layers (list of str or None): Filter connections by layer (e.g., 'IG->L4', 'L4->L4'); defaults to None, which includes all layers.
            min_spike_pairs (int): Minimum number of spike pairs to include; defaults to 150.
            xlim (tuple of int or None): The x-axis limits for the plot; defaults to None.
        """
        group_colors = {
            'No_CTZ': '#797979',  # Grey color
            'CTZ': '#5a00c2'  # Purple color
        }

        fig, ax = plt.subplots(figsize=(5, 5))  # figsize is (width, height)
        time_points = self.ccg['t']  # assuming this is the correct key for time points array
        groups = ['No_CTZ', 'CTZ']
        # Ensure the front group is plotted last by sorting the list based on the front_group parameter
        groups = sorted(groups, key=lambda x: x == front_group)

        for group in groups:
            df = self.access_ccgs(groupname=group, significance=significance, sd_sig=sd_sig, EorI=EorI, connectiontype=connectiontype, response=response, layers=layers, min_spike_pairs=min_spike_pairs, limit=limit)
            ccgs_data = df['ccgs'].tolist()

            if len(ccgs_data) == 0:
                continue  # Skip if no data to plot
            
            mean_ccg = np.mean(ccgs_data, axis=0)

            # Plotting the mean CCG
            ax.plot(time_points, mean_ccg, color=group_colors[group], label=f'Mean CCG {group}', linewidth=3)

            # Plotting individual traces
            for trace in ccgs_data:
                ax.plot(time_points, trace, color=group_colors[group], alpha=0.3, linewidth=0.5)  # Lower alpha for individual traces

        ax.set_title('Comparison of Mean CCGs by Group with Individual Traces')
        ax.set_xlabel('Time (ms)')
        ax.set_ylabel('Counts')

        # Change the x-axis limits if specified
        if xlim:
            ax.set_xlim(xlim)

        # Hide legend
        ax.legend().set_visible(False)
        plt.show()

    def plot_box_and_strip_ccg(self, groups=None, cell_types=None, show_outliers=True, hue_order=None):
        """
        Plots boxplots and stripplots for specified groups and cell types, with color adjustments made directly in the plotting calls.

        Args:
            groups (list of str, optional): List of groups to include in the plot.
            cell_types (list of str, optional): List of cell types to include in the plot.
            show_outliers (bool, optional): Whether to show outliers.
            hue_order (list, optional): Order of the hue levels.
        """
        # Define color mapping for groups
        group_colors = {
            'No_CTZ': '#797979',
            'CTZ': '#5a00c2'
        }

        # Generate lighter versions for the box face color
        lightened_colors = {k: mcolors.to_rgba(v, alpha=0.5) for k, v in group_colors.items()}

        # Boxplot customization
        boxprops = {'edgecolor': 'k', 'linewidth': 2}
        whiskerprops = {'color': 'k', 'linewidth': 2}
        boxplot_kwargs = {
            'boxprops': boxprops,
            'medianprops': whiskerprops,
            'whiskerprops': whiskerprops,
            'capprops': {'linewidth': 0},  # Hide the caps
            'showfliers': show_outliers,
            'palette': group_colors,
            'hue_order': hue_order,
            'width': 0.75
        }

        # Stripplot customization
        stripplot_kwargs = {
            'linewidth': 0.6,
            'size': 6,
            'alpha': 0.7,
            'jitter': True,
            'dodge': True,
            'marker': 'o' if show_outliers else 'd',
            'palette': lightened_colors,
            'hue_order': hue_order
        }

        # Prepare data for boxplot
        boxplot_df = self.prepare_for_boxplot_ccg()

        # Filter by specified groups and cell types
        if groups:
            boxplot_df = boxplot_df[boxplot_df['Group'].isin(groups)]
        if cell_types:
            boxplot_df = boxplot_df[boxplot_df['CellTypes'].isin(cell_types)]

        # Plotting
        plt.figure(figsize=(12, 6))
        ax = sns.boxplot(data=boxplot_df, x='CellTypes', y='Value', hue='Group', **boxplot_kwargs)

        # Manually set the facecolor for boxplot
        for i, artist in enumerate(ax.artists):
            col = lightened_colors[ax.get_legend_handles_labels()[1][i // len(cell_types)]]
            artist.set_facecolor(col)

        # Add stripplot on top of boxplot for raw data visualization
        sns.stripplot(data=boxplot_df, x='CellTypes', y='Value', hue='Group', **stripplot_kwargs)

        # Enhance the plot
        plt.title('Comparison of Values Across Groups and Cell Types')
        plt.ylabel('Value')
        plt.xlabel('Cell Type')
        ax.legend(title='Group')
        plt.show()
        
    def is_significant_ccg(self, sd_multiplier=7):
        # Access the time array
        t = self.ccg['t']
        
        # Create an empty list to store significance results temporarily
        significance_results = []

        # Iterate over all CCG entries
        for ccg_values in self.ccg['MonoConnectionsTable']['ccgs']:
            # Define the noise distribution from the flanks of the CCG
            noise_mask = (np.abs(t) >= 50) & (np.abs(t) <= 100)
            noise_values = ccg_values[noise_mask]

            # Calculate the mean and standard deviation of the noise distribution
            noise_mean = np.mean(noise_values)
            noise_std = np.std(noise_values)

            # Define the peak detection region within ±10ms of zero
            peak_mask = (t >= -10) & (t <= 10)
            peak_values = ccg_values[peak_mask]

            # Find the peak value in the defined region
            peak = np.max(peak_values)

            # Check if the peak is more than the specified multiplier of standard deviations above the noise mean
            significant = peak > noise_mean + sd_multiplier * noise_std

            # Append the significance result
            significance_results.append(significant)

        # Safely add the 'sd_sig' column to the DataFrame
        self.ccg['MonoConnectionsTable']['sd_sig'] = significance_results

        return self.ccg['MonoConnectionsTable']['sd_sig']
    
    def plot_group_excess_synchrony(self, significance=True, sd_sig=None, EorI=None, connectiontype=None, response=None, layers=None, min_spike_pairs=150,limit=None, directory=None, file_name=None, front_group='CTZ'):

        group_colors = {
            'No_CTZ': '#797979',  # Grey color
            'CTZ': '#5a00c2'  # Purple color
        }

        # Define lighter versions of the colors for SEM shading
        sem_colors = {
            'No_CTZ': mcolors.to_rgba(group_colors['No_CTZ'], alpha=0.3),
            'CTZ': mcolors.to_rgba(group_colors['CTZ'], alpha=0.3)
        }

        fig, ax = plt.subplots(figsize=(5, 5))  # figsize is (width, height)
        groups = ['No_CTZ', 'CTZ']
        # Ensure the front group is plotted last by sorting the list based on the front_group parameter
        groups = sorted(groups, key=lambda x: x == front_group)

        for group in groups:
            df = self.access_ccgs(groupname=group, significance=significance, sd_sig=sd_sig, EorI=EorI, connectiontype=connectiontype, response=response, layers=layers, min_spike_pairs=min_spike_pairs, limit=limit)
            excess_synchrony_data = df['ExcessSynchrony'].tolist()

            if len(excess_synchrony_data) == 0:
                continue  # Skip if no data to plot

            mean_excess_synchrony = np.mean(excess_synchrony_data)
            sem_excess_synchrony = np.std(excess_synchrony_data) / np.sqrt(len(excess_synchrony_data))

            # Plotting the mean excess synchrony
            ax.bar(group, mean_excess_synchrony, yerr=sem_excess_synchrony, color=group_colors[group], alpha=0.7, label=f'{group}')

        ax.set_title('Comparison of Mean Excess Synchrony by Group')
        ax.set_ylabel('Excess Synchrony')

        # Prompt user for directory and file name if not provided
        if directory is None:
            directory = input("Please enter the directory to save the plot: ")
        if file_name is None:
            file_name = input("Please enter the file name to save the plot: ")

        # Create directory if it does not exist
        os.makedirs(directory, exist_ok=True)

        # Save the figure as an SVG file in the specified directory
        file_path = os.path.join(directory, f'{file_name}.svg')
        plt.savefig(file_path, format='svg', transparent=True)

        plt.show()
    
    def compare_group_excess_synchrony(self, significance=True, sd_sig=None, EorI=None, connectiontype=None, response=None, layers=None, min_spike_pairs=150):
        # Filter data for No_CTZ group
        no_ctz_df = self.access_ccgs(groupname='No_CTZ', significance=significance, sd_sig=sd_sig, EorI=EorI, connectiontype=connectiontype, response=response, layers=layers, min_spike_pairs=min_spike_pairs)
        no_ctz_excess_synchrony = no_ctz_df['ExcessSynchrony'].apply(pd.to_numeric, errors='coerce').dropna()

        # Filter data for CTZ group
        ctz_df = self.access_ccgs(groupname='CTZ', significance=significance, sd_sig=sd_sig, EorI=EorI, connectiontype=connectiontype, response=response, layers=layers, min_spike_pairs=min_spike_pairs)
        ctz_excess_synchrony = ctz_df['ExcessSynchrony'].apply(pd.to_numeric, errors='coerce').dropna()

        # Summary statistics
        no_ctz_summary = no_ctz_excess_synchrony.describe()
        ctz_summary = ctz_excess_synchrony.describe()

        # Perform Mann-Whitney U test
        stat, p_value = mannwhitneyu(no_ctz_excess_synchrony, ctz_excess_synchrony)

        # Print summary statistics
        print("No_CTZ Summary Statistics:")
        print(no_ctz_summary)
        print("\nCTZ Summary Statistics:")
        print(ctz_summary)

        # Print Mann-Whitney U test results
        print("\nMann-Whitney U Test Results:")
        print(f"Statistic: {stat}")
        print(f"P-Value: {p_value}")

        # Return results for further use if needed
        return {
            'no_ctz_excess_synchrony': no_ctz_excess_synchrony, #returns the filtered data for No_CTZ group
            'ctz_excess_synchrony': ctz_excess_synchrony, #returns the filtered data for CTZ group
            'no_ctz_summary': no_ctz_summary,
            'ctz_summary': ctz_summary,
            'mannwhitneyu_stat': stat,
            'mannwhitneyu_p_value': p_value
        }

    def compare_group_excess_synchrony_welch(self, significance=True, sd_sig=None, EorI=None, connectiontype=None, response=None, layers=None, min_spike_pairs=150):
        no_ctz_df = self.access_ccgs(groupname='No_CTZ', significance=significance, sd_sig=sd_sig, EorI=EorI, connectiontype=connectiontype, response=response, layers=layers, min_spike_pairs=min_spike_pairs)
        no_ctz_excess_synchrony = no_ctz_df['ExcessSynchrony'].apply(pd.to_numeric, errors='coerce').dropna()

        ctz_df = self.access_ccgs(groupname='CTZ', significance=significance, sd_sig=sd_sig, EorI=EorI, connectiontype=connectiontype, response=response, layers=layers, min_spike_pairs=min_spike_pairs)
        ctz_excess_synchrony = ctz_df['ExcessSynchrony'].apply(pd.to_numeric, errors='coerce').dropna()

        no_ctz_summary = no_ctz_excess_synchrony.describe()
        ctz_summary = ctz_excess_synchrony.describe()

        stat, p_value = ttest_ind(no_ctz_excess_synchrony, ctz_excess_synchrony, equal_var=False)

        print("No_CTZ Summary Statistics:")
        print(no_ctz_summary)
        print("\nCTZ Summary Statistics:")
        print(ctz_summary)

        print("\nWelch's t-test Results:")
        print(f"Statistic: {stat}")
        print(f"P-Value: {p_value}")

        return {
            'no_ctz_summary': no_ctz_summary,
            'ctz_summary': ctz_summary,
            'ttest_stat': stat,
            'ttest_p_value': p_value
        }

    def compare_group_excess_synchrony_ks(self, significance=True, sd_sig=None, EorI=None, connectiontype=None, response=None, layers=None, min_spike_pairs=150):
        from scipy.stats import ks_2samp
        no_ctz_df = self.access_ccgs(groupname='No_CTZ', significance=significance, sd_sig=sd_sig, EorI=EorI, connectiontype=connectiontype, response=response, layers=layers, min_spike_pairs=min_spike_pairs)
        no_ctz_excess_synchrony = no_ctz_df['ExcessSynchrony'].apply(pd.to_numeric, errors='coerce').dropna()

        ctz_df = self.access_ccgs(groupname='CTZ', significance=significance, sd_sig=sd_sig, EorI=EorI, connectiontype=connectiontype, response=response, layers=layers, min_spike_pairs=min_spike_pairs)
        ctz_excess_synchrony = ctz_df['ExcessSynchrony'].apply(pd.to_numeric, errors='coerce').dropna()

        no_ctz_summary = no_ctz_excess_synchrony.describe()
        ctz_summary = ctz_excess_synchrony.describe()

        stat, p_value = ks_2samp(no_ctz_excess_synchrony, ctz_excess_synchrony)

        print("No_CTZ Summary Statistics:")
        print(no_ctz_summary)
        print("\nCTZ Summary Statistics:")
        print(ctz_summary)

        print("\nKolmogorov-Smirnov Test Results:")
        print(f"Statistic: {stat}")
        print(f"P-Value: {p_value}")

        return {
            'no_ctz_summary': no_ctz_summary,
            'ctz_summary': ctz_summary,
            'ks_stat': stat,
            'ks_p_value': p_value
        }
        
    def permutation_test(self, data1, data2, num_permutations=10000):
        # Calculate the observed difference in means
        observed_diff = np.mean(data1) - np.mean(data2)
        
        # Combine the data
        combined_data = np.concatenate([data1, data2])
        
        # Perform permutations
        perm_diffs = np.zeros(num_permutations)
        for i in range(num_permutations):
            np.random.shuffle(combined_data)
            perm_data1 = combined_data[:len(data1)]
            perm_data2 = combined_data[len(data1):]
            perm_diffs[i] = np.mean(perm_data1) - np.mean(perm_data2)
        
        # Calculate the p-value
        p_value = np.sum(np.abs(perm_diffs) >= np.abs(observed_diff)) / num_permutations
        
        return observed_diff, p_value, perm_diffs

    def compare_group_excess_synchrony_permutation(self, significance=True, sd_sig=None, EorI=None, connectiontype=None, response=None, layers=None, min_spike_pairs=150, num_permutations=10000, limit=None, save_path=None):
        """
        Compares group excess synchrony between No_CTZ and CTZ groups using a permutation test,
        and saves the results as a DataFrame. Handles cases where one group may have no data.

        Parameters:
            significance (bool): Filter connections by significance.
            sd_sig (bool): Filter connections based on SD significance.
            EorI (list of str): Filter connections by excitatory/inhibitory status.
            connectiontype (str): Filter connections by type.
            response (str): Filter connections by responsiveness.
            layers (list of str): Filter connections by layers.
            min_spike_pairs (int): Minimum number of spike pairs.
            num_permutations (int): Number of permutations for the permutation test.
            limit (bool): Filter connections by threshold.
            save_path (str): Path to save the results as a CSV file.

        Returns:
            DataFrame: A DataFrame containing summary statistics and permutation test results.
        """

        # Filter data for No_CTZ group
        no_ctz_df = self.access_ccgs(groupname='No_CTZ', significance=significance, sd_sig=sd_sig, EorI=EorI, connectiontype=connectiontype, response=response, layers=layers, min_spike_pairs=min_spike_pairs, limit=limit)
        no_ctz_excess_synchrony = no_ctz_df['ExcessSynchrony'].apply(pd.to_numeric, errors='coerce').dropna().values

        # Filter data for CTZ group
        ctz_df = self.access_ccgs(groupname='CTZ', significance=significance, sd_sig=sd_sig, EorI=EorI, connectiontype=connectiontype, response=response, layers=layers, min_spike_pairs=min_spike_pairs, limit=limit)
        ctz_excess_synchrony = ctz_df['ExcessSynchrony'].apply(pd.to_numeric, errors='coerce').dropna().values

        # Check if either group has no data
        if len(no_ctz_excess_synchrony) == 0:
            no_ctz_summary = pd.Series([None] * 8, index=['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max'])
        else:
            no_ctz_summary = pd.Series(no_ctz_excess_synchrony).describe()

        if len(ctz_excess_synchrony) == 0:
            ctz_summary = pd.Series([None] * 8, index=['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max'])
        else:
            ctz_summary = pd.Series(ctz_excess_synchrony).describe()

        # Perform the permutation test only if both groups have data
        if len(no_ctz_excess_synchrony) > 0 and len(ctz_excess_synchrony) > 0:
            observed_diff, p_value, perm_diffs = self.permutation_test(no_ctz_excess_synchrony, ctz_excess_synchrony, num_permutations)
        else:
            observed_diff = None
            p_value = None
            perm_diffs = []

        # Ensure all lists are the same length or adjust appropriately
        stats_labels = ['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']
        no_ctz_values = no_ctz_summary.values
        ctz_values = ctz_summary.values

        # Create a DataFrame to store the summary statistics
        results_df = pd.DataFrame({
            'Statistic': stats_labels,
            'No_CTZ': no_ctz_values,
            'CTZ': ctz_values
        })

        # Add the permutation test results to the DataFrame
        perm_results_df = pd.DataFrame({
            'Statistic': ['Observed Difference', 'P-Value'],
            'No_CTZ': [observed_diff, None],  # Observed difference applies to both groups, but we'll store it under No_CTZ
            'CTZ': [None, p_value]  # P-Value is only meaningful for the comparison
        })

        # Combine the results into a single DataFrame
        results_df = pd.concat([results_df, perm_results_df], ignore_index=True)

        # Save the DataFrame to a CSV file if a save path is provided
        if save_path:
            results_df.to_csv(save_path, index=False)

        # Return the DataFrame
        return results_df

    def compare_group_excess_synchrony_permutation_quartile(self, significance=True, sd_sig=None, EorI=None, connectiontype=None, response=None, layers=None, min_spike_pairs=150, num_permutations=10000, quartile='top', limit=None):
        # Filter data for No_CTZ group
        no_ctz_df = self.access_ccgs(groupname='No_CTZ', significance=significance, sd_sig=sd_sig, EorI=EorI, connectiontype=connectiontype, response=response, layers=layers, min_spike_pairs=min_spike_pairs, limit=limit)
        no_ctz_excess_synchrony = no_ctz_df['ExcessSynchrony'].apply(pd.to_numeric, errors='coerce').dropna()

        # Filter data for CTZ group
        ctz_df = self.access_ccgs(groupname='CTZ', significance=significance, sd_sig=sd_sig, EorI=EorI, connectiontype=connectiontype, response=response, layers=layers, min_spike_pairs=min_spike_pairs, limit=limit)
        ctz_excess_synchrony = ctz_df['ExcessSynchrony'].apply(pd.to_numeric, errors='coerce').dropna()

        # Calculate quartiles
        no_ctz_quartiles = no_ctz_excess_synchrony.quantile([0.25, 0.5, 0.75])
        ctz_quartiles = ctz_excess_synchrony.quantile([0.25, 0.5, 0.75])

        # Filter by specified quartile
        if quartile == 'top':
            no_ctz_filtered = no_ctz_excess_synchrony[no_ctz_excess_synchrony >= no_ctz_quartiles[0.75]]
            ctz_filtered = ctz_excess_synchrony[ctz_excess_synchrony >= ctz_quartiles[0.75]]
        elif quartile == 'bottom':
            no_ctz_filtered = no_ctz_excess_synchrony[no_ctz_excess_synchrony <= no_ctz_quartiles[0.25]]
            ctz_filtered = ctz_excess_synchrony[ctz_excess_synchrony <= ctz_quartiles[0.25]]
        elif quartile == 'middle':
            no_ctz_filtered = no_ctz_excess_synchrony[(no_ctz_excess_synchrony > no_ctz_quartiles[0.25]) & (no_ctz_excess_synchrony < no_ctz_quartiles[0.75])]
            ctz_filtered = ctz_excess_synchrony[(ctz_excess_synchrony > ctz_quartiles[0.25]) & (ctz_excess_synchrony < ctz_quartiles[0.75])]
        else:
            no_ctz_filtered = no_ctz_excess_synchrony
            ctz_filtered = ctz_excess_synchrony

        no_ctz_summary = no_ctz_filtered.describe()
        ctz_summary = ctz_filtered.describe()

        # Perform the permutation test
        observed_diff, p_value, perm_diffs = self.permutation_test(no_ctz_filtered, ctz_filtered, num_permutations)

        # Print summary statistics
        print("No_CTZ Summary Statistics:")
        print(no_ctz_summary)
        print("\nCTZ Summary Statistics:")
        print(ctz_summary)

        # Print Permutation Test results
        print("\nPermutation Test Results:")
        print(f"Observed Difference: {observed_diff}")
        print(f"P-Value: {p_value}")

        # Return results for further use if needed
        return {
            'no_ctz_summary': no_ctz_summary,
            'ctz_summary': ctz_summary,
            'observed_diff': observed_diff,
            'perm_diffs': perm_diffs,
            'perm_p_value': p_value
        }

    def run_permutation_test_between_groups(self, band_name, condition, quantile_low=None, quantile_high=None, cell_type=None, cell_type2=None, n_permutations=10000):
        """
        Runs a permutation test to compare the PPC2 values between groups for a specified frequency band.

        Parameters:
        band_name (str): The name of the frequency band to filter by (e.g., 'alpha', 'beta').
        condition (bool): The condition to filter by (True for evoked, False for spontaneous).
        quantile_low (float): The lower quantile threshold for filtering.
        quantile_high (float): The upper quantile threshold for filtering.
        cell_type (str): The cell type to filter by.
        cell_type2 (int): The secondary cell type to filter by.
        n_permutations (int): The number of permutations to perform.

        Returns:
        float: The p-value from the permutation test.
        """
        filtered_df = self.filter_by_frequency_band(band_name)

        # Filter by condition
        filtered_df = filtered_df[filtered_df['Condition'] == condition]

        # Filter by cell type if specified
        if cell_type:
            filtered_df = filtered_df[filtered_df['CellType'] == cell_type]
        if cell_type2 is not None:
            filtered_df = filtered_df[filtered_df['CellType2'] == cell_type2]

        # Apply quantile filtering if specified
        if quantile_low is not None and quantile_high is not None:
            filtered_df = self.filter_data_by_quantile(filtered_df, quantile_low, quantile_high)

        # Aggregate PPC2 values per CellID and frequency band midpoint
        aggregated_df = filtered_df.groupby(['CellID', 'Group']).agg({'PPC2': 'mean'}).reset_index()

        # Separate the data by groups
        df_ctz = aggregated_df[aggregated_df['Group'] == 'CTZ']['PPC2']
        df_no_ctz = aggregated_df[aggregated_df['Group'] == 'No_CTZ']['PPC2']

        # Calculate observed difference in means
        observed_diff = np.mean(df_ctz) - np.mean(df_no_ctz)

        # Concatenate the data for permutation
        all_data = np.concatenate([df_ctz, df_no_ctz])
        n_ctz = len(df_ctz)
        n_no_ctz = len(df_no_ctz)

        # Perform permutations
        perm_diffs = np.zeros(n_permutations)
        for i in range(n_permutations):
            np.random.shuffle(all_data)
            perm_ctz = all_data[:n_ctz]
            perm_no_ctz = all_data[n_ctz:]
            perm_diffs[i] = np.mean(perm_ctz) - np.mean(perm_no_ctz)

        # Calculate p-value
        p_value = np.mean(np.abs(perm_diffs) >= np.abs(observed_diff))
        return p_value
    
    def filter_by_ccg_threshold(self, time_range, threshold_range):
        """
        Filters CCGs based on whether their values exceed a given threshold within a specified time range.
        Adds a new column 'threshold' indicating True or False based on the condition.

        Parameters:
            time_range (int or tuple): An integer specifying a single time point, or a tuple specifying the start and end of the time range to check (e.g., (-5, 0) or (0, 1)).
            threshold_range (tuple): A tuple specifying the lower and upper bounds of the threshold (e.g., (0, 7)).

        Returns:
            pd.DataFrame: The updated DataFrame with a new 'threshold' column.
        """

        # Ensure 'MonoConnectionsTable' exists in 'self.ccg'
        if 'MonoConnectionsTable' not in self.ccg:
            raise KeyError("'MonoConnectionsTable' not found in 'self.ccg'")

        # Extract the MonoConnectionsTable DataFrame
        df = self.ccg['MonoConnectionsTable'].copy()

        # Ensure 'ccgs' and 't' exist in 'self.ccg'
        if 'ccgs' not in df or 't' not in self.ccg:
            raise KeyError("'ccgs' or 't' not found in 'self.ccg'")

        # Extract time points and CCG values
        time_points = self.ccg['t']
        ccgs = df['ccgs']

        # Convert time points to integers for ease of indexing
        time_points = time_points.astype(int)

        # Determine the time range
        if isinstance(time_range, int):
            start_time = end_time = time_range
        else:
            start_time, end_time = time_range

        # Find the indices of the time range
        start_idx = (time_points >= start_time).argmax()
        end_idx = (time_points <= end_time).argmin() + 1

        # Ensure the indices are valid
        if start_idx > end_idx:
            raise ValueError("Invalid time range specified")

        # Initialize the 'threshold' column with False
        df['threshold'] = False

        # Unpack the threshold range
        lower_threshold, upper_threshold = threshold_range

        # Check CCG values within the specified time and threshold ranges
        for index, row in df.iterrows():
            ccg_values = ccgs.iloc[index][start_idx:end_idx]
            if any((ccg_values > lower_threshold) & (ccg_values < upper_threshold)):
                df.at[index, 'threshold'] = True

        # Update the MonoConnectionsTable in self.ccg
        self.ccg['MonoConnectionsTable'] = df

        return df

    def nullify_specific_rows(self, indices):
        """
        Sets all values of specific rows in 'MonoConnectionsTable' to NaN based on the provided indices.
        
        Parameters:
            indices (list): List of indices of the rows to be nullified.
        
        Returns:
            pd.DataFrame: The updated DataFrame with the specified rows set to NaN.
        """
        # Ensure 'MonoConnectionsTable' exists in 'self.ccg'
        if 'MonoConnectionsTable' not in self.ccg:
            raise KeyError("'MonoConnectionsTable' not found in 'self.ccg'")

        # Extract the MonoConnectionsTable DataFrame
        df = self.ccg['MonoConnectionsTable']

        # Check if indices are valid
        invalid_indices = [i for i in indices if i not in df.index]
        if invalid_indices:
            raise KeyError(f"Invalid indices: {invalid_indices}")

        # Set specified rows to NaN
        df.loc[indices] = float('nan')

        # Update the MonoConnectionsTable in self.ccg
        self.ccg['MonoConnectionsTable'] = df

        return df

    def plot_group_excess_synchrony_with_dots(self, significance=True, sd_sig=None, EorI=None, connectiontype=None, response=None, layers=None, min_spike_pairs=150, limit=None, directory=None, file_name=None, front_group='CTZ'):
        """
        Plots the mean excess synchrony by group with SEM as error bars and overlays the underlying data as color-matched dots.

        Parameters:
            significance (bool): Filter connections by significance.
            sd_sig (bool): Filter connection based on 7*SD based on peak signal relative to noise of CCGs.
            EorI (list of str): Filter connections by excitatory/inhibitory status.
            connectiontype (str): Filter connections by type (e.g., 'FS->RS').
            response (str): Filter connections based on responsiveness.
            layers (list of str): Filter connections by layer.
            min_spike_pairs (int): Minimum number of spike pairs to include.
            limit (int): Limit the number of connections to include.
            directory (str): Directory to save the plot.
            file_name (str): File name to save the plot.
            front_group (str): Group to be plotted in front.
        """
        group_colors = {
            'No_CTZ': '#797979',  # Grey color
            'CTZ': '#5a00c2'  # Purple color
        }

        # Define lighter versions of the colors for SEM shading
        sem_colors = {
            'No_CTZ': mcolors.to_rgba(group_colors['No_CTZ'], alpha=0.3),
            'CTZ': mcolors.to_rgba(group_colors['CTZ'], alpha=0.3)
        }

        fig, ax = plt.subplots(figsize=(5, 5))  # figsize is (width, height)
        groups = ['No_CTZ', 'CTZ']
        # Ensure the front group is plotted last by sorting the list based on the front_group parameter
        groups = sorted(groups, key=lambda x: x == front_group)

        for group in groups:
            df = self.access_ccgs(groupname=group, significance=significance, sd_sig=sd_sig, EorI=EorI, connectiontype=connectiontype, response=response, layers=layers, min_spike_pairs=min_spike_pairs, limit=limit)
            excess_synchrony_data = df['ExcessSynchrony'].tolist()

            if len(excess_synchrony_data) == 0:
                continue  # Skip if no data to plot

            mean_excess_synchrony = np.mean(excess_synchrony_data)
            sem_excess_synchrony = np.std(excess_synchrony_data) / np.sqrt(len(excess_synchrony_data))

            # Plotting the mean excess synchrony
            ax.bar(group, mean_excess_synchrony, yerr=sem_excess_synchrony, color=group_colors[group], alpha=0.3, label=f'{group}')
            
            # Overlaying the underlying data as dots
            x_positions = np.full(len(excess_synchrony_data), group)  # x-positions are the same for each group
            ax.scatter(x_positions, excess_synchrony_data, color=group_colors[group], alpha=0.7, edgecolor=None)

        ax.set_title('Comparison of Mean Excess Synchrony by Group')
        ax.set_ylabel('Excess Synchrony')
        ax.legend()

        # Prompt user for directory and file name if not provided
        if directory is None:
            directory = input("Please enter the directory to save the plot: ")
        if file_name is None:
            file_name = input("Please enter the file name to save the plot: ")

        # Create directory if it does not exist
        os.makedirs(directory, exist_ok=True)

        # Save the figure as an SVG file in the specified directory
        file_path = os.path.join(directory, f'{file_name}.svg')
        plt.savefig(file_path, format='svg', transparent=True)

        plt.show()

    def plot_group_DCW_with_dots(self, significance=True, sd_sig=None, EorI=None, connectiontype=None, response=None, layers=None, min_spike_pairs=150, limit=None, directory=None, file_name=None, front_group='CTZ'):
            """
            Plots the mean excess synchrony by group with SEM as error bars and overlays the underlying data as color-matched dots.

            Parameters:
                significance (bool): Filter connections by significance.
                sd_sig (bool): Filter connection based on 7*SD based on peak signal relative to noise of CCGs.
                EorI (list of str): Filter connections by excitatory/inhibitory status.
                connectiontype (str): Filter connections by type (e.g., 'FS->RS').
                response (str): Filter connections based on responsiveness.
                layers (list of str): Filter connections by layer.
                min_spike_pairs (int): Minimum number of spike pairs to include.
                limit (int): Limit the number of connections to include.
                directory (str): Directory to save the plot.
                file_name (str): File name to save the plot.
                front_group (str): Group to be plotted in front.
            """
            group_colors = {
                'No_CTZ': '#797979',  # Grey color
                'CTZ': '#5a00c2'  # Purple color
            }

            # Define lighter versions of the colors for SEM shading
            sem_colors = {
                'No_CTZ': mcolors.to_rgba(group_colors['No_CTZ'], alpha=0.3),
                'CTZ': mcolors.to_rgba(group_colors['CTZ'], alpha=0.3)
            }

            fig, ax = plt.subplots(figsize=(5, 5))  # figsize is (width, height)
            groups = ['No_CTZ', 'CTZ']
            # Ensure the front group is plotted last by sorting the list based on the front_group parameter
            groups = sorted(groups, key=lambda x: x == front_group)

            for group in groups:
                df = self.access_ccgs(groupname=group, significance=significance, sd_sig=sd_sig, EorI=EorI, connectiontype=connectiontype, response=response, layers=layers, min_spike_pairs=min_spike_pairs, limit=limit)
                excess_synchrony_data = df['DCW'].tolist()

                if len(excess_synchrony_data) == 0:
                    continue  # Skip if no data to plot

                mean_excess_synchrony = np.mean(excess_synchrony_data)
                sem_excess_synchrony = np.std(excess_synchrony_data) / np.sqrt(len(excess_synchrony_data))

                # Plotting the mean excess synchrony
                ax.bar(group, mean_excess_synchrony, yerr=sem_excess_synchrony, color=group_colors[group], alpha=0.3, label=f'{group}')
                
                # Overlaying the underlying data as dots
                x_positions = np.full(len(excess_synchrony_data), group)  # x-positions are the same for each group
                ax.scatter(x_positions, excess_synchrony_data, color=group_colors[group], alpha=0.7, edgecolor=None)

            ax.set_title('Comparison of DCW by Group')
            ax.set_ylabel('DCW')
            ax.legend()

            # Prompt user for directory and file name if not provided
            if directory is None:
                directory = input("Please enter the directory to save the plot: ")
            if file_name is None:
                file_name = input("Please enter the file name to save the plot: ")

            # Create directory if it does not exist
            os.makedirs(directory, exist_ok=True)

            # Save the figure as an SVG file in the specified directory
            file_path = os.path.join(directory, f'{file_name}.svg')
            plt.savefig(file_path, format='svg')

            plt.show()

    def compare_group_DCW_permutation(self, significance=True, sd_sig=None, EorI=None, connectiontype=None, response=None, layers=None, min_spike_pairs=150, num_permutations=10000, limit=None):
        # Filter data for No_CTZ group
        no_ctz_df = self.access_ccgs(groupname='No_CTZ', significance=significance, sd_sig=sd_sig, EorI=EorI, connectiontype=connectiontype, response=response, layers=layers, min_spike_pairs=min_spike_pairs, limit=limit)
        no_ctz_excess_synchrony = no_ctz_df['DCW'].apply(pd.to_numeric, errors='coerce').dropna().values

        # Filter data for CTZ group
        ctz_df = self.access_ccgs(groupname='CTZ', significance=significance, sd_sig=sd_sig, EorI=EorI, connectiontype=connectiontype, response=response, layers=layers, min_spike_pairs=min_spike_pairs, limit=limit)
        ctz_excess_synchrony = ctz_df['DCW'].apply(pd.to_numeric, errors='coerce').dropna().values

        no_ctz_summary = pd.Series(no_ctz_excess_synchrony).describe()
        ctz_summary = pd.Series(ctz_excess_synchrony).describe()

        # Perform the permutation test
        observed_diff, p_value, perm_diffs = self.permutation_test(no_ctz_excess_synchrony, ctz_excess_synchrony, num_permutations)

        # Print summary statistics
        print("No_CTZ Summary Statistics:")
        print(no_ctz_summary)
        print("\nCTZ Summary Statistics:")
        print(ctz_summary)

        # Print Permutation Test results
        print("\nPermutation Test Results:")
        print(f"Observed Difference: {observed_diff}")
        print(f"P-Value: {p_value}")

        # Return results for further use if needed
        return {
            'no_ctz_summary': no_ctz_summary,
            'ctz_summary': ctz_summary,
            'observed_diff': observed_diff,
            'perm_diffs': perm_diffs,
            'perm_p_value': p_value
        }

In [ ]:
import mat73
import os

class PhaseDataHandler:
    
    FREQUENCY_BANDS = {
        'alpha': (8, 15),
        'beta': (15, 30),
        'gamma': (30, 55),
        'high_gamma': (66, 120)
    }
    
    def __init__(self, eed_object, phase_directory):
        self.eed = eed_object
        self.phase_directory = phase_directory
        self.phase = {}
        self.load_phase_files()
        self.unpack_phase_data()  # This will now directly update self.phase
    
    def load_phase_files(self):
        """
        Loads all required .mat files from the specified CCG directory and sets them as attributes under the `ccg` dictionary.

        Required Files:
            
        The data from each file is accessed by stripping the '.mat' and accessing the corresponding key in the loaded dictionary, except for specific exceptions noted.
        """
        required_files = [
            'phase_data_out.mat'
        ]

        # Check if the directory exists
        if not os.path.exists(self.phase_directory):
            print(f"Phase directory not found: {self.phase_directory}")
            return

        # Load each required .mat file from the directory
        for file_name in required_files:
            file_path = os.path.join(self.phase_directory, file_name)
            print(f"Loading Phase data from: {file_path}")
            try:
                # Load the file and extract the data using the base filename as the key
                data = mat73.loadmat(file_path)
                base_key = file_name.replace('.mat', '')
                # Handle special case for 'MonoConnectionsTable'
                if base_key == 'phase_data_out':
                    self.phase[base_key] = data['phase_data_out']
                elif base_key in data:
                    self.phase[base_key] = data[base_key]
                else:
                    print(f"Expected key '{base_key}' not found in {file_name}")
                    self.phase[base_key] = None
            except FileNotFoundError:
                print(f"File not found: {file_path}")
                self.phase[base_key] = None
            except Exception as e:
                print(f"Error loading file {file_path}: {e}")
                self.phase[base_key] = None
                
    def unpack_phase_data(self):
        frames = []
        for groupname, group_data in self.phase['phase_data_out'].items():
            for recording, records in group_data.items():
                df = pd.DataFrame(records)
                df['Group'] = groupname
                df['Recording'] = recording

                # Renaming group entries
                df['Group'] = df['Group'].replace({'Control': 'No_CTZ', 'ES': 'CTZ'})

                # Data cleaning and conversion
                df['Condition'] = df['Condition'].apply(lambda x: x[0] if isinstance(x, list) and len(x) == 1 else x)
                df['Condition'] = df['Condition'].map({'Stim': True, 'No Stim': False})
                df['CellType2'] = df['CellType2'].apply(lambda x: x[0] if isinstance(x, list) and len(x) == 1 else x)
                df['CellType2'] = df['CellType2'].map({'+FS': 1, 'nsFS': 0, '-FS': -1, '+RS': 1, 'nsRS': 0, '-RS': -1})

                # Flatten all columns that contain lists to scalar if they contain only one value
                for col in df.columns:
                    if df[col].apply(lambda x: isinstance(x, list)).any():
                        df[col] = df[col].apply(lambda x: x[0] if isinstance(x, list) and len(x) == 1 else x)

                frames.append(df)

        # Replace self.phase with the cleaned DataFrame
        self.phase = pd.concat(frames, ignore_index=True)
        
    def plot_ctz_noctz_ppc2(self, cell_type=None, cell_type2=None, show_individual_points=False, smoothing_window=1, min_spikes_evoked=1, min_spikes_spontaneous=1):
        # Prepare the subplots
        fig, axes = plt.subplots(1, 2, figsize=(14, 6))
        fig.suptitle('PPC2 Analysis for CTZ and No_CTZ Groups with SEM')

        # Define color mapping for groups
        group_colors = {
            'No_CTZ': '#797979',
            'CTZ': '#5a00c2'
        }

        # Generate lighter versions for the SEM face color
        lightened_colors = {k: v + '33' for k, v in group_colors.items()}

        # Function to apply simple moving average for smoothing
        def smooth_data(y, window_size):
            if window_size <= 1:
                return y
            window = np.ones(int(window_size)) / float(window_size)
            return np.convolve(y, window, 'same')

        # Common plotting function for a given condition
        def plot_data(df, group, ax, title):
            # Filter by cell type if specified
            if cell_type:
                df = df[df['CellType'] == cell_type]
            if cell_type2:
                df = df[df['CellType2'] == cell_type2]

            # Group by frequency band midpoint
            grouped = df.groupby('fband_midpoint')
            
            # Plot individual data points if toggled on
            if show_individual_points:
                for fband_midpoint, group_data in grouped:
                    ax.plot([fband_midpoint] * len(group_data), group_data['PPC2'], color=lightened_colors[group], alpha=0.7, markersize=4)
            
            # Calculate mean and SEM
            mean_values = grouped['PPC2'].mean()
            sem_values = grouped['PPC2'].sem()

            # Smooth the mean and SEM if smoothing_window is greater than 1
            x_values = mean_values.index
            mean_values_smoothed = smooth_data(mean_values, smoothing_window)
            sem_values_smoothed = smooth_data(sem_values, smoothing_window)

            # Plot mean line
            ax.plot(x_values, mean_values_smoothed, linestyle='-', color=group_colors[group], linewidth=3, label=f'Mean {group}')
            # Plot SEM as shaded area
            ax.fill_between(x_values, (mean_values_smoothed - sem_values_smoothed), (mean_values_smoothed + sem_values_smoothed), color=lightened_colors[group], alpha=0.1)
            
            ax.set_title(title)
            ax.set_xlabel('Frequency Band Midpoint (Hz)')
            ax.set_ylabel('PPC2')
            ax.grid(False)  # Turn off grid lines
            ax.legend()

        # Filter data for both groups
        df_ctz = self.phase[(self.phase['Group'] == 'CTZ')]
        df_no_ctz = self.phase[(self.phase['Group'] == 'No_CTZ')]

        #filter data based on spikes 
        list_of_df = [df_ctz, df_no_ctz]
        for df in list_of_df:
            df = self.filter_by_spikes(df, min_spikes_spontaneous=min_spikes_spontaneous, min_spikes_evoked=min_spikes_evoked)
            #rename the corected df
            if df.equals(df_ctz):
                df_ctz = df
            else:
                df_no_ctz = df

        
        # Separate the data based on the 'Condition'
        conditions = [False, True]  # False for Spontaneous, True for Evoked
        titles = ['Spontaneous', 'Evoked']

        for i, cond in enumerate(conditions):
            # Filter by condition for each group
            df_ctz_cond = df_ctz[df_ctz['Condition'] == cond]
            df_no_ctz_cond = df_no_ctz[df_no_ctz['Condition'] == cond]

            # Plot CTZ group data
            plot_data(df_ctz_cond, 'CTZ', axes[i], titles[i])

            # Plot No_CTZ group data
            plot_data(df_no_ctz_cond, 'No_CTZ', axes[i], titles[i])

        plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust layout to make room for the title
        plt.show()
        
    def filter_by_spikes(self, df, min_spikes_spontaneous, min_spikes_evoked):
        """
        Filters the DataFrame based on the 'Condition' column with two different
        minimum spike thresholds for Spontaneous and Evoked conditions.

        Parameters:
        df (DataFrame): The DataFrame to filter.
        min_spikes_spontaneous (int): Minimum number of spikes for Spontaneous condition.
        min_spikes_evoked (int): Minimum number of spikes for Evoked condition.

        Returns:
        DataFrame: A new DataFrame filtered according to the specified thresholds.
        """
        # Filter for Spontaneous condition (Condition == False)
        spontaneous_df = df[(df['Condition'] == False) & (df['NumSpikes'] >= min_spikes_spontaneous)]
        
        # Filter for Evoked condition (Condition == True)
        evoked_df = df[(df['Condition'] == True) & (df['NumSpikes'] >= min_spikes_evoked)]
        
        # Concatenate the two filtered DataFrames
        filtered_df = pd.concat([spontaneous_df, evoked_df])

        return filtered_df
    
    def filter_data_by_quantile(self, df, quantile_low, quantile_high, column='PPC2'):
        """
        Filters the DataFrame based on the quantile range of a specified column.

        Parameters:
        df (DataFrame): The DataFrame to filter.
        quantile_low (float): The lower quantile threshold (e.g., 0.25 for the 25th percentile).
        quantile_high (float): The upper quantile threshold (e.g., 0.75 for the 75th percentile).
        column (str): The column to apply the quantile filter on.

        Returns:
        DataFrame: A new DataFrame filtered according to the specified quantiles.
        """
        lower_bound = df[column].quantile(quantile_low)
        upper_bound = df[column].quantile(quantile_high)
        filtered_df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
        return filtered_df
    
    def plot_quantile_filtered_ppc2(self, quantile_low=0.05, quantile_high=0.95, show_individual_points=False, 
                                    smoothing_window=1, min_spikes_evoked=5, min_spikes_spontaneous=50):
        """
        Plots PPC2 data with quantile filtering, and saves the plots dynamically in subfolders based on cell type and sensory response.

        Parameters:
            quantile_low (float): The lower quantile threshold for filtering.
            quantile_high (float): The upper quantile threshold for filtering.
            show_individual_points (bool): Whether to show individual data points in the plot.
            smoothing_window (int): Window size for smoothing the data.
            min_spikes_evoked (int): Minimum number of spikes for evoked condition filtering.
            min_spikes_spontaneous (int): Minimum number of spikes for spontaneous condition filtering.

        Returns:
            None: Plots are saved in the specified directory with the appropriate subfolder structure.
        """

        # Default directory
        base_directory = '/Volumes/MannySSD/figures/thesis_final_figs/ppc2/'

        # Assume these attributes are set within the object
        cell_types = ['FS', 'RS']  # Example: or dynamically extracted from your data
        sensory_responses = [1, 0]  # 1 for Sensory Response, 0 for Non-Sensory Response

        # Define color mapping for groups
        group_colors = {
            'No_CTZ': '#797979',
            'CTZ': '#5a00c2'
        }

        # Generate lighter versions for the SEM face color
        lightened_colors = {k: v + '33' for k, v in group_colors.items()}

        # Function to apply simple moving average for smoothing
        def smooth_data(y, window_size):
            if window_size <= 1:
                return y
            window = np.ones(int(window_size)) / float(window_size)
            return np.convolve(y, window, 'same')

        # Common plotting function for a given condition
        def plot_data(df, group, ax, title):
            # Apply quantile filtering
            df = self.filter_data_by_quantile(df, quantile_low, quantile_high)

            # Group by frequency band midpoint
            grouped = df.groupby('fband_midpoint')

            # Plot individual data points if toggled on
            if show_individual_points:
                for fband_midpoint, group_data in grouped:
                    ax.plot([fband_midpoint] * len(group_data), group_data['PPC2'], color=lightened_colors[group], alpha=0.7, markersize=4)

            # Calculate mean and SEM
            mean_values = grouped['PPC2'].mean()
            sem_values = grouped['PPC2'].sem()

            # Smooth the mean and SEM if smoothing_window is greater than 1
            x_values = mean_values.index
            mean_values_smoothed = smooth_data(mean_values, smoothing_window)
            sem_values_smoothed = smooth_data(sem_values, smoothing_window)

            # Plot mean line
            ax.plot(x_values, mean_values_smoothed, linestyle='-', color=group_colors[group], linewidth=3, label=f'Mean {group}')
            # Plot SEM as shaded area
            ax.fill_between(x_values, (mean_values_smoothed - sem_values_smoothed), (mean_values_smoothed + sem_values_smoothed), color=lightened_colors[group], alpha=0.1)

            ax.set_title(title)
            ax.set_xlabel('Frequency Band Midpoint (Hz)')
            ax.set_ylabel('PPC2')
            ax.grid(False)  # Turn off grid lines
            ax.legend()

        # Iterate through all combinations of cell types and sensory responses
        for cell_type in cell_types:
            for cell_type2 in sensory_responses:
                cell_type2_label = 'Sensory Response' if cell_type2 == 1 else 'Non-Sensory Response'

                # Filter the DataFrame based on the current cell type and sensory response
                df_ctz_filtered = self.phase[(self.phase['Group'] == 'CTZ') & (self.phase['CellType'] == cell_type) & (self.phase['CellType2'] == cell_type2)]
                df_no_ctz_filtered = self.phase[(self.phase['Group'] == 'No_CTZ') & (self.phase['CellType'] == cell_type) & (self.phase['CellType2'] == cell_type2)]

                # Prepare the subplots (1 row by 2 columns for Spontaneous vs Evoked)
                fig, axes = plt.subplots(1, 2, figsize=(14, 6))
                fig.suptitle(f'PPC2 Analysis for {cell_type} - {cell_type2_label}', fontsize=16)

                conditions = [False, True]  # False for Spontaneous, True for Evoked
                titles = ['Spontaneous', 'Evoked']

                for i, cond in enumerate(conditions):
                    # Filter by condition for each group
                    df_ctz_cond = df_ctz_filtered[df_ctz_filtered['Condition'] == cond]
                    df_no_ctz_cond = df_no_ctz_filtered[df_no_ctz_filtered['Condition'] == cond]

                    # Plot CTZ group data
                    plot_data(df_ctz_cond, 'CTZ', axes[i], titles[i])

                    # Plot No_CTZ group data
                    plot_data(df_no_ctz_cond, 'No_CTZ', axes[i], titles[i])

                plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust layout to make room for the title

                # Create the directory structure based on cell type and sensory response
                subfolder = os.path.join(base_directory, cell_type, cell_type2_label)
                os.makedirs(subfolder, exist_ok=True)

                # Generate the filename based on cell type and sensory response
                file_name = f'{cell_type}_{cell_type2_label}_ppc2'
                file_path = os.path.join(subfolder, f'{file_name}.svg')

                # Save the figure as an SVG file in the specified subfolder
                fig.savefig(file_path, format='svg', transparent=True)
                plt.close(fig)  # Close the figure after saving to free up memory

    def filter_by_frequency_band(self, band_name):
        """
        Filters the data based on the specified frequency band.

        Parameters:
        band_name (str): The name of the frequency band to filter by (e.g., 'alpha', 'beta').

        Returns:
        DataFrame: A DataFrame filtered by the specified frequency band.
        """
        if band_name not in self.FREQUENCY_BANDS:
            raise ValueError(f"Invalid band name: {band_name}. Valid names are: {list(self.FREQUENCY_BANDS.keys())}")

        low, high = self.FREQUENCY_BANDS[band_name]
        filtered_df = self.phase[(self.phase['fband_midpoint'] >= low) & (self.phase['fband_midpoint'] < high)]
        return filtered_df
    
    def compare_frequency_bands_between_conditions(self, frequency_bands=None, cell_types1=None, cell_types2=None, quantile_low=None, quantile_high=None, show_outliers=None, hue_order=None):
        """
        Compares the data between groups for all specified frequency bands and conditions, and saves the plots dynamically.

        Parameters:
            frequency_bands (list of str): List of frequency bands to compare (e.g., ['alpha', 'beta']).
            cell_types1 (list of str): List of primary cell types to filter by (e.g., ['RS', 'FS']).
            cell_types2 (list of int): List of secondary cell types to filter by (1 for Sensory Response, 0 for Non-Sensory Response).
            quantile_low (float): The lower quantile threshold for filtering.
            quantile_high (float): The upper quantile threshold for filtering.
            show_outliers (bool): Whether to show outliers in the boxplot.
            hue_order (list of str): Order of the hue levels in the plot.

        Returns:
            None: Plots are saved in the specified directory.
        """

        # Base directory
        base_directory = '/Volumes/MannySSD/figures/thesis_final_figs/ppc2/'

        # Initialize default parameters if not provided
        if frequency_bands is None:
            frequency_bands = ['alpha', 'beta', 'gamma', 'high_gamma']
        if cell_types1 is None:
            cell_types1 = ['RS', 'FS']
        if cell_types2 is None:
            cell_types2 = [1, 0]

        # Define color mapping for groups
        group_colors = {
            'No_CTZ': '#797979',
            'CTZ': '#5a00c2'
        }

        # Generate lighter versions for the box face color
        lightened_colors = {k: v + '33' for k, v in group_colors.items()}

        # Boxplot customization
        boxprops = {'edgecolor': 'k', 'linewidth': 2}
        whiskerprops = {'color': 'k', 'linewidth': 2}
        boxplot_kwargs = {
            'boxprops': boxprops,
            'medianprops': whiskerprops,
            'whiskerprops': whiskerprops,
            'capprops': {'linewidth': 0},  # Hide the caps
            'showfliers': show_outliers,
            'palette': group_colors,
            'hue_order': hue_order,
            'width': 0.75
        }

        # Stripplot customization
        stripplot_kwargs = {
            'linewidth': 0.6,
            'size': 6,
            'alpha': 0.7,
            'jitter': True,
            'dodge': True,  # Removed dodge=True from the sns.stripplot() call
            'palette': lightened_colors,
            'hue_order': hue_order
        }

        # Loop over each combination of cell types
        for cell1 in cell_types1:
            for cell2 in cell_types2:
                cell_type2_label = 'Sensory Response' if cell2 == 1 else 'Non-Sensory Response'

                # Create subfolder path based on hierarchy
                subfolder = os.path.join(base_directory, cell1, cell_type2_label)
                os.makedirs(subfolder, exist_ok=True)

                # Prepare the 1x2 plot
                fig, axes = plt.subplots(1, 2, figsize=(14, 6), sharey=True)
                fig.suptitle(f'{cell1} - {cell_type2_label} Comparison Across Frequency Bands')

                conditions = [False, True]  # False for Spontaneous, True for Evoked
                titles = ['Spontaneous', 'Evoked']

                for i, cond in enumerate(conditions):
                    # Filter data for each condition
                    filtered_dfs = []
                    for band in frequency_bands:
                        filtered_df = self.filter_by_frequency_band(band)
                        filtered_df = filtered_df[filtered_df['Condition'] == cond]
                        if cell1:
                            filtered_df = filtered_df[filtered_df['CellType'] == cell1]
                        if cell2 is not None:
                            filtered_df = filtered_df[filtered_df['CellType2'] == cell2]

                        # Apply quantile filtering if specified
                        if quantile_low is not None and quantile_high is not None:
                            filtered_df = self.filter_data_by_quantile(filtered_df, quantile_low, quantile_high)

                        filtered_df['Frequency Band'] = band  # Add frequency band to DataFrame
                        filtered_dfs.append(filtered_df)

                    # Concatenate data across frequency bands
                    combined_df = pd.concat(filtered_dfs)

                    # Aggregate PPC2 values per CellID, Group, and Frequency Band
                    aggregated_df = combined_df.groupby(['CellID', 'Group', 'Frequency Band']).agg({'PPC2': 'mean'}).reset_index()

                    # Plot the comparison for this condition across all frequency bands
                    sns.boxplot(data=aggregated_df, x='Frequency Band', y='PPC2', hue='Group', **boxplot_kwargs, ax=axes[i])
                    sns.stripplot(data=aggregated_df, x='Frequency Band', y='PPC2', hue='Group', **stripplot_kwargs, ax=axes[i])

                    axes[i].set_title(titles[i])
                    axes[i].set_xlabel('Frequency Band')
                    if i == 0:
                        axes[i].set_ylabel('PPC2')
                    axes[i].legend(title='Group')

                # Save the figure
                file_name = f'{cell1}_{cell_type2_label}_FrequencyBandComparison.svg'
                file_path = os.path.join(subfolder, file_name)
                fig.savefig(file_path, format='svg', transparent=True)
                plt.close(fig)  # Close the figure after saving to free up memory

    def run_permutation_test_between_groups(self, band_name, condition, quantile_low=None, quantile_high=None, cell_type=None, cell_type2=None, min_num_spikes=None, n_permutations=10000):
        """
        Runs a permutation test to compare the PPC2 values between groups for a specified frequency band.

        Parameters:
        band_name (str): The name of the frequency band to filter by (e.g., 'alpha', 'beta').
        condition (bool): The condition to filter by (True for evoked, False for spontaneous).
        quantile_low (float): The lower quantile threshold for filtering.
        quantile_high (float): The upper quantile threshold for filtering.
        cell_type (str): The cell type to filter by.
        cell_type2 (int): The secondary cell type to filter by.
        min_num_spikes (int): The minimum number of spikes to filter by.
        n_permutations (int): The number of permutations to perform.

        Returns:
        float: The p-value from the permutation test.
        """
        filtered_df = self.filter_by_frequency_band(band_name)

        # Filter by condition
        filtered_df = filtered_df[filtered_df['Condition'] == condition]

        # Filter by cell type if specified
        if cell_type:
            filtered_df = filtered_df[filtered_df['CellType'] == cell_type]
        if cell_type2 is not None:
            filtered_df = filtered_df[filtered_df['CellType2'] == cell_type2]

        # Filter by minimum number of spikes if specified
        if min_num_spikes is not None:
            filtered_df = filtered_df[filtered_df['NumSpikes'] >= min_num_spikes]

        # Apply quantile filtering if specified
        if quantile_low is not None and quantile_high is not None:
            filtered_df = self.filter_data_by_quantile(filtered_df, quantile_low, quantile_high)

        # Aggregate PPC2 values per CellID and frequency band midpoint
        aggregated_df = filtered_df.groupby(['CellID', 'Group']).agg({'PPC2': 'mean'}).reset_index()

        # Separate the data by groups
        df_ctz = aggregated_df[aggregated_df['Group'] == 'CTZ']['PPC2']
        df_no_ctz = aggregated_df[aggregated_df['Group'] == 'No_CTZ']['PPC2']

        # Calculate observed difference in means
        observed_diff = np.mean(df_ctz) - np.mean(df_no_ctz)

        # Concatenate the data for permutation
        all_data = np.concatenate([df_ctz, df_no_ctz])
        n_ctz = len(df_ctz)
        n_no_ctz = len(df_no_ctz)

        # Perform permutations
        perm_diffs = np.zeros(n_permutations)
        for i in range(n_permutations):
            np.random.shuffle(all_data)
            perm_ctz = all_data[:n_ctz]
            perm_no_ctz = all_data[n_ctz:]
            perm_diffs[i] = np.mean(perm_ctz) - np.mean(perm_no_ctz)

        # Calculate p-value
        p_value = np.mean(np.abs(perm_diffs) >= np.abs(observed_diff))
        return p_value

In [ ]:
class CV_Analysis:
    def __init__(self, eed_object):
        """
        Initialize the CV_Analysis class.

        Parameters:
            eed_object (ExtractEphysData): An instance of ExtractEphysData which provides
                                           access to electrophysiological data methods and attributes.
        """
        self.eed = eed_object

    def get_spike_times_for_cell(self, groupname, recordingname, cid):
        """
        Retrieves the spike times for a specific cell based on groupname, recordingname, and cid.

        Parameters:
            groupname (str): The group name identifier.
            recordingname (str): The recording name identifier.
            cid (int): The unique cell identifier.

        Returns:
            np.array: The spike times in samples for the specified cell.
        """
        df = self.eed.dataframes['basic_metrics']
        row = df[(df['groupname'] == groupname) & 
                 (df['recordingname'] == recordingname) & 
                 (df['cid'] == cid)]
        
        if not row.empty:
            return row['SpikeTimes_all'].values[0]
        else:
            raise ValueError("No matching cell found.")

    def calculate_moving_window_firing_rate(self, spike_times, window_duration_seconds=180, step_size_seconds=1):
        """
        Calculates the firing rate across a moving window of a specified duration.

        Parameters:
            spike_times (np.array): The spike times in samples for a particular cell.
            window_duration_seconds (int): The duration of the moving window in seconds (default is 180 seconds).
            step_size_seconds (int): The step size for the moving window in seconds (default is 1 second).

        Returns:
            dict: A dictionary with the window start time (in seconds) as the key and the firing rate as the value.
        """
        sampling_rate = 30000  # 30 kHz
        window_size_samples = int(window_duration_seconds * sampling_rate)
        step_size_samples = int(step_size_seconds * sampling_rate)

        max_time_samples = int(spike_times[-1])
        firing_rate_dict = {}

        for start_time in range(0, max_time_samples - window_size_samples + 1, step_size_samples):
            end_time = start_time + window_size_samples
            spikes_in_window = np.sum((spike_times >= start_time) & (spike_times < end_time))
            firing_rate = spikes_in_window / window_duration_seconds
            firing_rate_dict[start_time / sampling_rate] = firing_rate

        return firing_rate_dict

    def analyze_single_cell(self, groupname, recordingname, cid):
        """
        Analyzes a single cell, calculating the moving window firing rate.

        Parameters:
            groupname (str): The group name identifier.
            recordingname (str): The recording name identifier.
            cid (int): The unique cell identifier.

        Returns:
            dict: A dictionary containing the moving window firing rate over time.
        """
        spike_times = self.get_spike_times_for_cell(groupname, recordingname, cid)
        firing_rate_dict = self.calculate_moving_window_firing_rate(spike_times)
        return firing_rate_dict


# import the data for analysis 

In [ ]:
# Initialize the class with the path to your .mat file 
#whisker = ExtractEphysData('/Volumes/MannySSD/', 'all_data_20ms_99CI_FINAL.mat')

In [ ]:
organoid = ExtractEphysData('/Volumes/Manny2TB/axion_mea_data_organoid/', )

## rbp4 opto and whikser starts here 

In [ ]:
rbp4_whisker = ExtractEphysData('/Volumes/MannySSD/', 'all_data.mat')

In [ ]:
#repeat for opto data
rbp4_opto = ExtractEphysData('/Volumes/MannySSD/rbp4_led/','all_data.mat')


In [ ]:
rbp4_opto_opto_df_manager = DataFrameManager(rbp4_opto)
rbp4_opto_opto_df_manager.create_psth_dataframe_opto()
rbp4_opto_opto_df_manager.dataframes['psth_dataframe_8 Hz LED']['PSTHs_raw'][0].shape

In [ ]:
df1, df2 = rbp4_opto_opto_df_manager.compare_groups('CTZ', 'No_CTZ', '8 Hz LED', cell_type=None, is_single_unit=None, stim_responsivity=None) 
df1


In [ ]:
x, y = rbp4_opto_opto_df_manager.extract_stim_signals_opto()
x['8 Hz LED'].shape

In [ ]:
rbp4_opto_opto_df_manager.plot_rasters_for_cid('CTZ','ctz_2847_rbp4_optostim','cid131')
rbp4_opto_opto_df_manager.plot_combined_psth_and_raster('CTZ','ctz_2847_rbp4_optostim','cid131', time_window=None)
rbp4_opto_opto_df_manager.plot_combined_psth_and_raster_normalized('CTZ','ctz_2847_rbp4_optostim','cid131', time_window=None, smoothing_window=5, normalize=False)



In [ ]:
rbp4_opto_opto_df_manager.create_dataframe(['Cell_Type', 'LaminarLabel','IsSingleUnit', 'StimResponsivity', 'SpikeTrains_for_PSTHs', 'PSTHs_raw'], 'psth_dataframe')

In [ ]:
rbp4_opto_opto_df_manager.dataframes['psth_dataframe']['SpikeTrains_for_PSTHs'][0][0].shape

In [ ]:
rbp4_opto_opto_df_manager.eed.trialTagsLabels['trialTagsLabels']

In [ ]:
rbp4_opto_opto_df_manager.plot_psth_comparison('CTZ', 'No_CTZ', '8 Hz LED', cell_type='RS', is_single_unit=1.0, stim_responsivity=1.0, time_range=(-15,250), plot_mode='traces', smoothing_window=10)
rbp4_opto_opto_df_manager.plot_psth_comparison('CTZ', 'No_CTZ', '8 Hz LED', cell_type='RS', is_single_unit=1.0, stim_responsivity=1.0, time_range=(-15,250), plot_mode='mean', smoothing_window=10)

In [ ]:
rbp4_opto_opto_df_manager.plot_psth_with_stim_opto('CTZ','No_CTZ',  stim_label='8 Hz LED', cell_type='RS', is_single_unit=1.0, stim_responsivity=1.0, time_range=(-15,600), plot_mode='traces', smoothing_window=10)

In [ ]:
def plot_opto_stim_responses(df, group_col='groupname', baseline_col='MeanFR_baseline', stim_col='MeanFR_stim', jitter=0.1, plot_individual_points=True):
    """
    Plot the mean, SEM, and optionally individual points with jitter of responses for baseline and stimulation conditions 
    for two groups present in the DataFrame, along with the count of observations.

    Parameters:
    df (pandas.DataFrame): DataFrame containing the data to plot.
    group_col (str): Name of the column in df that contains group names.
    baseline_col (str): Name of the column in df that contains baseline firing rates.
    stim_col (str): Name of the column in df that contains stimulated firing rates.
    jitter (float): The width of the jitter around the main x position.
    plot_individual_points (bool): Whether to plot individual points with jitter.
    """
    conditions = ['Baseline', 'Stimulation']
    groups = df[group_col].unique()
    colors = ['blue', 'orange']  # Specify more colors if there are more than two groups
    means = {group: [] for group in groups}
    sems = {group: [] for group in groups}
    counts = {group: [] for group in groups}

    # Plotting
    x = np.arange(len(conditions))  # the label locations
    width = 0.35  # the width of the bars
    offset = width / (len(groups) * 2)
    
    fig, ax = plt.subplots()
    for i, group in enumerate(groups):
        group_data = df[df[group_col] == group]
        for j, cond in enumerate([baseline_col, stim_col]):
            responses = group_data[cond]
            means[group].append(np.median(responses))
            sems[group].append(np.std(responses) / np.sqrt(len(responses)))
            counts[group].append(len(responses))
            
            if plot_individual_points:
                # Add individual points with jitter
                jittered_x = x[j] - offset + i * offset * 2 + np.random.uniform(-jitter, jitter, size=len(responses))
                ax.plot(jittered_x, responses, 'o', color=colors[i], alpha=0.2)
        
        # Add mean and SEM points without horizontal caps
        ax.errorbar(x - offset + i * offset * 2, means[group], yerr=sems[group], fmt='o', color=colors[i],
                    label=f'{group} (N={counts[group][0]})', capsize=0, elinewidth=2, capthick=2, markersize=5)

    # Add some text for labels, title, and custom x-axis tick labels, etc.
    ax.set_xlabel('Condition')
    ax.set_ylabel('Firing Rate (Hz)')
    ax.set_title('Comparison of Baseline vs. Stimulation Responses by Group')

    ax.set_xticks(x)
    ax.set_xticklabels(conditions)
    
    ax.legend()
    plt.show()
    
def compare_group_firing_rates_with_stats(df, group_col='groupname', baseline_col='MeanFR_baseline', stim_col='PeakEvokedFR'):
    """
    Performs Mann-Whitney U tests to compare baseline and stimulation firing rates between groups
    and includes descriptive statistics for each group.

    Parameters:
    df (pandas.DataFrame): DataFrame containing the data to analyze.
    group_col (str): Name of the column in df that contains group names.
    baseline_col (str): Name of the column in df that contains baseline firing rates.
    stim_col (str): Name of the column in df that contains peak evoked firing rates.

    Returns:
    pandas.DataFrame: A DataFrame with descriptive statistics and Mann-Whitney U test results.
    """
    # Group the DataFrame by 'groupname' and extract values
    grouped = df.groupby(group_col)

    group_stats = []
    for group_name, group_df in grouped:
        for condition in [baseline_col, stim_col]:
            # Extract values and compute statistics
            values = np.array([val.item() for val in group_df[condition] if isinstance(val, np.ndarray)])
            group_stats.append({
                'Group': group_name,
                'Condition': condition,
                'N': len(values),
                'Mean': np.mean(values),
                'Median': np.median(values),
                'SD': np.std(values)
            })

    stats_df = pd.DataFrame(group_stats)

    # Calculate Mann-Whitney U tests between groups
    test_results = []
    groups = list(grouped.groups.keys())
    for i in range(len(groups)):
        for j in range(i+1, len(groups)):
            group1_name, group2_name = groups[i], groups[j]
            for condition in [baseline_col, stim_col]:
                data1 = np.array([val.item() for val in grouped.get_group(group1_name)[condition] if isinstance(val, np.ndarray)])
                data2 = np.array([val.item() for val in grouped.get_group(group2_name)[condition] if isinstance(val, np.ndarray)])
                stat, p_value = stats.mannwhitneyu(data1, data2, alternative='two-sided')
                #change the 
                
                test_results.append({
                    'Group1': group1_name,
                    'Group2': group2_name,
                    'Condition': condition,
                    'U-Statistic': stat,
                    'p-Value': p_value,
                    f'{group1_name} N': len(data1),
                    f'{group1_name} Mean': np.mean(data1),
                    f'{group1_name} Median': np.median(data1),
                    f'{group1_name} SD': np.std(data1),
                    f'{group2_name} N': len(data2),
                    f'{group2_name} Mean': np.mean(data2),
                    f'{group2_name} Median': np.median(data2),
                    f'{group2_name} SD': np.std(data2)
                })

    results_df = pd.DataFrame(test_results)
    return stats_df, results_df

def compare_CTZ_vs_NoCTZ(df, group_col, value_col):
    # Filter data for the two specific groups
    ctz_data = df[df[group_col] == 'CTZ'][value_col].to_numpy()
    no_ctz_data = df[df[group_col] == 'No_CTZ'][value_col].to_numpy()


        # Calculate Welch's t-test
    stat, p_value = stats.ttest_ind(ctz_data, no_ctz_data, equal_var=False, alternative='greater')
        
        # Collect results
    results = {
            'Group1': 'CTZ',
            'Group2': 'No_CTZ',
            'P-Value': p_value,
            'CTZ_Mean': ctz_data.mean(),
            'No_CTZ_Mean': no_ctz_data.mean(),
            'CTZ_SD': ctz_data.std(),
            'No_CTZ_SD': no_ctz_data.std()
        }

    return pd.DataFrame([results])



In [ ]:
rbp4_opto_opto_df_manager = DataFrameManager(rbp4_opto)
rbp4_opto_opto_df_manager.eed
rbp4_opto_opto_df_manager.create_dataframe(['Cell_Type', 'IsSingleUnit', 'StimResponsivity', 'MeanFR_baseline', 'MeanFR_stim', 'PeakEvokedFR', 'PeakEvokedFR_Latency', 'FanoFactor_baseline', 'FanoFactor_stim'], 'basic_metrics')

###fast spiking sua for opto 
fastspiking_opto_df = rbp4_opto_opto_df_manager.get_filtered_data('basic_metrics', is_single_unit=None,  cell_type='FS', stim_responsivity=1.0)
#fastspiking_opto_df = fastspiking_opto_df[fastspiking_opto_df['MeanFR_baseline'] > 0.01]
plot_opto_stim_responses(fastspiking_opto_df, baseline_col='MeanFR_baseline', stim_col='MeanFR_stim', jitter=0.1, plot_individual_points=True)


regularspiking_opto_df = rbp4_opto_opto_df_manager.get_filtered_data('basic_metrics', is_single_unit=1.0, cell_type='RS', stim_responsivity=1.0)
#regularspiking_opto_df = regularspiking_opto_df[regularspiking_opto_df['MeanFR_baseline'] > 0.01]
plot_opto_stim_responses(regularspiking_opto_df, baseline_col='MeanFR_baseline', stim_col='PeakEvokedFR', jitter=0.1, plot_individual_points=True)


mua_opto_df = rbp4_opto_opto_df_manager.get_filtered_data('basic_metrics', is_single_unit=None, cell_type=None, stim_responsivity=1.0)
plot_opto_stim_responses(mua_opto_df, baseline_col='MeanFR_baseline', stim_col='MeanFR_stim', jitter=0.1, plot_individual_points=False)

mua_opto_df = rbp4_opto_opto_df_manager.get_filtered_data('basic_metrics', is_single_unit=0.0, cell_type=None, stim_responsivity=1.0)
plot_opto_stim_responses(mua_opto_df, baseline_col='MeanFR_baseline', stim_col='MeanFR_stim', jitter=0.1, plot_individual_points=True)


In [ ]:
rbp4_opto_opto_df_manager.dataframes['basic_metrics']

In [ ]:
rbp4_whisker_df_manager.create_psth_dataframe()
rbp4_whisker_df_manager.calculate_basic_stats('CTZ', 'No_CTZ', stim_label=None, baseline_range=(-100, -1), stim_range=(10,50), cell_type='FS', is_single_unit=1.0, stim_responsivity=1.0, smoothing_window=5)
rbp4_whisker_df_manager.prepare_for_boxplot()
rbp4_whisker_df_manager.plot_box_and_strip(groups=['CTZ', 'No_CTZ'], stimulations=None, show_outliers=True)

rbp4_whisker_df_manager.create_psth_dataframe()
rbp4_whisker_df_manager.calculate_basic_stats('CTZ', 'No_CTZ', stim_label=None, baseline_range=(-100, -1), stim_range=(10,50), cell_type='RS', is_single_unit=1.0, stim_responsivity=1.0, smoothing_window=5)
rbp4_whisker_df_manager.prepare_for_boxplot()
rbp4_whisker_df_manager.plot_box_and_strip(groups=['CTZ', 'No_CTZ'], stimulations=None, show_outliers=True)

rbp4_whisker_df_manager.create_psth_dataframe()
rbp4_whisker_df_manager.calculate_basic_stats('CTZ', 'No_CTZ', stim_label=None, baseline_range=(-100, -1), stim_range=(10,50), cell_type=None, is_single_unit=None, stim_responsivity=0.0, smoothing_window=5)
rbp4_whisker_df_manager.prepare_for_boxplot()
rbp4_whisker_df_manager.plot_box_and_strip(groups=['CTZ', 'No_CTZ'], stimulations=None, show_outliers=True)

In [ ]:
fastspiking_opto_df

In [ ]:
regularspiking_opto_df.info()

In [ ]:
# List of columns that should be numeric
numeric_columns = ['IsSingleUnit', 'StimResponsivity', 'MeanFR_baseline', 'MeanFR_stim',
                   'PeakEvokedFR', 'PeakEvokedFR_Latency', 'FanoFactor_baseline', 'FanoFactor_stim']

# Convert these columns to numeric, coercing errors to NaN
for col in numeric_columns:
    regularspiking_opto_df[col] = pd.to_numeric(regularspiking_opto_df[col], errors='coerce')

# After converting, it's good to check if there are any NaNs that were introduced
print(regularspiking_opto_df[numeric_columns].isna().sum())

In [ ]:
regularspiking_opto_df['MeanFR_stim'].to_numpy()

## start the CV analysis 

## Whisker CCG starts here

In [ ]:
#intialize the ccg handler
CrossCorrObj = CCGHandler(whisker, '/Volumes/MannySSD/Data/ccgs')
#CrossCorrObj = CCGHandler(whisker, '/Volumes/MannySSD/new_ccgs_manny')
CrossCorrObj.ccg['MonoConnectionsTable'].shape

### make plot and run stats on CCGs

In [ ]:
CrossCorrObj.ccg['MonoConnectionsTable']

In [ ]:
CrossCorrObj.ccg['MonoConnectionsTable'].columns


In [ ]:
#print the unique celltypes in the ccg table
CrossCorrObj.ccg['MonoConnectionsTable']['CellTypes'].unique()
print(f'The unique cell types in the CCG table are: {CrossCorrObj.ccg["MonoConnectionsTable"]["CellTypes"].unique()}')

#print the unique Significance in the ccg table
CrossCorrObj.ccg['MonoConnectionsTable']['Significance'].unique()
print(f'The unique significance values in the CCG table are: {CrossCorrObj.ccg["MonoConnectionsTable"]["Significance"].unique()}')

#print the unique sd_sig in the ccg table
CrossCorrObj.ccg['MonoConnectionsTable']['sd_sig'].unique()
print(f'The unique sd_sig values in the CCG table are: {CrossCorrObj.ccg["MonoConnectionsTable"]["sd_sig"].unique()}')

#print the unique groupnames in the ccg table
CrossCorrObj.ccg['MonoConnectionsTable']['groupname'].unique()
print(f'The unique groupnames in the CCG table are: {CrossCorrObj.ccg["MonoConnectionsTable"]["groupname"].unique()}')

#print the unique StimResp_A
CrossCorrObj.ccg['MonoConnectionsTable']['StimResp_A'].unique()
print(f'The unique StimResp_A values in the CCG table are: {CrossCorrObj.ccg["MonoConnectionsTable"]["StimResp_A"].unique()}')

#print the unique StimResp_B
CrossCorrObj.ccg['MonoConnectionsTable']['StimResp_B'].unique()
print(f'The unique StimResp_B values in the CCG table are: {CrossCorrObj.ccg["MonoConnectionsTable"]["StimResp_B"].unique()}')

#print the unique layers 
CrossCorrObj.ccg['MonoConnectionsTable']['layers'].unique()
print(f'The unique layers in the CCG table are: {CrossCorrObj.ccg["MonoConnectionsTable"]["layers"].unique()}')

In [ ]:
CrossCorrObj.ccg['PointwiseABs'].shape 
#print the details of what the shap emeans 
print(f'The shape of the pointwise abs is {CrossCorrObj.ccg["PointwiseABs"].shape}') # (101, 4878, 2), (time, pairs, and positive and negative PointwiseABs)
print(f'The shape represents time, pairs, and positive and negative PointwiseABs') 

CrossCorrObj.ccg['SimultaneousABs'].shape
print(f'The shape of the simultaneous abs is {CrossCorrObj.ccg["SimultaneousABs"].shape}') # (101, 4878, 2), (time, pairs, and positive and negative SimultaneousABs)
print(f'The shapre represents time, pairs, and positive and negative SimultaneousABs')

In [ ]:
def plot_ccg(CrossCorrObj, index=0, filter_criteria=None):
    # Filter data if criteria are provided
    if filter_criteria:
        mask = np.ones(len(CrossCorrObj.ccg['MonoConnectionsTable']), dtype=bool)
        for key, value in filter_criteria.items():
            mask &= CrossCorrObj.ccg['MonoConnectionsTable'][key] == value
        filtered_indices = np.where(mask)[0]
        if len(filtered_indices) == 0:
            print("No data matches the filter criteria.")
            return
        index = filtered_indices[0]  # Use the first matching index

    # Extract data
    ccf = CrossCorrObj.ccg['MonoConnectionsTable']['ccgs'][index]
    negative_pointwise_ab = CrossCorrObj.ccg['PointwiseABs'][:, index, 0]
    positive_pointwise_ab = CrossCorrObj.ccg['PointwiseABs'][:, index, 1]
    negative_simultaneous_ab = CrossCorrObj.ccg['SimultaneousABs'][:, index, 0]
    positive_simultaneous_ab = CrossCorrObj.ccg['SimultaneousABs'][:, index, 1]
    time = CrossCorrObj.ccg['t']

    # Create plot
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(time, ccf, label='CCF', color='black')
    ax.plot(time, negative_pointwise_ab, label='Negative Pointwise AB', color='red', linestyle='--')
    ax.plot(time, positive_pointwise_ab, label='Positive Pointwise AB', color='red', linestyle='--')
    ax.plot(time, negative_simultaneous_ab, label='Negative Simultaneous AB', color='blue', linestyle='-.')
    ax.plot(time, positive_simultaneous_ab, label='Positive Simultaneous AB', color='blue', linestyle='-.')

    ax.set_xlabel('Time (ms)')
    ax.set_ylabel('Correlation')
    ax.set_title(f'CCG Analysis for Index {index}')
    ax.legend()
    plt.tight_layout()
    plt.show()

    # Print some information about the plotted data
    print(f"Plotted CCG for index {index}")
    print(f"Cell Types: {CrossCorrObj.ccg['MonoConnectionsTable']['CellTypes'][index]}")
    print(f"Significance: {CrossCorrObj.ccg['MonoConnectionsTable']['Significance'][index]}")
    print(f"Group Name: {CrossCorrObj.ccg['MonoConnectionsTable']['groupname'][index]}")


In [ ]:
plot_ccg(CrossCorrObj)

# Example with filtering:
filter_criteria = {
    'CellTypes': 'RS->RS',
   'Significance': True,
    'groupname': 'CTZ'
}

plot_ccg(CrossCorrObj, filter_criteria=filter_criteria)

### plot figures and run stats, save dynamically

In [ ]:
CrossCorrObj.is_significant_ccg()


# Define the conditions you want to iterate over
responses = ['responsive', 'nonresponsive']
connection_types = ['RS->RS', 'FS->FS']

# Other parameters remain consistent
min_spike_pairs = 1000  # minimum number of spike pairs (wass 1000)upda
sd_sig = True  # use standard deviation to determine significance
significance = ['S']  # determine if the ccg is significant based on the old method
EorI = None
#layers = ['SG->SG', 'SG->IG', 'IG->SG']

layers = None

# Base directory for all figures
base_directory = '/Volumes/MannySSD/figures/synchrony_oddpairs2'

# Loop over the response types and connection types
for connectiontype in connection_types:
    # Create a top-level folder for each connection type
    connection_folder = os.path.join(base_directory, connectiontype.replace('->', '_'))
    os.makedirs(connection_folder, exist_ok=True)

    for response in responses:
        # Create subfolders within each connection type folder for each response type
        response_folder = os.path.join(connection_folder, response)
        os.makedirs(response_folder, exist_ok=True)

        # Dynamically create file names based on the current conditions
        file_name_base = f"{connectiontype.replace('->', '_')}_excess_synchrony_{min_spike_pairs}_{sd_sig}_{response}_{layers}_{significance}_{EorI}"
        file_name_dots = f"{file_name_base}_dots"
        file_name_no_dots = f"{file_name_base}_no_dots"

        # Plotting and saving figures
        CrossCorrObj.plot_group_ccgs_traces(significance=significance, sd_sig=sd_sig, EorI=EorI, connectiontype=connectiontype, response=response, layers=layers, min_spike_pairs=min_spike_pairs, xlim=(-10,10), front_group='No_CTZ', limit=None)

        CrossCorrObj.plot_group_ccgs(significance=significance, sd_sig=sd_sig, EorI=EorI, connectiontype=connectiontype, response=response, layers=layers, min_spike_pairs=min_spike_pairs, xlim=(-50,50), limit=None, front_group='No_CTZ', directory=response_folder, file_name=file_name_base)

        CrossCorrObj.plot_group_ccgs(significance=significance, sd_sig=sd_sig, EorI=EorI, connectiontype=connectiontype, response=response, layers=layers, min_spike_pairs=min_spike_pairs, xlim=None, limit=None, front_group='No_CTZ', directory=response_folder, file_name=f"{file_name_base}_full")

        CrossCorrObj.plot_group_ccgs(significance=significance, sd_sig=sd_sig, EorI=EorI, connectiontype=connectiontype, response=response, layers=layers, min_spike_pairs=min_spike_pairs, xlim=(-10,10), limit=None, front_group='No_CTZ', directory=response_folder, file_name=f"{file_name_base}_zoomed")

        CrossCorrObj.plot_group_excess_synchrony(significance=significance, sd_sig=sd_sig, EorI=EorI, connectiontype=connectiontype, response=response, layers=layers, min_spike_pairs=min_spike_pairs, limit=None, directory=response_folder, file_name=file_name_no_dots)

        CrossCorrObj.plot_group_excess_synchrony_with_dots(significance=significance, sd_sig=sd_sig, EorI=EorI, connectiontype=connectiontype, response=response, layers=layers, min_spike_pairs=min_spike_pairs, limit=None, directory=response_folder, file_name=file_name_dots)

        # Perform the permutation test and store results
        results_perm = CrossCorrObj.compare_group_excess_synchrony_permutation(
            significance=significance,
            sd_sig=sd_sig,
            EorI=EorI,
            connectiontype=connectiontype,
            response=response,
            layers=layers,
            min_spike_pairs=min_spike_pairs,
            num_permutations=10000, 
            limit=None
        )

        # Save the permutation results in the appropriate folder
        results_perm.to_csv(os.path.join(response_folder, f"{file_name_base}_results.csv"), index=False)


In [ ]:
FSFS_CTZ_synchdf = CrossCorrObj.access_ccgs('CTZ', significance=significance, sd_sig=sd_sig, EorI=EorI, connectiontype=connectiontype, response=response, layers=layers, min_spike_pairs=min_spike_pairs)
FSFS_NoCTZ_synchdf = CrossCorrObj.access_ccgs('No_CTZ', significance=significance, sd_sig=sd_sig, EorI=EorI, connectiontype=connectiontype, response=response, layers=layers, min_spike_pairs=min_spike_pairs)
RSRS_CTZ_synchdf = CrossCorrObj.access_ccgs('CTZ', significance=significance, sd_sig=sd_sig,  EorI=EorI, connectiontype=connectiontype, response=response, layers=layers, min_spike_pairs=min_spike_pairs)
RSRS_NoCTZ_synchdf = CrossCorrObj.access_ccgs('No_CTZ', significance=significance, sd_sig=sd_sig, EorI=EorI, connectiontype=connectiontype, response=response, layers=layers, min_spike_pairs=min_spike_pairs)

### this section is for directed weights E/I -- no differences to be seen here 

In [ ]:

CrossCorrObj.is_significant_ccg()

min_spike_pairs = 50; #minimum number of spike pairs
sd_sig = None #use standard deviation to determine significance
significance = None  #determine if the ccg is significant based old method
response = 'responsive' #response to filter by 'responsive' or 'nonresponsive'
connectiontype=['FS->FS']
EorI=['S']
layers = None

# change file name to allow more than one connection type
file_name = connectiontype[0].replace('->','_') + str(min_spike_pairs)+str(sd_sig)+response+str(layers)+str(significance)+str(EorI)
file_name_dots = connectiontype[0].replace('->','_') + str(min_spike_pairs)+str(sd_sig)+response+str(layers)+str(significance)+str(EorI)+'_dots'


CrossCorrObj.plot_group_ccgs_traces(significance=significance, sd_sig=sd_sig, EorI=EorI, connectiontype=connectiontype, response=response, layers=layers, min_spike_pairs=min_spike_pairs, xlim=(-5,5), front_group='No_CTZ', limit=None)
CrossCorrObj.plot_group_ccgs(significance=significance, sd_sig=sd_sig, EorI=EorI, connectiontype=connectiontype, response=response, layers=layers, min_spike_pairs=min_spike_pairs, xlim=(-50,50), limit=None, front_group='No_CTZ', directory='/Volumes/MannySSD/figures/synchrony', file_name=file_name)
CrossCorrObj.plot_group_ccgs(significance=significance, sd_sig=sd_sig, EorI=EorI, connectiontype=connectiontype, response=response, layers=layers, min_spike_pairs=min_spike_pairs, xlim=None, limit=None, front_group='No_CTZ', directory='/Volumes/MannySSD/figures/synchrony', file_name=file_name+'_full')
CrossCorrObj.plot_group_ccgs(significance=significance, sd_sig=sd_sig, EorI=EorI, connectiontype=connectiontype, response=response, layers=layers, min_spike_pairs=min_spike_pairs, xlim=(-4,4), limit=None, front_group='No_CTZ', directory='/Volumes/MannySSD/figures/synchrony', file_name=file_name+'_zoomed')
CrossCorrObj.plot_group_DCW_with_dots(significance=significance, sd_sig=sd_sig, EorI=EorI, connectiontype=connectiontype, response=response, layers=layers, min_spike_pairs=min_spike_pairs, limit=None, directory='/Volumes/MannySSD/figures/synchrony', file_name=file_name_dots)

#dcw_results = CrossCorrObj.compare_group_DCW_permutation(
#    significance=significance,
#    sd_sig=sd_sig,
#    EorI=EorI,
#    connectiontype=connectiontype,
#    response=response,
#    layers=layers,
#    min_spike_pairs=min_spike_pairs,
#    num_permutations=10000, 
#    limit=None
#)

results_perm = CrossCorrObj.compare_group_excess_synchrony_permutation(
    significance=significance,
    sd_sig=sd_sig,
    EorI=EorI,
    connectiontype=connectiontype,
    response=response,
    layers=layers,
    min_spike_pairs=min_spike_pairs,
    num_permutations=10000, 
    limit=None
)


## Whisker Dataframe initializer Starts Here

### intialize whisker df manager

In [ ]:
whisker_df_manager = DataFrameManager(whisker)
whisker_df_manager.create_dataframe(['Cell_Type', 'IsSingleUnit', 'StimResponsivity', 
                                     'StimProb', 'MeanFR_baseline', 'MeanFR_stim','LaminarLabel',
                                     'PeakEvokedFR', 'PeakEvokedFR_Latency', 'FanoFactor_baseline', 
                                     'FanoFactor_stim', 'SpikeTimes_all', 'MeanFR_inst_stim', 'Template_Channel', 
                                     'ModulationIndex', 'Normalized_Template_Waveform', 'TroughToPeak_duration', 
                                     'SpikeHalfWidth', 'UnNormalized_Template_Waveform', 'ISI_violations_percent', 
                                     'Recording_Duration', 'Sampling_Frequency', 'SpikeTimes_all', 'FirstSpikeLatency', 
                                     'Stim_Intensity', 'Stim_Onsets_samples', 'Stim_Offsets_samples'], 'basic_metrics')
whisker_df_manager.create_psth_dataframe_2() #create the psth dataframe

### Burst analysis: The run_mannwhitneyu_test and the plot_burst_counts_by_laminar are modiofied from the above, but it works for now, might need to change

In [ ]:
def calculate_bursts(row):
    """
    This function identifies burst events and associates them with the corresponding stimulation intensity.
    
    Parameters:
    row: A single row from the DataFrame containing spike times, sampling frequency, and stimulation details.

    Returns:
    bursts: A list of all identified bursts, where each burst is a list of spike times in samples.
    bursts_with_stim: A list of tuples, where each tuple contains:
        - A list of spike times that form a burst.
        - The corresponding stimulation intensity during which the burst occurred, if applicable.
    """
    spike_times = np.array(row['SpikeTimes_all'])  # Extract spike times in samples from the row
    sampling_freq = row['Sampling_Frequency']  # Extract the sampling frequency in Hz
    stim_intensities = row['Stim_Intensity']  # Extract the stimulus intensities (1, 2, 3, 4 for Zero, Low, Mid, Max)
    stim_onsets = np.array(row['Stim_Onsets_samples'])  # Extract stimulation onset times in samples
    stim_offsets = np.array(row['Stim_Offsets_samples'])  # Extract stimulation offset times in samples
    
    # Convert spike times from samples to milliseconds
    time_in_ms = spike_times / sampling_freq * 1000  
    
    # Calculate Interspike Intervals (ISIs) in milliseconds
    isi = np.diff(time_in_ms)
    
    # Identify burst events: ISIs ≤ 10 ms
    burst_mask = isi <= 10  # Boolean array where True indicates that the ISI is part of a burst
    
    # Initialize lists to store bursts and bursts with associated stimulation information
    bursts = []  # This will store all identified bursts
    bursts_with_stim = []  # This will store bursts along with their corresponding stimulation intensity
    current_burst = []  # Temporarily holds the spike times forming a burst
    
    for i in range(len(burst_mask)):
        if burst_mask[i]:  # If the ISI at index i indicates a burst (ISI ≤ 10 ms)
            if not current_burst:  # If no burst is currently being tracked
                current_burst.append(spike_times[i])  # Start the burst with the current spike time
            current_burst.append(spike_times[i + 1])  # Add the next spike time to the current burst
            
            # If this is the last ISI and it's part of a burst, finalize the burst
            if i == len(burst_mask) - 1:
                bursts.append(current_burst)  # Store the burst in the overall list of bursts
                bursts_with_stim.append((current_burst, get_stimulation_for_burst(current_burst, stim_onsets, stim_offsets, stim_intensities)))
        else:
            if current_burst:  # If a burst is being tracked and this ISI ends the burst
                bursts.append(current_burst)  # Store the burst in the overall list of bursts
                bursts_with_stim.append((current_burst, get_stimulation_for_burst(current_burst, stim_onsets, stim_offsets, stim_intensities)))
                current_burst = []  # Reset for the next potential burst
    
    # The 'bursts' list now contains all identified bursts (each as a list of spike times in samples).
    # The 'bursts_with_stim' list contains tuples, where each tuple includes:
    #   - A list of spike times forming a burst.
    #   - The stimulation intensity (1, 2, 3, 4) during which the burst occurred.
    
    return bursts, bursts_with_stim

def get_stimulation_for_burst(burst, stim_onsets, stim_offsets, stim_intensities):
    """
    Helper function to determine which stimulation intensity a burst occurred during.
    
    Parameters:
    burst: A list of spike times representing a burst.
    stim_onsets: An array of stimulation onset times in samples.
    stim_offsets: An array of stimulation offset times in samples.
    stim_intensities: An array of stimulation intensities corresponding to the onsets and offsets.

    Returns:
    The stimulation intensity (1, 2, 3, 4) during which the burst occurred, or None if it doesn't match any stimulation window.
    """
    burst_start = burst[0]  # The start of the burst (first spike time in the burst)
    burst_end = burst[-1]  # The end of the burst (last spike time in the burst)
    
    # Loop through all stimulation periods
    for onset, offset, intensity in zip(stim_onsets, stim_offsets, stim_intensities):
        if burst_start >= onset and burst_end <= offset:  # If the burst fits within a stimulation period
            return intensity  # Return the corresponding stimulation intensity
    
    return None  # If the burst doesn't fit within any stimulation period, return None

def count_bursts_by_stimulation(row):
    """
    Counts the number of segments with bursts during no stimulation and stimulation.

    Parameters:
    row: A single row from the DataFrame containing the bursts with stimulation information.

    Returns:
    A tuple containing the counts:
        - Count of segments with bursts during no stimulation (None or 1.0)
        - Count of segments with bursts during stimulation (2.0, 3.0, 4.0)
    """
    bursts_with_stim = row['Bursts_with_Stim']
    
    # Initialize counts
    no_stim_count = 0
    stim_count = 0
    
    # Iterate through the bursts_with_stim list
    for burst, stim in bursts_with_stim:
        if stim is None or stim == 1.0:
            no_stim_count += 1
        elif stim in [2.0, 3.0, 4.0]:
            stim_count += 1
    
    return no_stim_count, stim_count

def count_bursts_by_cid_and_laminar(row):
    """
    Counts the number of segments with bursts during no stimulation and stimulation per cid and LaminarLabel.

    Parameters:
    row: A single row from the DataFrame containing the bursts with stimulation information.

    Returns:
    A dictionary containing:
        - groupname: The group (CTZ or No_CTZ)
        - recordingname: The recording name
        - cid: The cell ID
        - LaminarLabel: The laminar label (SG, L4, IG)
        - No_Stim_Burst_Count: Count of segments with bursts during no stimulation (None or 1.0)
        - Stim_Burst_Count: Count of segments with bursts during stimulation (2.0, 3.0, 4.0)
    """
    bursts_with_stim = row['Bursts_with_Stim']
    
    # Initialize counts
    no_stim_count = 0
    stim_count = 0
    
    # Iterate through the bursts_with_stim list
    for burst, stim in bursts_with_stim:
        if stim is None or stim == 1.0:
            no_stim_count += 1
        elif stim in [2.0, 3.0, 4.0]:
            stim_count += 1
    
    return pd.Series({
        'groupname': row['groupname'],
        'recordingname': row['recordingname'],
        'cid': row['cid'],
        'LaminarLabel': row['LaminarLabel'],
        'Cell_Type': row['Cell_Type'],
        'ModulationIndex': row['ModulationIndex'],
        'IsSingleUnit': row['IsSingleUnit'],
        'StimResponsivity': row['StimResponsivity'],
        'StimProb': row['StimProb'],
        'No_Stim_Burst_Count': no_stim_count,
        'Stim_Burst_Count': stim_count
    })

def plot_burst_counts_by_laminar(melted_data, show_outliers=True):
    """
    Plots the burst counts broken down by LaminarLabel and optionally omits outliers.

    Parameters:
    melted_data: The melted DataFrame containing burst counts.
    show_outliers: Boolean indicating whether to display outliers in the box plot (default is True).
    """
    # Define the color palette
    palette = {"CTZ": "purple", "No_CTZ": "grey"}

    # Specify the order of the LaminarLabels
    laminar_order = ['SG', 'L4', 'IG']

    # Specify the hue order to ensure consistent plotting
    hue_order = ["CTZ", "No_CTZ"]

    # Create a FacetGrid to plot data for each LaminarLabel separately
    g = sns.FacetGrid(melted_data, col="LaminarLabel", height=5, aspect=1.5, col_order=laminar_order)

    # Map the boxplot to the grid, with control over outlier display
    g.map(sns.boxplot, "Condition", "Burst_Count", "groupname", 
          palette=palette, hue_order=hue_order, 
          order=['No_Stim_Burst_Count', 'Stim_Burst_Count'], showfliers=show_outliers)

    # Set the labels and title
    g.set_axis_labels('Condition', 'Burst Count')
    g.set_titles(col_template='{col_name}')
    g.add_legend(title='Group')

    # Adjust the layout and show the plot
    plt.subplots_adjust(top=0.85)
    g.fig.suptitle('Comparison of Burst Counts by Laminar Label between CTZ and No_CTZ Groups')
    plt.show()
#
# 
# 
# Apply the burst calculation function to each row in the DataFrame
whisker_df_manager.dataframes['basic_metrics'][['Bursts', 'Bursts_with_Stim']] = whisker_df_manager.dataframes['basic_metrics'].apply(calculate_bursts, axis=1, result_type='expand')

# Apply the function to count bursts for no stimulation and stimulation conditions
whisker_df_manager.dataframes['basic_metrics'][['No_Stim_Burst_Count', 'Stim_Burst_Count']] = whisker_df_manager.dataframes['basic_metrics'].apply(count_bursts_by_stimulation, axis=1, result_type='expand')

# Apply the function to calculate burst counts per cid and LaminarLabel
burst_counts_per_cid_laminar = whisker_df_manager.dataframes['basic_metrics'].apply(count_bursts_by_cid_and_laminar, axis=1)

# Prepare the data for plotting
#melted_data_laminar = pd.melt(burst_counts_per_cid_laminar, id_vars=['groupname', 'recordingname', 'cid', 'LaminarLabel'], 
#                              value_vars=['No_Stim_Burst_Count', 'Stim_Burst_Count'],
#                              var_name='Condition', value_name='Burst_Count')


# Example usage with the option to hide outliers
#plot_burst_counts_by_laminar(melted_data_laminar, show_outliers=False)


def filter_and_melt_data(df, cell_type=None, is_single_unit=None, stim_responsivity=None, modulation_index=None):
    """
    Filters the DataFrame based on the specified criteria and melts it for plotting,
    while ensuring that all necessary columns are carried over.

    Parameters:
    df: The DataFrame containing burst data.
    cell_type: Filter based on 'Cell_Type' ('FS', 'RS', or None for no filtering).
    is_single_unit: Filter based on 'IsSingleUnit' (1.0, 0.0, or None for no filtering).
    stim_responsivity: Filter based on 'StimResponsivity' (-1.0, 0.0, 1.0, or None for no filtering).
    modulation_index: Filter based on 'ModulationIndex' ('positive', 'negative', 'none', 'modulated', or None for no filtering).

    Returns:
    melted_data: The melted DataFrame, ready for plotting.
    """
    # Apply filters based on user input
    if cell_type is not None:
        df = df[df['Cell_Type'] == cell_type]
    
    if is_single_unit is not None:
        df = df[df['IsSingleUnit'] == is_single_unit]
    
    if stim_responsivity is not None:
        df = df[df['StimResponsivity'] == stim_responsivity]
    
    if modulation_index is not None:
        if modulation_index == 'modulated':
            df = df[df['ModulationIndex'].isin(['positive', 'negative'])]
        else:
            df = df[df['ModulationIndex'] == modulation_index]

    # Melt the DataFrame for plotting, ensuring all necessary columns are carried over
    melted_data = pd.melt(
        df,
        id_vars=['groupname', 'recordingname', 'cid', 'LaminarLabel', 'Cell_Type', 
                 'IsSingleUnit', 'StimResponsivity', 'ModulationIndex'],
        value_vars=['No_Stim_Burst_Count', 'Stim_Burst_Count'],
        var_name='Condition', value_name='Burst_Count'
    )
    
    return melted_data

melted_data_laminar_filtered = filter_and_melt_data(
    burst_counts_per_cid_laminar,
    cell_type=None,
    is_single_unit=0.0,
    stim_responsivity=None,
    modulation_index='modulated'  # Using the new "modulated" option
)
# Now you can plot the filtered data
plot_burst_counts_by_laminar(melted_data_laminar_filtered, show_outliers=False)


def run_mannwhitneyu_test(data, laminar_label, condition, cell_type=None, is_single_unit=None, stim_responsivity=None, modulation_index=None):
    """
    Perform the Mann-Whitney U test between CTZ and No_CTZ for a given LaminarLabel and Condition,
    with optional filtering based on Cell_Type, IsSingleUnit, StimResponsivity, and ModulationIndex.

    Parameters:
    data: The melted DataFrame containing burst counts.
    laminar_label: The specific LaminarLabel to filter by (e.g., 'SG', 'L4', 'IG').
    condition: The specific condition to filter by ('No_Stim_Burst_Count' or 'Stim_Burst_Count').
    cell_type: Filter based on 'Cell_Type' ('FS', 'RS', or None for no filtering).
    is_single_unit: Filter based on 'IsSingleUnit' (1.0, 0.0, or None for no filtering).
    stim_responsivity: Filter based on 'StimResponsivity' (-1.0, 0.0, 1.0, or None for no filtering).
    modulation_index: Filter based on 'ModulationIndex' ('positive', 'negative', 'none', 'modulated', or None for no filtering).

    Returns:
    A dictionary containing the test results: U statistic, p-value, and whether the result is significant.
    """
    # Filter the data for the specified LaminarLabel and Condition
    subset = data[(data['LaminarLabel'] == laminar_label) & (data['Condition'] == condition)]
    
    # Apply additional filters based on user input
    if cell_type is not None:
        subset = subset[subset['Cell_Type'] == cell_type]
    
    if is_single_unit is not None:
        subset = subset[subset['IsSingleUnit'] == is_single_unit]
    
    if stim_responsivity is not None:
        subset = subset[subset['StimResponsivity'] == stim_responsivity]
    
    if modulation_index is not None:
        if modulation_index == 'modulated':
            subset = subset[subset['ModulationIndex'].isin(['positive', 'negative'])]
        else:
            subset = subset[subset['ModulationIndex'] == modulation_index]

    # Separate the data into the two groups
    ctz_data = subset[subset['groupname'] == 'CTZ']['Burst_Count']
    no_ctz_data = subset[subset['groupname'] == 'No_CTZ']['Burst_Count']
    
    # Perform the Mann-Whitney U test
    u_stat, p_value = mannwhitneyu(ctz_data, no_ctz_data, alternative='two-sided')
    
    # Determine significance (typically p < 0.05 is considered significant)
    significance = p_value < 0.05
    
    return {
        'LaminarLabel': laminar_label,
        'Condition': condition,
        'Cell_Type': cell_type,
        'IsSingleUnit': is_single_unit,
        'StimResponsivity': stim_responsivity,
        'ModulationIndex': modulation_index,
        'U Statistic': u_stat,
        'p-value': p_value,
        'Significant': significance
    }


# Example usage with filtering and running the test for each combination of LaminarLabel and Condition
results = []
for laminar_label in ['SG', 'L4', 'IG']:
    for condition in ['No_Stim_Burst_Count', 'Stim_Burst_Count']:
        result = run_mannwhitneyu_test(
            melted_data_laminar_filtered, 
            laminar_label, 
            condition, 
            cell_type=None,
            is_single_unit=0.0,
            stim_responsivity=None,
            modulation_index='modulated')  # Using the new "modulated" option)

        results.append(result)

# Convert the results to a DataFrame for easier interpretation
results_df = pd.DataFrame(results)

# Display the results
results_df

In [ ]:
def run_mannwhitneyu_test(data, laminar_label, condition, plot_filename, cell_type=None, is_single_unit=None, stim_responsivity=None, modulation_index=None):
    """
    Perform the Mann-Whitney U test between CTZ and No_CTZ for a given LaminarLabel and Condition,
    with optional filtering based on Cell_Type, IsSingleUnit, StimResponsivity, and ModulationIndex.
    Also calculates descriptive statistics (N, mean, median, SD, min, max) for both groups.

    Parameters:
    data: The melted DataFrame containing burst counts.
    laminar_label: The specific LaminarLabel to filter by (e.g., 'SG', 'L4', 'IG').
    condition: The specific condition to filter by ('No_Stim_Burst_Count' or 'Stim_Burst_Count').
    plot_filename: The name of the plot file that was saved.
    cell_type: Filter based on 'Cell_Type' ('FS', 'RS', or None for no filtering).
    is_single_unit: Filter based on 'IsSingleUnit' (1.0, 0.0, or None for no filtering).
    stim_responsivity: Filter based on 'StimResponsivity' (-1.0, 0.0, 1.0, or None for no filtering).
    modulation_index: Filter based on 'ModulationIndex' ('positive', 'negative', 'none', 'modulated', or None for no filtering).

    Returns:
    A dictionary containing the test results: U statistic, p-value, whether the result is significant,
    and descriptive statistics for both No_CTZ and CTZ groups, and the name of the plot file.
    """
    # Filter the data for the specified LaminarLabel and Condition
    subset = data[(data['LaminarLabel'] == laminar_label) & (data['Condition'] == condition)]
    
    # Apply additional filters based on user input
    if cell_type is not None:
        subset = subset[subset['Cell_Type'] == cell_type]
    
    if is_single_unit is not None:
        subset = subset[subset['IsSingleUnit'] == is_single_unit]
    
    if stim_responsivity is not None:
        subset = subset[subset['StimResponsivity'] == stim_responsivity]
    
    if modulation_index is not None:
        if modulation_index == 'modulated':
            subset = subset[subset['ModulationIndex'].isin(['positive', 'negative'])]
        else:
            subset = subset[subset['ModulationIndex'] == modulation_index]

    # Separate the data into the two groups
    ctz_data = subset[subset['groupname'] == 'CTZ']['Burst_Count']
    no_ctz_data = subset[subset['groupname'] == 'No_CTZ']['Burst_Count']
    
    # Check if both groups have data
    if ctz_data.empty or no_ctz_data.empty:
        return {
            'LaminarLabel': laminar_label,
            'Condition': condition,
            'Cell_Type': cell_type,
            'IsSingleUnit': is_single_unit,
            'StimResponsivity': stim_responsivity,
            'ModulationIndex': modulation_index,
            'U Statistic': None,
            'p-value': None,
            'Significant': False,
            'CTZ_N': len(ctz_data),
            'CTZ_Mean': np.nan,
            'CTZ_Median': np.nan,
            'CTZ_SD': np.nan,
            'CTZ_Min': np.nan,
            'CTZ_Max': np.nan,
            'No_CTZ_N': len(no_ctz_data),
            'No_CTZ_Mean': np.nan,
            'No_CTZ_Median': np.nan,
            'No_CTZ_SD': np.nan,
            'No_CTZ_Min': np.nan,
            'No_CTZ_Max': np.nan,
            'Note': f"Test not performed due to insufficient data; Plot saved as {plot_filename}"
        }
    
    # Perform the Mann-Whitney U test
    u_stat, p_value = mannwhitneyu(ctz_data, no_ctz_data, alternative='two-sided')
    
    # Determine significance (typically p < 0.05 is considered significant)
    significance = p_value < 0.05

    # Calculate descriptive statistics for CTZ
    ctz_mean = ctz_data.mean()
    ctz_median = ctz_data.median()
    ctz_sd = ctz_data.std()
    ctz_min = ctz_data.min()
    ctz_max = ctz_data.max()
    ctz_n = len(ctz_data)
    
    # Calculate descriptive statistics for No_CTZ
    no_ctz_mean = no_ctz_data.mean()
    no_ctz_median = no_ctz_data.median()
    no_ctz_sd = no_ctz_data.std()
    no_ctz_min = no_ctz_data.min()
    no_ctz_max = no_ctz_data.max()
    no_ctz_n = len(no_ctz_data)
    
    return {
        'LaminarLabel': laminar_label,
        'Condition': condition,
        'Cell_Type': cell_type,
        'IsSingleUnit': is_single_unit,
        'StimResponsivity': stim_responsivity,
        'ModulationIndex': modulation_index,
        'U Statistic': u_stat,
        'p-value': p_value,
        'Significant': significance,
        'CTZ_N': ctz_n,
        'CTZ_Mean': ctz_mean,
        'CTZ_Median': ctz_median,
        'CTZ_SD': ctz_sd,
        'CTZ_Min': ctz_min,
        'CTZ_Max': ctz_max,
        'No_CTZ_N': no_ctz_n,
        'No_CTZ_Mean': no_ctz_mean,
        'No_CTZ_Median': no_ctz_median,
        'No_CTZ_SD': no_ctz_sd,
        'No_CTZ_Min': no_ctz_min,
        'No_CTZ_Max': no_ctz_max,
        'Note': f"Plot saved as {plot_filename}"
    }
def plot_burst_counts_by_laminar(melted_data, show_outliers=True):
    """
    Plots the burst counts broken down by LaminarLabel and optionally omits outliers.

    Parameters:
    melted_data: The melted DataFrame containing burst counts.
    show_outliers: Boolean indicating whether to display outliers in the box plot (default is True).

    Returns:
    g: The FacetGrid object, which can be used for saving the plot.
    """
    # Define the color palette
    palette = {"CTZ": "purple", "No_CTZ": "grey"}

    # Specify the order of the LaminarLabels
    laminar_order = ['SG', 'L4', 'IG']

    # Specify the hue order to ensure consistent plotting
    hue_order = ["CTZ", "No_CTZ"]

    # Create a FacetGrid to plot data for each LaminarLabel separately
    g = sns.FacetGrid(melted_data, col="LaminarLabel", height=5, aspect=1.5, col_order=laminar_order)

    # Map the boxplot to the grid, with control over outlier display
    g.map(sns.boxplot, "Condition", "Burst_Count", "groupname", 
          palette=palette, hue_order=hue_order, 
          order=['No_Stim_Burst_Count', 'Stim_Burst_Count'], showfliers=show_outliers)

    # Set the labels and title
    g.set_axis_labels('Condition', 'Burst Count')
    g.set_titles(col_template='{col_name}')
    g.add_legend(title='Group')

    # Adjust the layout
    plt.subplots_adjust(top=0.85)
    g.fig.suptitle('Comparison of Burst Counts by Laminar Label between CTZ and No_CTZ Groups')

    return g  # Return the FacetGrid object for saving the plot

# Define the output base directory
output_base_dir = "/Volumes/MannySSD/figures/laminaranalysis_bursts"

# Check if the base directory exists, and if not, create it
os.makedirs(output_base_dir, exist_ok=True)

# Define possible values for each filter
cell_types = [None]
is_single_units = [0.0, None]
stim_responsivities = [ 1.0, None]
modulation_indices = ['positive', 'modulated', None]

# Initialize a list to store all results
all_results = []

for cell_type in cell_types:
    for is_single_unit in is_single_units:
        for stim_responsivity in stim_responsivities:
            for modulation_index in modulation_indices:
                # Filter and melt the data
                melted_data_laminar_filtered = filter_and_melt_data(
                    burst_counts_per_cid_laminar,
                    cell_type=cell_type,
                    is_single_unit=is_single_unit,
                    stim_responsivity=stim_responsivity,
                    modulation_index=modulation_index
                )
                
                # Create a hierarchical directory structure for saving the plots
                dir_name = f"{cell_type}_{is_single_unit}_{stim_responsivity}_{modulation_index}"
                dir_name = dir_name.replace("None", "ALL")  # Replace None with ALL for directory naming
                plot_dir = os.path.join(output_base_dir, dir_name)
                os.makedirs(plot_dir, exist_ok=True)

                # Plot the data and save the plot
                plot_filename = f"plot_{cell_type}_{is_single_unit}_{stim_responsivity}_{modulation_index}.svg"
                plot_filepath = os.path.join(plot_dir, plot_filename)
                g = plot_burst_counts_by_laminar(melted_data_laminar_filtered, show_outliers=False)
                g.savefig(plot_filepath, format='svg', transparent=True)
    
                # Close the plot to prevent it from displaying in the notebook
                plt.close(g.fig)

                # Run the Mann-Whitney U test for each combination of LaminarLabel and Condition
                for laminar_label in ['SG', 'L4', 'IG']:
                    for condition in ['No_Stim_Burst_Count', 'Stim_Burst_Count']:
                        result = run_mannwhitneyu_test(
                            melted_data_laminar_filtered,
                            laminar_label,
                            condition,
                            plot_filename,  # Pass the plot filename to the function
                            cell_type=cell_type,
                            is_single_unit=is_single_unit,
                            stim_responsivity=stim_responsivity,
                            modulation_index=modulation_index
                        )
                        all_results.append(result)

# Convert all results to a DataFrame and save to CSV
results_df = pd.DataFrame(all_results)
results_filepath = os.path.join(output_base_dir, "mannwhitneyu_results.csv")
results_df.to_csv(results_filepath, index=False)

print(f"Plots saved to {output_base_dir}")
print(f"Statistical results saved to {results_filepath}")
# Convert all results to a DataFrame and save to CSV
results_df = pd.DataFrame(all_results)
results_filepath = os.path.join(output_base_dir, "mannwhitneyu_results.csv")
results_df.to_csv(results_filepath, index=False)

print(f"Plots saved to {output_base_dir}")
print(f"Statistical results saved to {results_filepath}")


In [ ]:


#repeat this but make it have clear scentences
# Print unique values in the 'Cell_Type' column
unique_cell_types = whisker_df_manager.dataframes['basic_metrics']['Cell_Type'].unique()
print(f"Unique values in 'Cell_Type' column: {unique_cell_types}")

# Print unique values in the 'IsSingleUnit' column
unique_is_single_units = whisker_df_manager.dataframes['basic_metrics']['IsSingleUnit'].unique()

# Print unique values in the 'StimResponsivity' column
unique_stim_responsivities = whisker_df_manager.dataframes['basic_metrics']['StimResponsivity'].unique()

# Print unique values in the 'ModulationIndex' column
unique_modulation_indices = whisker_df_manager.dataframes['basic_metrics']['ModulationIndex'].unique()

# Print unique values in the 'LaminarLabel' column
unique_laminar_labels = whisker_df_manager.dataframes['basic_metrics']['LaminarLabel'].unique()

# Print unique values in the 'IsSingleUnit' column
print(f"Unique values in 'IsSingleUnit' column: {unique_is_single_units}")
print(f"Unique values in 'StimResponsivity' column: {unique_stim_responsivities}")
print(f"Unique values in 'ModulationIndex' column: {unique_modulation_indices}")
print(f"Unique values in 'LaminarLabel' column: {unique_laminar_labels}")
print(f"Unique values in 'StimResponsivity' column: {unique_stim_responsivities}")


In [ ]:
whisker_df_manager.dataframes['basic_metrics'] 

#find the CTZ groups with the most cells in them 
whisker_df_manager.dataframes['basic_metrics'].groupby(['groupname', 'recordingname']).count()


In [ ]:
class RSCAnalysis:
    def __init__(self, whisker_df_manager):
        self.whisker_df_manager = whisker_df_manager
        self.df = whisker_df_manager.dataframes['basic_metrics']
        self.results_df = pd.DataFrame()
        self.color_map = {'CTZ': 'purple', 'No_CTZ': 'grey'}
        self.intensity_order = ['Zero', 'Low', 'Mid', 'Max']

    
    def _bin_spikes(self, spike_times, stim_windows, bin_size, pre_stim_samples, post_stim_samples):
        """Bin spike times into a histogram for each stimulation window."""
        max_window_length = max(offset - onset + pre_stim_samples + post_stim_samples for onset, offset in stim_windows)
        n_bins = int(np.ceil(max_window_length / bin_size))
        
        binned_counts = np.zeros((len(stim_windows), n_bins))
        
        for i, (onset, offset) in enumerate(stim_windows):
            window_start = onset - pre_stim_samples
            window_end = offset + post_stim_samples
            window_spikes = spike_times[(spike_times >= window_start) & (spike_times < window_end)] - window_start
            
            binned_counts[i], _ = np.histogram(window_spikes, bins=n_bins, range=(0, max_window_length))
        
        return binned_counts
    
    def calculate_rsc(self, recording_name, bin_size_ms=50, stim_intensity=None, 
                      pre_stim_ms=0, post_stim_ms=0, cell_type=None, 
                      is_single_unit=None, stim_responsivity=None, 
                      modulation_index=None, laminar_label=None):
        """Calculate rSC correlations between cells for a specific recording and stimulation condition."""
        recording_df = self.df[self.df['recordingname'] == recording_name]
        
        # Get the groupname for this recording
        groupname = recording_df['groupname'].iloc[0] if 'groupname' in recording_df.columns else 'Unknown'
        
        # Apply filters
        if cell_type is not None:
            recording_df = recording_df[recording_df['Cell_Type'] == cell_type]
        if is_single_unit is not None:
            recording_df = recording_df[recording_df['IsSingleUnit'] == is_single_unit]
        if stim_responsivity is not None:
            recording_df = recording_df[recording_df['StimResponsivity'] == stim_responsivity]
        if modulation_index is not None:
            recording_df = recording_df[recording_df['ModulationIndex'] == modulation_index]
        if laminar_label is not None:
            recording_df = recording_df[recording_df['LaminarLabel'] == laminar_label]
        
        if recording_df.empty:
            print(f"Warning: No data available for the specified filters in recording {recording_name}")
            return None, None
        
        sampling_freq = recording_df['Sampling_Frequency'].iloc[0]
        bin_size_samples = int(bin_size_ms * sampling_freq / 1000)
        pre_stim_samples = int(pre_stim_ms * sampling_freq / 1000)
        post_stim_samples = int(post_stim_ms * sampling_freq / 1000)
        
        binned_spikes = {}
        cell_info = {}
        
        for _, row in recording_df.iterrows():
            cid = row['cid']
            spike_times = row['SpikeTimes_all']
            stim_onsets = row['Stim_Onsets_samples']
            stim_offsets = row['Stim_Offsets_samples']
            stim_intensities = row['Stim_Intensity']
            
            # Filter stimulation windows based on intensity
            if stim_intensity is not None:
                stim_windows = [(onset, offset) for onset, offset, intensity in 
                                zip(stim_onsets, stim_offsets, stim_intensities) 
                                if intensity == stim_intensity]
            else:
                stim_windows = list(zip(stim_onsets, stim_offsets))
            
            if not stim_windows:
                print(f"Warning: No stimulation windows found for cell {cid} in recording {recording_name}")
                continue
            
            # Bin the spikes
            binned_counts = self._bin_spikes(spike_times, stim_windows, bin_size_samples, pre_stim_samples, post_stim_samples)
            binned_spikes[cid] = binned_counts.flatten()  # Flatten to 1D array for correlation calculation
            
            cell_info[cid] = {
                'Cell_Type': row['Cell_Type'],
                'LaminarLabel': row['LaminarLabel'],
                'MeanFR_baseline': row['MeanFR_baseline'],
                'MeanFR_stim': row['MeanFR_stim'],
                'IsSingleUnit': row['IsSingleUnit'],
                'StimResponsivity': row['StimResponsivity'],
                'ModulationIndex': row['ModulationIndex']
            }
        
        if len(binned_spikes) < 2:
            print(f"Warning: Not enough cells ({len(binned_spikes)}) for correlation analysis in recording {recording_name}")
            return None, None
        
        # Calculate correlations
        cids = list(binned_spikes.keys())
        num_cells = len(cids)
        corr_matrix = np.zeros((num_cells, num_cells))
        
        for i in range(num_cells):
            for j in range(i+1, num_cells):
                corr, _ = pearsonr(binned_spikes[cids[i]], binned_spikes[cids[j]])
                corr_matrix[i, j] = corr
                corr_matrix[j, i] = corr
        
        corr_df = pd.DataFrame(corr_matrix, index=cids, columns=cids)
        
        # Update results dataframe
        self._update_results_df(recording_name, corr_df, cell_info, bin_size_ms, stim_intensity, pre_stim_ms, post_stim_ms, groupname)
        
        return corr_df, cell_info

    def _update_results_df(self, recording_name, corr_df, cell_info, bin_size_ms, stim_intensity, pre_stim_ms, post_stim_ms, groupname):
        """Update the results dataframe with the calculated correlations and metadata."""
        intensity_labels = {1: 'Zero', 2: 'Low', 3: 'Mid', 4: 'Max'}
        stim_label = intensity_labels.get(stim_intensity, 'All')
        
        new_rows = []
        for cid1 in corr_df.index:
            for cid2 in corr_df.columns:
                if cid1 < cid2:  # Avoid duplicates and self-correlations
                    new_row = {
                        'recording_name': recording_name,
                        'groupname': groupname,  # Add groupname to the results
                        'cid1': cid1,
                        'cid2': cid2,
                        'cell_type1': cell_info[cid1]['Cell_Type'],
                        'cell_type2': cell_info[cid2]['Cell_Type'],
                        'laminar_label1': cell_info[cid1]['LaminarLabel'],
                        'laminar_label2': cell_info[cid2]['LaminarLabel'],
                        'mean_fr_baseline1': cell_info[cid1]['MeanFR_baseline'],
                        'mean_fr_baseline2': cell_info[cid2]['MeanFR_baseline'],
                        'mean_fr_stim1': cell_info[cid1]['MeanFR_stim'],
                        'mean_fr_stim2': cell_info[cid2]['MeanFR_stim'],
                        'is_single_unit1': cell_info[cid1]['IsSingleUnit'],
                        'is_single_unit2': cell_info[cid2]['IsSingleUnit'],
                        'stim_responsivity1': cell_info[cid1]['StimResponsivity'],
                        'stim_responsivity2': cell_info[cid2]['StimResponsivity'],
                        'modulation_index1': cell_info[cid1]['ModulationIndex'],
                        'modulation_index2': cell_info[cid2]['ModulationIndex'],
                        'rsc': corr_df.loc[cid1, cid2],
                        'bin_size_ms': bin_size_ms,
                        'stim_intensity': stim_label,
                        'pre_stim_ms': pre_stim_ms,
                        'post_stim_ms': post_stim_ms
                    }
                    new_rows.append(new_row)
        
        new_df = pd.DataFrame(new_rows)
        self.results_df = pd.concat([self.results_df, new_df], ignore_index=True)

    def process_all_recordings(self, bin_size_ms=50, pre_stim_ms=50, post_stim_ms=100, **filter_params):
        """
        Process all recordings and calculate rSC correlations for each stimulation intensity.
        
        :param bin_size_ms: Size of time bins in milliseconds (default: 50ms)
        :param pre_stim_ms: Time before stimulation onset to include (in ms)
        :param post_stim_ms: Time after stimulation offset to include (in ms)
        :param filter_params: Additional filtering parameters (cell_type, is_single_unit, etc.)
        """
        unique_recordings = self.df['recordingname'].unique()

        for recording_name in unique_recordings:
            print(f"Processing recording: {recording_name}")
            for stim_intensity in range(1, 5):  # 1: Zero, 2: Low, 3: Mid, 4: Max
                self.calculate_rsc(recording_name, bin_size_ms, stim_intensity, pre_stim_ms, post_stim_ms, **filter_params)

    def get_results_df(self):
        """Return the results dataframe."""
        return self.results_df
    
    
    
    def compare_groups_zero_max(self):
        """
        Compare correlations between CTZ and No_CTZ groups for FS-FS, RS-RS, and FS-RS pairs
        during Zero and Max trials.
        
        :return: matplotlib figure object with the plotted box plots
        """
        if 'groupname' not in self.results_df.columns:
            raise ValueError("The 'groupname' column is missing from the results DataFrame. "
                             "Make sure to process the recordings before calling this method.")

        # Filter for Zero and Max trials
        zero_max_df = self.results_df[self.results_df['stim_intensity'].isin(['Zero', 'Max'])]

        # Create cell pair types
        zero_max_df['pair_type'] = zero_max_df.apply(lambda row: 
            'FS-FS' if row['cell_type1'] == 'FS' and row['cell_type2'] == 'FS' else
            'RS-RS' if row['cell_type1'] == 'RS' and row['cell_type2'] == 'RS' else
            'FS-RS', axis=1)

        # Prepare data for plotting
        plot_data = []
        for group in zero_max_df['groupname'].unique():
            group_df = zero_max_df[zero_max_df['groupname'] == group]
            for intensity in ['Zero', 'Max']:
                intensity_df = group_df[group_df['stim_intensity'] == intensity]
                for pair_type in ['FS-FS', 'RS-RS', 'FS-RS']:
                    pair_data = intensity_df[intensity_df['pair_type'] == pair_type]['rsc']
                    plot_data.extend([
                        {'Group': group, 'Intensity': intensity, 'Pair Type': pair_type, 'RSC': rsc}
                        for rsc in pair_data
                    ])

        plot_df = pd.DataFrame(plot_data)

        # Create the box plot
        fig, axes = plt.subplots(1, 2, figsize=(16, 8))
        fig.suptitle('Comparison of RSC: CTZ vs No_CTZ')

        for i, intensity in enumerate(['Zero', 'Max']):
            sns.boxplot(x='Pair Type', y='RSC', hue='Group', 
                        data=plot_df[plot_df['Intensity'] == intensity], ax=axes[i])
            axes[i].set_title(f'{intensity} Intensity')
            axes[i].set_ylabel('RSC (Spike Count Correlation)')
            if i == 1:  # Only show legend for the second plot
                axes[i].legend(title='Group')
            else:
                axes[i].get_legend().remove()

        # Perform statistical tests
        for intensity in ['Zero', 'Max']:
            print(f"\nStatistical tests for {intensity} intensity:")
            for pair_type in ['FS-FS', 'RS-RS', 'FS-RS']:
                group_names = plot_df['Group'].unique()
                if len(group_names) == 2:
                    group1_data = plot_df[(plot_df['Group'] == group_names[0]) & 
                                          (plot_df['Intensity'] == intensity) & 
                                          (plot_df['Pair Type'] == pair_type)]['RSC']
                    group2_data = plot_df[(plot_df['Group'] == group_names[1]) & 
                                          (plot_df['Intensity'] == intensity) & 
                                          (plot_df['Pair Type'] == pair_type)]['RSC']
                    
                    t_stat, p_value = ttest_ind(group1_data, group2_data)
                    print(f"{pair_type}: t-statistic = {t_stat:.4f}, p-value = {p_value:.4f}")
                else:
                    print(f"Warning: Expected 2 groups, but found {len(group_names)}. Skipping statistical test.")

        plt.tight_layout()
        return fig
    
    def compare_all_intensities(self):
        """
        Compare correlations between CTZ and No_CTZ groups for FS-FS, RS-RS, and FS-RS pairs
        across all four intensities.
        
        :return: matplotlib figure object with the plotted box plots
        """
        if 'groupname' not in self.results_df.columns:
            raise ValueError("The 'groupname' column is missing from the results DataFrame. "
                             "Make sure to process the recordings before calling this method.")

        # Create cell pair types
        self.results_df['pair_type'] = self.results_df.apply(lambda row: 
            'FS-FS' if row['cell_type1'] == 'FS' and row['cell_type2'] == 'FS' else
            'RS-RS' if row['cell_type1'] == 'RS' and row['cell_type2'] == 'RS' else
            'FS-RS', axis=1)

        # Prepare data for plotting
        plot_data = []
        for group in self.results_df['groupname'].unique():
            group_df = self.results_df[self.results_df['groupname'] == group]
            for intensity in self.intensity_order:
                intensity_df = group_df[group_df['stim_intensity'] == intensity]
                for pair_type in ['FS-FS', 'RS-RS', 'FS-RS']:
                    pair_data = intensity_df[intensity_df['pair_type'] == pair_type]['rsc']
                    plot_data.extend([
                        {'Group': group, 'Intensity': intensity, 'Pair Type': pair_type, 'RSC': rsc}
                        for rsc in pair_data
                    ])

        plot_df = pd.DataFrame(plot_data)

        # Create the box plot
        fig, ax = plt.subplots(figsize=(16, 8))
        fig.suptitle('Comparison of RSC: CTZ vs No_CTZ (All Intensities)')

        sns.boxplot(x='Intensity', y='RSC', hue='Group', 
                    data=plot_df, palette=self.color_map, 
                    order=self.intensity_order, 
                    ax=ax)
        ax.set_ylabel('RSC (Spike Count Correlation)')
        ax.legend(title='Group')

        # Perform statistical tests
        for intensity in self.intensity_order:
            print(f"\nStatistical tests for {intensity} intensity:")
            for pair_type in ['FS-FS', 'RS-RS', 'FS-RS']:
                group_names = plot_df['Group'].unique()
                if len(group_names) == 2:
                    group1_data = plot_df[(plot_df['Group'] == group_names[0]) & 
                                          (plot_df['Intensity'] == intensity) & 
                                          (plot_df['Pair Type'] == pair_type)]['RSC']
                    group2_data = plot_df[(plot_df['Group'] == group_names[1]) & 
                                          (plot_df['Intensity'] == intensity) & 
                                          (plot_df['Pair Type'] == pair_type)]['RSC']
                    
                    t_stat, p_value = ttest_ind(group1_data, group2_data)
                    print(f"{pair_type}: t-statistic = {t_stat:.4f}, p-value = {p_value:.4f}")
                else:
                    print(f"Warning: Expected 2 groups, but found {len(group_names)}. Skipping statistical test.")

        plt.tight_layout()
        return fig

    def compare_zero_vs_pooled(self):
        """
        Compare correlations between CTZ and No_CTZ groups for FS-FS, RS-RS, and FS-RS pairs
        for Zero intensity vs pooled Low/Mid/Max intensities.
        
        :return: matplotlib figure object with the plotted box plots
        """
        if 'groupname' not in self.results_df.columns:
            raise ValueError("The 'groupname' column is missing from the results DataFrame. "
                             "Make sure to process the recordings before calling this method.")

        # Create cell pair types
        self.results_df['pair_type'] = self.results_df.apply(lambda row: 
            'FS-FS' if row['cell_type1'] == 'FS' and row['cell_type2'] == 'FS' else
            'RS-RS' if row['cell_type1'] == 'RS' and row['cell_type2'] == 'RS' else
            'FS-RS', axis=1)

        # Prepare data for plotting
        plot_data = []
        for group in self.results_df['groupname'].unique():
            group_df = self.results_df[self.results_df['groupname'] == group]
            
            # Zero intensity
            zero_df = group_df[group_df['stim_intensity'] == 'Zero']
            for pair_type in ['FS-FS', 'RS-RS', 'FS-RS']:
                pair_data = zero_df[zero_df['pair_type'] == pair_type]['rsc']
                plot_data.extend([
                    {'Group': group, 'Intensity': 'Zero', 'Pair Type': pair_type, 'RSC': rsc}
                    for rsc in pair_data
                ])
            
            # Pooled Low/Mid/Max intensities
            pooled_df = group_df[group_df['stim_intensity'].isin(['Low', 'Mid', 'Max'])]
            for pair_type in ['FS-FS', 'RS-RS', 'FS-RS']:
                pair_data = pooled_df[pooled_df['pair_type'] == pair_type]['rsc']
                plot_data.extend([
                    {'Group': group, 'Intensity': 'Pooled', 'Pair Type': pair_type, 'RSC': rsc}
                    for rsc in pair_data
                ])

        plot_df = pd.DataFrame(plot_data)

        # Create the box plot
        fig, ax = plt.subplots(figsize=(12, 6))
        fig.suptitle('Comparison of RSC: CTZ vs No_CTZ (Zero vs Pooled)')

        sns.boxplot(x='Intensity', y='RSC', hue='Group', 
                    data=plot_df, palette=self.color_map, 
                    order=['Zero', 'Pooled'], 
                    ax=ax)
        ax.set_ylabel('RSC (Spike Count Correlation)')
        ax.legend(title='Group')

        # Perform statistical tests
        for intensity in ['Zero', 'Pooled']:
            print(f"\nStatistical tests for {intensity} intensity:")
            for pair_type in ['FS-FS', 'RS-RS', 'FS-RS']:
                group_names = plot_df['Group'].unique()
                if len(group_names) == 2:
                    group1_data = plot_df[(plot_df['Group'] == group_names[0]) & 
                                          (plot_df['Intensity'] == intensity) & 
                                          (plot_df['Pair Type'] == pair_type)]['RSC']
                    group2_data = plot_df[(plot_df['Group'] == group_names[1]) & 
                                          (plot_df['Intensity'] == intensity) & 
                                          (plot_df['Pair Type'] == pair_type)]['RSC']
                    
                    t_stat, p_value = ttest_ind(group1_data, group2_data)
                    print(f"{pair_type}: t-statistic = {t_stat:.4f}, p-value = {p_value:.4f}")
                else:
                    print(f"Warning: Expected 2 groups, but found {len(group_names)}. Skipping statistical test.")

        plt.tight_layout()
        return fig

    def apply_filters(self, df, filters):
        """
        Apply filters to the DataFrame based on the provided filter dictionary.
        
        :param df: DataFrame to filter
        :param filters: Dictionary of filters to apply
        :return: Filtered DataFrame
        """
        filtered_df = df.copy()
        for column, values in filters.items():
            if isinstance(values, (list, tuple, np.ndarray)):
                filtered_df = filtered_df[filtered_df[column].isin(values)]
            else:
                filtered_df = filtered_df[filtered_df[column] == values]
        return filtered_df

    def compare_all_intensities(self, filters=None):
        """
        Compare correlations between CTZ and No_CTZ groups for FS-FS, RS-RS, and FS-RS pairs
        across all four intensities, with optional filtering.
        
        :param filters: Dictionary of filters to apply (e.g., {'IsSingleUnit': 1.0, 'StimResponsivity': [0.0, 1.0]})
        :return: matplotlib figure object with the plotted box plots
        """
        if 'groupname' not in self.results_df.columns:
            raise ValueError("The 'groupname' column is missing from the results DataFrame. "
                             "Make sure to process the recordings before calling this method.")

        # Apply filters if provided
        df = self.results_df if filters is None else self.apply_filters(self.results_df, filters)

        # Create cell pair types
        df['pair_type'] = df.apply(lambda row: 
            'FS-FS' if row['cell_type1'] == 'FS' and row['cell_type2'] == 'FS' else
            'RS-RS' if row['cell_type1'] == 'RS' and row['cell_type2'] == 'RS' else
            'FS-RS', axis=1)

        # Prepare data for plotting
        plot_data = []
        for group in df['groupname'].unique():
            group_df = df[df['groupname'] == group]
            for intensity in self.intensity_order:
                intensity_df = group_df[group_df['stim_intensity'] == intensity]
                for pair_type in ['FS-FS', 'RS-RS', 'FS-RS']:
                    pair_data = intensity_df[intensity_df['pair_type'] == pair_type]['rsc']
                    plot_data.extend([
                        {'Group': group, 'Intensity': intensity, 'Pair Type': pair_type, 'RSC': rsc}
                        for rsc in pair_data
                    ])

        plot_df = pd.DataFrame(plot_data)

        # Create the box plot
        fig, ax = plt.subplots(figsize=(16, 8))
        fig.suptitle('Comparison of RSC: CTZ vs No_CTZ (All Intensities)')

        sns.boxplot(x='Intensity', y='RSC', hue='Group', 
                    data=plot_df, palette=self.color_map, 
                    order=self.intensity_order, 
                    ax=ax)
        ax.set_ylabel('RSC (Spike Count Correlation)')
        ax.legend(title='Group')

        # Add filter information to the plot
        if filters:
            filter_text = "\n".join([f"{k}: {v}" for k, v in filters.items()])
            plt.text(0.95, 0.05, f"Filters applied:\n{filter_text}", 
                     horizontalalignment='right', verticalalignment='bottom', 
                     transform=ax.transAxes, fontsize=8, alpha=0.7)

        # Perform statistical tests
        for intensity in self.intensity_order:
            print(f"\nStatistical tests for {intensity} intensity:")
            for pair_type in ['FS-FS', 'RS-RS', 'FS-RS']:
                group_names = plot_df['Group'].unique()
                if len(group_names) == 2:
                    group1_data = plot_df[(plot_df['Group'] == group_names[0]) & 
                                          (plot_df['Intensity'] == intensity) & 
                                          (plot_df['Pair Type'] == pair_type)]['RSC']
                    group2_data = plot_df[(plot_df['Group'] == group_names[1]) & 
                                          (plot_df['Intensity'] == intensity) & 
                                          (plot_df['Pair Type'] == pair_type)]['RSC']
                    
                    t_stat, p_value = ttest_ind(group1_data, group2_data)
                    print(f"{pair_type}: t-statistic = {t_stat:.4f}, p-value = {p_value:.4f}")
                else:
                    print(f"Warning: Expected 2 groups, but found {len(group_names)}. Skipping statistical test.")

        plt.tight_layout()
        return fig

    def compare_zero_vs_pooled(self, filters=None):
        """
        Compare correlations between CTZ and No_CTZ groups for FS-FS, RS-RS, and FS-RS pairs
        for Zero intensity vs pooled Low/Mid/Max intensities, with optional filtering.
        
        :param filters: Dictionary of filters to apply (e.g., {'IsSingleUnit': 1.0, 'StimResponsivity': [0.0, 1.0]})
        :return: matplotlib figure object with the plotted box plots
        """
        if 'groupname' not in self.results_df.columns:
            raise ValueError("The 'groupname' column is missing from the results DataFrame. "
                             "Make sure to process the recordings before calling this method.")

        # Apply filters if provided
        df = self.results_df if filters is None else self.apply_filters(self.results_df, filters)

        # Create cell pair types
        df['pair_type'] = df.apply(lambda row: 
            'FS-FS' if row['cell_type1'] == 'FS' and row['cell_type2'] == 'FS' else
            'RS-RS' if row['cell_type1'] == 'RS' and row['cell_type2'] == 'RS' else
            'FS-RS', axis=1)

        # Prepare data for plotting
        plot_data = []
        for group in df['groupname'].unique():
            group_df = df[df['groupname'] == group]
            
            # Zero intensity
            zero_df = group_df[group_df['stim_intensity'] == 'Zero']
            for pair_type in ['FS-FS', 'RS-RS', 'FS-RS']:
                pair_data = zero_df[zero_df['pair_type'] == pair_type]['rsc']
                plot_data.extend([
                    {'Group': group, 'Intensity': 'Zero', 'Pair Type': pair_type, 'RSC': rsc}
                    for rsc in pair_data
                ])
            
            # Pooled Low/Mid/Max intensities
            pooled_df = group_df[group_df['stim_intensity'].isin(['Low', 'Mid', 'Max'])]
            for pair_type in ['FS-FS', 'RS-RS', 'FS-RS']:
                pair_data = pooled_df[pooled_df['pair_type'] == pair_type]['rsc']
                plot_data.extend([
                    {'Group': group, 'Intensity': 'Pooled', 'Pair Type': pair_type, 'RSC': rsc}
                    for rsc in pair_data
                ])

        plot_df = pd.DataFrame(plot_data)

        # Create the box plot
        fig, ax = plt.subplots(figsize=(12, 6))
        fig.suptitle('Comparison of RSC: CTZ vs No_CTZ (Zero vs Pooled)')

        sns.boxplot(x='Intensity', y='RSC', hue='Group', 
                    data=plot_df, palette=self.color_map, 
                    order=['Zero', 'Pooled'], 
                    ax=ax)
        ax.set_ylabel('RSC (Spike Count Correlation)')
        ax.legend(title='Group')

        # Add filter information to the plot
        if filters:
            filter_text = "\n".join([f"{k}: {v}" for k, v in filters.items()])
            plt.text(0.95, 0.05, f"Filters applied:\n{filter_text}", 
                     horizontalalignment='right', verticalalignment='bottom', 
                     transform=ax.transAxes, fontsize=8, alpha=0.7)

        # Perform statistical tests
        for intensity in ['Zero', 'Pooled']:
            print(f"\nStatistical tests for {intensity} intensity:")
            for pair_type in ['FS-FS', 'RS-RS', 'FS-RS']:
                group_names = plot_df['Group'].unique()
                if len(group_names) == 2:
                    group1_data = plot_df[(plot_df['Group'] == group_names[0]) & 
                                          (plot_df['Intensity'] == intensity) & 
                                          (plot_df['Pair Type'] == pair_type)]['RSC']
                    group2_data = plot_df[(plot_df['Group'] == group_names[1]) & 
                                          (plot_df['Intensity'] == intensity) & 
                                          (plot_df['Pair Type'] == pair_type)]['RSC']
                    
                    t_stat, p_value = ttest_ind(group1_data, group2_data)
                    print(f"{pair_type}: t-statistic = {t_stat:.4f}, p-value = {p_value:.4f}")
                else:
                    print(f"Warning: Expected 2 groups, but found {len(group_names)}. Skipping statistical test.")

        plt.tight_layout()
        return fig



# Example usage:
rsc_analyzer = RSCAnalysis(whisker_df_manager)
rsc_analyzer.process_all_recordings()

# Define filters
filters = {
    'IsSingleUnit': 1.0,
    'StimResponsivity': [0.0, 1.0],
    'ModulationIndex': 'positive',
    'LaminarLabel': ['IG', 'SG']
}

In [ ]:
# Compare Zero vs Pooled
fig_pooled = rsc_analyzer.compare_zero_vs_pooled()

Well need to revisit if you wan to work with CV

In [ ]:
cv_analysis = CV_Analysis(whisker_df_manager)

# Analyzing a single cell
groupname = 'CTZ'
recordingname = 'ctz_3836'
cid = 'cid107'  # Replace with the actual CID of the cell you're interested in

firing_rate_dict = cv_analysis.analyze_single_cell(groupname, recordingname, cid)

#plot the firing_rate_dict where he results are stored in a dictionary where the keys are the start times of each window (in seconds), and the values are the corresponding firing rates.
plt.figure(figsize=(12, 6))
plt.plot(list(firing_rate_dict.keys()), list(firing_rate_dict.values()), color='b')
plt.xlabel('Time (s)')

In [ ]:
# Filter the DataFrame to include only single units (IsSingleUnit == 1.0) and FS or RS cell types
filtered_df = whisker_df_manager.dataframes['basic_metrics'][
    (whisker_df_manager.dataframes['basic_metrics']['IsSingleUnit'] == 1.0) & 
    (whisker_df_manager.dataframes['basic_metrics']['Cell_Type'].isin(['FS', 'RS']))
]

# Define a function to extract the first element from a numpy array, or return the value as-is if it's already a scalar
def extract_first_element(value):
    if isinstance(value, np.ndarray):
        return value.item()  # Extract the scalar from a single-element numpy array
    return value

# Apply this function to the relevant columns in the filtered DataFrame
filtered_df['StimResponsivity'] = filtered_df['StimResponsivity'].apply(extract_first_element)
filtered_df['ModulationIndex'] = filtered_df['ModulationIndex'].apply(extract_first_element)

# Define the combinations
groupnames = ['CTZ', 'No_CTZ']
cell_types = ['FS', 'RS']
stim_responsivity_values = [1.0, 0.0, -1.0]
modulation_label_values = ['positive', 'negative', 'none']

# Initialize an empty list to store the results
results = []

# Iterate over all combinations
for groupname in groupnames:
    for cell_type in cell_types:
        for stim_responsivity in stim_responsivity_values:
            for modulation_label in modulation_label_values:
                    # Apply detailed filtering logic
                    filtered_specific_df = filtered_df[
                        (filtered_df['groupname'] == groupname) &
                        (filtered_df['Cell_Type'] == cell_type) &
                        (filtered_df['StimResponsivity'] == stim_responsivity) &
                        (filtered_df['ModulationIndex'] == modulation_label)
                    ]
                    
                    # If there are no records matching the filter, skip to the next iteration
                    if filtered_specific_df.empty:
                        continue
                    
                    # Extract the recording names and cell ids
                    recordings_and_cells = filtered_specific_df[['recordingname', 'cid']]
                    
                    # Store the number and identity of the recordings and cells
                    num_unique_recordings = recordings_and_cells['recordingname'].nunique()
                    unique_recording_names = recordings_and_cells['recordingname'].unique()
                    num_unique_cells = recordings_and_cells['cid'].nunique()
                    unique_cell_ids = recordings_and_cells['cid'].unique()
                    
                    # Append the results to the list
                    results.append({
                        'groupname': groupname,
                        'CellType': cell_type,
                        'StimResponsivity': stim_responsivity,
                        'ModulationIndex': modulation_label,
                        'NumUniqueRecordings': num_unique_recordings,
                        'UniqueRecordingNames': unique_recording_names,
                        'NumUniqueCells': num_unique_cells,
                        'UniqueCellIDs': unique_cell_ids
                    })

# Convert the results list into a DataFrame
SUA_breakdown_df = pd.DataFrame(results)
SUA_breakdown_df.to_csv('/Volumes/MannySSD/figures/comparing_conditions_dynamic/all_stimulations/SUA_breakdown_df.csv', index=False)

#### get the breakdown of how the untis are being broken up


In [ ]:
whisker_df_manager.generate_summary_statistics('IsSingleUnit', 'basic_metrics') # was the ISI violation less than 1.5%? If True then it is a single unit, if False then it is a multiunit

### this is important reminder as to how the data is being stored, notice the repetition, need to fix later, 
Use the print statement to help you remember how things are stored

In [ ]:
# this accesses the data from the dataframe['psths_dataframe_{label}]['SpikeTrains_for_PSTHs'][i] 
# where i is the index of the cell in the dataframe and returns numpy.ndarray (trial_n x time=1500 ms, 1ms bins) of the spike trains
# in rows and time in columns. Columns will always be 1ms bins at 1500ms duration
# zero
print(type(whisker_df_manager.dataframes['psth_dataframe_Zero']['SpikeTrains_for_PSTHs'][0]))
print(whisker_df_manager.dataframes['psth_dataframe_Zero']['SpikeTrains_for_PSTHs'][0].shape)
# low
print(type(whisker_df_manager.dataframes['psth_dataframe_Low']['SpikeTrains_for_PSTHs'][0]))
print(whisker_df_manager.dataframes['psth_dataframe_Low']['SpikeTrains_for_PSTHs'][0].shape)
# mid
print(type(whisker_df_manager.dataframes['psth_dataframe_Mid']['SpikeTrains_for_PSTHs'][0]))
print(whisker_df_manager.dataframes['psth_dataframe_Mid']['SpikeTrains_for_PSTHs'][0].shape)
# max
print(type(whisker_df_manager.dataframes['psth_dataframe_Max']['SpikeTrains_for_PSTHs'][0]))
print(whisker_df_manager.dataframes['psth_dataframe_Max']['SpikeTrains_for_PSTHs'][0].shape)

In [ ]:
#this is a pandas.core.series.Series , contains the raw calculated PSTHs from Matlab 
print(type(whisker_df_manager.dataframes['psth_dataframe_Zero']['PSTHs_raw'])) 
# shape of (n, ) where n is the number of cells in the DataFrame
print(whisker_df_manager.dataframes['psth_dataframe_Zero']['PSTHs_raw'].shape) 

#this is a numpy.ndarray, contains the raw calculated PSTHs from MATLAB for the first cell id in the DataFrame
print(type(whisker_df_manager.dataframes['psth_dataframe_Zero']['PSTHs_raw'][0]))
# shape is (n,) where n is the number of bins which is 1500 1 ms bins and the values are the calculated PSTHs 
print(whisker_df_manager.dataframes['psth_dataframe_Zero']['PSTHs_raw'][0].shape) 

### plot Wavefornms (norm or unormalized) aplot raster, or rasters with PSTHS --> to to make more efficients to plot dynamically

In [ ]:
def iterate_and_plot_waveforms(df_manager, output_dir, show=False):
    # Define the parameters to iterate over
    cell_types = ['FS', 'RS']
    is_single_units = [1.0, 0.0, None]
    stim_responsivities = [1.0, 0.0, None]

    # Iterate over each combination of parameters
    for cell_type in cell_types:
        for is_single_unit in is_single_units:
            for stim_responsivity in stim_responsivities:
                # Filter the DataFrame based on current parameters using the df_manager
                df_filtered = df_manager.get_filtered_data(
                    'basic_metrics', 
                    is_single_unit=is_single_unit, 
                    cell_type=cell_type, 
                    stim_responsivity=stim_responsivity
                )
                
                # Skip iteration if the filtered DataFrame is empty
                if df_filtered.empty:
                    continue

                # Define folder names based on the current parameters
                is_single_unit_str = 'SUA' if is_single_unit == 1.0 else 'MUA' if is_single_unit == 0.0 else 'ALL'
                stim_responsivity_str = 'sensory_responsive' if stim_responsivity == 1.0 else 'non_responsive' if stim_responsivity == 0.0 else 'ALL'
                folder_suffix = f"{cell_type}_{is_single_unit_str}_stim_{stim_responsivity_str}"

                # Plot normalized waveforms
                iterate_and_plot_normalizedwaveforms(df_filtered, output_dir, f'{folder_suffix}_normalized_waveforms', show)

                # Plot unnormalized waveforms
                iterate_and_plot_unnormalizedwaveforms(df_filtered, output_dir, f'{folder_suffix}_unnormalized_waveforms', show)

def iterate_and_plot_normalizedwaveforms(df, output_dir, folder_name, show=False):
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Create the folder within the output directory
    save_dir = os.path.join(output_dir, folder_name)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    unique_combinations = df[['groupname', 'recordingname', 'cid']].drop_duplicates()

    for idx, row in unique_combinations.iterrows():
        groupname = row['groupname']
        recordingname = row['recordingname']
        cid = row['cid']

        condition = (df['groupname'] == groupname) & (df['recordingname'] == recordingname) & (df['cid'] == cid)
        waveforms = df.loc[condition, 'Normalized_Template_Waveform'].values
        

        if waveforms.size > 0:
            waveform = waveforms[0]
            
            #convert to microvolts by multiplying by 0.25
            waveform = waveform * 0.25

            fig, ax = plt.subplots(figsize=(10, 6))
            ax.plot(waveform)
            ax.set_title(f'Normalized Template Waveform for CID: {cid}, Group: {groupname}, Recording: {recordingname}')
            ax.set_xlabel('Time')
            ax.set_ylabel('Amplitude')

            if show:
                plt.show()
            else:
                save_path = os.path.join(save_dir, f'normalized_waveform_{groupname}_{recordingname}_{cid}.svg')
                fig.savefig(save_path, transparent=True)
            plt.close(fig)          

def iterate_and_plot_unnormalizedwaveforms(df, output_dir, folder_name, show=False):
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Create the folder within the output directory
    save_dir = os.path.join(output_dir, folder_name)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    unique_combinations = df[['groupname', 'recordingname', 'cid']].drop_duplicates()

    for idx, row in unique_combinations.iterrows():
        groupname = row['groupname']
        recordingname = row['recordingname']
        cid = row['cid']

        condition = (df['groupname'] == groupname) & (df['recordingname'] == recordingname) & (df['cid'] == cid)
        waveforms = df.loc[condition, 'UnNormalized_Template_Waveform'].values
        

        if waveforms.size > 0:
            waveform = waveforms[0]
            
            #convert to microvolts by multiplying by 0.25
            waveform = waveform * 0.25

            fig, ax = plt.subplots(figsize=(10, 6))
            ax.plot(waveform)
            ax.set_title(f'UnNormalized Template Waveform for CID: {cid}, Group: {groupname}, Recording: {recordingname}')
            ax.set_xlabel('Time')
            ax.set_ylabel('Amplitude')

            if show:
                plt.show()
            else:
                save_path = os.path.join(save_dir, f'unnormalized_waveform_{groupname}_{recordingname}_{cid}.svg')
                fig.savefig(save_path, transparent=True)
            plt.close(fig)

# Example of how to call the function
iterate_and_plot_waveforms(whisker_df_manager, '/Volumes/MannySSD/figures/waveforms', show=False)

plot rasters or rasters with Raw AND smmoothe PSTH for a specific recording and cid 

In [ ]:
whisker_df_manager.plot_rasters_for_cid_changebinsize('No_CTZ','veh_3062_2','cid109', time_window=(-10, 50), bin_size=1)
whisker_df_manager.plot_rasters_for_cid_changebinsize('No_CTZ','veh_3062_2','cid109', time_window=(-10, 50), bin_size=2)
whisker_df_manager.plot_rasters_for_cid_changebinsize('No_CTZ','veh_3062_2','cid109', time_window=(-10, 50), bin_size=2, filter_empty_trials=True)
whisker_df_manager.plot_combined_psth_and_raster_changebinsize('No_CTZ','veh_3062_2','cid109', time_window=(-20, 50), smoothing_window=3, bin_size=1)
whisker_df_manager.plot_combined_psth_and_raster_changebinsize('No_CTZ','veh_3062_2','cid109', time_window=(-100, 100), smoothing_window=3, bin_size=1)

### plot the raster, raw and smoothed PSTHs in a nested way by group, recording, and cell id that meet the criteria dynamically 

In [ ]:
# Define the parameter sets
cell_types = ['FS', 'RS']
is_single_units = [True, False, None]  # True for SUA, False for MUA, None for ALL
stim_responsivities = [1.0, 0.0, None]  # 1.0 for sensory_responsive, 0.0 for non_responsive, None for ALL

# Map is_single_unit and stim_responsivity to meaningful strings
is_single_unit_map = {True: 'SUA', False: 'MUA', None: 'ALL'}
stim_responsivity_map = {1.0: 'sensory_responsive', 0.0: 'non_responsive', None: 'ALL'}

# Loop over the parameter combinations
for cell_type in cell_types:
    for is_single_unit in is_single_units:
        for stim_responsivity in stim_responsivities:
            
            # Process and store PSTH and raster data
            stored_data = whisker_df_manager.process_and_store_psth_raster_data(
                is_single_unit=is_single_unit, 
                cell_type=cell_type, 
                stim_responsivity=stim_responsivity, 
                time_window=(-100, 600), 
                bin_size=1, 
                smoothing_window=3
            )
            
            
            # Construct folder name
            folder_name = f'PSTH_and_Rasters_{cell_type}_{is_single_unit_map[is_single_unit]}_{stim_responsivity_map[stim_responsivity]}'
            
            # Plot and save the PSTH and raster data
            whisker_df_manager.plot_all_psth_and_raster(
                stored_data, 
                output_dir='/Volumes/MannySSD/figures/psth_raster_data_wholestimulation', 
                folder_name=folder_name, 
                file_format='svg', 
                show=False
            )

## SUA or MUA plotting Then creates descriptive stat, runs kruskall, and permutation (sores in df)

### mass plot different combinations of stim for neurometric curves with box plots and mean/sem plots between groups(can specific any number of stimulation)

In [ ]:
# plot the box and line plots for all combinations of cell types, stim ranges, is_single_units, and stim_responsivities
def dynamic_plot_saving(whisker_df_manager, cell_types, stim_ranges, is_single_units, stim_responsivities, base_directory, stimulations=None):
    """
    Dynamically saves plots across different conditions for specified cell types and parameter combinations.

    Args:
        whisker_df_manager: The data manager object with methods to calculate stats and plot.
        cell_types (list of str): List of cell types (e.g., ['FS', 'RS']).
        stim_ranges (list of tuple): List of stimulation ranges (e.g., [(0, 500), (0, 1000)]).
        is_single_units (list of float): List of is_single_unit values (e.g., [1.0, 0.0, None]).
        stim_responsivities (list of float): List of stim_responsivity values (e.g., [1.0, 0.0, None]).
        base_directory (str): Base directory to save the plots.
        stimulations (list of str, optional): List of stimulations to include in the plot. If None, include all stimulations.
    """
    required_columns = ['mean_stimulation', 'Stimulation', 'Group']
    
    for cell_type in cell_types:
        for stim_range in stim_ranges:
            for is_single_unit in is_single_units:
                for stim_responsivity in stim_responsivities:
                    # Define file name suffix based on parameters
                    is_single_unit_str = 'All' if is_single_unit is None else str(is_single_unit)
                    stim_responsivity_str = 'All' if stim_responsivity is None else str(stim_responsivity)
                    suffix = f"{cell_type}_IsSingleUnit_{is_single_unit_str}_IsStimResponsive_{stim_responsivity_str}_StimRange_{stim_range[0]}to{stim_range[1]}"
                    
                    # Calculate basic stats
                    print(f"Calculating basic stats for: {suffix}")
                    whisker_df_manager.calculate_basic_stats(
                        'CTZ', 'No_CTZ', 
                        baseline_range=(-100, -1), stim_range=stim_range, 
                        cell_type=cell_type, is_single_unit=is_single_unit, 
                        stim_responsivity=stim_responsivity, 
                        smoothing_window=3, modulation_label='positive'
                    )
                    
                    # Prepare data for boxplot
                    print("Preparing data for boxplot")
                    df = whisker_df_manager.prepare_for_boxplot()
                    print(f"DataFrame shape after preparation: {df.shape}")
                    print(f"DataFrame columns: {df.columns}")
                    print(f"First few rows of DataFrame: {df.head()}")

                    # Check if the required columns are present
                    missing_columns = [col for col in required_columns if col not in df.columns]
                    if missing_columns:
                        print(f"Skipping combination {suffix} due to missing columns: {missing_columns}")
                        continue
                    
                    # Define directory for saving plots
                    directory = os.path.join(base_directory, 'boxplots_and_line', suffix)
                    os.makedirs(directory, exist_ok=True)
                    print(f"Directory created: {directory}")
                    
                    # Plot box and strip plots
                    print("Plotting box and strip plots")
                    #whisker_df_manager.plot_box_and_strip(df, 
                    #    groups=['CTZ', 'No_CTZ'], stimulations=stimulations, show_outliers=True,
                    #    directory=directory, file_name=f'{suffix}_boxplot', ylim=None, 
                    #    modulation_label='positive'
                    #)
                    
                    whisker_df_manager.plot_box_and_strip_with_controls(df, 
                        groups=['CTZ', 'No_CTZ'], stimulations=stimulations, show_outliers=True, show_scatter=True, 
                        directory=directory, file_name=f'{suffix}_boxplot', 
                        modulation_label='positive'
                    )
                    
                    # Plot mean and SEM line plots
                    #print("Plotting mean and SEM line plots")
                    #whisker_df_manager.plot_mean_and_sem_lineplot(df, 
                    #    groups=['CTZ', 'No_CTZ'], stimulations=stimulations, directory=directory,
                    #    file_name=f'{suffix}_lineplot', ylim=None, 
                    #    modulation_label='positive'
                    #)

#all possible combinations of cell types, stim ranges, is_single_units, and stim_responsivities

cell_types = ['FS', 'RS']
stim_ranges =  [(0,20)]
is_single_units = [1.0]
stim_responsivities = [1.0]
stimulations = ['Zero', 'Low', 'Mid', 'Max']

# includes all stimulations, default
base_directory = '/Volumes/MannySSD/figures/comparing_conditions_dynamic/all_stimulations'
dynamic_plot_saving(whisker_df_manager, cell_types, stim_ranges, is_single_units, stim_responsivities, base_directory, stimulations)

LAMINAR: repeat this process but sperate it by laminar labels 

In [ ]:
def dynamic_plot_saving_laminar(whisker_df_manager, cell_types, stim_ranges, is_single_units, stim_responsivities, base_directory, laminarlabel, stimulations=None):
    """
    Dynamically saves plots across different conditions for specified cell types, laminar labels, and parameter combinations.

    Args:
        whisker_df_manager: The data manager object with methods to calculate stats and plot.
        cell_types (list of str): List of cell types (e.g., ['FS', 'RS', 'None']).
        stim_ranges (list of tuple): List of stimulation ranges (e.g., [(0, 500), (0,50), (0,20)]).
        is_single_units (list of float): List of is_single_unit values (e.g., [1.0, 0.0, None]).
        stim_responsivities (list of float): List of stim_responsivity values (e.g., [1.0, 0.0, None]).
        base_directory (str): Base directory to save the plots.
        laminarlabel (list of str): List of laminar labels (e.g., ['SG', 'IG', 'L4']).
        stimulations (list of str, optional): List of stimulations to include in the plot. If None, include all stimulations.
    """
    required_columns = ['mean_stimulation', 'Stimulation', 'Group', 'LaminarLabel']
    
    for cell_type in cell_types:
        for stim_range in stim_ranges:
            for is_single_unit in is_single_units:
                for stim_responsivity in stim_responsivities:
                    for laminar in laminarlabel:
                        # Handle the 'None' case for cell_type
                        if cell_type == 'None':
                            suffix = f"AllCells_IsSingleUnit_{str(is_single_unit) if is_single_unit is not None else 'All'}_IsStimResponsive_{str(stim_responsivity) if stim_responsivity is not None else 'All'}_StimRange_{stim_range[0]}to{stim_range[1]}_{laminar}"
                            cell_type_filter = None
                        else:
                            suffix = f"{cell_type}_IsSingleUnit_{str(is_single_unit) if is_single_unit is not None else 'All'}_IsStimResponsive_{str(stim_responsivity) if stim_responsivity is not None else 'All'}_StimRange_{stim_range[0]}to{stim_range[1]}_{laminar}"
                            cell_type_filter = cell_type

                        # Calculate basic stats
                        print(f"Calculating basic stats for: {suffix}")
                        whisker_df_manager.calculate_basic_stats(
                            'CTZ', 'No_CTZ', 
                            baseline_range=(-100, -1), stim_range=stim_range, 
                            cell_type=cell_type_filter, is_single_unit=is_single_unit, 
                            stim_responsivity=stim_responsivity, 
                            smoothing_window=3, modulation_label=None
                        )
                        
                        # Prepare data for boxplot
                        print(f"Preparing data for boxplot, filtering by laminar label: {laminar}")
                        df = whisker_df_manager.prepare_for_boxplot()
                        df = df[df['LaminarLabel'] == laminar]
                        
                        # If not 'All', further filter by cell type
                        if cell_type_filter is not None:
                            df = df[df['Cell_Type'] == cell_type_filter]

                        print(f"DataFrame shape after filtering: {df.shape}")
                        print(f"DataFrame columns: {df.columns}")
                        print(f"First few rows of DataFrame: {df.head()}")

                        # Check if the required columns are present
                        missing_columns = [col for col in required_columns if col not in df.columns]
                        if missing_columns:
                            print(f"Skipping combination {suffix} due to missing columns: {missing_columns}")
                            continue
                        
                        # Define directory for saving plots
                        directory = os.path.join(base_directory, laminar, 'boxplots_and_line', suffix)
                        os.makedirs(directory, exist_ok=True)
                        print(f"Directory created: {directory}")
                        
                        # Plot box and strip plots
                        print("Plotting box and strip plots")
                        whisker_df_manager.plot_box_and_strip(df, 
                            groups=['CTZ', 'No_CTZ'], stimulations=stimulations, show_outliers=True,
                            directory=directory, file_name=f'{suffix}_boxplot', ylim=None, 
                            modulation_label=None
                        )
                        
                        # Plot mean and SEM line plots
                        print("Plotting mean and SEM line plots")
                        whisker_df_manager.plot_mean_and_sem_lineplot(df, 
                            groups=['CTZ', 'No_CTZ'], stimulations=stimulations, directory=directory,
                            file_name=f'{suffix}_lineplot', ylim=None, 
                            modulation_label=None
                        )
                        
cell_types = ['FS', 'RS', None]
stim_ranges =  [(0, 500), (0,50), (0,20)]
is_single_units = [1.0, 0.0,  None]
stim_responsivities = [1.0, None]
laminarlabel = ['SG', 'IG', 'L4']
stimulations = ['Zero', 'Low', 'Mid', 'Max']

# includes all stimulations, default
base_directory = '/Volumes/MannySSD/figures/comparing_conditions_dynamic/all_stimulations_laminar'
dynamic_plot_saving_laminar(whisker_df_manager, cell_types, stim_ranges, is_single_units, stim_responsivities, base_directory, laminarlabel, stimulations)


### debug or mass run statsistical analysis across different combinations of stim for bow plots and mean/sem plots between groups as a function of stimualtion 

ALL PSTHs (not laminar) only PSTHs and nothign else...look below for more 

In [ ]:
import pandas as pd
from scipy.stats import iqr
from scipy.stats import sem, t

def dynamic_statistical_analysis(whisker_df_manager, cell_types, stim_ranges, is_single_units, stim_responsivities, stimulations=None):
    """
    Dynamically collects descriptive statistics across different combinations for specified cell types and parameter combinations.
    
    # it currently filters based on modulation_label='positive' and firstspike_latency=True so keep that in mind

    Args:
        whisker_df_manager: The data manager object with methods to calculate stats and plot.
        cell_types (list of str): List of cell types (e.g., ['FS', 'RS']).
        stim_ranges (list of tuple): List of stimulation ranges (e.g., [(0, 500), (0, 1000)]).
        is_single_units (list of float): List of is_single_unit values (e.g., [1.0, 0.0, None]).
        stim_responsivities (list of float): List of stim_responsivity values (e.g., [1.0, 0.0, None]).
        stimulations (list of str, optional): List of stimulations to include in the analysis. If None, include all stimulations.
    """
    # Initialize results dictionary
    results = {
        'CellType': [],
        'StimRange': [],
        'IsSingleUnit': [],
        'StimResponsivity': [],
        'Stimulation': [],
        'Group': [],
        'N': [],
        'Mean': [],
        'Median': [],
        'SD': [],
        'SEM': [],
        'CI Lower': [],
        'CI Upper': [],
        'Range': []
    }
    
    for cell_type in cell_types:
        for stim_range in stim_ranges:
            for is_single_unit in is_single_units:
                for stim_responsivity in stim_responsivities:
                    # Define file name suffix based on parameters
                    is_single_unit_str = 'All' if is_single_unit is None else str(is_single_unit)
                    stim_responsivity_str = 'All' if stim_responsivity is None else str(stim_responsivity)
                    suffix = f"{cell_type}_IsSingleUnit_{is_single_unit_str}_IsStimResponsive_{stim_responsivity_str}_StimRange_{stim_range[0]}to{stim_range[1]}"
                    
                    # Calculate basic stats
                    print(f"Calculating basic stats for: {suffix}")
                    whisker_df_manager.calculate_basic_stats(
                        'CTZ', 'No_CTZ', stim_label=None, 
                        baseline_range=(-100, -1), stim_range=stim_range, 
                        cell_type=cell_type, is_single_unit=is_single_unit, 
                        stim_responsivity=stim_responsivity, 
                        smoothing_window=3, modulation_label='positive'
                    )
                    
                    # Prepare data for boxplot
                    print("Preparing data for boxplot")
                    df = whisker_df_manager.prepare_for_boxplot()
                    print(f"DataFrame shape after preparation: {df.shape}")
                    print(f"DataFrame columns: {df.columns}")
                    print(f"First few rows of DataFrame: {df.head()}")

                    # Filter data for the specified stimulations
                    if stimulations:
                        df = df[df['Stimulation'].isin(stimulations)]
                    
                    
                    # Check if the required columns are present
                    required_columns = ['mean_stimulation', 'Stimulation', 'Group']
                    missing_columns = [col for col in required_columns if col not in df.columns]
                    if missing_columns:
                        print(f"Skipping combination {suffix} due to missing columns: {missing_columns}")
                        continue

                    # Loop through each stimulation and perform analysis
                    for stim in stimulations or df['Stimulation'].unique():
                        for group in ['CTZ', 'No_CTZ']:
                            group_data = df[(df['Stimulation'] == stim) & (df['Group'] == group)]['mean_stimulation']
                            
                            if group_data.empty:
                                print(f"Skipping {stim} for group {group} due to lack of data")
                                continue
                            
                            # Calculate IQR
                            Q1 = group_data.quantile(0.25)
                            Q3 = group_data.quantile(0.75)
                            IQR = Q3 - Q1
                            
                            print(f"Before IQR filtering: count={group_data.count()}, mean={group_data.mean()}, std={group_data.std()}")
                            
                            
                            # Filter data within IQR
                            filtered_data = group_data[(group_data >= Q1 - 1.5 * IQR) & (group_data <= Q3 + 1.5 * IQR)]
                            
                            print(f"After IQR filtering: count={filtered_data.count()}, mean={filtered_data.mean()}, std={filtered_data.std()}")

                            if filtered_data.empty:
                                print(f"Skipping {stim} for group {group} due to all data being outside IQR")
                                continue
                            
                            # Calculate descriptive statistics
                            group_desc = filtered_data.describe()
                            group_sem = sem(filtered_data, nan_policy='omit')
                            confidence_level = 0.95
                            degrees_freedom = len(filtered_data) - 1
                            confidence_interval = t.interval(confidence_level, degrees_freedom, loc=group_desc['mean'], scale=group_sem)
                            
                            # Store results
                            results['CellType'].append(cell_type)
                            results['StimRange'].append(f"{stim_range[0]}to{stim_range[1]}")
                            results['IsSingleUnit'].append(is_single_unit_str)
                            results['StimResponsivity'].append(stim_responsivity_str)
                            results['Stimulation'].append(stim)
                            results['Group'].append(group)
                            results['N'].append(group_desc['count'])
                            results['Mean'].append(group_desc['mean'])
                            results['Median'].append(filtered_data.median())
                            results['SD'].append(group_desc['std'])
                            results['SEM'].append(group_sem)
                            results['CI Lower'].append(confidence_interval[0])
                            results['CI Upper'].append(confidence_interval[1])
                            results['Range'].append((filtered_data.min(), filtered_data.max()))
    
    # Convert results dictionary to DataFrame
    results_df = pd.DataFrame(results)
    print(f"Results DataFrame:\n{results_df}")

    return results_df
def generate_actual_data(whisker_df_manager, cell_types, stim_ranges, is_single_units, stim_responsivities, stimulations=None):
    """
    Collect actual data across different combinations for specified cell types and parameter combinations.

    Args:
        whisker_df_manager: The data manager object with methods to calculate stats and plot.
        cell_types (list of str): List of cell types (e.g., ['FS', 'RS']).
        stim_ranges (list of tuple): List of stimulation ranges (e.g., [(0, 500), (0, 1000)]).
        is_single_units (list of float): List of is_single_unit values (e.g., [1.0, 0.0, None]).
        stim_responsivities (list of float): List of stim_responsivity values (e.g., [1.0, 0.0, None]).
        stimulations (list of str, optional): List of stimulations to include in the analysis. If None, include all stimulations.
    """
    # Initialize list to collect actual data
    actual_data = []

    for cell_type in cell_types:
        for stim_range in stim_ranges:
            for is_single_unit in is_single_units:
                for stim_responsivity in stim_responsivities:
                    # Define file name suffix based on parameters
                    is_single_unit_str = 'All' if is_single_unit is None else str(is_single_unit)
                    stim_responsivity_str = 'All' if stim_responsivity is None else str(stim_responsivity)
                    suffix = f"{cell_type}_IsSingleUnit_{is_single_unit_str}_IsStimResponsive_{stim_responsivity_str}_StimRange_{stim_range[0]}to{stim_range[1]}"

                    # Calculate basic stats
                    print(f"Calculating basic stats for: {suffix}")
                    whisker_df_manager.calculate_basic_stats(
                        'CTZ', 'No_CTZ', stim_label=None,
                        baseline_range=(-100, -1), stim_range=stim_range,
                        cell_type=cell_type, is_single_unit=is_single_unit,
                        stim_responsivity=stim_responsivity,
                        smoothing_window=3, modulation_label='positive'
                    )

                    # Prepare data for boxplot
                    print("Preparing data for boxplot")
                    df = whisker_df_manager.prepare_for_boxplot()
                    print(f"DataFrame shape after preparation: {df.shape}")
                    print(f"DataFrame columns: {df.columns}")
                    print(f"First few rows of DataFrame: {df.head()}")

                    # Filter data for the specified stimulations
                    if stimulations:
                        df = df[df['Stimulation'].isin(stimulations)]


                    # Check if the required columns are present
                    required_columns = ['mean_stimulation', 'Stimulation', 'Group']
                    missing_columns = [col for col in required_columns if col not in df.columns]
                    if missing_columns:
                        print(f"Skipping combination {suffix} due to missing columns: {missing_columns}")
                        continue

                    # Loop through each stimulation and perform analysis
                    for stim in stimulations or df['Stimulation'].unique():
                        for group in ['CTZ', 'No_CTZ']:
                            group_data = df[(df['Stimulation'] == stim) & (df['Group'] == group)]
                            
                            if group_data.empty:
                                print(f"Skipping {stim} for group {group} due to lack of data")
                                continue

                            # Filter data within IQR
                            Q1 = group_data['mean_stimulation'].quantile(0.25)
                            Q3 = group_data['mean_stimulation'].quantile(0.75)
                            IQR = Q3 - Q1
                            filtered_data = group_data[(group_data['mean_stimulation'] >= Q1 - 1.5 * IQR) & (group_data['mean_stimulation'] <= Q3 + 1.5 * IQR)]
                            
                            if filtered_data.empty:
                                print(f"Skipping {stim} for group {group} due to all data being outside IQR")
                                continue

                            # Add additional columns for metadata
                            filtered_data['CellType'] = cell_type
                            filtered_data['StimRange'] = f"{stim_range[0]}to{stim_range[1]}"
                            filtered_data['IsSingleUnit'] = is_single_unit_str
                            filtered_data['StimResponsivity'] = stim_responsivity_str
                            
                            # Collect the filtered data
                            actual_data.append(filtered_data)

    # Combine all collected data into a single DataFrame
    actual_data_df = pd.concat(actual_data, ignore_index=True)
    print(f"Actual Data DataFrame:\n{actual_data_df}")

    return actual_data_df
def kruskal_wallis_test(data, group_col, value_col):
    """
    Perform Kruskal-Wallis H-test for independent samples.

    Args:
        data (pd.DataFrame): DataFrame containing the data.
        group_col (str): Column name to group by (e.g., 'Group').
        value_col (str): Column name containing the values (e.g., 'mean_stimulation').

    Returns:
        tuple: Kruskal-Wallis H-statistic and p-value.
    """
    # Group data by the specified column and extract values
    groups = [group[value_col].values for name, group in data.groupby(group_col)]

    # Perform Kruskal-Wallis test
    h_stat, p_value = kruskal(*groups)

    return h_stat, p_value
def perform_kruskal_wallis_on_actual_data(actual_data_df, stimulations):
    """
    Perform Kruskal-Wallis test on the actual data DataFrame.

    Args:
        actual_data_df (pd.DataFrame): DataFrame containing the actual data.
        stimulations (list of str): List of stimulations to include in the analysis.

    Returns:
        pd.DataFrame: DataFrame containing the Kruskal-Wallis test results.
    """
    # Initialize results dictionary for Kruskal-Wallis test results
    kruskal_results = {
        'CellType': [],
        'StimRange': [],
        'IsSingleUnit': [],
        'StimResponsivity': [],
        'Stimulation': [],
        'N_CTZ': [],
        'N_No_CTZ': [],
        'Mean_CTZ': [],
        'Mean_No_CTZ': [],
        'H_statistic': [],
        'p_value': []
    }

    for cell_type in actual_data_df['CellType'].unique():
        for stim_range in actual_data_df['StimRange'].unique():
            for is_single_unit in actual_data_df['IsSingleUnit'].unique():
                for stim_responsivity in actual_data_df['StimResponsivity'].unique():
                    for stim in stimulations:
                        subset_df = actual_data_df[
                            (actual_data_df['CellType'] == cell_type) &
                            (actual_data_df['StimRange'] == stim_range) &
                            (actual_data_df['IsSingleUnit'] == is_single_unit) &
                            (actual_data_df['StimResponsivity'] == stim_responsivity) &
                            (actual_data_df['Stimulation'] == stim)
                        ]

                        if subset_df.empty:
                            continue

                        # Separate data for each group
                        group_ctz = subset_df[subset_df['Group'] == 'CTZ']
                        group_no_ctz = subset_df[subset_df['Group'] == 'No_CTZ']

                        # Calculate N and mean for each group
                        n_ctz = len(group_ctz)
                        n_no_ctz = len(group_no_ctz)
                        mean_ctz = group_ctz['mean_stimulation'].mean()
                        mean_no_ctz = group_no_ctz['mean_stimulation'].mean()

                        # Perform Kruskal-Wallis test
                        h_stat, p_value = kruskal_wallis_test(subset_df, group_col='Group', value_col='mean_stimulation')

                        # Store Kruskal-Wallis results
                        kruskal_results['CellType'].append(cell_type)
                        kruskal_results['StimRange'].append(stim_range)
                        kruskal_results['IsSingleUnit'].append(is_single_unit)
                        kruskal_results['StimResponsivity'].append(stim_responsivity)
                        kruskal_results['Stimulation'].append(stim)
                        kruskal_results['N_CTZ'].append(n_ctz)
                        kruskal_results['N_No_CTZ'].append(n_no_ctz)
                        
                        #convert to a float with 3 decimal places
                        mean_ctz = float("{:.3f}".format(mean_ctz))
                        kruskal_results['Mean_CTZ'].append(mean_ctz)
                        
                        #convert to a float with 3 decimal places 
                        mean_no_ctz = float("{:.3f}".format(mean_no_ctz))
                        kruskal_results['Mean_No_CTZ'].append(mean_no_ctz)
                        
                        #convert to a float with 3 decimal places for h_stat
                        h_stat = float("{:.3f}".format(h_stat))
                        kruskal_results['H_statistic'].append(h_stat)
                        
                        
                        #convert to a float with 3 decimal places
                        p_value = float("{:.3f}".format(p_value))
                        kruskal_results['p_value'].append(p_value)
                    

    # Convert Kruskal-Wallis results dictionary to DataFrame
    kruskal_results_df = pd.DataFrame(kruskal_results)
    print(f"Kruskal-Wallis Results DataFrame:\n{kruskal_results_df}")

    return kruskal_results_df
def permutation_test(data, group_col, value_col, n_permutations=1000):
    """
    Perform a permutation test for the specified groups.

    Args:
        data (pd.DataFrame): DataFrame containing the data.
        group_col (str): Column name to group by (e.g., 'Group').
        value_col (str): Column name containing the values (e.g., 'mean_stimulation').
        n_permutations (int): Number of permutations to perform.

    Returns:
        float: p-value from the permutation test.
    """
    # Calculate the actual Kruskal-Wallis test statistic
    groups = [group[value_col].values for name, group in data.groupby(group_col)]
    actual_stat, _ = kruskal(*groups)
    
    # Combine all data values and shuffle group labels to generate the null distribution
    combined_values = data[value_col].values
    group_labels = data[group_col].values
    permuted_stats = []

    for _ in range(n_permutations):
        np.random.shuffle(group_labels)
        permuted_groups = [combined_values[group_labels == label] for label in np.unique(group_labels)]
        perm_stat, _ = kruskal(*permuted_groups)
        permuted_stats.append(perm_stat)
    
    # Calculate the p-value as the proportion of permuted stats greater than or equal to the actual stat
    permuted_stats = np.array(permuted_stats)
    p_value = np.sum(permuted_stats >= actual_stat) / n_permutations

    return p_value
def perform_permutation_test_on_actual_data(actual_data_df, stimulations, n_permutations=1000):
    """
    Perform a permutation test on the actual data DataFrame.

    Args:
        actual_data_df (pd.DataFrame): DataFrame containing the actual data.
        stimulations (list of str): List of stimulations to include in the analysis.
        n_permutations (int): Number of permutations to perform.

    Returns:
        pd.DataFrame: DataFrame containing the permutation test results.
    """
    # Initialize results dictionary for permutation test results
    perm_results = {
        'CellType': [],
        'StimRange': [],
        'IsSingleUnit': [],
        'StimResponsivity': [],
        'Stimulation': [],
        'N_CTZ': [],
        'N_No_CTZ': [],
        'Mean_CTZ': [],
        'Mean_No_CTZ': [],
        'Permutation_p_value': []
    }

    for cell_type in actual_data_df['CellType'].unique():
        for stim_range in actual_data_df['StimRange'].unique():
            for is_single_unit in actual_data_df['IsSingleUnit'].unique():
                for stim_responsivity in actual_data_df['StimResponsivity'].unique():
                    for stim in stimulations:
                        subset_df = actual_data_df[
                            (actual_data_df['CellType'] == cell_type) &
                            (actual_data_df['StimRange'] == stim_range) &
                            (actual_data_df['IsSingleUnit'] == is_single_unit) &
                            (actual_data_df['StimResponsivity'] == stim_responsivity) &
                            (actual_data_df['Stimulation'] == stim)
                        ]

                        if subset_df.empty:
                            continue

                        # Separate data for each group
                        group_ctz = subset_df[subset_df['Group'] == 'CTZ']
                        group_no_ctz = subset_df[subset_df['Group'] == 'No_CTZ']

                        # Calculate N and mean for each group
                        n_ctz = len(group_ctz)
                        n_no_ctz = len(group_no_ctz)
                        mean_ctz = group_ctz['mean_stimulation'].mean()
                        mean_no_ctz = group_no_ctz['mean_stimulation'].mean()

                        # Perform permutation test
                        p_value = permutation_test(subset_df, group_col='Group', value_col='mean_stimulation', n_permutations=n_permutations)

                        # Store permutation test results
                        perm_results['CellType'].append(cell_type)
                        perm_results['StimRange'].append(stim_range)
                        perm_results['IsSingleUnit'].append(is_single_unit)
                        perm_results['StimResponsivity'].append(stim_responsivity)
                        perm_results['Stimulation'].append(stim)
                        perm_results['N_CTZ'].append(n_ctz)
                        perm_results['N_No_CTZ'].append(n_no_ctz)
                        
                        #convert to a float with 3 decimal places
                        mean_ctz = float("{:.3f}".format(mean_ctz))
                        perm_results['Mean_CTZ'].append(mean_ctz)
                        
                        #convert to a float with 3 decimal places 
                        mean_no_ctz = float("{:.3f}".format(mean_no_ctz))
                        perm_results['Mean_No_CTZ'].append(mean_no_ctz)
                        
                        #convert to a float with 3 decimal places
                        p_value = float("{:.3f}".format(p_value))
                        perm_results['Permutation_p_value'].append(p_value)

    # Convert permutation test results dictionary to DataFrame
    perm_results_df = pd.DataFrame(perm_results)
    print(f"Permutation Test Results DataFrame:\n{perm_results_df}")

    return perm_results_df


descriptive_stats_df = dynamic_statistical_analysis(whisker_df_manager, cell_types, stim_ranges, is_single_units, stim_responsivities, stimulations)
input_df_4_kruskal_and_permutation = generate_actual_data(whisker_df_manager, cell_types, stim_ranges, is_single_units, stim_responsivities, stimulations)
kruskal_results_df = perform_kruskal_wallis_on_actual_data(input_df_4_kruskal_and_permutation, stimulations)
perm_results_df = perform_permutation_test_on_actual_data(input_df_4_kruskal_and_permutation, stimulations, n_permutations=1000)

descriptive_stats_df.to_csv('/Volumes/MannySSD/figures/comparing_conditions_dynamic/all_stimulations/descriptive_stats.csv', index=False)
input_df_4_kruskal_and_permutation.to_csv('/Volumes/MannySSD/figures/comparing_conditions_dynamic/all_stimulations/input_df_4_kruskal_and_permutation.csv', index=False)
kruskal_results_df.to_csv('/Volumes/MannySSD/figures/comparing_conditions_dynamic/all_stimulations/kruskal_results.csv', index=False)
perm_results_df.to_csv('/Volumes/MannySSD/figures/comparing_conditions_dynamic/all_stimulations/perm_results.csv', index=False)

LAMINAR: This section will run the analysis for PSTHs based on a laminar profiles, similar to the above but can handle the laminar label

In [ ]:
import pandas as pd
from scipy.stats import iqr
from scipy.stats import sem, t

## dynamic_statistical_analysis and generate_actual_data in this cell is modified form the above to allow for 
## the inclusion of laminar labels in the analysis -- will need to change the function names 
def dynamic_statistical_analysis(whisker_df_manager, cell_types, stim_ranges, is_single_units, stim_responsivities, laminarlabel, stimulations=None):
    """
    Dynamically collects descriptive statistics across different combinations for specified cell types and parameter combinations.

    Args:
        whisker_df_manager: The data manager object with methods to calculate stats and plot.
        cell_types (list of str): List of cell types (e.g., ['FS', 'RS', 'None']).
        stim_ranges (list of tuple): List of stimulation ranges (e.g., [(0, 500), (0, 1000)]).
        is_single_units (list of float): List of is_single_unit values (e.g., [1.0, 0.0, None]).
        stim_responsivities (list of float): List of stim_responsivity values (e.g., [1.0, 0.0, None]).
        laminarlabel (list of str): List of laminar labels (e.g., ['SG', 'IG', 'L4']).
        stimulations (list of str, optional): List of stimulations to include in the analysis. If None, include all stimulations.
    """
    # Initialize results dictionary
    results = {
        'Cell_Type': [],
        'StimRange': [],
        'IsSingleUnit': [],
        'StimResponsivity': [],
        'Stimulation': [],
        'Group': [],
        'N': [],
        'Mean': [],
        'Median': [],
        'SD': [],
        'SEM': [],
        'CI Lower': [],
        'CI Upper': [],
        'Range': []
    }

    for cell_type in cell_types:
        for stim_range in stim_ranges:
            for is_single_unit in is_single_units:
                for stim_responsivity in stim_responsivities:
                    for laminar in laminarlabel:
                        # Prepare data for boxplot
                        df = whisker_df_manager.prepare_for_boxplot()

                        # Handle the 'None' case: if None, include both 'FS' and 'RS'
                        if cell_type is not None:
                            df = df[df['Cell_Type'] == cell_type]
                            cell_type_str = cell_type
                        else:
                            cell_type_str = 'All'  # This means include all cell types, so no filtering

                        if is_single_unit is not None:
                            df = df[df['IsSingleUnit'] == is_single_unit]
                            is_single_unit_str = str(is_single_unit)
                        else:
                            is_single_unit_str = 'All'

                        if stim_responsivity is not None:
                            df = df[df['StimResponsivity'] == stim_responsivity]
                            stim_responsivity_str = str(stim_responsivity)
                        else:
                            stim_responsivity_str = 'All'

                        # Filter by laminar label
                        df = df[df['LaminarLabel'] == laminar]
                        print(f"DataFrame shape after filtering by laminar label '{laminar}': {df.shape}")

                        # Filter data for the specified stimulations
                        if stimulations:
                            df = df[df['Stimulation'].isin(stimulations)]

                        # Check if the required columns are present
                        required_columns = ['mean_stimulation', 'Stimulation', 'Group']
                        missing_columns = [col for col in required_columns if col not in df.columns]
                        if missing_columns:
                            print(f"Skipping combination {cell_type_str} due to missing columns: {missing_columns}")
                            continue

                        # Loop through each stimulation and perform analysis
                        for stim in stimulations or df['Stimulation'].unique():
                            for group in ['CTZ', 'No_CTZ']:
                                group_data = df[(df['Stimulation'] == stim) & (df['Group'] == group)]['mean_stimulation']

                                if group_data.empty:
                                    print(f"Skipping {stim} for group {group} due to lack of data")
                                    continue

                                # Calculate IQR
                                Q1 = group_data.quantile(0.25)
                                Q3 = group_data.quantile(0.75)
                                IQR = Q3 - Q1

                                # Filter data within IQR
                                #filtered_data = group_data[(group_data >= Q1 - 1.5 * IQR) & (group_data <= Q3 + 1.5 * IQR)]
                                filtered_data = group_data 

                                if filtered_data.empty:
                                    print(f"Skipping {stim} for group {group} due to all data being outside IQR")
                                    continue

                                # Calculate descriptive statistics
                                group_desc = filtered_data.describe()
                                group_sem = sem(filtered_data, nan_policy='omit')
                                confidence_level = 0.95
                                degrees_freedom = len(filtered_data) - 1
                                confidence_interval = t.interval(confidence_level, degrees_freedom, loc=group_desc['mean'], scale=group_sem)

                                # Store results
                                results['Cell_Type'].append(cell_type_str)
                                results['StimRange'].append(f"{stim_range[0]}to{stim_range[1]}")
                                results['IsSingleUnit'].append(is_single_unit_str)
                                results['StimResponsivity'].append(stim_responsivity_str)
                                results['Stimulation'].append(stim)
                                results['Group'].append(group)
                                results['N'].append(group_desc['count'])
                                results['Mean'].append(group_desc['mean'])
                                results['Median'].append(filtered_data.median())
                                results['SD'].append(group_desc['std'])
                                results['SEM'].append(group_sem)
                                results['CI Lower'].append(confidence_interval[0])
                                results['CI Upper'].append(confidence_interval[1])
                                results['Range'].append((filtered_data.min(), filtered_data.max()))

    # Convert results dictionary to DataFrame
    results_df = pd.DataFrame(results)
    print(f"Results DataFrame:\n{results_df}")

    return results_df

def generate_actual_data(whisker_df_manager, cell_types, stim_ranges, is_single_units, stim_responsivities, laminarlabel, stimulations=None):
    """
    Collect actual data across different combinations for specified cell types, laminar labels, and parameter combinations.

    Args:
        whisker_df_manager: The data manager object with methods to calculate stats and plot.
        cell_types (list of str): List of cell types (e.g., ['FS', 'RS', None] - None means include both 'FS' and 'RS').
        stim_ranges (list of tuple): List of stimulation ranges (e.g., [(0, 500), (0, 1000)]).
        is_single_units (list of float): List of is_single_unit values (e.g., [1.0, 0.0, None]).
        stim_responsivities (list of float): List of stim_responsivity values (e.g., [1.0, 0.0, None]).
        laminarlabel (list of str): List of laminar labels (e.g., ['SG', 'IG', 'L4']).
        stimulations (list of str, optional): List of stimulations to include in the analysis. If None, include all stimulations.
    """
    # Initialize list to collect actual data
    actual_data = []

    for cell_type in cell_types:
        for stim_range in stim_ranges:
            for is_single_unit in is_single_units:
                for stim_responsivity in stim_responsivities:
                    for laminar in laminarlabel:
                        # Handle the 'None' case for cell_type
                        df = whisker_df_manager.prepare_for_boxplot()

                        # If cell_type is None, include all cell types ('FS' and 'RS')
                        if cell_type is not None:
                            df = df[df['Cell_Type'] == cell_type]
                            cell_type_str = cell_type
                        else:
                            cell_type_str = 'All'  # Include all cell types, so no filtering

                        if is_single_unit is not None:
                            df = df[df['IsSingleUnit'] == is_single_unit]
                            is_single_unit_str = str(is_single_unit)
                        else:
                            is_single_unit_str = 'All'

                        if stim_responsivity is not None:
                            df = df[df['StimResponsivity'] == stim_responsivity]
                            stim_responsivity_str = str(stim_responsivity)
                        else:
                            stim_responsivity_str = 'All'

                        # Filter by laminar label
                        df = df[df['LaminarLabel'] == laminar]
                        print(f"DataFrame shape after filtering by laminar label '{laminar}': {df.shape}")

                        # If the DataFrame is empty after filtering, print a message and continue
                        if df.empty:
                            print(f"No data available for laminar label '{laminar}' with current filters.")
                            continue

                        # Filter data for the specified stimulations
                        if stimulations:
                            df = df[df['Stimulation'].isin(stimulations)]

                        # Check if the required columns are present
                        required_columns = ['mean_stimulation', 'Stimulation', 'Group']
                        missing_columns = [col for col in required_columns if col not in df.columns]
                        if missing_columns:
                            print(f"Skipping combination {cell_type_str} due to missing columns: {missing_columns}")
                            continue

                        # Loop through each stimulation and collect data
                        for stim in stimulations or df['Stimulation'].unique():
                            for group in ['CTZ', 'No_CTZ']:
                                group_data = df[(df['Stimulation'] == stim) & (df['Group'] == group)]
                                
                                if group_data.empty:
                                    print(f"Skipping {stim} for group {group} due to lack of data")
                                    continue

                                # Filter data within IQR
                                Q1 = group_data['mean_stimulation'].quantile(0.25)
                                Q3 = group_data['mean_stimulation'].quantile(0.75)
                                IQR = Q3 - Q1
                                #filtered_data = group_data[(group_data['mean_stimulation'] >= Q1 - 1.5 * IQR) & (group_data['mean_stimulation'] <= Q3 + 1.5 * IQR)]
                                filtered_data = group_data
                                if filtered_data.empty:
                                    print(f"Skipping {stim} for group {group} due to all data being outside IQR")
                                    continue

                                # Add additional columns for metadata
                                filtered_data['Cell_Type'] = cell_type_str
                                filtered_data['StimRange'] = f"{stim_range[0]}to{stim_range[1]}"
                                filtered_data['IsSingleUnit'] = is_single_unit_str
                                filtered_data['StimResponsivity'] = stim_responsivity_str
                                filtered_data['LaminarLabel'] = laminar  # Include laminar label
                                
                                # Collect the filtered data
                                actual_data.append(filtered_data)

    # Check if there's any data to concatenate
    if not actual_data:
        print("No data was collected, returning an empty DataFrame.")
        return pd.DataFrame()  # Return an empty DataFrame if no data was collected

    # Combine all collected data into a single DataFrame
    actual_data_df = pd.concat(actual_data, ignore_index=True)
    print(f"Actual Data DataFrame:\n{actual_data_df}")

    return actual_data_df


def wilcoxon_rank_sum_test(data, group_col, value_col):
    """
    Perform Wilcoxon rank-sum test (Mann-Whitney U test) for independent samples.

    Args:
        data (pd.DataFrame): DataFrame containing the data.
        group_col (str): Column name to group by (e.g., 'Group').
        value_col (str): Column name containing the values (e.g., 'mean_stimulation').

    Returns:
        tuple: Wilcoxon rank-sum U-statistic and p-value.
    """
    # Ensure that there are exactly two groups to compare
    unique_groups = data[group_col].unique()
    if len(unique_groups) != 2:
        raise ValueError(f"Wilcoxon rank-sum test requires exactly two groups, but {len(unique_groups)} groups were found.")

    # Extract values for each group
    group1_values = data[data[group_col] == unique_groups[0]][value_col].values
    group2_values = data[data[group_col] == unique_groups[1]][value_col].values

    # Perform the Wilcoxon rank-sum test
    u_stat, p_value = mannwhitneyu(group1_values, group2_values, alternative='two-sided')

    return u_stat, p_value
def perform_wilcoxon_test_on_actual_data(actual_data_df, stimulations):
    """
    Perform Wilcoxon rank-sum test on the actual data DataFrame.

    Args:
        actual_data_df (pd.DataFrame): DataFrame containing the actual data.
        stimulations (list of str): List of stimulations to include in the analysis.

    Returns:
        pd.DataFrame: DataFrame containing the Wilcoxon test results.
    """
    # Initialize results dictionary for Wilcoxon test results
    wilcoxon_results = {
        'Cell_Type': [],
        'StimRange': [],
        'IsSingleUnit': [],
        'StimResponsivity': [],
        'Stimulation': [],
        'N_CTZ': [],
        'N_No_CTZ': [],
        'Mean_CTZ': [],
        'Mean_No_CTZ': [],
        'U_statistic': [],
        'p_value': []
    }

    for cell_type in actual_data_df['Cell_Type'].unique():
        for stim_range in actual_data_df['StimRange'].unique():
            for is_single_unit in actual_data_df['IsSingleUnit'].unique():
                for stim_responsivity in actual_data_df['StimResponsivity'].unique():
                    for stim in stimulations:
                        subset_df = actual_data_df[
                            (actual_data_df['Cell_Type'] == cell_type) &
                            (actual_data_df['StimRange'] == stim_range) &
                            (actual_data_df['IsSingleUnit'] == is_single_unit) &
                            (actual_data_df['StimResponsivity'] == stim_responsivity) &
                            (actual_data_df['Stimulation'] == stim)
                        ]

                        if subset_df.empty:
                            continue

                        # Separate data for each group
                        group_ctz = subset_df[subset_df['Group'] == 'CTZ']
                        group_no_ctz = subset_df[subset_df['Group'] == 'No_CTZ']

                        # Calculate N and mean for each group
                        n_ctz = len(group_ctz)
                        n_no_ctz = len(group_no_ctz)
                        mean_ctz = group_ctz['mean_stimulation'].mean()
                        mean_no_ctz = group_no_ctz['mean_stimulation'].mean()

                        # Perform Wilcoxon rank-sum test
                        u_stat, p_value = wilcoxon_rank_sum_test(subset_df, group_col='Group', value_col='mean_stimulation')

                        # Store Wilcoxon test results
                        wilcoxon_results['Cell_Type'].append(cell_type)
                        wilcoxon_results['StimRange'].append(stim_range)
                        wilcoxon_results['IsSingleUnit'].append(is_single_unit)
                        wilcoxon_results['StimResponsivity'].append(stim_responsivity)
                        wilcoxon_results['Stimulation'].append(stim)
                        wilcoxon_results['N_CTZ'].append(n_ctz)
                        wilcoxon_results['N_No_CTZ'].append(n_no_ctz)
                        
                        # Convert to a float with 3 decimal places
                        mean_ctz = float("{:.3f}".format(mean_ctz))
                        wilcoxon_results['Mean_CTZ'].append(mean_ctz)
                        
                        # Convert to a float with 3 decimal places 
                        mean_no_ctz = float("{:.3f}".format(mean_no_ctz))
                        wilcoxon_results['Mean_No_CTZ'].append(mean_no_ctz)
                        
                        # Convert to a float with 3 decimal places for u_stat
                        u_stat = float("{:.3f}".format(u_stat))
                        wilcoxon_results['U_statistic'].append(u_stat)
                        
                        # Convert to a float with 3 decimal places
                        p_value = float("{:.3f}".format(p_value))
                        wilcoxon_results['p_value'].append(p_value)

    # Convert Wilcoxon test results dictionary to DataFrame
    wilcoxon_results_df = pd.DataFrame(wilcoxon_results)
    print(f"Wilcoxon Test Results DataFrame:\n{wilcoxon_results_df}")

    return wilcoxon_results_df

def kruskal_wallis_test(data, group_col, value_col):
    """
    Perform Kruskal-Wallis H-test for independent samples.

    Args:
        data (pd.DataFrame): DataFrame containing the data.
        group_col (str): Column name to group by (e.g., 'Group').
        value_col (str): Column name containing the values (e.g., 'mean_stimulation').

    Returns:
        tuple: Kruskal-Wallis H-statistic and p-value. If all values are identical, return NaN for both.
    """
    # Group data by the specified column and extract values
    groups = [group[value_col].values for name, group in data.groupby(group_col)]
    
    # Check if all values across groups are identical
    all_values = np.concatenate(groups)
    if np.all(all_values == all_values[0]):
        return np.nan, np.nan
    
    # Perform Kruskal-Wallis test
    h_stat, p_value = kruskal(*groups)

    return h_stat, p_value


def perform_kruskal_wallis_on_actual_data(actual_data_df, stimulations):
    """
    Perform Kruskal-Wallis test on the actual data DataFrame.

    Args:
        actual_data_df (pd.DataFrame): DataFrame containing the actual data.
        stimulations (list of str): List of stimulations to include in the analysis.

    Returns:
        pd.DataFrame: DataFrame containing the Kruskal-Wallis test results.
    """
    # Initialize results dictionary for Kruskal-Wallis test results
    kruskal_results = {
        'Cell_Type': [],
        'StimRange': [],
        'IsSingleUnit': [],
        'StimResponsivity': [],
        'Stimulation': [],
        'N_CTZ': [],
        'N_No_CTZ': [],
        'Mean_CTZ': [],
        'Mean_No_CTZ': [],
        'H_statistic': [],
        'p_value': []
    }

    for cell_type in actual_data_df['Cell_Type'].unique():
        for stim_range in actual_data_df['StimRange'].unique():
            for is_single_unit in actual_data_df['IsSingleUnit'].unique():
                for stim_responsivity in actual_data_df['StimResponsivity'].unique():
                    for stim in stimulations:
                        # Create a subset of the data for the current combination
                        subset_df = actual_data_df[
                            ((actual_data_df['Cell_Type'] == cell_type) | (cell_type == 'All')) &
                            (actual_data_df['StimRange'] == stim_range) &
                            ((actual_data_df['IsSingleUnit'] == is_single_unit) | (is_single_unit == 'All')) &
                            ((actual_data_df['StimResponsivity'] == stim_responsivity) | (stim_responsivity == 'All')) &
                            (actual_data_df['Stimulation'] == stim)
                        ]

                        if subset_df.empty:
                            continue

                        # Separate data for each group
                        group_ctz = subset_df[subset_df['Group'] == 'CTZ']
                        group_no_ctz = subset_df[subset_df['Group'] == 'No_CTZ']

                        # Calculate N and mean for each group
                        n_ctz = len(group_ctz)
                        n_no_ctz = len(group_no_ctz)
                        mean_ctz = group_ctz['mean_stimulation'].mean()
                        mean_no_ctz = group_no_ctz['mean_stimulation'].mean()

                        # Perform Kruskal-Wallis test
                        h_stat, p_value = kruskal_wallis_test(subset_df, group_col='Group', value_col='mean_stimulation')

                        # Store Kruskal-Wallis results
                        kruskal_results['Cell_Type'].append(cell_type)
                        kruskal_results['StimRange'].append(stim_range)
                        kruskal_results['IsSingleUnit'].append(is_single_unit)
                        kruskal_results['StimResponsivity'].append(stim_responsivity)
                        kruskal_results['Stimulation'].append(stim)
                        kruskal_results['N_CTZ'].append(n_ctz)
                        kruskal_results['N_No_CTZ'].append(n_no_ctz)
                        
                        # Convert to a float with 3 decimal places
                        mean_ctz = float("{:.3f}".format(mean_ctz))
                        kruskal_results['Mean_CTZ'].append(mean_ctz)
                        
                        # Convert to a float with 3 decimal places 
                        mean_no_ctz = float("{:.3f}".format(mean_no_ctz))
                        kruskal_results['Mean_No_CTZ'].append(mean_no_ctz)
                        
                        # Convert to a float with 3 decimal places for h_stat
                        h_stat = float("{:.3f}".format(h_stat))
                        kruskal_results['H_statistic'].append(h_stat)
                        
                        # Convert to a float with 3 decimal places
                        p_value = float("{:.3f}".format(p_value))
                        kruskal_results['p_value'].append(p_value)
                    

    # Convert Kruskal-Wallis results dictionary to DataFrame
    kruskal_results_df = pd.DataFrame(kruskal_results)
    print(f"Kruskal-Wallis Results DataFrame:\n{kruskal_results_df}")

    return kruskal_results_df


def permutation_test(data, group_col, value_col, n_permutations=1000):
    """
    Perform a permutation test for the specified groups.

    Args:
        data (pd.DataFrame): DataFrame containing the data.
        group_col (str): Column name to group by (e.g., 'Group').
        value_col (str): Column name containing the values (e.g., 'mean_stimulation').
        n_permutations (int): Number of permutations to perform.

    Returns:
        float: p-value from the permutation test.
    """
    # Calculate the actual Kruskal-Wallis test statistic
    groups = [group[value_col].values for name, group in data.groupby(group_col)]
    actual_stat, _ = kruskal(*groups)
    
    # Combine all data values and shuffle group labels to generate the null distribution
    combined_values = data[value_col].values
    group_labels = data[group_col].values
    permuted_stats = []

    for _ in range(n_permutations):
        np.random.shuffle(group_labels)
        permuted_groups = [combined_values[group_labels == label] for label in np.unique(group_labels)]
        perm_stat, _ = kruskal(*permuted_groups)
        permuted_stats.append(perm_stat)
    
    # Calculate the p-value as the proportion of permuted stats greater than or equal to the actual stat
    permuted_stats = np.array(permuted_stats)
    p_value = np.sum(permuted_stats >= actual_stat) / n_permutations

    return p_value
def perform_permutation_test_on_actual_data(actual_data_df, stimulations, n_permutations=1000):
    """
    Perform a permutation test on the actual data DataFrame.

    Args:
        actual_data_df (pd.DataFrame): DataFrame containing the actual data.
        stimulations (list of str): List of stimulations to include in the analysis.
        n_permutations (int): Number of permutations to perform.

    Returns:
        pd.DataFrame: DataFrame containing the permutation test results.
    """
    # Initialize results dictionary for permutation test results
    perm_results = {
        'Cell_Type': [],
        'StimRange': [],
        'IsSingleUnit': [],
        'StimResponsivity': [],
        'Stimulation': [],
        'N_CTZ': [],
        'N_No_CTZ': [],
        'Mean_CTZ': [],
        'Mean_No_CTZ': [],
        'Permutation_p_value': []
    }

    for cell_type in actual_data_df['Cell_Type'].unique():
        for stim_range in actual_data_df['StimRange'].unique():
            for is_single_unit in actual_data_df['IsSingleUnit'].unique():
                for stim_responsivity in actual_data_df['StimResponsivity'].unique():
                    for stim in stimulations:
                        subset_df = actual_data_df[
                            (actual_data_df['Cell_Type'] == cell_type) &
                            (actual_data_df['StimRange'] == stim_range) &
                            (actual_data_df['IsSingleUnit'] == is_single_unit) &
                            (actual_data_df['StimResponsivity'] == stim_responsivity) &
                            (actual_data_df['Stimulation'] == stim)
                        ]

                        if subset_df.empty:
                            continue

                        # Separate data for each group
                        group_ctz = subset_df[subset_df['Group'] == 'CTZ']
                        group_no_ctz = subset_df[subset_df['Group'] == 'No_CTZ']

                        # Calculate N and mean for each group
                        n_ctz = len(group_ctz)
                        n_no_ctz = len(group_no_ctz)
                        mean_ctz = group_ctz['mean_stimulation'].mean()
                        mean_no_ctz = group_no_ctz['mean_stimulation'].mean()

                        # Perform permutation test
                        p_value = permutation_test(subset_df, group_col='Group', value_col='mean_stimulation', n_permutations=n_permutations)

                        # Store permutation test results
                        perm_results['Cell_Type'].append(cell_type)
                        perm_results['StimRange'].append(stim_range)
                        perm_results['IsSingleUnit'].append(is_single_unit)
                        perm_results['StimResponsivity'].append(stim_responsivity)
                        perm_results['Stimulation'].append(stim)
                        perm_results['N_CTZ'].append(n_ctz)
                        perm_results['N_No_CTZ'].append(n_no_ctz)
                        
                        #convert to a float with 3 decimal places
                        mean_ctz = float("{:.3f}".format(mean_ctz))
                        perm_results['Mean_CTZ'].append(mean_ctz)
                        
                        #convert to a float with 3 decimal places 
                        mean_no_ctz = float("{:.3f}".format(mean_no_ctz))
                        perm_results['Mean_No_CTZ'].append(mean_no_ctz)
                        
                        #convert to a float with 3 decimal places
                        p_value = float("{:.3f}".format(p_value))
                        perm_results['Permutation_p_value'].append(p_value)

    # Convert permutation test results dictionary to DataFrame
    perm_results_df = pd.DataFrame(perm_results)
    print(f"Permutation Test Results DataFrame:\n{perm_results_df}")

    return perm_results_df

def bootstrap_statistic_difference(data, group_col, value_col, n_bootstraps=10000, alpha=0.05, statistic='mean'):
    """
    Perform a bootstrap statistic-difference test between two groups (mean or median).

    Args:
        data (pd.DataFrame): DataFrame containing the data.
        group_col (str): Column name to group by (e.g., 'Group').
        value_col (str): Column name containing the values (e.g., 'mean_stimulation').
        n_bootstraps (int): Number of bootstrap samples to generate.
        alpha (float): Significance level for confidence intervals.
        statistic (str): Statistic to use ('mean' or 'median').

    Returns:
        tuple: Observed statistic difference, bootstrap statistic difference confidence interval, and p-value.
    """
    # Ensure that there are exactly two groups to compare
    unique_groups = data[group_col].unique()
    if len(unique_groups) != 2:
        raise ValueError(f"Bootstrap statistic-difference test requires exactly two groups, but {len(unique_groups)} groups were found.")
    
    # Extract values for each group
    group1_values = data[data[group_col] == unique_groups[0]][value_col].values
    group2_values = data[data[group_col] == unique_groups[1]][value_col].values
    
    # Select the appropriate statistic function
    if statistic == 'mean':
        stat_func = np.mean
    elif statistic == 'median':
        stat_func = np.median
    else:
        raise ValueError(f"Unsupported statistic '{statistic}'. Use 'mean' or 'median'.")
    
    # Calculate the observed statistic difference
    observed_stat_diff = stat_func(group1_values) - stat_func(group2_values)
    
    # Bootstrap resampling
    stat_diffs = []
    for _ in range(n_bootstraps):
        group1_sample = np.random.choice(group1_values, size=len(group1_values), replace=True)
        group2_sample = np.random.choice(group2_values, size=len(group2_values), replace=True)
        stat_diffs.append(stat_func(group1_sample) - stat_func(group2_sample))
    
    # Calculate the p-value
    stat_diffs = np.array(stat_diffs)
    p_value = np.sum(np.abs(stat_diffs) >= np.abs(observed_stat_diff)) / n_bootstraps
    
    # Calculate the confidence interval
    ci_lower = np.percentile(stat_diffs, 100 * alpha / 2)
    ci_upper = np.percentile(stat_diffs, 100 * (1 - alpha / 2))
    
    return observed_stat_diff, (ci_lower, ci_upper), p_value
def perform_bootstrap_statistic_test_on_actual_data(actual_data_df, stimulations, n_bootstraps=10000, alpha=0.05, statistic='mean'):
    """
    Perform a bootstrap statistic-difference test (mean or median) on the actual data DataFrame.

    Args:
        actual_data_df (pd.DataFrame): DataFrame containing the actual data.
        stimulations (list of str): List of stimulations to include in the analysis.
        n_bootstraps (int): Number of bootstrap samples to generate.
        alpha (float): Significance level for confidence intervals.
        statistic (str): Statistic to use ('mean' or 'median').

    Returns:
        pd.DataFrame: DataFrame containing the bootstrap test results.
    """
    # Initialize results dictionary for bootstrap statistic-difference test results
    bootstrap_results = {
        'Cell_Type': [],
        'StimRange': [],
        'IsSingleUnit': [],
        'StimResponsivity': [],
        'Stimulation': [],
        'N_CTZ': [],
        'N_No_CTZ': [],
        f'{statistic.capitalize()}_CTZ': [],
        f'{statistic.capitalize()}_No_CTZ': [],
        'Observed_Statistic_Difference': [],
        'CI_Lower': [],
        'CI_Upper': [],
        'Bootstrap_p_value': []
    }

    for cell_type in actual_data_df['Cell_Type'].unique():
        for stim_range in actual_data_df['StimRange'].unique():
            for is_single_unit in actual_data_df['IsSingleUnit'].unique():
                for stim_responsivity in actual_data_df['StimResponsivity'].unique():
                    for stim in stimulations:
                        subset_df = actual_data_df[
                            (actual_data_df['Cell_Type'] == cell_type) &
                            (actual_data_df['StimRange'] == stim_range) &
                            (actual_data_df['IsSingleUnit'] == is_single_unit) &
                            (actual_data_df['StimResponsivity'] == stim_responsivity) &
                            (actual_data_df['Stimulation'] == stim)
                        ]

                        if subset_df.empty:
                            continue

                        # Separate data for each group
                        group_ctz = subset_df[subset_df['Group'] == 'CTZ']
                        group_no_ctz = subset_df[subset_df['Group'] == 'No_CTZ']

                        # Calculate N and statistic for each group
                        n_ctz = len(group_ctz)
                        n_no_ctz = len(group_no_ctz)
                        stat_ctz = np.mean(group_ctz['mean_stimulation']) if statistic == 'mean' else np.median(group_ctz['mean_stimulation'])
                        stat_no_ctz = np.mean(group_no_ctz['mean_stimulation']) if statistic == 'mean' else np.median(group_no_ctz['mean_stimulation'])

                        # Perform bootstrap statistic-difference test
                        observed_stat_diff, (ci_lower, ci_upper), p_value = bootstrap_statistic_difference(
                            subset_df, group_col='Group', value_col='mean_stimulation', 
                            n_bootstraps=n_bootstraps, alpha=alpha, statistic=statistic
                        )

                        # Store bootstrap test results
                        bootstrap_results['Cell_Type'].append(cell_type)
                        bootstrap_results['StimRange'].append(stim_range)
                        bootstrap_results['IsSingleUnit'].append(is_single_unit)
                        bootstrap_results['StimResponsivity'].append(stim_responsivity)
                        bootstrap_results['Stimulation'].append(stim)
                        bootstrap_results['N_CTZ'].append(n_ctz)
                        bootstrap_results['N_No_CTZ'].append(n_no_ctz)
                        
                        # Convert to a float with 3 decimal places
                        stat_ctz = float("{:.3f}".format(stat_ctz))
                        bootstrap_results[f'{statistic.capitalize()}_CTZ'].append(stat_ctz)
                        
                        # Convert to a float with 3 decimal places 
                        stat_no_ctz = float("{:.3f}".format(stat_no_ctz))
                        bootstrap_results[f'{statistic.capitalize()}_No_CTZ'].append(stat_no_ctz)
                        
                        # Store observed statistic difference and confidence interval
                        bootstrap_results['Observed_Statistic_Difference'].append(float("{:.3f}".format(observed_stat_diff)))
                        bootstrap_results['CI_Lower'].append(float("{:.3f}".format(ci_lower)))
                        bootstrap_results['CI_Upper'].append(float("{:.3f}".format(ci_upper)))
                        
                        # Convert to a float with 3 decimal places
                        p_value = float("{:.3f}".format(p_value))
                        bootstrap_results['Bootstrap_p_value'].append(p_value)

    # Convert bootstrap test results dictionary to DataFrame
    bootstrap_results_df = pd.DataFrame(bootstrap_results)
    print(f"Bootstrap {statistic.capitalize()}-Difference Test Results DataFrame:\n{bootstrap_results_df}")

    return bootstrap_results_df




# The same approach can be applied to the other functions for generating data and performing tests. 
# Now, loop through each laminar label and call the statistical analysis functions, saving the results dynamically.
cell_types = ['FS', 'RS', None]
laminarlabel = ['SG', 'IG', 'L4']
base_directory = '/Volumes/MannySSD/figures/comparing_conditions_dynamic/all_stimulations_laminar_NOIQR'
stimulations = ['Zero', 'Low', 'Mid', 'Max']  # Correct stimulation labels
stim_ranges =  [(0, 500), (0,50), (0,20)]
is_single_units = [1.0, 0.0,  None]
stim_responsivities = [1.0, None]


for laminar in laminarlabel:
    print(f"Running analysis for laminar label: {laminar}")
    
    # Run the descriptive statistics
    #descriptive_stats_df = dynamic_statistical_analysis(whisker_df_manager, cell_types, stim_ranges, is_single_units, stim_responsivities, [laminar], stimulations)
    
    
    # Generate actual data
    print(f"Generating actual data for laminar label: {laminar}")
    input_df_4_kruskal_and_permutation = generate_actual_data(whisker_df_manager, 
                                                              cell_types, 
                                                              stim_ranges, 
                                                              is_single_units, 
                                                              stim_responsivities, 
                                                              [laminar],
                                                              stimulations)
    
    
    ### run statistical test
    
    #perform bootstrap test
    #bootstrap_results_df = perform_bootstrap_statistic_test_on_actual_data(input_df_4_kruskal_and_permutation, stimulations, n_bootstraps=10000, alpha=0.05, statistic='median')
    
    # Perform Kruskal-Wallis test
    kruskal_results_df = perform_kruskal_wallis_on_actual_data(input_df_4_kruskal_and_permutation, stimulations)
    
    # perform Wilcoxon rank-sum test
    #wilcoxon_results_df = perform_wilcoxon_test_on_actual_data(input_df_4_kruskal_and_permutation, stimulations)
    
    # Perform permutation test
    #perm_results_df = perform_permutation_test_on_actual_data(input_df_4_kruskal_and_permutation, stimulations, n_permutations=1000)
    
    # Save the results dynamically based on laminar label
    laminar_directory = os.path.join(base_directory, laminar)
    os.makedirs(laminar_directory, exist_ok=True)
    
    #descriptive_stats_df.to_csv(os.path.join(laminar_directory, f'descriptive_stats_{laminar}.csv'), index=False)
    input_df_4_kruskal_and_permutation.to_csv(os.path.join(laminar_directory, f'input_df_4_kruskal_and_permutation_{laminar}.csv'), index=False)
    kruskal_results_df.to_csv(os.path.join(laminar_directory, f'kruskal_results_{laminar}.csv'), index=False)
    #perm_results_df.to_csv(os.path.join(laminar_directory, f'perm_results_{laminar}.csv'), index=False)
    #wilcoxon_results_df.to_csv(os.path.join(laminar_directory, f'wilcoxon_results_{laminar}.csv'), index=False)
    #bootstrap_results_df.to_csv(os.path.join(laminar_directory, f'bootstrap_results_{laminar}.csv'), index=False)
#
    print(f"Completed analysis for laminar label: {laminar}\n")


Run a smilar plotting and analysis approach for other analysis beyond PSTHS--- but only for FirstSpikeLatency -- you will have to change the ylim within the dynamic_plot_saving_beyond_PSTHs based on the FS and RS--- need to fix later 

In [ ]:
# plot the box and line plots for all combinations of cell types, stim ranges, is_single_units, and stim_responsivities
def replace_mean_stimulation_with_column_value(df, target_column):
    """
    Replaces the values in the 'mean_stimulation' column with the corresponding values
    from a specified column based on the 'Stimulation' entry.

    Args:
        df (pd.DataFrame): The input DataFrame.
        target_column (str): The name of the column to use for replacing 'mean_stimulation' values.
                             This column should contain arrays with four elements corresponding to
                             'Zero', 'Low', 'Mid', and 'Max' stimulations.

    Returns:
        pd.DataFrame: The modified DataFrame with 'mean_stimulation' values replaced.
    """
    # Define a mapping from Stimulation labels to indices
    stimulation_mapping = {'Zero': 0, 'Low': 1, 'Mid': 2, 'Max': 3}
    
    # Apply the replacement row by row
    def replace_row(row):
        stim_label = row['Stimulation']
        if stim_label in stimulation_mapping:
            index = stimulation_mapping[stim_label]
            # Replace mean_stimulation with the corresponding value from the target column
            row['mean_stimulation'] = row[target_column][index]
        return row
    
    # Apply the function to each row of the DataFrame
    df = df.apply(replace_row, axis=1)
    
    return df
def dynamic_statistical_analysis_beyond_PSTHs(whisker_df_manager, target_column, cell_types, stim_ranges, is_single_units, stim_responsivities, stimulations=None):
    """
    Dynamically collects descriptive statistics across different combinations for specified cell types and parameter combinations.
    This function is an extension of the dynamic_statistical_analysis function to handle beyond PSTH data analysis so 
    that it can be used for other data types beyond PSTHs. It used a helper to replace the mean_stimulation column with the target column data and unpacks the target column data for analysis.
    The target columns must have four indexed within their array (zero, low, mid, and max) correspondign to the stimulations and then replaced.
    
    # it currently filters based on modulation_label='positive' and firstspike_latency=True so keep that in mind

    Args:
        whisker_df_manager: The data manager object with methods to calculate stats and plot.
        cell_types (list of str): List of cell types (e.g., ['FS', 'RS']).
        stim_ranges (list of tuple): List of stimulation ranges (e.g., [(0, 500), (0, 1000)]).
        is_single_units (list of float): List of is_single_unit values (e.g., [1.0, 0.0, None]).
        stim_responsivities (list of float): List of stim_responsivity values (e.g., [1.0, 0.0, None]).
        stimulations (list of str, optional): List of stimulations to include in the analysis. If None, include all stimulations.
        target_column (str): The target column to use for the analysis.
    """
    # Initialize results dictionary
    results = {
        'CellType': [],
        'StimRange': [],
        'IsSingleUnit': [],
        'StimResponsivity': [],
        'Stimulation': [],
        'Group': [],
        'N': [],
        'Mean': [],
        'Median': [],
        'SD': [],
        'SEM': [],
        'CI Lower': [],
        'CI Upper': [],
        'Range': []
    }
    
    for cell_type in cell_types:
        for stim_range in stim_ranges:
            for is_single_unit in is_single_units:
                for stim_responsivity in stim_responsivities:
                    # Define file name suffix based on parameters
                    is_single_unit_str = 'All' if is_single_unit is None else str(is_single_unit)
                    stim_responsivity_str = 'All' if stim_responsivity is None else str(stim_responsivity)
                    suffix = f"{cell_type}_IsSingleUnit_{is_single_unit_str}_IsStimResponsive_{stim_responsivity_str}_StimRange_{stim_range[0]}to{stim_range[1]}"
                    
                    # Calculate basic stats
                    print(f"Calculating basic stats for: {suffix}")
                    whisker_df_manager.calculate_basic_stats(
                        'CTZ', 'No_CTZ', stim_label=None, 
                        baseline_range=(-100, -1), stim_range=stim_range, 
                        cell_type=cell_type, is_single_unit=is_single_unit, 
                        stim_responsivity=stim_responsivity, 
                        smoothing_window=3, modulation_label='positive'
                    )
                    
                    # Prepare data for boxplot
                    print("Preparing data for boxplot")
                    df = whisker_df_manager.prepare_for_boxplot()
                    
                    # replace the data with the actual data of the target column
                    df = replace_mean_stimulation_with_column_value(df, target_column)
                    
                    print(f"DataFrame shape after preparation: {df.shape}")
                    print(f"DataFrame columns: {df.columns}")
                    print(f"First few rows of DataFrame: {df.head()}")

                    # Filter data for the specified stimulations
                    if stimulations:
                        df = df[df['Stimulation'].isin(stimulations)]
                    
                    
                    # Check if the required columns are present
                    required_columns = ['mean_stimulation', 'Stimulation', 'Group']
                    missing_columns = [col for col in required_columns if col not in df.columns]
                    if missing_columns:
                        print(f"Skipping combination {suffix} due to missing columns: {missing_columns}")
                        continue

                    # Loop through each stimulation and perform analysis
                    for stim in stimulations or df['Stimulation'].unique():
                        for group in ['CTZ', 'No_CTZ']:
                            group_data = df[(df['Stimulation'] == stim) & (df['Group'] == group)]['mean_stimulation']
                            
                            if group_data.empty:
                                print(f"Skipping {stim} for group {group} due to lack of data")
                                continue
                            
                            # Calculate IQR
                            Q1 = group_data.quantile(0.25)
                            Q3 = group_data.quantile(0.75)
                            IQR = Q3 - Q1
                            
                            # Filter data within IQR
                            filtered_data = group_data[(group_data >= Q1 - 1.5 * IQR) & (group_data <= Q3 + 1.5 * IQR)]
                            
                            if filtered_data.empty:
                                print(f"Skipping {stim} for group {group} due to all data being outside IQR")
                                continue
                            
                            # Calculate descriptive statistics
                            group_desc = filtered_data.describe()
                            group_sem = sem(filtered_data, nan_policy='omit')
                            confidence_level = 0.95
                            degrees_freedom = len(filtered_data) - 1
                            confidence_interval = t.interval(confidence_level, degrees_freedom, loc=group_desc['mean'], scale=group_sem)
                            
                            # Store results
                            results['CellType'].append(cell_type)
                            results['StimRange'].append(f"{stim_range[0]}to{stim_range[1]}")
                            results['IsSingleUnit'].append(is_single_unit_str)
                            results['StimResponsivity'].append(stim_responsivity_str)
                            results['Stimulation'].append(stim)
                            results['Group'].append(group)
                            results['N'].append(group_desc['count'])
                            results['Mean'].append(group_desc['mean'])
                            results['Median'].append(filtered_data.median())
                            results['SD'].append(group_desc['std'])
                            results['SEM'].append(group_sem)
                            results['CI Lower'].append(confidence_interval[0])
                            results['CI Upper'].append(confidence_interval[1])
                            results['Range'].append((filtered_data.min(), filtered_data.max()))
    
    # Convert results dictionary to DataFrame
    results_df = pd.DataFrame(results)
    print(f"Results DataFrame:\n{results_df}")

    return results_df
def dynamic_plot_saving_beyond_PSTHs(whisker_df_manager, target_column, cell_types, stim_ranges, is_single_units, stim_responsivities, base_directory, stimulations=None):
    """
    
    ### you might have to change the ylim values for the different cell types!!!!!! 
    
    Dynamically saves plots across different conditions for specified cell types and parameter combinations.
    
    This function is an extension of the dynamic_plot_saving_beyond_PSTHs function to handle beyond PSTH data analysis so 
    that it can be used for other data types beyond PSTHs. It used a helper to replace the mean_stimulation column with the target column data and unpacks the target column data for analysis.
    The target columns must have four indexed within their array (zero, low, mid, and max) correspondign to the stimulations and then replaced.

    Args:
        whisker_df_manager: The data manager object with methods to calculate stats and plot.
        cell_types (list of str): List of cell types (e.g., ['FS', 'RS']).
        stim_ranges (list of tuple): List of stimulation ranges (e.g., [(0, 500), (0, 1000)]).
        is_single_units (list of float): List of is_single_unit values (e.g., [1.0, 0.0, None]).
        stim_responsivities (list of float): List of stim_responsivity values (e.g., [1.0, 0.0, None]).
        base_directory (str): Base directory to save the plots.
        stimulations (list of str, optional): List of stimulations to include in the plot. If None, include all stimulations.
    """
    required_columns = ['mean_stimulation', 'Stimulation', 'Group']
    
    for cell_type in cell_types:
        for stim_range in stim_ranges:
            for is_single_unit in is_single_units:
                for stim_responsivity in stim_responsivities:
                    # Define file name suffix based on parameters
                    is_single_unit_str = 'All' if is_single_unit is None else str(is_single_unit)
                    stim_responsivity_str = 'All' if stim_responsivity is None else str(stim_responsivity)
                    suffix = f"{cell_type}_IsSingleUnit_{is_single_unit_str}_IsStimResponsive_{stim_responsivity_str}_StimRange_{stim_range[0]}to{stim_range[1]}"
                    
                    # Calculate basic stats
                    print(f"Calculating basic stats for: {suffix}")
                    whisker_df_manager.calculate_basic_stats(
                        'CTZ', 'No_CTZ', 
                        baseline_range=(-100, -1), stim_range=stim_range, 
                        cell_type=cell_type, is_single_unit=is_single_unit, 
                        stim_responsivity=stim_responsivity, 
                        smoothing_window=3, modulation_label='positive'
                    )
                    
                    # Prepare data for boxplot
                    print("Preparing data for boxplot")
                    df = whisker_df_manager.prepare_for_boxplot()
                    df = replace_mean_stimulation_with_column_value(df, target_column)
                    
                    
                    print(f"DataFrame shape after preparation: {df.shape}")
                    print(f"DataFrame columns: {df.columns}")
                    print(f"First few rows of DataFrame: {df.head()}")

                    # Check if the required columns are present
                    missing_columns = [col for col in required_columns if col not in df.columns]
                    if missing_columns:
                        print(f"Skipping combination {suffix} due to missing columns: {missing_columns}")
                        continue
                    
                    # Define directory for saving plots
                    directory = os.path.join(base_directory, 'boxplots_and_line', suffix)
                    os.makedirs(directory, exist_ok=True)
                    print(f"Directory created: {directory}")
                    
                    
                    #if cell_type == 'FS':
                    #    ylim_FS = (0, 40)
                    #if cell_type == 'RS':
                    #    ylim = (0, 300)
                        
                    # Plot box and strip plots
                    print("Plotting box and strip plots")
                    whisker_df_manager.plot_box_and_strip_with_controls(df, 
                        groups=['CTZ', 'No_CTZ'], stimulations=stimulations, show_outliers=False, show_scatter=True, 
                        directory=directory, file_name=f'{suffix}_boxplot', 
                        modulation_label='positive'
                    )
                    
                    #whisker_df_manager.plot_box_and_strip(df, 
                    #    groups=['CTZ', 'No_CTZ'], stimulations=stimulations, show_outliers=True,
                    #    directory=directory, file_name=f'{suffix}_boxplot', ylim=ylim, 
                    #    modulation_label='positive'
                    #)
                    
                    # Plot mean and SEM line plots
                    #print("Plotting mean and SEM line plots")
                    #whisker_df_manager.plot_mean_and_sem_lineplot(df, 
                    #    groups=['CTZ', 'No_CTZ'], stimulations=stimulations, directory=directory,
                    #    file_name=f'{suffix}_lineplot', ylim=None, 
                    #    modulation_label='positive'
                    #)

#all possible combinations of cell types, stim ranges, is_single_units, and stim_responsivities
cell_types = ['FS', 'RS']
stim_ranges =  [(0,20)]
is_single_units = [1.0]
stim_responsivities = [1.0]
stimulations = ['Low', 'Mid', 'Max']
target_column='FirstSpikeLatency'


base_directory = '/Volumes/MannySSD/figures/comparing_conditions_dynamic/all_stimulations_FirstSpikeLatency'
dynamic_plot_saving_beyond_PSTHs(whisker_df_manager, target_column, cell_types, stim_ranges, is_single_units, stim_responsivities, base_directory, stimulations = ['Low', 'Mid', 'Max'])

In [ ]:
from scipy.stats import iqr
from scipy.stats import sem, t

cell_types = ['FS', 'RS']
stim_ranges =  [(0,20)]
is_single_units = [1.0]
stim_responsivities = [1.0]
stimulations = ['Low', 'Mid', 'Max']
target_column='FirstSpikeLatency'

descriptive_stats_df_FirstSpikeLatency = dynamic_statistical_analysis_beyond_PSTHs(whisker_df_manager, target_column, 
                                                                                   cell_types, stim_ranges, is_single_units, stim_responsivities, stimulations)
input_df_4_kruskal_and_permutation_FirstSpikeLatency = generate_actual_data(whisker_df_manager, 
                                                              cell_types, stim_ranges, is_single_units, stim_responsivities, stimulations)
kruskal_results_df_FirstSpikeLatency = perform_kruskal_wallis_on_actual_data(input_df_4_kruskal_and_permutation, stimulations)
perm_results_df_FirstSpikeLatency  = perform_permutation_test_on_actual_data(input_df_4_kruskal_and_permutation, stimulations, n_permutations=1000)

descriptive_stats_df_FirstSpikeLatency.to_csv('/Volumes/MannySSD/figures/comparing_conditions_dynamic/all_stimulations_FirstSpikeLatency/descriptive_stats_FirstSpikeLatency.csv', index=False)
input_df_4_kruskal_and_permutation_FirstSpikeLatency.to_csv('/Volumes/MannySSD/figures/comparing_conditions_dynamic/all_stimulations_FirstSpikeLatency/input_df_4_kruskal_and_permutation_FirstSpikeLatency.csv', index=False)
kruskal_results_df_FirstSpikeLatency.to_csv('/Volumes/MannySSD/figures/comparing_conditions_dynamic/all_stimulations_FirstSpikeLatency/kruskal_results_FirstSpikeLatency.csv', index=False)
perm_results_df_FirstSpikeLatency.to_csv('/Volumes/MannySSD/figures/comparing_conditions_dynamic/all_stimulations_FirstSpikeLatency/perm_results_FirstSpikeLatency.csv', index=False)

Run a smilar plotting and analysis approach for other analysis beyond PSTHS--- but only for FirstSpikeLatency_Reliability--- 

In [ ]:
cell_types = ['FS', 'RS']
stim_ranges =  [(0,20)]
is_single_units = [1.0]
stim_responsivities = [1.0]
stimulations = ['Low', 'Mid', 'Max']
target_column='FirstSpikeLatency_Reliability'

### hard coded to change ylim for FirstSpikeLatency_Reliability !!!!! 
def dynamic_plot_saving_beyond_PSTHs(whisker_df_manager, target_column, cell_types, stim_ranges, is_single_units, stim_responsivities, base_directory, stimulations=None):
    """
    
    ### you might have to change the ylim values for the different cell types!!!!!! 
    
    Dynamically saves plots across different conditions for specified cell types and parameter combinations.
    
    This function is an extension of the dynamic_plot_saving_beyond_PSTHs function to handle beyond PSTH data analysis so 
    that it can be used for other data types beyond PSTHs. It used a helper to replace the mean_stimulation column with the target column data and unpacks the target column data for analysis.
    The target columns must have four indexed within their array (zero, low, mid, and max) correspondign to the stimulations and then replaced.

    Args:
        whisker_df_manager: The data manager object with methods to calculate stats and plot.
        cell_types (list of str): List of cell types (e.g., ['FS', 'RS']).
        stim_ranges (list of tuple): List of stimulation ranges (e.g., [(0, 500), (0, 1000)]).
        is_single_units (list of float): List of is_single_unit values (e.g., [1.0, 0.0, None]).
        stim_responsivities (list of float): List of stim_responsivity values (e.g., [1.0, 0.0, None]).
        base_directory (str): Base directory to save the plots.
        stimulations (list of str, optional): List of stimulations to include in the plot. If None, include all stimulations.
    """
    required_columns = ['mean_stimulation', 'Stimulation', 'Group']
    
    for cell_type in cell_types:
        for stim_range in stim_ranges:
            for is_single_unit in is_single_units:
                for stim_responsivity in stim_responsivities:
                    # Define file name suffix based on parameters
                    is_single_unit_str = 'All' if is_single_unit is None else str(is_single_unit)
                    stim_responsivity_str = 'All' if stim_responsivity is None else str(stim_responsivity)
                    suffix = f"{cell_type}_IsSingleUnit_{is_single_unit_str}_IsStimResponsive_{stim_responsivity_str}_StimRange_{stim_range[0]}to{stim_range[1]}"
                    
                    # Calculate basic stats
                    print(f"Calculating basic stats for: {suffix}")
                    whisker_df_manager.calculate_basic_stats(
                        'CTZ', 'No_CTZ', 
                        baseline_range=(-100, -1), stim_range=stim_range, 
                        cell_type=cell_type, is_single_unit=is_single_unit, 
                        stim_responsivity=stim_responsivity, 
                        smoothing_window=3, modulation_label='positive'
                    )
                    
                    # Prepare data for boxplot
                    print("Preparing data for boxplot")
                    df = whisker_df_manager.prepare_for_boxplot()
                    df = replace_mean_stimulation_with_column_value(df, target_column)
                    
                    
                    print(f"DataFrame shape after preparation: {df.shape}")
                    print(f"DataFrame columns: {df.columns}")
                    print(f"First few rows of DataFrame: {df.head()}")

                    # Check if the required columns are present
                    missing_columns = [col for col in required_columns if col not in df.columns]
                    if missing_columns:
                        print(f"Skipping combination {suffix} due to missing columns: {missing_columns}")
                        continue
                    
                    # Define directory for saving plots
                    directory = os.path.join(base_directory, 'boxplots_and_line', suffix)
                    os.makedirs(directory, exist_ok=True)
                    print(f"Directory created: {directory}")
                    
                    
                    #if cell_type == 'FS':
                    #    ylim = (0.0, 0.08)
                    #if cell_type == 'RS':
                    #   ylim = (0.0, 0.03)
                        
                    #whisker_df_manager.plot_box_and_strip(df, 
                    #    groups=['CTZ', 'No_CTZ'], stimulations=stimulations, show_outliers=True,
                    #    directory=directory, file_name=f'{suffix}_boxplot', ylim=ylim, 
                    #    modulation_label='positive'
                    #)
                    
                    
                    # Plot box and strip plots
                    print("Plotting box and strip plots")
                    whisker_df_manager.plot_box_and_strip_with_controls(df, 
                        groups=['CTZ', 'No_CTZ'], stimulations=stimulations, show_outliers=False, show_scatter=True, 
                        directory=directory, file_name=f'{suffix}_boxplot', 
                        modulation_label='positive'
                    )
                    
                    # Plot mean and SEM line plots
                    #print("Plotting mean and SEM line plots")
                    #whisker_df_manager.plot_mean_and_sem_lineplot(df, 
                    #    groups=['CTZ', 'No_CTZ'], stimulations=stimulations, directory=directory,
                    #    file_name=f'{suffix}_lineplot', ylim=None, 
                    #    modulation_label='positive'
                    #)


base_directory = '/Volumes/MannySSD/figures/comparing_conditions_dynamic/all_stimulations_FirstSpikeLatency_Reliability'
dynamic_plot_saving_beyond_PSTHs(whisker_df_manager, target_column, cell_types, stim_ranges, is_single_units, stim_responsivities, base_directory, stimulations = ['Low', 'Mid', 'Max'])


descriptive_stats_df_FirstSpikeLatency_Reliability = dynamic_statistical_analysis_beyond_PSTHs(whisker_df_manager, target_column, 
                                                                                   cell_types, stim_ranges, is_single_units, stim_responsivities, stimulations)
input_df_4_kruskal_and_permutation_FirstSpikeLatency_Reliability = generate_actual_data(whisker_df_manager, 
                                                              cell_types, stim_ranges, is_single_units, stim_responsivities, stimulations)
kruskal_results_df_FirstSpikeLatency_Reliability = perform_kruskal_wallis_on_actual_data(input_df_4_kruskal_and_permutation, stimulations)
perm_results_df_FirstSpikeLatency_Reliability  = perform_permutation_test_on_actual_data(input_df_4_kruskal_and_permutation, stimulations, n_permutations=1000)

descriptive_stats_df_FirstSpikeLatency_Reliability.to_csv('/Volumes/MannySSD/figures/comparing_conditions_dynamic/all_stimulations_FirstSpikeLatency_Reliability/descriptive_stats_FirstSpikeLatency_Reliability.csv', index=False)
input_df_4_kruskal_and_permutation_FirstSpikeLatency_Reliability.to_csv('/Volumes/MannySSD/figures/comparing_conditions_dynamic/all_stimulations_FirstSpikeLatency_Reliability/input_df_4_kruskal_and_permutation_FirstSpikeLatency_Reliability.csv', index=False)
kruskal_results_df_FirstSpikeLatency_Reliability.to_csv('/Volumes/MannySSD/figures/comparing_conditions_dynamic/all_stimulations_FirstSpikeLatency_Reliability/kruskal_results_FirstSpikeLatency_Reliability.csv', index=False)
perm_results_df_FirstSpikeLatency_Reliability.to_csv('/Volumes/MannySSD/figures/comparing_conditions_dynamic/all_stimulations_FirstSpikeLatency_Reliability/perm_results_FirstSpikeLatency_Reliability.csv', index=False)

Run a smilar plotting and analysis approach for other analysis beyond PSTHS--- but only for FanoFactor_stim 

In [ ]:
cell_types = ['FS', 'RS']
stim_ranges =  [(0, 500), (0,50), (0,20)]
is_single_units = [1.0, 0.0]
stim_responsivities = [1.0]
stimulations = ['Zero', 'Low', 'Mid', 'Max']
target_column='FanoFactor_stim'

def dynamic_plot_saving_beyond_PSTHs(whisker_df_manager, target_column, cell_types, stim_ranges, is_single_units, stim_responsivities, base_directory, stimulations=None):
    """
    
    ### you might have to change the ylim values for the different cell types!!!!!! 
    
    Dynamically saves plots across different conditions for specified cell types and parameter combinations.
    
    This function is an extension of the dynamic_plot_saving_beyond_PSTHs function to handle beyond PSTH data analysis so 
    that it can be used for other data types beyond PSTHs. It used a helper to replace the mean_stimulation column with the target column data and unpacks the target column data for analysis.
    The target columns must have four indexed within their array (zero, low, mid, and max) correspondign to the stimulations and then replaced.

    Args:
        whisker_df_manager: The data manager object with methods to calculate stats and plot.
        cell_types (list of str): List of cell types (e.g., ['FS', 'RS']).
        stim_ranges (list of tuple): List of stimulation ranges (e.g., [(0, 500), (0, 1000)]).
        is_single_units (list of float): List of is_single_unit values (e.g., [1.0, 0.0, None]).
        stim_responsivities (list of float): List of stim_responsivity values (e.g., [1.0, 0.0, None]).
        base_directory (str): Base directory to save the plots.
        stimulations (list of str, optional): List of stimulations to include in the plot. If None, include all stimulations.
    """
    required_columns = ['mean_stimulation', 'Stimulation', 'Group']
    
    for cell_type in cell_types:
        for stim_range in stim_ranges:
            for is_single_unit in is_single_units:
                for stim_responsivity in stim_responsivities:
                    # Define file name suffix based on parameters
                    is_single_unit_str = 'All' if is_single_unit is None else str(is_single_unit)
                    stim_responsivity_str = 'All' if stim_responsivity is None else str(stim_responsivity)
                    suffix = f"{cell_type}_IsSingleUnit_{is_single_unit_str}_IsStimResponsive_{stim_responsivity_str}_StimRange_{stim_range[0]}to{stim_range[1]}"
                    
                    # Calculate basic stats
                    print(f"Calculating basic stats for: {suffix}")
                    whisker_df_manager.calculate_basic_stats(
                        'CTZ', 'No_CTZ', 
                        baseline_range=(-100, -1), stim_range=stim_range, 
                        cell_type=cell_type, is_single_unit=is_single_unit, 
                        stim_responsivity=stim_responsivity, 
                        smoothing_window=3, modulation_label='positive'
                    )
                    
                    # Prepare data for boxplot
                    print("Preparing data for boxplot")
                    df = whisker_df_manager.prepare_for_boxplot()
                    df = replace_mean_stimulation_with_column_value(df, target_column)
                    
                    
                    print(f"DataFrame shape after preparation: {df.shape}")
                    print(f"DataFrame columns: {df.columns}")
                    print(f"First few rows of DataFrame: {df.head()}")

                    # Check if the required columns are present
                    missing_columns = [col for col in required_columns if col not in df.columns]
                    if missing_columns:
                        print(f"Skipping combination {suffix} due to missing columns: {missing_columns}")
                        continue
                    
                    # Define directory for saving plots
                    directory = os.path.join(base_directory, 'boxplots_and_line', suffix)
                    os.makedirs(directory, exist_ok=True)
                    print(f"Directory created: {directory}")
                    
                    
                    if cell_type == 'FS':
                        ylim = (0.80, 1.02)
                    if cell_type == 'RS':
                       ylim = (0.95, 1.02)
                        
                    # Plot box and strip plots
                    print("Plotting box and strip plots")
                    whisker_df_manager.plot_box_and_strip(df, 
                        groups=['CTZ', 'No_CTZ'], stimulations=stimulations, show_outliers=True,
                        directory=directory, file_name=f'{suffix}_boxplot', ylim=ylim, 
                        modulation_label='positive'
                    )
                    
                    # Plot mean and SEM line plots
                    #print("Plotting mean and SEM line plots")
                    #whisker_df_manager.plot_mean_and_sem_lineplot(df, 
                    #    groups=['CTZ', 'No_CTZ'], stimulations=stimulations, directory=directory,
                    #    file_name=f'{suffix}_lineplot', ylim=None, 
                    #    modulation_label='positive'
                    #)

base_directory = '/Volumes/MannySSD/figures/comparing_conditions_dynamic/all_stimulations_FanoFactor_stim'

dynamic_plot_saving_beyond_PSTHs(whisker_df_manager, target_column, cell_types, stim_ranges, is_single_units, stim_responsivities, base_directory, stimulations = ['Low', 'Mid', 'Max'])


descriptive_stats_df_FanoFactor_stim = dynamic_statistical_analysis_beyond_PSTHs(whisker_df_manager, target_column, 
                                                                                   cell_types, stim_ranges, is_single_units, stim_responsivities, stimulations)
input_df_4_kruskal_and_permutation_FanoFactor_stim = generate_actual_data(whisker_df_manager, 
                                                              cell_types, stim_ranges, is_single_units, stim_responsivities, stimulations)
kruskal_results_df_FanoFactor_stim = perform_kruskal_wallis_on_actual_data(input_df_4_kruskal_and_permutation, stimulations)
perm_results_df_FanoFactor_stim  = perform_permutation_test_on_actual_data(input_df_4_kruskal_and_permutation, stimulations, n_permutations=1000)



descriptive_stats_df_FanoFactor_stim.to_csv('/Volumes/MannySSD/figures/comparing_conditions_dynamic/all_stimulations_FirstSpikeLatency_Reliability/descriptive_stats_FanoFactor_stim.csv', index=False)
input_df_4_kruskal_and_permutation_FanoFactor_stim.to_csv('/Volumes/MannySSD/figures/comparing_conditions_dynamic/all_stimulations_FanoFactor_stim/input_df_4_kruskal_and_permutation_FanoFactor_stim.csv', index=False)
kruskal_results_df_FanoFactor_stim.to_csv('/Volumes/MannySSD/figures/comparing_conditions_dynamic/all_stimulations_FanoFactor_stim/kruskal_results_FanoFactor_stim.csv', index=False)
perm_results_df_FanoFactor_stim.to_csv('/Volumes/MannySSD/figures/comparing_conditions_dynamic/all_stimulations_FanoFactor_stim/perm_results_FanoFactor_stim.csv', index=False)


## plot and analyze bases on location of unit on probe 

In [ ]:
# revert the modulation 
whisker_df_manager.revert_modulation_index_to_numeric()
print(whisker_df_manager.dataframes['basic_metrics']['ModulationIndex'].head())
print(whisker_df_manager.dataframes['basic_metrics'][['groupname', 'recordingname', 'cid', 'ModulationIndex_Numeric']].head())
print(whisker_df_manager.dataframes['basic_metrics'][['groupname', 'recordingname', 'cid', 'ModulationIndex']].head())


If you want to plot per group, laminar profile with basefline FR

In [ ]:
whisker_df_manager.plot_modulation_index_with_firing_rate_color_and_size('basic_metrics', group_name='CTZ', is_single_unit=0.0, cell_type=None, stim_responsivity=None, jitter=0.9)
whisker_df_manager.plot_modulation_index_with_firing_rate_color_and_size('basic_metrics', group_name='No_CTZ', is_single_unit=0.0, cell_type=None, stim_responsivity=None, jitter=0.9)

In [ ]:
whisker_df_manager.plot_modulation_index_with_firing_rate_color_and_size_groupcomparison('basic_metrics', 
                                                                                         is_single_unit=None,
                                                                                         cell_type=None, 
                                                                                         stim_responsivity=None, 
                                                                                         jitter=0.5, 
                                                                                         size_multiplier=50, 
                                                                                         min_size=5, 
                                                                                         min_fr_threshold=0.0001)

In [ ]:
whisker_df_manager.plot_modulation_index_with_firing_rate_color_and_size_groupcomparison('basic_metrics', 
                                                                                         is_single_unit=1.0,
                                                                                         cell_type='FS', 
                                                                                         stim_responsivity=None, 
                                                                                         jitter=0.5, 
                                                                                         size_multiplier=50, 
                                                                                         min_size=5, 
                                                                                         min_fr_threshold=0.0001)

In [ ]:
def plot_modulation_index_with_stim_prob_color_and_size_groupcomparison(data_obj, df_name, is_single_unit=None, cell_type=None, stim_responsivity=None, jitter=0.1, size_multiplier=1, min_size=20, min_prob_threshold=0.5):
    """
    Plot the modulation index as a function of electrode location for CTZ and No_CTZ groups side by side.
    StimProb AUROC (first value in the array) is represented by both color and size of circles, with improved scaling for AUROC values.
    The 'plasma' colormap is used for values > 0.5, and black triangles are used for values == 0.5.
    Circles have black outlines and are slightly offset horizontally to reduce overlap. 
    Horizontal lines connect each point to x=0.

    Parameters:
    data_obj (object): Object containing the data and methods for data preparation.
    df_name (str): Name of the DataFrame to filter and plot.
    is_single_unit (float or None): If 1.0, filter for single units, if 0.0, filter for non-single units, 
                                    if None, do not filter by this criterion.
    cell_type (str or None): Filter for 'FS' or 'RS' cell types. If None, no filtering by cell type.
    stim_responsivity (float or None): Value to filter by in the StimResponsivity column. Can be 1.0, 0.0, or -1.0.
                                    If None, no filtering by StimResponsivity.
    jitter (float): Amount of horizontal jitter to apply to points. Default is 0.1.
    size_multiplier (float): Multiplier for circle sizes. Default is 1.
    min_size (float): Minimum size for circles with StimProb at the threshold. Default is 20.
    min_prob_threshold (float): StimProb threshold for minimum size. Default is 0.5.
    """
    groups = ['CTZ', 'No_CTZ']
    fig, axes = plt.subplots(1, 2, figsize=(6, 10), sharey=True)

    all_data = []
    for group in groups:
        data = data_obj.prepare_plotting_data(df_name, group, is_single_unit, cell_type, stim_responsivity)
        if data is not None and not data.empty:
            all_data.append(data)

    if not all_data:
        print("No data available for plotting.")
        return

    # Combine all data for global color scaling
    combined_data = pd.concat(all_data)

    # Extract the first value (AUROC) from each StimProb array
    combined_data['StimProb_AUROC'] = combined_data['StimProb'].apply(lambda x: x[0] if isinstance(x, (list, np.ndarray)) and len(x) > 0 else x)
    combined_data['StimProb_AUROC'] = pd.to_numeric(combined_data['StimProb_AUROC'], errors='coerce')

    # Ensure StimProb_AUROC values are between 0.5 and 1
    combined_data['StimProb_AUROC'] = combined_data['StimProb_AUROC'].clip(0.5, 1)

    min_prob = combined_data['StimProb_AUROC'].min()
    max_prob = combined_data['StimProb_AUROC'].max()

    # Define size calculation function with improved scaling
    def calculate_size(prob):
        return min_size + (prob - min_prob_threshold) * size_multiplier * 1000

    for ax, group in zip(axes, groups):
        plotting_data = combined_data[combined_data['groupname'] == group]

        if plotting_data.empty:
            ax.text(0.5, 0.5, f'No data for {group}', ha='center', va='center', transform=ax.transAxes)
            continue

        # Add horizontal jitter
        plotting_data['Jittered_ElectrodeOrder'] = plotting_data['ElectrodeOrder'] + np.random.uniform(-jitter, jitter, len(plotting_data))

        # Calculate sizes
        plotting_data['sizes'] = plotting_data['StimProb_AUROC'].apply(calculate_size)

        # Plot horizontal lines from x=0 to each point
        for _, row in plotting_data.iterrows():
            ax.plot([0, row['ModulationIndex_Numeric']], [row['Jittered_ElectrodeOrder'], row['Jittered_ElectrodeOrder']], 
                    color='gray', alpha=0.3, linewidth=0.5)

        # Separate data for AUROC == 0.5 and AUROC > 0.5
        data_05 = plotting_data[plotting_data['StimProb_AUROC'] == 0.5]
        data_gt_05 = plotting_data[plotting_data['StimProb_AUROC'] > 0.5]

        # Plot circles for AUROC > 0.5
        scatter = ax.scatter(data_gt_05['ModulationIndex_Numeric'], data_gt_05['Jittered_ElectrodeOrder'], 
                            c=data_gt_05['StimProb_AUROC'], cmap='plasma', 
                            s=data_gt_05['sizes'], alpha=0.8, vmin=0.5, vmax=1.0, 
                            edgecolors='black', linewidths=0.5)

        # Plot black triangles for AUROC == 0.5
        ax.scatter(data_05['ModulationIndex_Numeric'], data_05['Jittered_ElectrodeOrder'], 
                   c='black', marker='^', s=data_05['sizes'], 
                   edgecolors='black', linewidths=0.5)

        # Add a vertical line at x=0
        ax.axvline(x=0, color='black', linestyle='--', linewidth=0.5)

        ax.set_xlabel('Spontaneous MI')
        ax.set_title(f'{group}\n{cell_type or "All"} {"single" if is_single_unit == 1.0 else "multi" if is_single_unit == 0.0 else "all"} units')
        ax.invert_yaxis()  # To match the order from the provided list
        ax.set_xlim(-1.25, 1.25)
        ax.grid(True, linestyle=':', alpha=0.6)
        ax.set_yticks(np.arange(0, 32, 1))

    # Add a common y-label
    fig.text(0.04, 0.5, 'Electrode Order', va='center', rotation='vertical')

    # Add colorbar for StimProb AUROC with improved scaling
    cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])  # [left, bottom, width, height]
    cbar = fig.colorbar(scatter, cax=cbar_ax, extend='max')
    cbar.set_label('StimProb AUROC')
    
    # Create more ticks for the colorbar
    tick_locator = plt.LinearLocator(numticks=6)
    cbar.locator = tick_locator
    cbar.update_ticks()

    # Add legend for black triangles
    legend_elements = [plt.Line2D([0], [0], marker='^', color='w', label='AUROC = 0.5',
                                  markerfacecolor='black', markersize=10)]
    ax.legend(handles=legend_elements, loc='lower right')

    # Adjust layout
    plt.tight_layout()
    plt.subplots_adjust(right=0.9)  # Make room for the colorbar

    # Save the plot
    directory = '/Volumes/MannySSD/figures/laminar_plots'
    try:
        os.makedirs(directory, exist_ok=True)
    except OSError as e:
        print(f"Error creating directory: {e}")
        return

    file_name = f'CTZ_No_CTZ_comparison_{"single" if is_single_unit == 1.0 else "multi" if is_single_unit == 0.0 else "all"}_units_stim_prob_auroc_color_size_jittered_plasma_triangle'
    if cell_type:
        file_name += f'_{cell_type}'
    if stim_responsivity is not None:
        file_name += f'_stim_{stim_responsivity}'

    file_path = os.path.join(directory, f'{file_name}.svg')

    try:
        plt.savefig(file_path, format='svg', dpi=300, bbox_inches='tight')
        print(f"Plot saved to {file_path}")
        plt.show()
    except Exception as e:
        print(f"Error saving plot: {e}")
     
plot_modulation_index_with_stim_prob_color_and_size_groupcomparison(
    whisker_df_manager, 
    'basic_metrics', 
    is_single_unit=1.0, 
    cell_type='FS', 
    stim_responsivity=None, 
    size_multiplier=2, 
    jitter=0.5, 
    min_prob_threshold=0.5, 
    min_size=20
)

In [ ]:
def plot_modulation_index_with_stim_prob_color_and_size_groupcomparison(data_obj, df_name, is_single_unit=None, cell_type=None, stim_responsivity=None, jitter=0.1, size_multiplier=1, min_size=20, min_prob_threshold=0.001):
    """
    Plot the modulation index as a function of electrode location for CTZ and No_CTZ groups side by side.
    StimProb is represented by both color and size of circles, with customizable size controls.
    Circles have black outlines and are slightly offset horizontally to reduce overlap. 
    Horizontal lines connect each point to x=0.

    Parameters:
    data_obj (object): Object containing the data and methods for data preparation.
    df_name (str): Name of the DataFrame to filter and plot.
    is_single_unit (float or None): If 1.0, filter for single units, if 0.0, filter for non-single units, 
                                    if None, do not filter by this criterion.
    cell_type (str or None): Filter for 'FS' or 'RS' cell types. If None, no filtering by cell type.
    stim_responsivity (float or None): Value to filter by in the StimResponsivity column. Can be 1.0, 0.0, or -1.0.
                                    If None, no filtering by StimResponsivity.
    jitter (float): Amount of horizontal jitter to apply to points. Default is 0.1.
    size_multiplier (float): Multiplier for circle sizes. Default is 1.
    min_size (float): Minimum size for circles with StimProb below the threshold. Default is 20.
    min_prob_threshold (float): StimProb threshold below which circles will have the minimum size. Default is 0.001.
    """
    groups = ['CTZ', 'No_CTZ']
    fig, axes = plt.subplots(1, 2, figsize=(6, 8), sharey=True)

    all_data = []
    for group in groups:
        data = data_obj.prepare_plotting_data(df_name, group, is_single_unit, cell_type, stim_responsivity)
        if data is not None and not data.empty:
            all_data.append(data)

    if not all_data:
        print("No data available for plotting.")
        return

    # Combine all data for global color scaling
    combined_data = pd.concat(all_data)

    # Convert StimProb to scalar values
    combined_data['StimProb'] = combined_data['StimProb'].apply(lambda x: x.item() if isinstance(x, np.ndarray) else x)
    combined_data['StimProb'] = pd.to_numeric(combined_data['StimProb'], errors='coerce')

    max_prob = combined_data['StimProb'].max()

    # Define size calculation function
    def calculate_size(prob):
        if prob <= min_prob_threshold:
            return min_size
        else:
            return min_size + (prob - min_prob_threshold) * size_multiplier

    for ax, group in zip(axes, groups):
        plotting_data = combined_data[combined_data['groupname'] == group]

        if plotting_data.empty:
            ax.text(0.5, 0.5, f'No data for {group}', ha='center', va='center', transform=ax.transAxes)
            continue

        # Add horizontal jitter
        plotting_data['Jittered_ElectrodeOrder'] = plotting_data['ElectrodeOrder'] + np.random.uniform(-jitter, jitter, len(plotting_data))

        # Calculate sizes
        plotting_data['sizes'] = plotting_data['StimProb'].apply(calculate_size)

        # Plot horizontal lines from x=0 to each point
        for _, row in plotting_data.iterrows():
            ax.plot([0, row['ModulationIndex_Numeric']], [row['Jittered_ElectrodeOrder'], row['Jittered_ElectrodeOrder']], 
                    color='gray', alpha=0.3, linewidth=0.5)

        # Plot the modulation index points
        scatter = ax.scatter(plotting_data['ModulationIndex_Numeric'], plotting_data['Jittered_ElectrodeOrder'], 
                            c=plotting_data['StimProb'], cmap='hot', 
                            s=plotting_data['sizes'], alpha=0.8, vmin=0, vmax=max_prob, 
                            edgecolors='black', linewidths=0.5)

        # Add a vertical line at x=0
        ax.axvline(x=0, color='black', linestyle='--', linewidth=0.5)

        ax.set_xlabel('Spontaneous MI')
        ax.set_title(f'{group}\n{cell_type} {"single" if is_single_unit == 1.0 else "multi"}-units')
        ax.invert_yaxis()  # To match the order from the provided list
        ax.set_xlim(-1.0, 1.0)
        ax.grid(False)
        ax.set_yticks(np.arange(0, 32, 1))

    # Add a common y-label
    fig.text(0.04, 0.5, 'Electrode Order', va='center', rotation='vertical')

    # Add colorbar for StimProb
    cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])  # [left, bottom, width, height]
    cbar = fig.colorbar(scatter, cax=cbar_ax)
    cbar.set_label('StimProb')
    cbar.set_ticks([0, max_prob])
    cbar.set_ticklabels(['0', f'{max_prob:.2f}'])

    # Adjust layout
    plt.tight_layout()
    plt.subplots_adjust(right=0.9)  # Make room for the colorbar

    # Save the plot
    directory = '/Volumes/MannySSD/figures/laminar_plots'
    try:
        os.makedirs(directory, exist_ok=True)
    except OSError as e:
        print(f"Error creating directory: {e}")
        return

    file_name = f'CTZ_No_CTZ_comparison_{"single" if is_single_unit == 1.0 else "multi"}_units_stim_prob_color_size_jittered_hot'
    if cell_type:
        file_name += f'_{cell_type}'
    if stim_responsivity is not None:
        file_name += f'_stim_{stim_responsivity}'

    file_path = os.path.join(directory, f'{file_name}.svg')

    try:
        plt.savefig(file_path, format='svg', dpi=300, bbox_inches='tight')
        print(f"Plot saved to {file_path}")
        plt.show()
    except Exception as e:
        print(f"Error saving plot: {e}")
        

plot_modulation_index_with_stim_prob_color_and_size_groupcomparison(whisker_df_manager, 'basic_metrics', is_single_unit=None, cell_type=None, stim_responsivity=None, jitter=0.1, size_multiplier=1, min_size=20, min_prob_threshold=0.001)
        


In [ ]:
whisker_df_manager.plot_modulation_index_with_firing_rate_color_and_size_groupcomparison('basic_metrics', 
                                                                                         is_single_unit=0.0,
                                                                                         cell_type=None, 
                                                                                         stim_responsivity=None, 
                                                                                         jitter=0.5, 
                                                                                         size_multiplier=50, 
                                                                                         min_size=5, 
                                                                                         min_fr_threshold=0.0001)

In [ ]:
whisker_df_manager.plot_modulation_index_with_firing_rate_color_and_size_groupcomparison('basic_metrics', 
                                                                                         is_single_unit=None,
                                                                                         cell_type='RS', 
                                                                                         stim_responsivity=None, 
                                                                                         jitter=0.5, 
                                                                                         size_multiplier=100, 
                                                                                         min_size=5, 
                                                                                         min_fr_threshold=0.01)

In [ ]:
whisker_df_manager.plot_modulation_index_with_firing_rate_color_and_size_groupcomparison('basic_metrics', 
                                                                                         is_single_unit=0.0,
                                                                                         cell_type=None, 
                                                                                         stim_responsivity=None, 
                                                                                         jitter=0.5, 
                                                                                         size_multiplier=30, 
                                                                                         min_size=2, 
                                                                                         min_fr_threshold=0.01)

In [ ]:
whisker_df_manager.plot_modulation_index_with_firing_rate_color_and_size_groupcomparison_evoked('basic_metrics', 
                                                                                         is_single_unit=0.0,
                                                                                         stim_index=3,  # Use max stimulation (index 4) 
                                                                                         cell_type=None, 
                                                                                         stim_responsivity=None, 
                                                                                         jitter=0.5, 
                                                                                         size_multiplier=5, 
                                                                                         min_size=2, 
                                                                                         min_fr_threshold=0.001)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import os

def plot_modulation_index_comparison(obj, df_name, is_single_unit=None, cell_type=None, stim_responsivity=None, save_dir='/Volumes/MannySSD/figures/laminaranalysis'):
    """
    Plot the modulation index comparison between two groups as a function of electrode location using the object's methods.
    Ensures the y-axis always represents 32 electrodes in the specified order and saves the plot dynamically based on input parameters.

    Parameters:
    obj (object): The object containing the prepare_plotting_data method.
    df_name (str): Name of the DataFrame to filter and plot.
    is_single_unit (float or None): If 1.0, filter for single units, if 0.0, filter for non-single units, 
                                    if None, do not filter by this criterion.
    cell_type (str or None): Filter for 'FS' or 'RS' cell types. If None, no filtering by cell type.
    stim_responsivity (float or None): Value to filter by in the StimResponsivity column. Can be 1.0, 0.0, or -1.0.
                                       If None, no filtering by Stim Responsivity.
    save_dir (str): Directory where the plot will be saved. Defaults to './plots'.
    """

    # Define the groups
    group1 = 'CTZ'
    group2 = 'No_CTZ'
    
    # Define the custom electrode order
    electrodes_order = [14, 20, 16, 18, 1, 31, 3, 29, 5, 27, 7, 25, 9, 23, 11, 21, 13, 19, 15, 17, 12, 22, 10, 24, 8, 26, 6, 28, 4, 30, 2, 32]
    
    # Prepare the data for each group using the object's method
    plotting_data1 = obj.prepare_plotting_data(df_name, group1, is_single_unit, cell_type, stim_responsivity)
    plotting_data2 = obj.prepare_plotting_data(df_name, group2, is_single_unit, cell_type, stim_responsivity)

    # Map the electrode order to the correct positions
    electrode_mapping = {electrode: idx for idx, electrode in enumerate(electrodes_order)}

    # Adjust the ElectrodeOrder column to reflect the custom order
    plotting_data1['ElectrodeOrder'] = plotting_data1['ElectrodeOrder'].map(electrode_mapping)
    plotting_data2['ElectrodeOrder'] = plotting_data2['ElectrodeOrder'].map(electrode_mapping)

    # Create a 1x2 subplot with narrower plots
    fig, axes = plt.subplots(1, 2, figsize=(4, 6))

    for ax, (plotting_data, group) in zip(axes, [(plotting_data1, group1), (plotting_data2, group2)]):
        if plotting_data is None or plotting_data.empty:
            ax.set_title(f'No data available for {group}')
            continue

        # Drop rows with NaN in relevant columns
        plotting_data = plotting_data.dropna(subset=['MeanFR_baseline', 'ModulationIndex_Numeric', 'ElectrodeOrder'])

        # Convert 'MeanFR_baseline' from array-like to scalar values
        plotting_data['MeanFR_baseline'] = plotting_data['MeanFR_baseline'].apply(lambda x: x.item() if isinstance(x, np.ndarray) else x)

        # Convert to numeric and ensure correct length
        plotting_data['MeanFR_baseline'] = pd.to_numeric(plotting_data['MeanFR_baseline'], errors='coerce')
        plotting_data['ModulationIndex_Numeric'] = pd.to_numeric(plotting_data['ModulationIndex_Numeric'], errors='coerce')
        plotting_data['ElectrodeOrder'] = pd.to_numeric(plotting_data['ElectrodeOrder'], errors='coerce')

        # Drop rows where any of these columns have NaNs after conversion
        plotting_data = plotting_data.dropna(subset=['MeanFR_baseline', 'ModulationIndex_Numeric', 'ElectrodeOrder'])

        # Ensure all columns have the same length
        if not (len(plotting_data['MeanFR_baseline']) == len(plotting_data['ModulationIndex_Numeric']) == len(plotting_data['ElectrodeOrder'])):
            print(f"Mismatch in data sizes for group {group}. Skipping plot.")
            continue

        # Normalize size values if necessary
        size_values = plotting_data['MeanFR_baseline'].values
        size_values = size_values / np.max(size_values) * 200  # Normalize to range suitable for plotting, to calculate back to the 
        
        # Plot the modulation index
        scatter = ax.scatter(plotting_data['ModulationIndex_Numeric'], plotting_data['ElectrodeOrder'], 
                             s=size_values,  # Use normalized sizes
                             c=plotting_data['ModulationIndex_Numeric'], 
                             cmap='bwr', alpha=0.6)

        # Set title and labels
        ax.set_title(f'{group} ({cell_type} units)')
        ax.set_xlabel('Spontaneous MI')
        ax.set_ylabel('Electrode Order')
        ax.invert_yaxis()  # To match the order from the provided list

        # Ensure the y-axis has all 32 electrodes listed 
        ax.set_yticks(np.arange(0, 32, 1))  # Set y-ticks to represent all 32 electrodes
        ax.set_ylim(31, 0)  # Set y-limits to cover all 32 electrodes
        ax.set_xlim(-1.1, 1.1)  # Enforce the color bar is always from -1 to 1

        # Enforce the color bar is always from -1 to 1 
        scatter.set_clim(-1, 1)

        # Add a light grey dotted vertical line at x = -0.3
        ax.axvline(x=-0.3, color='lightgrey', linestyle='--', linewidth=1)
        ax.axvline(x=0.3, color='lightgrey', linestyle='--', linewidth=1)

    # Add a shared colorbar
    cbar = fig.colorbar(scatter, ax=axes, orientation='vertical', label='Modulation Index')

    # Create a custom legend for circle sizes using the proxy artist approach
    legend_labels = [50, 100, 200]  # Example sizes to show in the legend
    handles = [plt.scatter([], [], s=size, color='gray', alpha=0.6, label=f'{size:.2f}') for size in legend_labels]
    plt.legend(handles=handles, title="MeanFR Baseline", loc='upper right', frameon=True)

    # Adjust layout to avoid overlap and make the plots more narrow
    plt.subplots_adjust(wspace=0.3)  # Increase the space between the subplots
    fig.tight_layout(pad=2)  # Adjust the overall padding around the figure

    # Create a dynamic file name based on the input parameters
    file_name = f"{df_name}_{cell_type}_SU{is_single_unit}_SR{stim_responsivity}.svg"
    file_path = os.path.join(save_dir, file_name)

    # Ensure the save directory exists
    os.makedirs(save_dir, exist_ok=True)

    # Save the plot
    plt.savefig(file_path, format='svg', transparent=True)
    print(f"Plot saved to {file_path}")

    plt.show()


#whisker_df_manager.plot_modulation_index('basic_metrics', is_single_unit=None, cell_type='RS', stim_responsivity=None)
#whisker_df_manager.plot_modulation_index('basic_metrics', is_single_unit=None, cell_type='FS', stim_responsivity=None)
#whisker_df_manager.plot_modulation_index('basic_metrics', is_single_unit=None, cell_type=None, stim_responsivity=None)



# Assuming whisker_df_manager is your object with prepare_plotting_data defined
plot_modulation_index_comparison(whisker_df_manager, df_name='basic_metrics', is_single_unit=1.0, cell_type='RS', stim_responsivity=1.0)
plot_modulation_index_comparison(whisker_df_manager, df_name='basic_metrics', is_single_unit=1.0, cell_type='FS', stim_responsivity=1.0)
plot_modulation_index_comparison(whisker_df_manager, df_name='basic_metrics', is_single_unit=1.0, cell_type='FS', stim_responsivity=None)
plot_modulation_index_comparison(whisker_df_manager, df_name='basic_metrics', is_single_unit=None, cell_type='FS', stim_responsivity=None)
plot_modulation_index_comparison(whisker_df_manager, df_name='basic_metrics', is_single_unit=None, cell_type='RS', stim_responsivity=None)
plot_modulation_index_comparison(whisker_df_manager, df_name='basic_metrics', is_single_unit=0.0, cell_type='FS', stim_responsivity=None)
plot_modulation_index_comparison(whisker_df_manager, df_name='basic_metrics', is_single_unit=0.0, cell_type='RS', stim_responsivity=None)



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import (
    ks_2samp,
    mannwhitneyu,
    cramervonmises_2samp,
    sem,
    t
)
import os

def calculate_confidence_interval(data, confidence=0.95):
    """Calculate confidence interval for a data series."""
    n = len(data)
    mean = np.mean(data)
    std_err = sem(data)
    h = std_err * t.ppf((1 + confidence) / 2., n-1)
    return mean - h, mean + h

def plot_modulation_index_comparison(
    obj,
    df_name,
    is_single_unit=None,
    cell_type=None,
    stim_responsivity=None,
    save_dir='/Volumes/MannySSD/figures/laminaranalysis'
):
    """
    Plot and analyze the modulation index comparison between two groups as a function of electrode location.
    Performs statistical tests to determine if distributions differ significantly.
    
    Parameters:
    obj (object): The object containing the prepare_plotting_data method.
    df_name (str): Name of the DataFrame to filter and plot.
    is_single_unit (float or None): If 1.0, filter for single units, if 0.0, filter for non-single units, 
                                    if None, do not filter by this criterion.
    cell_type (str or None): Filter for 'FS' or 'RS' cell types. If None, no filtering by cell type.
    stim_responsivity (float or None): Value to filter by in the StimResponsivity column. Can be 1.0, 0.0, or -1.0.
                                       If None, no filtering by Stim Responsivity.
    save_dir (str): Directory where the plot and results will be saved.
    """
    # Define the groups
    group1 = 'CTZ'
    group2 = 'No_CTZ'
    
    # Prepare the data for each group using the object's method
    data1 = obj.prepare_plotting_data(df_name, group1, is_single_unit, cell_type, stim_responsivity)
    data2 = obj.prepare_plotting_data(df_name, group2, is_single_unit, cell_type, stim_responsivity)
    
    # Check if data is available
    if data1 is None or data1.empty:
        print(f"No data available for group {group1} with the specified filters.")
        return
    if data2 is None or data2.empty:
        print(f"No data available for group {group2} with the specified filters.")
        return
    
    # Check for the presence of ElectrodeOrder instead of Template_Channel
    if 'ElectrodeOrder' not in data1.columns or 'ElectrodeOrder' not in data2.columns:
        print("The column 'ElectrodeOrder' is not present in one of the datasets.")
        return
    
    # Ensure necessary columns are present and drop NaNs
    required_columns = ['ModulationIndex_Numeric', 'ElectrodeOrder', 'MeanFR_baseline']
    data1 = data1.dropna(subset=required_columns)
    data2 = data2.dropna(subset=required_columns)
    
    # Convert columns to appropriate types
    for col in required_columns:
        data1[col] = pd.to_numeric(data1[col], errors='coerce')
        data2[col] = pd.to_numeric(data2[col], errors='coerce')
    
    # Descriptive Statistics
    def get_descriptive_stats(data, group_name):
        positions = data['ElectrodeOrder']
        modulation_indices = data['ModulationIndex_Numeric']
        mean_pos = positions.mean()
        median_pos = positions.median()
        std_pos = positions.std()
        range_pos = positions.max() - positions.min()
        ci_low, ci_high = calculate_confidence_interval(positions)
        n_units = len(positions)
        
        stats = {
            'Group': group_name,
            'N_Units': n_units,
            'Mean_Position': mean_pos,
            'Median_Position': median_pos,
            'Std_Position': std_pos,
            'Range_Position': range_pos,
            'CI_Low': ci_low,
            'CI_High': ci_high
        }
        return stats
    
    stats1 = get_descriptive_stats(data1, group1)
    stats2 = get_descriptive_stats(data2, group2)
    
    descriptive_stats_df = pd.DataFrame([stats1, stats2])
    
    # Statistical Tests
    # Kolmogorov-Smirnov Test
    ks_stat, ks_p = ks_2samp(data1['ElectrodeOrder'], data2['ElectrodeOrder'])
    
    # Mann-Whitney U Test
    mw_stat, mw_p = mannwhitneyu(data1['ElectrodeOrder'], data2['ElectrodeOrder'], alternative='two-sided')
    
    # Cramér–von Mises Test
    cvm_result = cramervonmises_2samp(data1['ElectrodeOrder'], data2['ElectrodeOrder'])
    
    statistical_tests_results = {
        'Kolmogorov-Smirnov': {'Statistic': ks_stat, 'p-value': ks_p},
        'Mann-Whitney U': {'Statistic': mw_stat, 'p-value': mw_p},
        'Cramér–von Mises': {'Statistic': cvm_result.statistic, 'p-value': cvm_result.pvalue}
    }
    
    statistical_results_df = pd.DataFrame(statistical_tests_results).T
    
    # Visualization
    sns.set(style='whitegrid')
    plt.figure(figsize=(12, 6))
    
    # Histogram Plot
    bins = np.arange(min(data1['ElectrodeOrder'].min(), data2['ElectrodeOrder'].min()),
                     max(data1['ElectrodeOrder'].max(), data2['ElectrodeOrder'].max()) + 1)
    
    plt.hist(
        data1['ElectrodeOrder'],
        bins=bins,
        alpha=0.5,
        label=f'{group1} (N={stats1["N_Units"]})',
        color='skyblue',
        edgecolor='black'
    )
    plt.hist(
        data2['ElectrodeOrder'],
        bins=bins,
        alpha=0.5,
        label=f'{group2} (N={stats2["N_Units"]})',
        color='salmon',
        edgecolor='black'
    )
    
    plt.xlabel('Electrode Order')
    plt.ylabel('Unit Count')
    plt.title(f'Distribution of Units Across Probe\nCell Type: {cell_type} | Single Unit: {is_single_unit} | Stim Responsivity: {stim_responsivity}')
    plt.legend()
    
    # Annotate with Statistical Test Results
    annotation_text = (
        f"Kolmogorov-Smirnov Test:\n"
        f"  Statistic = {ks_stat:.3f}\n"
        f"  p-value = {ks_p:.3e}\n"
        f"Mann-Whitney U Test:\n"
        f"  Statistic = {mw_stat:.3f}\n"
        f"  p-value = {mw_p:.3e}\n"
        f"Cramér–von Mises Test:\n"
        f"  Statistic = {cvm_result.statistic:.3f}\n"
        f"  p-value = {cvm_result.pvalue:.3e}"
    )
    
    plt.gcf().text(0.75, 0.5, annotation_text, bbox=dict(boxstyle="round", facecolor='white', alpha=0.5))
    
    # Save Plot
    os.makedirs(save_dir, exist_ok=True)
    plot_filename = f"Unit_Distribution_{df_name}_CellType_{cell_type}_SU_{is_single_unit}_SR_{stim_responsivity}.png"
    plot_path = os.path.join(save_dir, plot_filename)
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"Plot saved at {plot_path}")
    
    # Save Descriptive Statistics
    desc_stats_filename = f"Descriptive_Stats_{df_name}_CellType_{cell_type}_SU_{is_single_unit}_SR_{stim_responsivity}.csv"
    desc_stats_path = os.path.join(save_dir, desc_stats_filename)
    descriptive_stats_df.to_csv(desc_stats_path, index=False)
    
    # Save Statistical Test Results
    stats_results_filename = f"Statistical_Results_{df_name}_CellType_{cell_type}_SU_{is_single_unit}_SR_{stim_responsivity}.csv"
    stats_results_path = os.path.join(save_dir, stats_results_filename)
    statistical_results_df.to_csv(stats_results_path)
    
    print(f"Descriptive statistics saved at {desc_stats_path}")
    print(f"Statistical test results saved at {stats_results_path}")
    
    # Print Results
    print("\nDescriptive Statistics:")
    print(descriptive_stats_df.to_string(index=False))
    
    print("\nStatistical Test Results:")
    print(statistical_results_df.to_string())
    
    # Example function call

plot_modulation_index_comparison(
    whisker_df_manager,  
    df_name='basic_metrics',  # DataFrame name
    is_single_unit=None,  # Set filter for single units
    cell_type=None,  # Set filter for Regular-Spiking (RS) cells
    stim_responsivity=0.0,  # Filter for responsive cells
    save_dir='/Volumes/MannySSD/figures/laminaranalysis'  # Directory to save plots and results
)



do not re-run unless you need to re process

In [ ]:

df = whisker_df_manager.dataframes['basic_metrics']
#exposrt and save the dataframe but onluy for the columns that are needed for the plotting: groupname, recordingname, cid, Cell_Type, IsSingleUnit, StimResponsivity, ModulationIndex, ModulationIndex_Numeric, save the dataframe as a csv file in my directory/Volumes/MannySSD/figures/laminaranalysis/basic_metrics_for_plotting_baseline_andspikes.csv
df[['groupname', 'recordingname', 'cid', 'Cell_Type', 'IsSingleUnit', 'StimResponsivity', 'ModulationIndex', 'ModulationIndex_Numeric', 'MeanFR_baseline', 'SpikeTimes_all']].to_csv('', index=False)


### Analyze the AUROC data by group and within groups 

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import sem, t, ranksums

def calculate_statistics(data):
    """Calculate statistics for the given data."""
    N = len(data)
    mean = np.mean(data)
    sd = np.std(data)
    median = np.median(data)
    min_val = np.min(data)
    max_val = np.max(data)
    
    # Calculate the 95% confidence interval
    ci_low, ci_high = calculate_confidence_interval(data)
    
    return {
        'N': N,
        'Mean': mean,
        'SD': sd,
        'Median': median,
        'CI_Low': ci_low,
        'CI_High': ci_high,
        'Min': min_val,
        'Max': max_val
    }

def calculate_confidence_interval(data, confidence=0.95):
    """Calculate the confidence interval for a data series."""
    n = len(data)
    mean = np.mean(data)
    std_err = sem(data)
    h = std_err * t.ppf((1 + confidence) / 2., n-1)
    return mean - h, mean + h

def compare_auroc_stats(obj, df_name, groupnames, is_single_unit=None, cell_type=None):
    """
    Compare the AUROC values between responsive and non-responsive neurons within and between groups.
    
    Parameters:
    obj (object): The object containing the data and methods.
    df_name (str): The name of the DataFrame to use.
    groupnames (list of str): List of groups to analyze (e.g., ['CTZ', 'No_CTZ']).
    is_single_unit (float or None): Filter by single-unit activity (1.0 for SUA, 0.0 for MUA, None for both).
    cell_type (str or None): Filter by cell type ('FS' for Fast-Spiking, 'RS' for Regular-Spiking, None for both).
    
    Returns:
    pd.DataFrame: DataFrame containing the calculated statistics and comparison results.
    """
    results = []

    for groupname in groupnames:
        # Retrieve data for the specified group with optional filters
        df = obj.get_filtered_data(df_name, groupname=groupname, is_single_unit=is_single_unit, cell_type=cell_type, stim_responsivity=None)
        
        # Pool StimResponsivity = -1.0 and 1.0 as responsive
        responsive_df = df[(df['StimResponsivity'] == -1.0) | (df['StimResponsivity'] == 1.0)]
        non_responsive_df = df[df['StimResponsivity'] == 0.0]
        
        # Extract StimProb values
        responsive_stimprob = responsive_df['StimProb'].apply(lambda x: x[0]).dropna().values
        non_responsive_stimprob = non_responsive_df['StimProb'].apply(lambda x: x[0]).dropna().values
        
        # Calculate statistics for each group
        responsive_stats = calculate_statistics(responsive_stimprob)
        non_responsive_stats = calculate_statistics(non_responsive_stimprob)
        
        # Perform statistical comparison within the group
        if len(responsive_stimprob) > 0 and len(non_responsive_stimprob) > 0:
            test_stat, p_value = ranksums(responsive_stimprob, non_responsive_stimprob)
        else:
            test_stat, p_value = np.nan, np.nan  # Handle cases where one or both groups have no data
        
        # Store the results in the list
        results.append({
            'Group': groupname,
            'Comparison': 'Responsive vs Non-Responsive',
            'N_Responsive': responsive_stats['N'],
            'N_Non_Responsive': non_responsive_stats['N'],
            'Mean_Responsive': responsive_stats['Mean'],
            'Mean_Non_Responsive': non_responsive_stats['Mean'],
            'Median_Responsive': responsive_stats['Median'],
            'Median_Non_Responsive': non_responsive_stats['Median'],
            'SD_Responsive': responsive_stats['SD'],
            'SD_Non_Responsive': non_responsive_stats['SD'],
            'Min_Responsive': responsive_stats['Min'],
            'Max_Responsive': responsive_stats['Max'],
            'Min_Non_Responsive': non_responsive_stats['Min'],
            'Max_Non_Responsive': non_responsive_stats['Max'],
            'CI_Low_Responsive': responsive_stats['CI_Low'],
            'CI_High_Responsive': responsive_stats['CI_High'],
            'CI_Low_Non_Responsive': non_responsive_stats['CI_Low'],
            'CI_High_Non_Responsive': non_responsive_stats['CI_High'],
            'Test_Statistic': test_stat,
            'p_value': p_value
        })
    
    # Between-group comparison for responsive neurons only
    if len(groupnames) == 2:
        group1_df = obj.get_filtered_data(df_name, groupname=groupnames[0], is_single_unit=is_single_unit, cell_type=cell_type, stim_responsivity=None)
        group2_df = obj.get_filtered_data(df_name, groupname=groupnames[1], is_single_unit=is_single_unit, cell_type=cell_type, stim_responsivity=None)
        
        group1_responsive = group1_df[(group1_df['StimResponsivity'] == -1.0) | (group1_df['StimResponsivity'] == 1.0)]['StimProb'].apply(lambda x: x[0]).dropna().values
        group2_responsive = group2_df[(group2_df['StimResponsivity'] == -1.0) | (group2_df['StimResponsivity'] == 1.0)]['StimProb'].apply(lambda x: x[0]).dropna().values
        
        if len(group1_responsive) > 0 and len(group2_responsive) > 0:
            test_stat, p_value = ranksums(group1_responsive, group2_responsive)
        else:
            test_stat, p_value = np.nan, np.nan  # Handle cases where one or both groups have no data
        
        # Calculate statistics for each responsive group
        group1_stats = calculate_statistics(group1_responsive)
        group2_stats = calculate_statistics(group2_responsive)
        
        results.append({
            'Group': f'{groupnames[0]} vs {groupnames[1]}',
            'Comparison': 'Responsive Neurons Between Groups',
            'N_Responsive_Group1': group1_stats['N'],
            'N_Responsive_Group2': group2_stats['N'],
            'Mean_Responsive_Group1': group1_stats['Mean'],
            'Mean_Responsive_Group2': group2_stats['Mean'],
            'Median_Responsive_Group1': group1_stats['Median'],
            'Median_Responsive_Group2': group2_stats['Median'],
            'SD_Responsive_Group1': group1_stats['SD'],
            'SD_Responsive_Group2': group2_stats['SD'],
            'Min_Responsive_Group1': group1_stats['Min'],
            'Max_Responsive_Group1': group1_stats['Max'],
            'Min_Responsive_Group2': group2_stats['Min'],
            'Max_Responsive_Group2': group2_stats['Max'],
            'CI_Low_Responsive_Group1': group1_stats['CI_Low'],
            'CI_High_Responsive_Group1': group1_stats['CI_High'],
            'CI_Low_Responsive_Group2': group2_stats['CI_Low'],
            'CI_High_Responsive_Group2': group2_stats['CI_High'],
            'Test_Statistic': test_stat,
            'p_value': p_value
        })
    
    # Convert the results list into a DataFrame
    results_df = pd.DataFrame(results)
    
    return results_df

# Perform the analysis for FS and RS neurons separately
cell_types = ['FS', 'RS']
results_dfs = []

for cell_type in cell_types:
    # Run the comparison analysis for each cell type
    results_df = compare_auroc_stats(
        whisker_df_manager,
        df_name='basic_metrics',
        groupnames=['CTZ', 'No_CTZ'],
        is_single_unit=None,  # No filter on single units for this analysis
        cell_type=cell_type  # Filter by cell type (FS or RS)
    )
    # Add a column for cell type in the results
    results_df['Cell_Type'] = cell_type
    results_dfs.append(results_df)

# Combine the results into a single DataFrame
combined_results_df = pd.concat(results_dfs, ignore_index=True)

# Display the combined results
combined_results_df.head(), combined_results_df.describe()

# Extract the p-values from the combined results DataFrame
p_values = combined_results_df['p_value']

# Check the p-values to confirm significance
p_values_significant = p_values < 0.05

# Display p-values along with their significance
p_values, p_values_significant

## Begin the PSTH shit show

### visualize the mean PSTH for all recording easily to visualize in notebook

In [ ]:
ctz_df1, noctz_df2 = whisker_df_manager.compare_groups('CTZ', 'No_CTZ', 'Max', 'RS', is_single_unit=1.0, stim_responsivity=None)

def process_data(df, time_range):
    
    time_array = whisker_df_manager.eed.relative_time_ms['relative_time_ms']
    
    if time_range:
        time_mask = (time_array >= time_range[0]) & (time_array <= time_range[1])
        time_array = time_array[time_mask]
    else:
        time_mask = slice(None)
    
    grouped = df.groupby('recordingname')
    grouped_data = grouped['PSTHs_raw'].apply(lambda x: np.mean([np.array(i)[time_mask] for i in x], axis=0))
    
    return grouped_data

ctz_df1_grouped = process_data(ctz_df1, time_range=(0, 50))
noctz_df2_grouped = process_data(noctz_df2, time_range=(0, 50))

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# Create a color map
colors = cm.rainbow(np.linspace(0, 1, len(ctz_df1_grouped)))

# Create a 1xN subplot
fig, axs = plt.subplots(1, len(ctz_df1_grouped), figsize=(15, 5))

# Iterate over the grouped data
for ax, data, color, name in zip(axs, ctz_df1_grouped, colors, ctz_df1_grouped.index):
    ax.plot(data, color=color)
    ax.set_title(name)

# Show the plot
plt.tight_layout()
plt.show()



In [ ]:
# Create a color map
colors = cm.rainbow(np.linspace(0, 1, len(noctz_df2_grouped)))

# Create a 1xN subplot
fig, axs = plt.subplots(1, len(noctz_df2_grouped), figsize=(15, 5))

# Iterate over the grouped data
for ax, data, color, name in zip(axs, noctz_df2_grouped, colors, noctz_df2_grouped.index):
    ax.plot(data, color=color)
    ax.set_title(name)

# Show the plot
plt.tight_layout()
plt.show()

### plot the AUROC distribution

plot the distriution for ALL singlue units (FS and RS combined) for each group

In [ ]:
whisker_df_manager.inspect_raw_data('basic_metrics', is_single_unit=1.0, cell_type=None, groupname=None)


plot the distibution for ALL FS and RS single units

In [ ]:
whisker_df_manager.inspect_categorized_data('basic_metrics', is_single_unit=1.0, cell_type=None, groupname=None)


In [ ]:
whisker_df_manager.check_stim_responsivity_distribution_before_after('basic_metrics', is_single_unit=1.0, cell_type=None, groupname=None)
whisker_df_manager.plot_raw_auroc_distribution('basic_metrics', is_single_unit=1.0, cell_type=None, groupname=None)


In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import sem, t, ranksums

def calculate_statistics(data):
    """Calculate statistics for the given data."""
    N = len(data)
    mean = np.mean(data)
    sd = np.std(data)
    median = np.median(data)
    min_val = np.min(data)
    max_val = np.max(data)
    
    # Calculate the 95% confidence interval
    ci_low, ci_high = calculate_confidence_interval(data)
    
    return {
        'N': N,
        'Mean': mean,
        'SD': sd,
        'Median': median,
        'CI_Low': ci_low,
        'CI_High': ci_high,
        'Min': min_val,
        'Max': max_val
    }

def calculate_confidence_interval(data, confidence=0.95):
    """Calculate the confidence interval for a data series."""
    n = len(data)
    mean = np.mean(data)
    std_err = sem(data)
    h = std_err * t.ppf((1 + confidence) / 2., n-1)
    return mean - h, mean + h

def analyze_stimprob_by_responsiveness(obj, df_name, groupname, is_single_unit=None, cell_type=None):
    """
    Analyze and compare the StimProb values for responsive and non-responsive neurons in a specified group.
    Allows filtering by IsSingleUnit and Cell_Type.
    
    Parameters:
    obj (object): The object containing the data and methods.
    df_name (str): The name of the DataFrame to use.
    groupname (str): The treatment group to analyze (e.g., 'CTZ' or 'No_CTZ').
    is_single_unit (float or None): Filter by single-unit activity (1.0 for SUA, 0.0 for MUA, None for both).
    cell_type (str or None): Filter by cell type ('FS' for Fast-Spiking, 'RS' for Regular-Spiking, None for both).
    
    Returns:
    dict: A dictionary containing the statistics for responsive and non-responsive groups.
    """
    # Retrieve data for the specified group with optional filters
    df = obj.get_filtered_data(df_name, groupname=groupname, is_single_unit=is_single_unit, cell_type=cell_type, stim_responsivity=None)
    
    # Pool StimResponsivity = -1.0 and 1.0 as responsive
    responsive_df = df[(df['StimResponsivity'] == -1.0) | (df['StimResponsivity'] == 1.0)]
    non_responsive_df = df[df['StimResponsivity'] == 0.0]
    
    # Extract StimProb values
    responsive_stimprob = responsive_df['StimProb'].apply(lambda x: x[0]).dropna().values
    non_responsive_stimprob = non_responsive_df['StimProb'].apply(lambda x: x[0]).dropna().values
    
    # Calculate statistics for each group
    responsive_stats = calculate_statistics(responsive_stimprob)
    non_responsive_stats = calculate_statistics(non_responsive_stimprob)
    
    # Return the results
    return {
        'Responsive': responsive_stats,
        'Non_Responsive': non_responsive_stats
    }

# Example usage: Analyze for CTZ group, filtering for SUA (IsSingleUnit=1.0) and FS cells
ctz_stats_sua_fs = analyze_stimprob_by_responsiveness(
    whisker_df_manager, 
    df_name='basic_metrics', 
    groupname='CTZ', 
    is_single_unit=1.0,  # Filter for single-unit activity (SUA)
    cell_type='FS'  # Filter for Fast-Spiking cells (FS)
)

# Example usage: Analyze for No_CTZ group, without filtering by SUA or cell type
no_ctz_stats = analyze_stimprob_by_responsiveness(
    whisker_df_manager, 
    df_name='basic_metrics', 
    groupname='No_CTZ'
)

# Display the results
print("CTZ Group Analysis (SUA, FS):")
print(pd.DataFrame(ctz_stats_sua_fs))

print("\nNo_CTZ Group Analysis (No Filters):")
print(pd.DataFrame(no_ctz_stats))


In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import sem, t, ranksums

def calculate_statistics(data):
    """Calculate statistics for the given data."""
    N = len(data)
    mean = np.mean(data)
    sd = np.std(data)
    median = np.median(data)
    min_val = np.min(data)
    max_val = np.max(data)
    
    # Calculate the 95% confidence interval
    ci_low, ci_high = calculate_confidence_interval(data)
    
    return {
        'N': N,
        'Mean': mean,
        'SD': sd,
        'Median': median,
        'CI_Low': ci_low,
        'CI_High': ci_high,
        'Min': min_val,
        'Max': max_val
    }

def calculate_confidence_interval(data, confidence=0.95):
    """Calculate the confidence interval for a data series."""
    n = len(data)
    mean = np.mean(data)
    std_err = sem(data)
    h = std_err * t.ppf((1 + confidence) / 2., n-1)
    return mean - h, mean + h

def compare_auroc_stats(obj, df_name, groupnames, is_single_unit=None, cell_type=None):
    """
    Compare the AUROC values between responsive and non-responsive neurons within and between groups.
    
    Parameters:
    obj (object): The object containing the data and methods.
    df_name (str): The name of the DataFrame to use.
    groupnames (list of str): List of groups to analyze (e.g., ['CTZ', 'No_CTZ']).
    is_single_unit (float or None): Filter by single-unit activity (1.0 for SUA, 0.0 for MUA, None for both).
    cell_type (str or None): Filter by cell type ('FS' for Fast-Spiking, 'RS' for Regular-Spiking, None for both).
    
    Returns:
    pd.DataFrame: DataFrame containing the calculated statistics and comparison results.
    """
    results = []

    for groupname in groupnames:
        # Retrieve data for the specified group with optional filters
        df = obj.get_filtered_data(df_name, groupname=groupname, is_single_unit=is_single_unit, cell_type=cell_type, stim_responsivity=None)
        
        # Pool StimResponsivity = -1.0 and 1.0 as responsive
        responsive_df = df[(df['StimResponsivity'] == -1.0) | (df['StimResponsivity'] == 1.0)]
        non_responsive_df = df[df['StimResponsivity'] == 0.0]
        
        # Extract StimProb values
        responsive_stimprob = responsive_df['StimProb'].apply(lambda x: x[0]).dropna().values
        non_responsive_stimprob = non_responsive_df['StimProb'].apply(lambda x: x[0]).dropna().values
        
        # Calculate statistics for each group
        responsive_stats = calculate_statistics(responsive_stimprob)
        non_responsive_stats = calculate_statistics(non_responsive_stimprob)
        
        # Perform statistical comparison within the group
        if len(responsive_stimprob) > 0 and len(non_responsive_stimprob) > 0:
            test_stat, p_value = ranksums(responsive_stimprob, non_responsive_stimprob)
        else:
            test_stat, p_value = np.nan, np.nan  # Handle cases where one or both groups have no data
        
        # Store the results in the list
        results.append({
            'Group': groupname,
            'Comparison': 'Responsive vs Non-Responsive',
            'N_Responsive': responsive_stats['N'],
            'N_Non_Responsive': non_responsive_stats['N'],
            'Mean_Responsive': responsive_stats['Mean'],
            'Mean_Non_Responsive': non_responsive_stats['Mean'],
            'Median_Responsive': responsive_stats['Median'],
            'Median_Non_Responsive': non_responsive_stats['Median'],
            'SD_Responsive': responsive_stats['SD'],
            'SD_Non_Responsive': non_responsive_stats['SD'],
            'Min_Responsive': responsive_stats['Min'],
            'Max_Responsive': responsive_stats['Max'],
            'Min_Non_Responsive': non_responsive_stats['Min'],
            'Max_Non_Responsive': non_responsive_stats['Max'],
            'CI_Low_Responsive': responsive_stats['CI_Low'],
            'CI_High_Responsive': responsive_stats['CI_High'],
            'CI_Low_Non_Responsive': non_responsive_stats['CI_Low'],
            'CI_High_Non_Responsive': non_responsive_stats['CI_High'],
            'Test_Statistic': test_stat,
            'p_value': p_value
        })
    
    # Between-group comparison for responsive neurons only
    if len(groupnames) == 2:
        group1_df = obj.get_filtered_data(df_name, groupname=groupnames[0], is_single_unit=is_single_unit, cell_type=cell_type, stim_responsivity=None)
        group2_df = obj.get_filtered_data(df_name, groupname=groupnames[1], is_single_unit=is_single_unit, cell_type=cell_type, stim_responsivity=None)
        
        group1_responsive = group1_df[(group1_df['StimResponsivity'] == -1.0) | (group1_df['StimResponsivity'] == 1.0)]['StimProb'].apply(lambda x: x[0]).dropna().values
        group2_responsive = group2_df[(group2_df['StimResponsivity'] == -1.0) | (group2_df['StimResponsivity'] == 1.0)]['StimProb'].apply(lambda x: x[0]).dropna().values
        
        if len(group1_responsive) > 0 and len(group2_responsive) > 0:
            test_stat, p_value = ranksums(group1_responsive, group2_responsive)
        else:
            test_stat, p_value = np.nan, np.nan  # Handle cases where one or both groups have no data
        
        # Calculate statistics for each responsive group
        group1_stats = calculate_statistics(group1_responsive)
        group2_stats = calculate_statistics(group2_responsive)
        
        results.append({
            'Group': f'{groupnames[0]} vs {groupnames[1]}',
            'Comparison': 'Responsive Neurons Between Groups',
            'N_Responsive_Group1': group1_stats['N'],
            'N_Responsive_Group2': group2_stats['N'],
            'Mean_Responsive_Group1': group1_stats['Mean'],
            'Mean_Responsive_Group2': group2_stats['Mean'],
            'Median_Responsive_Group1': group1_stats['Median'],
            'Median_Responsive_Group2': group2_stats['Median'],
            'SD_Responsive_Group1': group1_stats['SD'],
            'SD_Responsive_Group2': group2_stats['SD'],
            'Min_Responsive_Group1': group1_stats['Min'],
            'Max_Responsive_Group1': group1_stats['Max'],
            'Min_Responsive_Group2': group2_stats['Min'],
            'Max_Responsive_Group2': group2_stats['Max'],
            'CI_Low_Responsive_Group1': group1_stats['CI_Low'],
            'CI_High_Responsive_Group1': group1_stats['CI_High'],
            'CI_Low_Responsive_Group2': group2_stats['CI_Low'],
            'CI_High_Responsive_Group2': group2_stats['CI_High'],
            'Test_Statistic': test_stat,
            'p_value': p_value
        })
    
    # Convert the results list into a DataFrame
    results_df = pd.DataFrame(results)
    
    return results_df

# Example usage:
groupnames = ['CTZ', 'No_CTZ']
results_df = compare_auroc_stats(
    whisker_df_manager, 
    df_name='basic_metrics', 
    groupnames=groupnames, 
    is_single_unit=1.0,  # Example filter for single-unit activity (SUA)
    cell_type='FS'  # Example filter for Fast-Spiking cells (FS)
)

# Display the results
print(results_df)


In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import sem, t, ranksums

def calculate_statistics(data):
    """Calculate statistics for the given data."""
    N = len(data)
    mean = np.mean(data)
    sd = np.std(data)
    median = np.median(data)
    min_val = np.min(data)
    max_val = np.max(data)
    
    # Calculate the 95% confidence interval
    ci_low, ci_high = calculate_confidence_interval(data)
    
    return {
        'N': N,
        'Mean': mean,
        'SD': sd,
        'Median': median,
        'CI_Low': ci_low,
        'CI_High': ci_high,
        'Min': min_val,
        'Max': max_val
    }

def calculate_confidence_interval(data, confidence=0.95):
    """Calculate the confidence interval for a data series."""
    n = len(data)
    mean = np.mean(data)
    std_err = sem(data)
    h = std_err * t.ppf((1 + confidence) / 2., n-1)
    return mean - h, mean + h

def compare_auroc_stats(obj, df_name, groupnames, is_single_unit=None, cell_type=None):
    """
    Compare the AUROC values between responsive and non-responsive neurons within and between groups.
    
    Parameters:
    obj (object): The object containing the data and methods.
    df_name (str): The name of the DataFrame to use.
    groupnames (list of str): List of groups to analyze (e.g., ['CTZ', 'No_CTZ']).
    is_single_unit (float or None): Filter by single-unit activity (1.0 for SUA, 0.0 for MUA, None for both).
    cell_type (str or None): Filter by cell type ('FS' for Fast-Spiking, 'RS' for Regular-Spiking, None for both).
    
    Returns:
    pd.DataFrame: DataFrame containing the calculated statistics and comparison results.
    """
    results = []

    for groupname in groupnames:
        # Retrieve data for the specified group with optional filters
        df = obj.get_filtered_data(df_name, groupname=groupname, is_single_unit=is_single_unit, cell_type=cell_type, stim_responsivity=None)
        
        # Pool StimResponsivity = -1.0 and 1.0 as responsive
        responsive_df = df[(df['StimResponsivity'] == -1.0) | (df['StimResponsivity'] == 1.0)]
        non_responsive_df = df[df['StimResponsivity'] == 0.0]
        
        # Extract StimProb values
        responsive_stimprob = responsive_df['StimProb'].apply(lambda x: x[0]).dropna().values
        non_responsive_stimprob = non_responsive_df['StimProb'].apply(lambda x: x[0]).dropna().values
        
        # Calculate statistics for each group
        responsive_stats = calculate_statistics(responsive_stimprob)
        non_responsive_stats = calculate_statistics(non_responsive_stimprob)
        
        # Perform statistical comparison within the group
        if len(responsive_stimprob) > 0 and len(non_responsive_stimprob) > 0:
            test_stat, p_value = ranksums(responsive_stimprob, non_responsive_stimprob)
        else:
            test_stat, p_value = np.nan, np.nan  # Handle cases where one or both groups have no data
        
        # Store the results in the list
        results.append({
            'Group': groupname,
            'Comparison': 'Responsive vs Non-Responsive',
            'N_Responsive': responsive_stats['N'],
            'N_Non_Responsive': non_responsive_stats['N'],
            'Mean_Responsive': responsive_stats['Mean'],
            'Mean_Non_Responsive': non_responsive_stats['Mean'],
            'Median_Responsive': responsive_stats['Median'],
            'Median_Non_Responsive': non_responsive_stats['Median'],
            'SD_Responsive': responsive_stats['SD'],
            'SD_Non_Responsive': non_responsive_stats['SD'],
            'Min_Responsive': responsive_stats['Min'],
            'Max_Responsive': responsive_stats['Max'],
            'Min_Non_Responsive': non_responsive_stats['Min'],
            'Max_Non_Responsive': non_responsive_stats['Max'],
            'CI_Low_Responsive': responsive_stats['CI_Low'],
            'CI_High_Responsive': responsive_stats['CI_High'],
            'CI_Low_Non_Responsive': non_responsive_stats['CI_Low'],
            'CI_High_Non_Responsive': non_responsive_stats['CI_High'],
            'Test_Statistic': test_stat,
            'p_value': p_value
        })
    
    # Between-group comparison for responsive neurons only
    if len(groupnames) == 2:
        group1_df = obj.get_filtered_data(df_name, groupname=groupnames[0], is_single_unit=is_single_unit, cell_type=cell_type, stim_responsivity=None)
        group2_df = obj.get_filtered_data(df_name, groupname=groupnames[1], is_single_unit=is_single_unit, cell_type=cell_type, stim_responsivity=None)
        
        group1_responsive = group1_df[(group1_df['StimResponsivity'] == -1.0) | (group1_df['StimResponsivity'] == 1.0)]['StimProb'].apply(lambda x: x[0]).dropna().values
        group2_responsive = group2_df[(group2_df['StimResponsivity'] == -1.0) | (group2_df['StimResponsivity'] == 1.0)]['StimProb'].apply(lambda x: x[0]).dropna().values
        
        if len(group1_responsive) > 0 and len(group2_responsive) > 0:
            test_stat, p_value = ranksums(group1_responsive, group2_responsive)
        else:
            test_stat, p_value = np.nan, np.nan  # Handle cases where one or both groups have no data
        
        # Calculate statistics for each responsive group
        group1_stats = calculate_statistics(group1_responsive)
        group2_stats = calculate_statistics(group2_responsive)
        
        results.append({
            'Group': f'{groupnames[0]} vs {groupnames[1]}',
            'Comparison': 'Responsive Neurons Between Groups',
            'N_Responsive_Group1': group1_stats['N'],
            'N_Responsive_Group2': group2_stats['N'],
            'Mean_Responsive_Group1': group1_stats['Mean'],
            'Mean_Responsive_Group2': group2_stats['Mean'],
            'Median_Responsive_Group1': group1_stats['Median'],
            'Median_Responsive_Group2': group2_stats['Median'],
            'SD_Responsive_Group1': group1_stats['SD'],
            'SD_Responsive_Group2': group2_stats['SD'],
            'Min_Responsive_Group1': group1_stats['Min'],
            'Max_Responsive_Group1': group1_stats['Max'],
            'Min_Responsive_Group2': group2_stats['Min'],
            'Max_Responsive_Group2': group2_stats['Max'],
            'CI_Low_Responsive_Group1': group1_stats['CI_Low'],
            'CI_High_Responsive_Group1': group1_stats['CI_High'],
            'CI_Low_Responsive_Group2': group2_stats['CI_Low'],
            'CI_High_Responsive_Group2': group2_stats['CI_High'],
            'Test_Statistic': test_stat,
            'p_value': p_value
        })
    
    # Convert the results list into a DataFrame
    results_df = pd.DataFrame(results)
    
    return results_df

# Perform the analysis for FS and RS neurons separately
cell_types = ['FS', 'RS']
results_dfs = []

for cell_type in cell_types:
    # Run the comparison analysis for each cell type
    results_df = compare_auroc_stats(
        whisker_df_manager,
        df_name='basic_metrics',
        groupnames=['CTZ', 'No_CTZ'],
        is_single_unit=1.0,  # No filter on single units for this analysis
        cell_type=cell_type  # Filter by cell type (FS or RS)
    )
    # Add a column for cell type in the results
    results_df['Cell_Type'] = cell_type
    results_dfs.append(results_df)

# Combine the results into a single DataFrame
combined_results_df = pd.concat(results_dfs, ignore_index=True)

# Display the combined results
combined_results_df.head(), combined_results_df.describe()
combined_results_df

In [ ]:
whisker_df_manager.plot_raw_auroc_distribution('basic_metrics', is_single_unit=None, cell_type='RS', groupname='CTZ')

In [ ]:
whisker_df_manager.check_stim_responsivity_distribution_before_after('basic_metrics', is_single_unit=1.0, cell_type='RS', groupname='CTZ')


In [ ]:
whisker_df_manager.inspect_categorized_data('basic_metrics', is_single_unit=1.0, cell_type='RS', groupname='CTZ')


In [ ]:
whisker_df_manager.plot_stim_prob_distribution('basic_metrics', is_single_unit=1.0, cell_type='RS', groupname=None, include_no_change=True, xlim_range=(0.48, 0.57), plot_stacked_bar=True)


plot the distributio for RS or FS cells per group

In [ ]:
whisker_df_manager.plot_stim_prob_distribution('basic_metrics', is_single_unit=1.0, cell_type='RS', groupname='CTZ', include_no_change=False, xlim_range=(0.45, 0.7), plot_stacked_bar=True)
whisker_df_manager.plot_stim_prob_distribution('basic_metrics', is_single_unit=1.0, cell_type='RS', groupname='No_CTZ', include_no_change=False, xlim_range=(0.45, 0.7), plot_stacked_bar=True)

whisker_df_manager.plot_stim_prob_distribution('basic_metrics', is_single_unit=1.0, cell_type='FS', groupname='CTZ', include_no_change=True, xlim_range=(0.45, 0.7), plot_stacked_bar=True)
whisker_df_manager.plot_stim_prob_distribution('basic_metrics', is_single_unit=1.0, cell_type='FS', groupname='No_CTZ', include_no_change=True, xlim_range=(0.45, 0.7), plot_stacked_bar=True)


In [ ]:
whisker_df_manager.plot_stim_prob_distribution('basic_metrics', is_single_unit=1.0, cell_type='FS', groupname='CTZ')
whisker_df_manager.plot_stim_prob_distribution('basic_metrics', is_single_unit=1.0, cell_type='FS', groupname='No_CTZ')
whisker_df_manager.plot_stim_prob_distribution('basic_metrics', is_single_unit=1.0, cell_type='RS', groupname='CTZ')
whisker_df_manager.plot_stim_prob_distribution('basic_metrics', is_single_unit=1.0, cell_type='RS', groupname='No_CTZ')


### Plot PSTHs based on modulation index, per group as mean, traces, or per recordings, using the older PSTHs

In [ ]:
## add the modulation index to the dataframe psth_dataframe
whisker_df_manager.add_modulation_label_column('psth_dataframe')
whisker_df_manager.add_modulation_label_column('psth_dataframe_Zero')
whisker_df_manager.add_modulation_label_column('psth_dataframe_Low')
whisker_df_manager.add_modulation_label_column('psth_dataframe_Mid')
whisker_df_manager.add_modulation_label_column('psth_dataframe_Max')

### FINAL FIGURES Below is all possible combinations of FS and RS MUA PSTHs -- Final 

### try to scale up for easier plotting

In [ ]:
def dynamic_plot_generator(
    stimulations, 
    cell_types=['FS', 'RS'], 
    is_single_unit_options=[1.0, 0.0, None], 
    stim_responsivity_options=[1.0, 0.0, None], 
    groupnames=['CTZ', 'No_CTZ'], 
    modulation_label='positive', 
    time_ranges={
        'plot_group_comparison_meansem': [-500, 1000], 
        'plot_all_stimulations_with_stim': [(-15, 50), (-500, 1000)]
    }, 
    smoothing_window=1, 
    base_directory='/Volumes/MannySSD/figures/thesis_final_figs/PSTHs/'
):
    # Helper function to translate the options to meaningful folder names
    def get_folder_name(option, option_type):
        if option_type == 'is_single_unit':
            return 'SUA' if option == 1.0 else 'MUA' if option == 0.0 else 'ALL'
        elif option_type == 'stim_responsivity':
            return 'sensory_responsive' if option == 1.0 else 'non_responsive' if option == 0.0 else 'ALL'
    
    # Loop through each combination of cell_type, is_single_unit, and stim_responsivity
    for cell_type in cell_types:
        for is_single_unit in is_single_unit_options:
            for stim_responsivity in stim_responsivity_options:
                
                # Create the folder structure with meaningful names
                su_folder = get_folder_name(is_single_unit, 'is_single_unit')
                sr_folder = get_folder_name(stim_responsivity, 'stim_responsivity')
                
                cell_type_folder = os.path.join(base_directory, cell_type)
                is_single_unit_folder = os.path.join(cell_type_folder, su_folder)
                stim_responsivity_folder = os.path.join(is_single_unit_folder, sr_folder)
                os.makedirs(stim_responsivity_folder, exist_ok=True)

                # File name base with meaningful names
                base_file_name = f"{cell_type}_{su_folder}_{sr_folder}"
                
                # Debug: Display the current filtering parameters
                print(f"Attempting plot with {cell_type}, SU: {su_folder}, SR: {sr_folder}")
                
                # Plot group comparison meansem
                file_name_meansem = f"{base_file_name}_meansem"

                try:
                    whisker_df_manager.plot_group_comparison_meansem(
                        stimulations,  # Passed as a positional argument
                        cell_type=cell_type,
                        is_single_unit=is_single_unit,
                        stim_responsivity=stim_responsivity,
                        groupnames=groupnames,
                        modulation_label=modulation_label,
                        time_range=time_ranges['plot_group_comparison_meansem'],
                        smoothing_window=smoothing_window,
                        directory=stim_responsivity_folder,
                        file_name=file_name_meansem
                    )

                except Exception as e:
                    print(f"Error plotting meansem for {cell_type}, SU: {su_folder}, SR: {sr_folder}: {str(e)}")
                    continue

                # Plot all stimulations with stim (traces and sem) for both time ranges
                for time_range in time_ranges['plot_all_stimulations_with_stim']:
                    time_label = f"{time_range[0]}to{time_range[1]}"

                    file_name_traces = f"{base_file_name}_traces_{time_label}"
                    try:
                        whisker_df_manager.plot_all_stimulations_with_stim(
                            groupnames[0],
                            groupnames[1],
                            cell_type=cell_type,
                            is_single_unit=is_single_unit,
                            stim_responsivity=stim_responsivity,
                            time_range=time_range,
                            plot_mode='traces',
                            smoothing_window=smoothing_window,
                            directory=stim_responsivity_folder,
                            file_name=file_name_traces
                        )
                    except Exception as e:
                        print(f"Error plotting traces for {cell_type}, SU: {su_folder}, SR: {sr_folder}, Time: {time_label}: {str(e)}")
                        continue

                    file_name_sem = f"{base_file_name}_sem_{time_label}"
                    try:
                        whisker_df_manager.plot_all_stimulations_with_stim(
                            groupnames[0],
                            groupnames[1],
                            cell_type=cell_type,
                            is_single_unit=is_single_unit,
                            stim_responsivity=stim_responsivity,
                            time_range=time_range,
                            plot_mode='sem',
                            smoothing_window=smoothing_window,
                            directory=stim_responsivity_folder,
                            file_name=file_name_sem
                        )
                    except Exception as e:
                        print(f"Error plotting SEM for {cell_type}, SU: {su_folder}, SR: {sr_folder}, Time: {time_label}: {str(e)}")
                        continue

# Example call
dynamic_plot_generator(
    stimulations=['Zero', 'Low', 'Mid', 'Max'],
    time_ranges={
        'plot_group_comparison_meansem': [-500, 1000], 
        'plot_all_stimulations_with_stim': [(-15, 50), (-500, 1000)]
    },
    base_directory='/Volumes/MannySSD/figures/thesis_final_figs/PSTHs/'
)

### IMPORT TEMPORAL SPIKING METRICS

In [ ]:
whisker_df_manager.dataframes['basic_metrics'].columns

In [ ]:
    # Filter the DataFrame based on user inputs
filtered_df = whisker_df_manager.dataframes['basic_metrics'][
        (whisker_df_manager.dataframes['basic_metrics']['IsSingleUnit'] == 1.0) & 
        (whisker_df_manager.dataframes['basic_metrics']['Cell_Type'] == 'FS')
    ]

filtered_df

In [ ]:
def plot_filtered_metrics(dataframes_obj, column_to_plot, stimulations=None, is_single_unit=1.0, cell_type='RS', stim_responsivity=1.0, directory=None, file_name=None, ylim=None):
    """
    Plots the specified metric with detailed filtering based on user inputs.

    Args:
        dataframes_obj (object): The object containing the dataframes, accessed as dataframes_obj.dataframes['basic_metrics'].
        column_to_plot (str): The column name to plot (like 'FirstSpikeLatency' or 'FirstSpikeLatency_Reliability').
        stimulations (list of str, optional): List of stimulations to include in the plot.
        is_single_unit (float, optional): Filter for single units (1.0) or multi-units (0.0).
        cell_type (str, optional): Filter for specific cell types ('FS' or 'RS').
        stim_responsivity (float, optional): Filter by stimulus responsivity (1.0, 0.0, or -1.0).
        modulation_label (str, optional): Filter by modulation label ('positive', 'negative', 'none').
        directory (str, optional): Directory to save the plot.
        file_name (str, optional): File name to save the plot.
        ylim (tuple, optional): Limits for the y-axis.
    """
    # Extract the relevant DataFrame
    df = dataframes_obj.dataframes['basic_metrics']

    # Filter the DataFrame based on user inputs
    filtered_df = df[
        (df['IsSingleUnit'] == is_single_unit) & 
        (df['Cell_Type'] == cell_type) & 
        (df['StimResponsivity'] == stim_responsivity)
    ]
    
    # If no data is found, print a message and return
    if filtered_df.empty:
        print("No data found for the specified filtering criteria.")
        return
    
    # Initialize a list to hold data for plotting
    plot_data = []

    # Loop through each row in the filtered DataFrame
    for index, row in filtered_df.iterrows():
        for i, stim in enumerate(stimulations):
            plot_data.append({
                'Group': row['groupname'],
                'Stimulation': stim,
                'Value': row[column_to_plot][i]  # Extract the value for each stimulation
            })

    # Convert the list to a DataFrame for easy plotting
    plot_df = pd.DataFrame(plot_data)

    # Plotting
    plt.figure(figsize=(8, 6))
    sns.boxplot(data=plot_df, x='Stimulation', y='Value', hue='Group')
    sns.stripplot(data=plot_df, x='Stimulation', y='Value', hue='Group', dodge=True, jitter=True, color='black')

    # Control the upper and lower limits of the y-axis
    if ylim is not None:
        plt.ylim(ylim)

    # Enhance the plot
    plt.title(f'Comparison of {column_to_plot} Across Stimulations for {cell_type} Units')
    plt.ylabel(column_to_plot)
    plt.xlabel('Stimulation Type')
    plt.legend(title='Group')

    # Save the figure as an SVG file in the specified directory
    if directory and file_name:
        file_path = os.path.join(directory, f'{file_name}.svg')
        plt.savefig(file_path, format='svg', transparent=True)
    else:
        plt.show()


plot_filtered_metrics(
    dataframes_obj=whisker_df_manager,  # Your data object
    column_to_plot='FirstSpikeLatency',  # The column you want to plot
    stimulations=['Zero', 'Low', 'Mid', 'Max'],  # List of stimulations
    is_single_unit=1.0,  # Filter for single units (1.0) or multi-units (0.0)
    cell_type='FS',  # Filter for specific cell types ('FS' or 'RS')
    stim_responsivity=1.0,  # Filter by stimulus responsivity
    directory='/Volumes/MannySSD/figures/temporal_spiking_metrics/FirstSpikeLatency', 
    file_name='boxplot_RS_FirstSpikeLatency', 
    ylim=None # Adjust as needed
)

### THIS WILL BE CONSIDERED THE FINAL FOR FirstSpikeLatency AND  FOR RS SUA

In [ ]:
whisker_df_manager.dataframes['basic_metrics'].columns

In [ ]:
whisker_df_manager.plot_box_and_strip_default(groups=['CTZ', 'No_CTZ'], stimulations=['Low', 'Max'], show_outliers=True, dataframe_key='temporal_spiking_metrics', 
                                              list_column='FirstSpikeLatency', cell_type_filter='RS', is_single_unit=1.0, stim_responsivity_filter=1.0, 
                                              modulation_label='positive', remove_outliers_option=False, 
                                              directory='/Volumes/MannySSD/figures/temporal_spiking_metrics/FirstSpikeLatency', 
                                              file_name='boxplot_RS_FirstSpikeLatency')
#now do FS cells
whisker_df_manager.plot_box_and_strip_default(groups=['CTZ', 'No_CTZ'], stimulations=['Low', 'Max'], show_outliers=True, dataframe_key='temporal_spiking_metrics',
                                              list_column='FirstSpikeLatency', cell_type_filter='FS', is_single_unit=1.0, stim_responsivity_filter=1.0,
                                                modulation_label='positive', remove_outliers_option=False, 
                                                directory='/Volumes/MannySSD/figures/temporal_spiking_metrics/FirstSpikeLatency',
                                                file_name='boxplot_FS_FirstSpikeLatency')


whisker_df_manager.plot_box_and_strip_default(groups=['CTZ', 'No_CTZ'], stimulations=['Low', 'Max'], show_outliers=True, dataframe_key='temporal_spiking_metrics', 
                                              list_column='FirstSpikeLatency_Reliability', cell_type_filter='RS', is_single_unit=1.0, stim_responsivity_filter=1.0, 
                                              modulation_label='positive', remove_outliers_option=False, 
                                              directory='/Volumes/MannySSD/figures/temporal_spiking_metrics/FirstSpikeLatency_Reliability', 
                                              file_name='boxplot_RS_FirstSpikeLatency_Reliability')

### now do FS cells
whisker_df_manager.plot_box_and_strip_default(groups=['CTZ', 'No_CTZ'], stimulations=['Low','Mid', 'Max'], show_outliers=True, dataframe_key='temporal_spiking_metrics', 
                                              list_column='FirstSpikeLatency_Reliability', cell_type_filter='FS', is_single_unit=1.0, stim_responsivity_filter=1.0, 
                                              modulation_label='positive', remove_outliers_option=False, 
                                              directory='/Volumes/MannySSD/figures/temporal_spiking_metrics/FirstSpikeLatency_Reliability', 
                                              file_name='boxplot_FS_FirstSpikeLatency_Reliability')

## plot the raster only or the raster with the PSTH for a specific cell id 

In [ ]:
#whisker_df_manager.plot_rasters_for_cid('CTZ', 'ctz_2303_1', 'cid530', time_window=(-50,50))
#whisker_df_manager.plot_combined_psth_and_raster('CTZ', 'ctz_2303_1', 'cid530', time_window=(-50,50))
whisker_df_manager.plot_combined_psth_and_raster_normalized('CTZ', 'ctz_2303_1', 'cid530', time_window=(-50,50), smoothing_window=3, normalize=False, show=False)



In [ ]:
whisker_df_manager.plot_combined_psth_and_raster_normalized('No_CTZ', 'veh_3062_2', 'cid109', time_window=(-50,50), smoothing_window=3, normalize=False, show=False)


In [ ]:
#whisker_df_manager.plot_rasters_for_cid_changebinsize('CTZ', 'ctz_2303_1', 'cid365', time_window=(-50,50), bin_size=1, filter_empty_trials=True)
#whisker_df_manager.plot_rasters_for_cid_changebinsize('CTZ', 'ctz_2303_1', 'cid530', time_window=(-50,50), bin_size=1, filter_empty_trials=True)
whisker_df_manager.plot_rasters_for_cid_changebinsize('CTZ', 'ctz_2303_2', 'cid208', time_window=(-50,50), bin_size=1, filter_empty_trials=True, recording_dir='/Volumes/MannySSD/figures' )
#whisker_df_manager.plot_rasters_for_cid_changebinsize('CTZ', 'ctz_3689_2', 'cid108', time_window=(-50,50), bin_size=1, filter_empty_trials=True)


In [ ]:
whisker_df_manager.plot_rasters_for_cid_changebinsize('No_CTZ', 'veh_3062_2', 'cid109', time_window=(-50,50), bin_size=1, filter_empty_trials=False, recording_dir='/Volumes/MannySSD/figures')
whisker_df_manager.plot_rasters_for_cid_changebinsize('No_CTZ', 'veh_3142_2', 'cid456', time_window=(-50,50), bin_size=1, filter_empty_trials=False, recording_dir='/Volumes/MannySSD/figures')
whisker_df_manager.plot_rasters_for_cid_changebinsize('No_CTZ', 'veh_2836_2', 'cid72', time_window=(-50,50), bin_size=1, filter_empty_trials=False, recording_dir='/Volumes/MannySSD/figures')

## now do rbp4 

### more whisker rbp4 plotting and data manager creatr

In [ ]:
rbp4_whisker_df_manager = DataFrameManager(rbp4_whisker)
rbp4_whisker_df_manager.eed
rbp4_whisker_df_manager.create_dataframe(['Cell_Type', 'IsSingleUnit', 'StimResponsivity', 'MeanFR_baseline', 'MeanFR_stim','LaminarLabel', 'PeakEvokedFR', 'PeakEvokedFR_Latency', 'FanoFactor_baseline', 'FanoFactor_stim', 'SpikeTimes_all', 'MeanFR_inst_stim' ], 'basic_metrics')


In [ ]:
rbp4_whisker_df_manager.create_psth_dataframe()
rbp4_whisker_df_manager.calculate_basic_stats('CTZ', 'No_CTZ', stim_label=None, baseline_range=(-100, -1), stim_range=(10,50), cell_type='RS', is_single_unit=1.0, stim_responsivity=1.0, smoothing_window=5)
rbp4_whisker_df_manager.prepare_for_boxplot()
rbp4_whisker_df_manager.plot_box_and_strip(groups=['CTZ', 'No_CTZ'], stimulations=['Low'], show_outliers=True, directory='/Volumes/MannySSD/figures', file_name='boxplot_RS_baseline_stim_rbp4')

In [ ]:
rbp4_whisker_df_manager.create_psth_dataframe()
rbp4_whisker_df_manager.calculate_basic_stats('CTZ', 'No_CTZ', stim_label=None, baseline_range=(-100, -1), stim_range=(10,50), cell_type='FS', is_single_unit=1.0, stim_responsivity=1.0, smoothing_window=5)
rbp4_whisker_df_manager.prepare_for_boxplot()
rbp4_whisker_df_manager.plot_box_and_strip(groups=['CTZ', 'No_CTZ'], stimulations=None, show_outliers=True, directory='/Volumes/MannySSD/figures', file_name='boxplot_FS_baseline_stim_rbp4')

rbp4_whisker_df_manager.create_psth_dataframe()
rbp4_whisker_df_manager.calculate_basic_stats('CTZ', 'No_CTZ', stim_label=None, baseline_range=(-100, -1), stim_range=(10,50), cell_type='RS', is_single_unit=1.0, stim_responsivity=1.0, smoothing_window=5)
rbp4_whisker_df_manager.prepare_for_boxplot()
rbp4_whisker_df_manager.plot_box_and_strip(groups=['CTZ', 'No_CTZ'], stimulations=None, show_outliers=True, directory='/Volumes/MannySSD/figures', file_name='boxplot_RS_baseline_stim_rbp4')

rbp4_whisker_df_manager.create_psth_dataframe()
rbp4_whisker_df_manager.calculate_basic_stats('CTZ', 'No_CTZ', stim_label=None, baseline_range=(-100, -1), stim_range=(10,50), cell_type=None, is_single_unit=None, stim_responsivity=0.0, smoothing_window=5)
rbp4_whisker_df_manager.prepare_for_boxplot()
rbp4_whisker_df_manager.plot_box_and_strip(groups=['CTZ', 'No_CTZ'], stimulations=None, show_outliers=True, directory='/Volumes/MannySSD/figures', file_name='boxplot_MUA_baseline_stim_rbp4')

In [ ]:
rbp4_whisker_df_manager.plot_all_stimulations_with_stim('CTZ', 'No_CTZ', cell_type='RS', is_single_unit=None, stim_responsivity=None, time_range=(-10,200), plot_mode='sem', smoothing_window=5)
rbp4_whisker_df_manager.plot_all_stimulations_with_stim('CTZ', 'No_CTZ', cell_type='FS', is_single_unit=None, stim_responsivity=None, time_range=(-10,200), plot_mode='sem', smoothing_window=5)

rbp4_whisker_df_manager.plot_all_stimulations_with_stim('CTZ', 'No_CTZ', cell_type='RS', is_single_unit=None, stim_responsivity=None, time_range=(-10,50), plot_mode='sem', smoothing_window=5)
rbp4_whisker_df_manager.plot_all_stimulations_with_stim('CTZ', 'No_CTZ', cell_type='FS', is_single_unit=None, stim_responsivity=None, time_range=(-10,50), plot_mode='sem', smoothing_window=5)

rbp4_whisker_df_manager.plot_all_stimulations('CTZ', 'No_CTZ', cell_type='RS', is_single_unit=1.0, stim_responsivity=1.0, time_range=(-10,600), plot_mode='sem', smoothing_window=8)
rbp4_whisker_df_manager.plot_all_stimulations('CTZ', 'No_CTZ', cell_type='FS', is_single_unit=1.0, stim_responsivity=1.0, time_range=(-10,600), plot_mode='sem', smoothing_window=8)
rbp4_whisker_df_manager.plot_all_stimulations('CTZ', 'No_CTZ', cell_type='RS', is_single_unit=0.0, stim_responsivity=1.0, time_range=(-10,600), plot_mode='sem', smoothing_window=8)
rbp4_whisker_df_manager.plot_all_stimulations('CTZ', 'No_CTZ', cell_type='FS', is_single_unit=0.0, stim_responsivity=1.0, time_range=(-10,600), plot_mode='sem', smoothing_window=8)

In [ ]:
rbp4_whisker_df_manager.plot_all_stimulations('CTZ', 'No_CTZ', cell_type='RS', is_single_unit=1.0, stim_responsivity=1.0, time_range=(-10,100), plot_mode='traces', smoothing_window=3)
rbp4_whisker_df_manager.plot_all_stimulations('CTZ', 'No_CTZ', cell_type='FS', is_single_unit=1.0, stim_responsivity=1.0, time_range=(-10,100), plot_mode='traces', smoothing_window=3)

In [ ]:
rbp4_whisker_df_manager.create_dataframe(['Cell_Type', 'IsSingleUnit', 'StimResponsivity', 'SpikeTimes_all', 'FirstSpikeLatency', 'FirstSpikeLatency_Reliability', 'ISI_baseline_CV', 'ISI_baseline_vec', 'ISI_pdf_peak_xy', 'ISI_pdf_x', 'ISI_pdf_y'  ], 'temporal_spiking_metrics')

rbp4_whisker_df_manager.plot_box_and_strip_default(groups=['CTZ', 'No_CTZ'], stimulations=['Low', 'Mid', 'Max'], show_outliers=False, dataframe_key='temporal_spiking_metrics', list_column='FirstSpikeLatency', cell_type_filter='RS', is_single_unit=True, stim_responsivity_filter=1, remove_outliers_option=False)
rbp4_whisker_df_manager.plot_box_and_strip_default(groups=['CTZ', 'No_CTZ'], stimulations=['Low', 'Mid', 'Max'], show_outliers=False, dataframe_key='temporal_spiking_metrics', list_column='FirstSpikeLatency_Reliability', cell_type_filter='RS', is_single_unit=True, stim_responsivity_filter=1, remove_outliers_option=False)
rbp4_whisker_df_manager.plot_box_and_strip_default(groups=['CTZ', 'No_CTZ'], stimulations=['Low', 'Mid', 'Max'], show_outliers=False, dataframe_key='temporal_spiking_metrics', list_column='FirstSpikeLatency', cell_type_filter='FS', is_single_unit=True, stim_responsivity_filter=1, remove_outliers_option=False)
rbp4_whisker_df_manager.plot_box_and_strip_default(groups=['CTZ', 'No_CTZ'], stimulations=['Low', 'Mid', 'Max'], show_outliers=False, dataframe_key='temporal_spiking_metrics', list_column='FirstSpikeLatency_Reliability', cell_type_filter='FS', is_single_unit=True, stim_responsivity_filter=1, remove_outliers_option=False)

## begin stats analysis (bonus laminar analysis)

In [ ]:
#whisker_df_manager.add_modulation_label_column('basic_metrics') 
#whisker_df_manager.dataframes['basic_metrics'].head()
#print(whisker_df_manager.dataframes['basic_metrics'].shape)

#mean_df, detailed_df, _ = whisker_df_manager.calculate_all_psths(is_single_unit=True, cell_type='RS', stim_responsivity=0.0, time_window=(25,50), normalize=True, filter_empty_trials=False)


### process and run the stats 

In [ ]:
def filter_mean_df_by_modulation_label(mean_df, basic_metrics_df):
    """
    Filters mean_df to retain only entries with 'positive' ModulationLabel in basic_metrics_df.
    
    Args:
    mean_df (pd.DataFrame): DataFrame containing mean stimulation data.
    basic_metrics_df (pd.DataFrame): DataFrame containing modulation label and other metrics.
    
    Returns:
    pd.DataFrame: Filtered mean_df with only 'positive' ModulationLabel entries.
    """
    # Merge mean_df with basic_metrics_df on the shared columns
    merged_df = mean_df.merge(basic_metrics_df, how='inner', 
                              left_on=['cid', 'LaminarLabel', 'recordingname', 'Group'], 
                              right_on=['cid', 'LaminarLabel', 'recordingname', 'groupname'])
    
    # Filter to retain only entries where ModulationLabel is 'positive'
    filtered_df = merged_df[merged_df['ModulationLabel'] == 'positive']
    
    # Drop columns from basic_metrics_df that were added during the merge
    filtered_df = filtered_df[mean_df.columns]
    
    return filtered_df

# Example usage
mean_df_filtered = filter_mean_df_by_modulation_label(mean_df, whisker_df_manager.dataframes['basic_metrics'])
mean_df_filtered

### maake plots 

In [ ]:
SG_stats_df, _= whisker_df_manager.plot_mean_stimulation_box_and_strip2(mean_df_filtered, groups=['CTZ', 'No_CTZ'], stimulations=['Zero','Max'], remove_outliers_option=False, laminar_labels=['SG'], ylim=None, directory='/Volumes/MannySSD/figures', file_name='mean_stimulations_boxplot_SG_MUA')
L4_stats_df, _= whisker_df_manager.plot_mean_stimulation_box_and_strip2(mean_df_filtered, groups=['CTZ', 'No_CTZ'], stimulations=['Zero','Max'],remove_outliers_option=False, laminar_labels=['L4'], ylim=None, directory='/Volumes/MannySSD/figures', file_name='mean_stimulations_boxplot_L4_MUA')
IG_stats_df, _= whisker_df_manager.plot_mean_stimulation_box_and_strip2(mean_df_filtered, groups=['CTZ', 'No_CTZ'], stimulations=['Zero','Max'],remove_outliers_option=False, laminar_labels=['IG'], ylim=None, directory='/Volumes/MannySSD/figures', file_name='mean_stimulations_boxplot_IG_MUA')
all_layers_df, _ = whisker_df_manager.plot_mean_stimulation_box_and_strip2(mean_df_filtered, groups=['CTZ', 'No_CTZ'], stimulations=['Zero','Low', 'Mid', 'Max'],remove_outliers_option=False, laminar_labels=None, ylim=None, directory='/Volumes/MannySSD/figures', file_name='mean_stimulations_boxplot_all_layers_MUA')

In [ ]:
SG_stats_df

In [ ]:
L4_stats_df

In [ ]:
IG_stats_df

### random stuff to be sorted 

In [ ]:

#### do not delete this cell neeed to work on the code below fopr custom box plot
def run_group_comparisons(df, group_column='Group', value_column='mean_stimulation', stim_column='Stimulation'):
    """
    Performs statistical comparisons between groups for each type of stimulation, checks for normality,
    uses the appropriate non-parametric tests, and includes detailed descriptive statistics.
    """
    results = []

    for stim in df[stim_column].unique():
        sub_df = df[df[stim_column] == stim]
        groups = sub_df[group_column].unique()

        if len(groups) < 2:
            continue  # Skip if not enough groups for comparison

        group_data = [sub_df[sub_df[group_column] == g][value_column].dropna() for g in groups]
        normality_results = [shapiro(data) for data in group_data]
        normality_p_values = [result.pvalue for result in normality_results]

        # Choose the appropriate statistical test based on group count
        if len(groups) == 2:
            stat, p_value = mannwhitneyu(*group_data)
            test_used = 'Mann-Whitney U'
        elif len(groups) > 2:
            stat, p_value = kruskal(*group_data)
            test_used = 'Kruskal-Wallis'
            # Note: If using Kruskal-Wallis, consider post-hoc tests for detailed group comparisons
        else:
            continue

        descriptive_stats = [{
            'mean': data.mean(),
            'SD': data.std(),
            'median': data.median(),
            'range_min': data.min(),
            'range_max': data.max(),
        } for data in group_data]

        result_entry = {
            'Stimulation': stim,
            'Test Used': test_used,
            'Test Statistic': stat,
            'p-value': p_value,
            **{f'N Group{i+1}': len(data) for i, data in enumerate(group_data)},
            **{f'Normality p-value Group{i+1}': p for i, p in enumerate(normality_p_values)},
            **{f'{stat_key} Group{i+1}': stat_val for i, stats in enumerate(descriptive_stats) for stat_key, stat_val in stats.items()}
        }

        results.append(result_entry)

    return pd.DataFrame(results)



# Example usage in your data processing workflow
cleaned_df = whisker_df_manager.remove_outliers_by_stimulation(mean_df)  # Assuming outliers have been removed
stats_results = run_group_comparisons(cleaned_df)

stats_results

In [ ]:
whisker_df_manager.plot_group_cell_distribution()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Function to pad arrays to the maximum length found in any array
def pad_array_to_max_length(arr, max_length):
    padding = max_length - len(arr)
    if padding > 0:
        return np.pad(arr, (0, padding), 'constant', constant_values=(0,))
    return arr

# Function to plot the heatmaps with time bins on the x-axis and adjustable colormap range
def plot_group_stimulation_heatmaps(df, vmin=None, vmax=None):
    groupnames = df['groupname'].unique()
    stimulations = ['Zero', 'Low', 'Mid', 'Max']
    num_groups = len(groupnames)
    
    # Create a subplot grid with 2 rows (or more if more groups) and 4 columns
    fig, axes = plt.subplots(num_groups, 4, figsize=(20, 5 * num_groups), sharex='col', sharey='row')
    
    # If no specific vmin and vmax are provided, determine them from the data
    if vmin is None or vmax is None:
        vmin = df['counts'].apply(np.min).min()  # Minimum value across all counts
        vmax = df['counts'].apply(np.max).max()  # Maximum value

    # Iterate over each group and stimulation to create each subplot
    for i, group in enumerate(groupnames):
        for j, stim in enumerate(stimulations):
            ax = axes[i, j] if num_groups > 1 else axes[j]
            
            # Filter DataFrame for the current group and stimulation
            stim_df = df[(df['groupname'] == group) & (df['Stimulation'] == stim)]

            if not stim_df.empty:
                # Find the maximum length of arrays in the 'counts' column for current group and stimulation
                max_length = stim_df['counts'].map(len).max()

                # Pad all 'counts' arrays to this maximum length
                padded_counts = stim_df['counts'].apply(lambda x: pad_array_to_max_length(x, max_length))

                # Stack the padded counts to create a matrix
                counts_matrix = np.stack(padded_counts.values)
                
                # Extract the time_array from the first row of the filtered DataFrame
                time_bins = stim_df.iloc[0]['time_array']
                num_ticks = min(len(time_bins), 10)
                tick_positions = np.linspace(0, len(time_bins) - 1, num_ticks, dtype=int)
                tick_labels = [f"{time_bins[pos]:.0f}" for pos in tick_positions]

                # Plot the heatmap with user-defined or data-driven color range
                sns.heatmap(counts_matrix, ax=ax, cmap="viridis", vmin=vmin, vmax=vmax, cbar=(j == len(stimulations) - 1))
                
                # Set x-ticks to align with time bins
                ax.set_xticks(tick_positions)
                ax.set_xticklabels(tick_labels, rotation=45, ha='right')
                
                ax.set_title(f'{group} - {stim}')
                ax.set_xlabel('Time (ms)')
                if j == 0:
                    ax.set_ylabel('Cells')

    plt.tight_layout()
    plt.show()




In [ ]:
unfiltered_mean_df, unfiltered_detailed_df = whisker_df_manager.calculate_all_psths(is_single_unit=None, cell_type=None, stim_responsivity=None, time_window=(0, 100), normalize=True, filter_empty_trials=True)


In [ ]:
whisker_df_manager.dataframes['psth_dataframe_Zero']

In [ ]:
_, detailed_df_entirepsth= whisker_df_manager.calculate_all_psths(is_single_unit=1.0, cell_type='RS', stim_responsivity=1.0, time_window=(0, 100), normalize=False, filter_empty_trials=False)
plot_group_stimulation_heatmaps(detailed_df_entirepsth)


In [ ]:
_, detailed_df_entirepsth= whisker_df_manager.calculate_all_psths(is_single_unit=1.0, cell_type='FS', stim_responsivity=1.0, time_window=(0, 100), normalize=True, filter_empty_trials=True)
plot_group_stimulation_heatmaps(detailed_df_entirepsth)

testing correlations 

In [ ]:
whisker_df_manager.dataframes['psth_dataframe_Zero']

In [ ]:
# Assuming 'dataframe' is your DataFrame
def find_recording_with_most_single_units(data):
    # Filter data for single units
    single_unit_data = data[data['IsSingleUnit'] == 1.0]

    # Group by recording name and count entries
    counts = single_unit_data.groupby('recordingname').size()

    # Find the recording with the maximum count
    max_single_units = counts.idxmax()
    max_count = counts.max()

    return max_single_units, max_count

max_recording, max_count = find_recording_with_most_single_units(whisker_df_manager.dataframes['psth_dataframe_Zero'])
print(f"The recording with the most single units is {max_recording} with {max_count} single units.")

In [ ]:
whisker_df_manager.extract_spike_trains('veh_3141_2')

# waveform plots and analysis -- plot dynamically

In [ ]:
def dynamic_plot_saving_waveforms(whisker_df_manager, base_directory):
    """
    Dynamically generates and saves plots across different conditions for waveforms and trough-to-peak histograms.

    Args:
        whisker_df_manager: The data manager object with methods to calculate stats and plot.
        base_directory (str): Base directory to save the plots.
    """
    groupnames = [None, 'CTZ', 'No_CTZ']
    is_single_units = [None, 1.0, 0.0]
    stim_responsivities = [1.0, 0.0, -1.0, None]
    
    for groupname in groupnames:
        # Determine the top-level folder based on groupname
        if groupname is None:
            top_folder = 'All'
        else:
            top_folder = groupname
        
        for is_single_unit in is_single_units:
            # Determine the subfolder based on is_single_unit
            if is_single_unit is None:
                subfolder = 'All_Units'
            elif is_single_unit == 1.0:
                subfolder = 'SUA'
            else:
                subfolder = 'MUA'
            
            #print the current status of the loop
            print(f"Generating plots for: {top_folder} - {subfolder}")
            
            
            
            for stim_responsivity in stim_responsivities:
                # Define the directory for saving plots
                directory = os.path.join(base_directory, top_folder, subfolder, f"StimResponsivity_{stim_responsivity}")
                os.makedirs(directory, exist_ok=True)
                
                # Create a suffix for filenames
                suffix = f"{top_folder}_{subfolder}_StimResponsivity_{stim_responsivity}"
                
                # Generate and save the trough-to-peak histogram
                print(f"Generating trough-to-peak histogram for: {suffix}")
                whisker_df_manager.plot_trough_to_peak_histogram(
                    groupname=groupname, is_single_unit=is_single_unit, 
                    stim_responsivity=stim_responsivity, modulation_label='positive'
                )
                plt.savefig(os.path.join(directory, f"{suffix}_trough_to_peak_histogram.svg"), transparent=True)
                plt.close()  # Ensure the plot is closed and saved correctly
                
                # Generate and save the combined waveforms (mean+SEM)
                print(f"Generating combined waveforms (mean+SEM) for: {suffix}")
                whisker_df_manager.plot_combined_waveforms_meansem(
                    groupname=groupname, modulation_label='positive', 
                     stim_responsivity=stim_responsivity, is_single_unit=is_single_unit
                )
                plt.savefig(os.path.join(directory, f"{suffix}_combined_waveforms_meansem.svg"), transparent=True)
                plt.close()  # Ensure the plot is closed and saved correctly
                
                # Generate and save the combined waveforms
                print(f"Generating combined waveforms for: {suffix}")
                whisker_df_manager.plot_combined_waveforms(
                    groupname=groupname, modulation_label='positive', 
                    stim_responsivity=stim_responsivity, is_single_unit=is_single_unit
                )
                plt.savefig(os.path.join(directory, f"{suffix}_combined_waveforms.svg"), transparent=True)
                plt.close()  # Ensure the plot is closed and saved correctly
                
                #print the current status of the loop
                print(f"Plots saved for: {top_folder} - {subfolder} - StimResponsivity_{stim_responsivity}")

    print("All plots have been generated and saved.")
    
    
# Set the base directory
base_directory = '/Volumes/MannySSD/figures/comparing_waveforms_dynamic'

# Call the function to generate and save the plots
dynamic_plot_saving_waveforms(whisker_df_manager, base_directory)

### lets try to use plotly all of these worked except for the last one -- still need to modify 

In [ ]:
pip install plotly

In [ ]:
import plotly.express as px
import plotly.offline as pyo

def plot_3d_waveform_metrics_with_stimulation(obj, x_metric='TroughToPeak_duration', y_metric='MeanFR_baseline', z_metric='SpikeHalfWidth', is_single_unit=None, stim_responsivity=None, modulation_label=None):
    """
    Creates an interactive 3D scatter plot of specified waveform metrics for FS and RS cells,
    with dot size based on mean_stimulation and color-coded by Cell_Type.

    Parameters:
    obj: The object containing the `calculate_basic_stats` and `prepare_for_boxplot` methods.
    x_metric (str): The metric to plot on the x-axis.
    y_metric (str): The metric to plot on the y-axis.
    z_metric (str): The metric to plot on the z-axis.
    is_single_unit (float or None): Filter by single units (1.0), non-single units (0.0), or None for no filtering.
    stim_responsivity (float or None): Filter by StimResponsivity (1.0, 0.0, or -1.0). If None, no filtering by this criterion.
    modulation_label (str or None): Filter by ModulationLabel ('positive', 'negative', 'none'). If None, no filtering by this criterion.
    firstspike_latency (bool or None): Whether to include first spike latency in the calculations.

    Returns:
    None: The function generates an interactive 3D plot in a new browser window and returns the combined DataFrame.
    """

    # Run the necessary calculations
    obj.calculate_basic_stats(
        'CTZ', 'No_CTZ', stim_label='Max', 
        baseline_range=(-100, -1), stim_range=(0, 500), 
        cell_type=None, is_single_unit=is_single_unit, 
        stim_responsivity=stim_responsivity, 
        smoothing_window=3, modulation_label=modulation_label
    )
    
    boxplot_df = obj.prepare_for_boxplot()

    # Combine FS and RS into a single DataFrame, adding a column to distinguish them
    fs_df = boxplot_df[boxplot_df['Cell_Type'] == 'FS']
    rs_df = boxplot_df[boxplot_df['Cell_Type'] == 'RS']
    combined_df = pd.concat([fs_df, rs_df])

    # Drop NaN values for the selected metrics and ensure `mean_stimulation` is included
    combined_df = combined_df[[x_metric, y_metric, z_metric, 'mean_stimulation', 'Cell_Type', 'Group', 'recordingname', 'cid', 'StimResponsivity']].dropna()

    # Create a 3D scatter plot with size based on `mean_stimulation` and color by `Cell_Type`
    fig = px.scatter_3d(combined_df, 
                        x=x_metric, 
                        y=y_metric, 
                        z=z_metric, 
                        color='Cell_Type',  # Color by Cell_Type (FS or RS)
                        size='mean_stimulation',  # Size by mean_stimulation
                        size_max=20,  # Max size for the dots
                        labels={
                            x_metric: x_metric.replace('_', ' ').title(),
                            y_metric: y_metric.replace('_', ' ').title(),
                            z_metric: z_metric.replace('_', ' ').title(),
                        },
                        hover_data={
                            'Group': True,  # Corrected the column name from 'groupname' to 'Group'
                            'recordingname': True,
                            'cid': True,
                            'StimResponsivity': True,
                            'mean_stimulation': True,
                        },
                        title=f'3D Scatter Plot with Mean Stimulation Intensity of {x_metric}, {y_metric}, and {z_metric}')

    # Update the layout to make the figure larger
    fig.update_layout(
        width=1000,  # Adjust width as needed
        height=800,  # Adjust height as needed
    )
    
    # Show the figure in a standalone browser window
    #pyo.plot(fig, auto_open=True)
    
    #show the figure in the notebook
    fig.show()

    return combined_df

combined_df = plot_3d_waveform_metrics_with_stimulation(
    whisker_df_manager,
    x_metric='Peak2ToTrough_ratio', 
    y_metric='TroughToPeak_duration', 
    z_metric='Peak1ToTrough_ratio', 
    is_single_unit=None,
    stim_responsivity=None,
    modulation_label=None
)

In [ ]:
import plotly.graph_objs as go

def plot_3d_waveform_metrics_with_stimulation(obj, x_metric='TroughToPeak_duration', y_metric='MeanFR_baseline', z_metric='SpikeHalfWidth', is_single_unit=None, stim_responsivity=None, modulation_label=None):
    """
    Creates an interactive 3D scatter plot of specified waveform metrics for FS and RS cells,
    with dot size based on mean_stimulation and color-coded by Cell_Type.

    Parameters:
    obj: The object containing the `calculate_basic_stats` and `prepare_for_boxplot` methods.
    x_metric (str): The metric to plot on the x-axis.
    y_metric (str): The metric to plot on the y-axis.
    z_metric (str): The metric to plot on the z-axis.
    is_single_unit (float or None): Filter by single units (1.0), non-single units (0.0), or None for no filtering.
    stim_responsivity (float or None): Filter by StimResponsivity (1.0, 0.0, or -1.0). If None, no filtering by this criterion.
    modulation_label (str or None): Filter by ModulationLabel ('positive', 'negative', 'none'). If None, no filtering by this criterion.

    Returns:
    None: The function generates an interactive 3D plot in a new browser window and returns the combined DataFrame.
    """

    # Run the necessary calculations
    obj.calculate_basic_stats(
        'CTZ', 'No_CTZ', stim_label='Max', 
        baseline_range=(-100, -1), stim_range=(0, 500), 
        cell_type=None, is_single_unit=is_single_unit, 
        stim_responsivity=stim_responsivity, 
        smoothing_window=3, modulation_label=modulation_label
    )
    
    boxplot_df = obj.prepare_for_boxplot()

    # Combine FS and RS into a single DataFrame
    fs_df = boxplot_df[boxplot_df['Cell_Type'] == 'FS']
    rs_df = boxplot_df[boxplot_df['Cell_Type'] == 'RS']
    combined_df = pd.concat([fs_df, rs_df])

    # Drop NaN values for the selected metrics and ensure `mean_stimulation` is included
    combined_df = combined_df[[x_metric, y_metric, z_metric, 'mean_stimulation', 'Cell_Type', 'Group', 'recordingname', 'cid', 'StimResponsivity']].dropna()

    # Create traces for FS and RS
    trace_fs = go.Scatter3d(
        x=fs_df[x_metric],
        y=fs_df[y_metric],
        z=fs_df[z_metric],
        mode='markers',
        marker=dict(
            size=fs_df['mean_stimulation'] / 5,  # Adjust size scaling as needed
            color='darkgoldenrod',  # FS color
            opacity=0.8,
        ),
        name='FS',
        text=[f"Group: {g}<br>Recording: {r}<br>cid: {c}<br>StimResponsivity: {s}<br>Mean Stimulation: {m:.2f}" 
              for g, r, c, s, m in zip(fs_df['Group'], fs_df['recordingname'], fs_df['cid'], fs_df['StimResponsivity'], fs_df['mean_stimulation'])],
        hoverinfo='text'
    )

    trace_rs = go.Scatter3d(
        x=rs_df[x_metric],
        y=rs_df[y_metric],
        z=rs_df[z_metric],
        mode='markers',
        marker=dict(
            size=rs_df['mean_stimulation'] / 5,  # Adjust size scaling as needed
            color='sienna',  # RS color
            opacity=0.8,
        ),
        name='RS',
        text=[f"Group: {g}<br>Recording: {r}<br>cid: {c}<br>StimResponsivity: {s}<br>Mean Stimulation: {m:.2f}" 
              for g, r, c, s, m in zip(rs_df['Group'], rs_df['recordingname'], rs_df['cid'], rs_df['StimResponsivity'], rs_df['mean_stimulation'])],
        hoverinfo='text'
    )

    # Combine traces
    fig = go.Figure(data=[trace_fs, trace_rs])

    # Update the layout
    fig.update_layout(
        scene=dict(
            xaxis_title=x_metric.replace('_', ' ').title(),
            yaxis_title=y_metric.replace('_', ' ').title(),
            zaxis_title=z_metric.replace('_', ' ').title(),
        ),
        width=1000,  # Adjust width as needed
        height=800,  # Adjust height as needed
        title=f'3D Scatter Plot with Mean Stimulation Intensity of {x_metric}, {y_metric}, and {z_metric}',
    )

    # Show the figure in the notebook
    fig.show()

    return combined_df

# Example usage:
combined_df = plot_3d_waveform_metrics_with_stimulation(
    whisker_df_manager,
    x_metric='Peak2ToTrough_ratio', 
    y_metric='TroughToPeak_duration', 
    z_metric='Peak1ToTrough_ratio', 
    is_single_unit=None,
    stim_responsivity=None,
    modulation_label=None
)




In [ ]:
import sys
import os
import contextlib

@contextlib.contextmanager # Context manager to suppress output but just for below function
def suppress_output():
    """
    Suppress all output (stdout and stderr) within the context.
    """
    with open(os.devnull, 'w') as devnull:
        old_stdout = sys.stdout
        old_stderr = sys.stderr
        sys.stdout = devnull
        sys.stderr = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout
            sys.stderr = old_stderr

import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

def plot_static_3d_waveform_metrics_with_stimulation(obj, x_metric='TroughToPeak_duration', 
                                                     y_metric='MeanFR_baseline', z_metric='SpikeHalfWidth', 
                                                     is_single_unit=None, stim_responsivity=None, 
                                                     modulation_label=None, 
                                                     output_path='3d_plot.svg', elev=30, azim=30):
    """
    Creates a static 3D scatter plot of specified waveform metrics for FS and RS cells,
    with dot size based on mean_stimulation and color-coded by Cell_Type, and saves as an SVG.

    Parameters:
    obj: The object containing the `calculate_basic_stats` and `prepare_for_boxplot` methods.
    x_metric (str): The metric to plot on the x-axis.
    y_metric (str): The metric to plot on the y-axis.
    z_metric (str): The metric to plot on the z-axis.
    is_single_unit (float or None): Filter by single units (1.0), non-single units (0.0), or None for no filtering.
    stim_responsivity (float or None): Filter by StimResponsivity (1.0, 0.0, or -1.0). If None, no filtering by this criterion.
    modulation_label (str or None): Filter by ModulationLabel ('positive', 'negative', 'none'). If None, no filtering by this criterion.
    output_path (str): Path to save the output SVG file.
    elev (int or float): Elevation angle for the 3D plot (default is 30).
    azim (int or float): Azimuth angle for the 3D plot (default is 30).

    Returns:
    None: The function generates a static 3D plot and saves it as an SVG.
    """

    # Suppress output during calculations and data preparation
    with suppress_output():
        # Run the necessary calculations
        obj.calculate_basic_stats(
            'CTZ', 'No_CTZ', stim_label='Max', 
            baseline_range=(-100, -1), stim_range=(0, 500), 
            cell_type=None, is_single_unit=is_single_unit, 
            stim_responsivity=stim_responsivity, 
            smoothing_window=3, modulation_label=modulation_label
        )
    
        boxplot_df = obj.prepare_for_boxplot()

    # Combine FS and RS into a single DataFrame
    fs_df = boxplot_df[boxplot_df['Cell_Type'] == 'FS']
    rs_df = boxplot_df[boxplot_df['Cell_Type'] == 'RS']
    combined_df = pd.concat([fs_df, rs_df])

    # Drop NaN values for the selected metrics and ensure `mean_stimulation` is included
    combined_df = combined_df[[x_metric, y_metric, z_metric, 'mean_stimulation', 'Cell_Type', 'Group', 'recordingname', 'cid', 'StimResponsivity']].dropna()

    # Create a 3D scatter plot
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Plot FS cells
    fs = combined_df[combined_df['Cell_Type'] == 'FS']
    scatter_fs = ax.scatter(
        fs[x_metric], fs[y_metric], fs[z_metric],
        s=fs['mean_stimulation'] * 2,  # Scale marker size
        c='darkgoldenrod', label='FS', alpha=0.8
    )

    # Plot RS cells
    rs = combined_df[combined_df['Cell_Type'] == 'RS']
    scatter_rs = ax.scatter(
        rs[x_metric], rs[y_metric], rs[z_metric],
        s=rs['mean_stimulation'] * 2,  # Scale marker size
        c='sienna', label='RS', alpha=0.8
    )

    # Set axis labels
    ax.set_xlabel(x_metric.replace('_', ' ').title())
    ax.set_ylabel(y_metric.replace('_', ' ').title())
    ax.set_zlabel(z_metric.replace('_', ' ').title())

    # Set title
    ax.set_title(f'3D Scatter Plot with Mean Stimulation Intensity of {x_metric}, {y_metric}, and {z_metric}')

    # Set viewing angle
    ax.view_init(elev=elev, azim=azim)

    # Add a legend for FS and RS
    ax.legend(loc='upper right')

    # Add a custom legend for dot sizes
    handles, labels = scatter_fs.legend_elements(prop="sizes", alpha=0.6, num=4)
    size_legend = plt.legend(handles, labels, title="Mean Stimulation", loc="lower right", bbox_to_anchor=(1.1, 0.5))

    # Add the custom legend to the plot
    ax.add_artist(size_legend)
    
    # Save the figure as an SVG file
    plt.savefig(output_path, format='svg', bbox_inches='tight', transparent=True)

    # Show the plot
    plt.show()


# Example usage:
plot_static_3d_waveform_metrics_with_stimulation(
    whisker_df_manager,
    x_metric='Peak2ToTrough_ratio', 
    y_metric='TroughToPeak_duration', 
    z_metric='Peak1ToTrough_ratio', 
    is_single_unit=1.0,
    stim_responsivity=None,
    modulation_label=None,
    output_path='/Volumes/MannySSD/figures/3dwaveformplots_SUA.svg',
    elev=30,  # Adjust elevation angle
    azim=5  # Adjust azimuth angle
)

In [ ]:
import plotly.express as px
import plotly.offline as pyo
import os

### this used to work but need to fix 

def plot_box_and_strip_with_plotly(obj, groups=None, stimulations=None, show_outliers=True, hue_order=None, 
                                   directory=None, file_name=None, ylim=None, 
                                   modulation_label=None, cell_type=None, 
                                   is_single_unit=None, stim_responsivity=None):
    """
    Plots interactive boxplots and stripplots for specified groups and stimulations using Plotly, with metadata accessible via hover.

    Args:
        obj: The object containing the `prepare_for_boxplot` method.
        groups (list of str, optional): List of groups to include in the plot.
        stimulations (list of str, optional): List of stimulations to include in the plot.
        show_outliers (bool, optional): Whether to show outliers.
        hue_order (list, optional): Order of the hue levels.
        directory (str, optional): Directory to save the plot.
        file_name (str, optional): File name for saving the plot.
        ylim (tuple, optional): Limits for the y-axis.
        modulation_label (str, optional): Filter by modulation label ('positive', 'negative', 'none').

        cell_type (str, optional): Filter for specific cell types ('FS', 'RS', etc.).
        is_single_unit (float, optional): Filter for single units (1.0), multi-units (0.0), or all (None).
        stim_responsivity (float, optional): Filter by stimulus responsivity (1.0, 0.0, -1.0).

    Returns:
        pandas.DataFrame: The DataFrame used for the plot.
    """
    # Prepare data for boxplot
    boxplot_df = obj.prepare_for_boxplot()

    # Apply filtering as needed
    if groups:
        boxplot_df = boxplot_df[boxplot_df['Group'].isin(groups)]
    if stimulations:
        boxplot_df = boxplot_df[boxplot_df['Stimulation'].isin(stimulations)]
    if modulation_label:
        boxplot_df = boxplot_df[boxplot_df['ModulationIndex'] == modulation_label]
    if cell_type:
        boxplot_df = boxplot_df[boxplot_df['Cell_Type'] == cell_type]
    if is_single_unit is not None:
        boxplot_df = boxplot_df[boxplot_df['IsSingleUnit'] == is_single_unit]
    if stim_responsivity is not None:
        boxplot_df = boxplot_df[boxplot_df['StimResponsivity'] == stim_responsivity]
    
    # Create a boxplot with stripplot overlay using Plotly
    fig = px.box(boxplot_df, 
                 x='Stimulation', 
                 y='mean_stimulation', 
                 color='Group', 
                 points='all' if show_outliers else 'outliers',
                 hover_data=['recordingname', 'ModulationLabel', 'cid', 'Cell_Type', 'Template_Channel', 'LaminarLabel', 'IsSingleUnit', 'TroughToPeak_duration', 
                             'PeakToPeak_ratio', 'peak1_normalized_amplitude', 'Peak1ToTrough_ratio', 'Peak2ToTrough_ratio', 'SpikeHalfWidth', 'MeanFR_baseline', 'StimResponsivity'],
                 color_discrete_map={
                     'No_CTZ': '#797979',
                     'CTZ': '#5a00c2'
                 })
    
    # Adjust y-axis limits if provided
    if ylim:
        fig.update_yaxes(range=ylim)

    # Add title and labels


    # Save the figure if directory and file name are provided
    if directory and file_name:
        os.makedirs(directory, exist_ok=True)
        file_path = os.path.join(directory, f'{file_name}.html')
        fig.write_html(file_path)

    fig.update_layout(
        title='Comparison of Mean Stimulation Across Groups and Stimulations',
        xaxis_title='Stimulation Type',
        yaxis_title='Mean Stimulation',
        width=1000,  # Adjust width as needed
        height=800,  # Adjust height as needed
    )
    
        # Update the layout to make the figure larger

    
    # Show the figure in a standalone browser window
    #pyo.plot(fig, auto_open=True)
    #plot the figure in the notebook
    fig.show()
    
# Call the function with your object and desired filtering options
boxplot_df = plot_box_and_strip_with_plotly(
    whisker_df_manager,  # Your object containing the data and methods
    groups=['CTZ', 'No_CTZ'],  # Filter by groups
    stimulations=['8Hz'],  # Filter by stimulations
    show_outliers=True,  # Show or hide outliers
    directory='/Volumes/MannySSD/figures/comparing_conditions_dynamic_plotly',  # Directory to save the plot
    file_name='plot_name',  # File name for saving the plot
    ylim=None,  # Set y-axis limits
    modulation_label='positive',  # Filter by modulation label
    cell_type='FS',  # Filter by cell type ('FS', 'RS')
    is_single_unit=1.0,  # Filter for single units
    stim_responsivity=1.0  # Filter by stimulus responsivity
)

# opto Emx data analysis 

### import the opto ephys data 

In [ ]:
#repeat for opto data
opto = ExtractEphysData('/Volumes/MannySSD/emx_led/','all_data.mat')

### intialize the opto dataframe object 

In [ ]:
opto_df_manager = DataFrameManager(opto)
opto_df_manager.create_dataframe(['Cell_Type', 'IsSingleUnit', 'StimResponsivity', 'MeanFR_baseline', 'MeanFR_stim', 
                                  'PeakEvokedFR', 'PeakEvokedFR_Latency', 'FanoFactor_baseline', 'FanoFactor_stim', 'SpikeTimes_all', 
                                  'MeanFR_inst_stim', 'Template_Channel', 'ModulationIndex', 'Normalized_Template_Waveform', 'TroughToPeak_duration', 
                                  'UnNormalized_Template_Waveform', 'ISI_violations_percent', 'Recording_Duration', 'Sampling_Frequency', 'SpikeTimes_all' ], 'basic_metrics')
opto_df_manager.create_psth_dataframe_opto()


In [ ]:

opto_df_manager.plot_optogenetic_psth_comparison_grid('CTZ', 'No_CTZ', '8 Hz LED',
                                      cell_types=['RS', 'FS'], is_single_unit=None, 
                                      stim_responsivity=1.0, modulation_label='positive',
                                      time_range=(-200, 800), plot_mode='sem', smoothing_window=1)


In [ ]:
melted_data_laminar

In [ ]:
# Usage example

import plotly.graph_objects as go
from plotly.subplots import make_subplots
fig = opto_df_manager.plot_interactive_psth_comparison_grid('CTZ', 'No_CTZ', '8 Hz LED',
                                                      cell_types=['RS', 'FS'], is_single_unit=1.0,
                                                      stim_responsivity=1.0, modulation_label='positive',
                                                      time_range=(-200, 800), smoothing_window=1)

In [ ]:
opto_df_manager.plot_optogenetic_psth_comparison_subplot_with_led('No_CTZ', 'CTZ', '8 Hz LED', 
                                      cell_type='FS', 
                                      is_single_unit=1.0, 
                                      stim_responsivity=1.0, 
                                      time_range=None, 
                                      plot_mode='sem', 
                                      smoothing_window=1)

In [ ]:
def plot_custom_boxplot_grid(opto_df_manager, column_to_analyze, 
                             single_unit_only=None, stim_responsivity=1.0, 
                             show_outliers=False, show_scatter=True, 
                             modulation_index='positive', jitter=True, jitter_amount=0.1):
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 16), sharex='col')
    color_palette = {'CTZ': 'purple', 'No_CTZ': 'grey'}
    
    def plot_for_cell_type(ax, cell_type, column):
        # Filter data
        data = opto_df_manager.dataframes['basic_metrics'].copy()
        
        # Convert IsSingleUnit to boolean
        data['IsSingleUnit'] = data['IsSingleUnit'].apply(lambda x: 1.0 in x or 1 in x if isinstance(x, (np.ndarray, list)) else x == 1.0 or x == 1)
        
        # Extract numeric values from the specified column
        data[f'{column}_numeric'] = data[column].apply(lambda x: float(x) if not pd.isna(x) else np.nan)
        
        if single_unit_only:
            data = data[data['IsSingleUnit']]
        
        if stim_responsivity is not None:
            data = data[data['StimResponsivity'] == stim_responsivity]
        
        if modulation_index is not None:
            data = data[data['ModulationIndex'] == modulation_index]
        
        # Remove any NaN or infinite values
        data = data.dropna(subset=[f'{column}_numeric'])
        data = data[~np.isinf(data[f'{column}_numeric'])]
        
        data = data[data['Cell_Type'] == cell_type]
        
        # Create box plot
        sns.boxplot(x='groupname', y=f'{column}_numeric', data=data, 
                    palette=color_palette, showfliers=show_outliers, ax=ax)
        
        # Add scatter plot if requested
        if show_scatter:
            positions = {group: i for i, group in enumerate(data['groupname'].unique())}
            
            for group in data['groupname'].unique():
                group_data = data[data['groupname'] == group]
                
                if show_outliers:
                    x = [positions[group]] * len(group_data)
                    y = group_data[f'{column}_numeric']
                else:
                    q1, q3 = group_data[f'{column}_numeric'].quantile([0.25, 0.75])
                    iqr = q3 - q1
                    lower_bound, upper_bound = q1 - 1.5 * iqr, q3 + 1.5 * iqr
                    non_outliers = group_data[(group_data[f'{column}_numeric'] >= lower_bound) & 
                                              (group_data[f'{column}_numeric'] <= upper_bound)]
                    x = [positions[group]] * len(non_outliers)
                    y = non_outliers[f'{column}_numeric']
                
                if jitter:
                    x = np.array(x) + np.random.uniform(-jitter_amount, jitter_amount, len(x))
                
                ax.scatter(x, y, color=color_palette[group], alpha=0.5, edgecolor='none', zorder=3)
        
        # Perform Mann-Whitney U test
        group1 = data[data['groupname'] == 'CTZ'][f'{column}_numeric']
        group2 = data[data['groupname'] == 'No_CTZ'][f'{column}_numeric']
        _, p_value = stats.mannwhitneyu(group1, group2, alternative='two-sided')
        
        # Add statistical annotation
        ax.text(0.5, 0.95, f'p = {p_value:.4f}', 
                horizontalalignment='center', verticalalignment='top',
                transform=ax.transAxes, fontsize=10)
        
        # Customize the plot
        ax.set_title(f"{cell_type} cells - {column}")
        if ax.get_subplotspec().is_last_row():
            ax.set_xlabel('Group')
        else:
            ax.set_xlabel('')
        if ax.get_subplotspec().is_first_col():
            ax.set_ylabel(column)
        else:
            ax.set_ylabel('')
        
        return ax.get_ylim()  # Return y-axis limits for synchronization
    
    # Plot for RS cells - MeanFR_baseline
    ylim1 = plot_for_cell_type(ax1, 'RS', 'MeanFR_baseline')
    
    # Plot for FS cells - MeanFR_baseline
    ylim2 = plot_for_cell_type(ax2, 'FS', 'MeanFR_baseline')
    
    # Synchronize y-axis for top row
    ymin_top = min(ylim1[0], ylim2[0])
    ymax_top = max(ylim1[1], ylim2[1])
    ax1.set_ylim(ymin_top, ymax_top)
    ax2.set_ylim(ymin_top, ymax_top)
    
    # Plot for RS cells - user specified column
    ylim3 = plot_for_cell_type(ax3, 'RS', column_to_analyze)
    
    # Plot for FS cells - user specified column
    ylim4 = plot_for_cell_type(ax4, 'FS', column_to_analyze)
    
    # Synchronize y-axis for bottom row
    ymin_bottom = min(ylim3[0], ylim4[0])
    ymax_bottom = max(ylim3[1], ylim4[1])
    ax3.set_ylim(ymin_bottom, ymax_bottom)
    ax4.set_ylim(ymin_bottom, ymax_bottom)
    
    # Set overall title
    fig.suptitle(f"Comparison of MeanFR_baseline and {column_to_analyze} for RS and FS cells", fontsize=16)
    
    # Adjust layout and show plot
    plt.tight_layout()
    
    #save the plot in the normal directory 
    plt.savefig(f'/Volumes/MannySSD/figures/opto_psths_comparison/BoxplotsforPSTHsCTZ_No_CTZ_comparison_{"single" if is_single_unit == 1.0 else "multi"}_units_evoked_FR.svg', format='svg', dpi=300, bbox_inches='tight', transparent=True)

    plt.show()
# Usage
columname = 'MeanFR_stim'  # This can be changed to any other column name
plot_custom_boxplot_grid(opto_df_manager, column_to_analyze=columname, 
                         single_unit_only=None, stim_responsivity=1.0, 
                         show_outliers=False, show_scatter=True, 
                         modulation_index='positive', jitter=True, jitter_amount=0.1)


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
from scipy import stats

def plot_custom_boxplot_grid_plotly(opto_df_manager, column_to_analyze, 
                                    single_unit_only=None, stim_responsivity=1.0, 
                                    show_outliers=False, show_scatter=True, 
                                    modulation_index='positive', jitter=True, jitter_amount=0.1):

    fig = make_subplots(rows=2, cols=2, 
                        subplot_titles=("RS cells - MeanFR_baseline", "FS cells - MeanFR_baseline",
                                        f"RS cells - {column_to_analyze}", f"FS cells - {column_to_analyze}"),
                        shared_xaxes='columns')

    color_palette = {'CTZ': 'purple', 'No_CTZ': 'grey'}

    def plot_for_cell_type(row, col, cell_type, column):
        # Filter data
        data = opto_df_manager.dataframes['basic_metrics'].copy()

        # Convert IsSingleUnit to boolean
        data['IsSingleUnit'] = data['IsSingleUnit'].apply(lambda x: 1.0 in x or 1 in x if isinstance(x, (np.ndarray, list)) else x == 1.0 or x == 1)

        # Extract numeric values from the specified column
        column_numeric = f"{column}_numeric"
        data[column_numeric] = pd.to_numeric(data[column], errors='coerce')

        if single_unit_only:
            data = data[data['IsSingleUnit']]

        if stim_responsivity is not None:
            data = data[data['StimResponsivity'] == stim_responsivity]

        if modulation_index is not None:
            data = data[data['ModulationIndex'] == modulation_index]

        # Remove any NaN or infinite values
        data = data.dropna(subset=[column_numeric])
        data = data[~data[column_numeric].isin([np.inf, -np.inf])]

        data = data[data['Cell_Type'] == cell_type]

        for group_index, group in enumerate(['CTZ', 'No_CTZ']):
            group_data = data[data['groupname'] == group]
            
            # Create box plot
            fig.add_trace(go.Box(
                y=group_data[column_numeric],
                name=group,
                boxpoints=False,  # Hide individual points in boxplot
                marker_color=color_palette[group],
                showlegend=False,
                x0=group_index,
                opacity=0.6,  # Make boxplot slightly transparent
                line=dict(width=2),  # Increase line width for visibility
                zorder=1  # Set lower z-order for boxplot
            ), row=row, col=col)

            # Add scatter plot if requested
            if show_scatter:
                x = [group_index] * len(group_data)
                y = group_data[column_numeric]

                if jitter:
                    x = [xi + np.random.uniform(-jitter_amount, jitter_amount) for xi in x]

                fig.add_trace(go.Scatter(
                    x=x,
                    y=y,
                    mode='markers',
                    marker=dict(color=color_palette[group], size=5, opacity=0.7),
                    name=group,
                    showlegend=False,
                    hovertemplate=f"Group: {group}<br>Value: %{{y}}<br>Cell ID: %{{customdata}}",
                    customdata=group_data['cid'],
                    zorder=2  # Set higher z-order for scatter plot
                ), row=row, col=col)

        # Perform Mann-Whitney U test
        group1 = data[data['groupname'] == 'CTZ'][column_numeric].dropna().astype(float)
        group2 = data[data['groupname'] == 'No_CTZ'][column_numeric].dropna().astype(float)
        
        if len(group1) > 0 and len(group2) > 0:
            _, p_value = stats.mannwhitneyu(group1, group2, alternative='two-sided')
            
            # Add statistical annotation
            fig.add_annotation(
                xref="x domain", yref="y domain",
                x=0.5, y=1,
                text=f'p = {p_value:.4f}',
                showarrow=False,
                row=row, col=col
            )
        else:
            print(f"Warning: Not enough data to perform Mann-Whitney U test for {cell_type} cells, {column}")

        # Update x-axis
        fig.update_xaxes(tickvals=[0, 1], ticktext=['CTZ', 'No_CTZ'], row=row, col=col)

    # Plot for each cell type and column
    plot_for_cell_type(1, 1, 'RS', 'MeanFR_baseline')
    plot_for_cell_type(1, 2, 'FS', 'MeanFR_baseline')
    plot_for_cell_type(2, 1, 'RS', column_to_analyze)
    plot_for_cell_type(2, 2, 'FS', column_to_analyze)

    # Update layout
    fig.update_layout(
        title=f"Comparison of MeanFR_baseline and {column_to_analyze} for RS and FS cells",
        height=800,
        width=1200,
        boxmode='group'
    )

    # Update axes labels
    fig.update_xaxes(title_text="Group", row=2, col=1)
    fig.update_xaxes(title_text="Group", row=2, col=2)
    fig.update_yaxes(title_text="MeanFR_baseline", row=1, col=1)
    fig.update_yaxes(title_text=column_to_analyze, row=2, col=1)

    return fig

# Usage
columname = 'MeanFR_stim'  # This can be changed to any other column name
fig = plot_custom_boxplot_grid_plotly(opto_df_manager, column_to_analyze=columname, 
                                      single_unit_only=None, stim_responsivity=1.0, 
                                      show_outliers=False, show_scatter=True, 
                                      modulation_index='positive', jitter=True, jitter_amount=0.1)
fig.show()

In [ ]:
opto_df_manager.dataframes['basic_metrics'].columns

In [ ]:
def plot_optogenetic_psth_per_recording(self, group1, group2, opto_freq, 
                                        cell_type=None, is_single_unit=None, 
                                        stim_responsivity=None, modulation_label=None,
                                        time_range=None, plot_mode='mean', smoothing_window=None):
    """
    Plots PSTHs for optogenetic experiments, filtered at the recording level for each group,
    and saves the results in a specified directory structure.

    Args:
        group1 (str): First group name.
        group2 (str): Second group name.
        opto_freq (str): Optogenetic stimulation frequency (e.g., '8 Hz LED').
        cell_type (str, optional): Cell type to filter.
        is_single_unit (float, optional): Single unit filter.
        stim_responsivity (float, optional): Stimulus responsivity filter.
        modulation_label (str, optional): Modulation label filter.
        time_range (tuple, optional): Tuple specifying the start and end of the time range (e.g., (-100, 200)).
        plot_mode (str, optional): Plotting mode ('mean', 'traces', 'sem').
        smoothing_window (int, optional): Size of the smoothing window; if None, no smoothing is applied.
    """
    # Define colors
    group_colors = {
        'No_CTZ': '#797979',
        'CTZ': '#5a00c2'
    }

    # Fetch data
    df1, df2 = self.compare_optogenetic_groups(group1, group2, opto_freq, cell_type, is_single_unit, stim_responsivity, modulation_label)
    if df1.empty and df2.empty:
        print("Both groups have no data after filtering.")
        return
    
    # Get the time array and adjust for the specified time range
    time_array = self.eed.relative_time_ms['relative_time_ms']
    if time_range:
        time_mask = (time_array >= time_range[0]) & (time_range[1] >= time_array)
        time_array = time_array[time_mask]
    else:
        time_mask = slice(None)

    # Extract LED signal
    extracted_led_signal = self.extract_stim_signals_opto()
    led_array = extracted_led_signal[0][opto_freq]
    led_array = led_array[time_mask]  # Apply the same time mask to LED signal

    # Base directory for saving figures
    base_dir = "/Volumes/MannySSD/figures/opto_psths_perrecording"

    # Process each group
    for group, df in zip([group1, group2], [df1, df2]):
        if df.empty:
            print(f"No data for group {group} after filtering.")
            continue

        # Create group directory
        group_dir = os.path.join(base_dir, group)
        os.makedirs(group_dir, exist_ok=True)

        # Process each unique recording
        for recording in df['recording'].unique():
            recording_df = df[df['recording'] == recording]
            
            # Create recording directory
            recording_dir = os.path.join(group_dir, recording)
            os.makedirs(recording_dir, exist_ok=True)

            # Create a new figure for this recording
            fig, ax = plt.subplots(figsize=(10, 5))

            # Plot PSTH data
            data = recording_df['PSTHs_conv'].apply(lambda x: np.array(x)[time_mask])
            if smoothing_window:
                window = np.ones(smoothing_window) / smoothing_window
                data = data.apply(lambda x: np.convolve(x, window, mode='same'))

            mean_psth = data.apply(pd.Series).mean(axis=0)

            if plot_mode == 'sem':
                sem = data.apply(pd.Series).sem(axis=0)
                ax.fill_between(time_array, mean_psth - sem, mean_psth + sem, color=group_colors[group], alpha=0.2)

            elif plot_mode == 'traces':
                for trace in data:
                    ax.plot(time_array, trace, color=group_colors[group]+'33', alpha=0.2)

            ax.plot(time_array, mean_psth, label=f'{group}', color=group_colors[group])

            # Plot LED signal
            ax_led = ax.twinx()
            ax_led.plot(time_array, led_array, color='blue', linestyle='-', linewidth=1.5, alpha=0.7, label='LED Signal')

            # Set plot attributes
            ax.set_title(f'{group} - {opto_freq} - Recording: {recording}')
            ax.set_xlabel('Time (ms)')
            ax.set_ylabel('Average Spike Rate')
            ax_led.set_ylabel('LED Signal (a.u.)', color='blue')
            ax_led.tick_params(axis='y', labelcolor='blue')

            # Add legends
            ax.legend(loc='upper left')
            ax_led.legend(loc='upper right')

            # Save the figure
            plt.tight_layout()
            fig_path = os.path.join(recording_dir, f'PSTH_{opto_freq}.png')
            plt.savefig(fig_path)
            plt.close(fig)

            print(f"Saved figure for {group} - {recording} at {fig_path}")

    print("Finished processing all recordings.")

opto_df_manager.plot_optogenetic_psth_per_recording('No_CTZ', 'CTZ', '8 Hz LED', 
                                      cell_type='RS', 
                                      is_single_unit=None, 
                                      stim_responsivity=None, 
                                      time_range=None, 
                                      plot_mode='sem', 
                                      smoothing_window=1)
    


## plot the waveforms for the opto LED df

In [ ]:
## need to do 

### plot the distriubtion of units 

In [ ]:
import plotly.express as px
import plotly.offline as pyo
import os

### this used to work but need to fix 

def plot_box_and_strip_with_plotly(obj, groups=None, stimulations=None, show_outliers=True, hue_order=None, 
                                   directory=None, file_name=None, ylim=None, 
                                   modulation_label=None, cell_type=None, 
                                   is_single_unit=None, stim_responsivity=None):
    """
    Plots interactive boxplots and stripplots for specified groups and stimulations using Plotly, with metadata accessible via hover.

    Args:
        obj: The object containing the `prepare_for_boxplot` method.
        groups (list of str, optional): List of groups to include in the plot.
        stimulations (list of str, optional): List of stimulations to include in the plot.
        show_outliers (bool, optional): Whether to show outliers.
        hue_order (list, optional): Order of the hue levels.
        directory (str, optional): Directory to save the plot.
        file_name (str, optional): File name for saving the plot.
        ylim (tuple, optional): Limits for the y-axis.
        modulation_label (str, optional): Filter by modulation label ('positive', 'negative', 'none').

        cell_type (str, optional): Filter for specific cell types ('FS', 'RS', etc.).
        is_single_unit (float, optional): Filter for single units (1.0), multi-units (0.0), or all (None).
        stim_responsivity (float, optional): Filter by stimulus responsivity (1.0, 0.0, -1.0).

    Returns:
        pandas.DataFrame: The DataFrame used for the plot.
    """
    # Prepare data for boxplot
    boxplot_df = obj.prepare_for_boxplot()

    # Apply filtering as needed
    if groups:
        boxplot_df = boxplot_df[boxplot_df['Group'].isin(groups)]
    if stimulations:
        boxplot_df = boxplot_df[boxplot_df['Stimulation'].isin(stimulations)]
    if modulation_label:
        boxplot_df = boxplot_df[boxplot_df['ModulationIndex'] == modulation_label]
    if cell_type:
        boxplot_df = boxplot_df[boxplot_df['Cell_Type'] == cell_type]
    if is_single_unit is not None:
        boxplot_df = boxplot_df[boxplot_df['IsSingleUnit'] == is_single_unit]
    if stim_responsivity is not None:
        boxplot_df = boxplot_df[boxplot_df['StimResponsivity'] == stim_responsivity]
    
    # Create a boxplot with stripplot overlay using Plotly
    fig = px.box(boxplot_df, 
                 x='Stimulation', 
                 y='mean_stimulation', 
                 color='Group', 
                 points='all' if show_outliers else 'outliers',
                 hover_data=['recordingname', 'ModulationLabel', 'cid', 'Cell_Type', 'Template_Channel', 'LaminarLabel', 'IsSingleUnit', 'TroughToPeak_duration', 
                             'PeakToPeak_ratio', 'peak1_normalized_amplitude', 'Peak1ToTrough_ratio', 'Peak2ToTrough_ratio', 'SpikeHalfWidth', 'MeanFR_baseline', 'StimResponsivity'],
                 color_discrete_map={
                     'No_CTZ': '#797979',
                     'CTZ': '#5a00c2'
                 })
    
    # Adjust y-axis limits if provided
    if ylim:
        fig.update_yaxes(range=ylim)

    # Add title and labels


    # Save the figure if directory and file name are provided
    if directory and file_name:
        os.makedirs(directory, exist_ok=True)
        file_path = os.path.join(directory, f'{file_name}.html')
        fig.write_html(file_path)

    fig.update_layout(
        title='Comparison of Mean Stimulation Across Groups and Stimulations',
        xaxis_title='Stimulation Type',
        yaxis_title='Mean Stimulation',
        width=1000,  # Adjust width as needed
        height=800,  # Adjust height as needed
    )
    
        # Update the layout to make the figure larger

    
    # Show the figure in a standalone browser window
    #pyo.plot(fig, auto_open=True)
    #plot the figure in the notebook
    fig.show()
    
# Call the function with your object and desired filtering options
boxplot_df = plot_box_and_strip_with_plotly(
    whisker_df_manager,  # Your object containing the data and methods
    groups=['CTZ', 'No_CTZ'],  # Filter by groups
    stimulations=['8Hz'],  # Filter by stimulations
    show_outliers=True,  # Show or hide outliers
    directory='/Volumes/MannySSD/figures/comparing_conditions_dynamic_plotly',  # Directory to save the plot
    file_name='plot_name',  # File name for saving the plot
    ylim=None,  # Set y-axis limits
    modulation_label='positive',  # Filter by modulation label
    cell_type='FS',  # Filter by cell type ('FS', 'RS')
    is_single_unit=1.0,  # Filter for single units
    stim_responsivity=1.0  # Filter by stimulus responsivity
)

### see distribution of FS and RS for opto 

In [ ]:
opto_df_manager.plot_trough_to_peak_histogram()

### look at each indidual recording 

In [ ]:
ctz_df1, noctz_df2 = opto_df_manager.compare_groups('CTZ', 'No_CTZ', '8 Hz LED', 'RS', is_single_unit=1.0, stim_responsivity=1.0)


import matplotlib.pyplot as plt
import matplotlib.cm as cm

def process_data(df, time_range):
    
    time_array = whisker_df_manager.eed.relative_time_ms['relative_time_ms']
    
    if time_range:
        time_mask = (time_array >= time_range[0]) & (time_array <= time_range[1])
        time_array = time_array[time_mask]
    else:
        time_mask = slice(None)
    
    grouped = df.groupby('recordingname')
    grouped_data = grouped['PSTHs_raw'].apply(lambda x: np.mean([np.array(i)[time_mask] for i in x], axis=0))
    
    return grouped_data

ctz_df1_grouped = process_data(ctz_df1, time_range=(-10, 600))
noctz_df2_grouped = process_data(noctz_df2, time_range=(-10, 600))


# Create a color map
colors = cm.rainbow(np.linspace(0, 1, len(ctz_df1_grouped)))

# Create a 1xN subplot
fig, axs = plt.subplots(1, len(ctz_df1_grouped), figsize=(15, 5))

# Iterate over the grouped data
for ax, data, color, name in zip(axs, ctz_df1_grouped, colors, ctz_df1_grouped.index):
    ax.plot(data, color=color)
    ax.set_title(name)

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
# Create a color map
colors = cm.rainbow(np.linspace(0, 1, len(noctz_df2_grouped)))

# Create a 1xN subplot
fig, axs = plt.subplots(1, len(noctz_df2_grouped), figsize=(15, 5))

# Iterate over the grouped data
for ax, data, color, name in zip(axs, noctz_df2_grouped, colors, noctz_df2_grouped.index):
    ax.plot(data, color=color)
    ax.set_title(name)

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
### plot and save rasters and Wavefornms 
RS_SUA_df = opto_df_manager.get_filtered_data('basic_metrics', is_single_unit=1.0, cell_type='RS', stim_responsivity=None)

def iterate_and_plot(df, plot_combined_psth_and_raster, output_dir, folder_name, time_window=None, smoothing_window=10, show=False):
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Create the folder within the output directory
    save_dir = os.path.join(output_dir, folder_name)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    unique_combinations = df[['groupname', 'recordingname', 'cid']].drop_duplicates()

    for idx, row in unique_combinations.iterrows():
        groupname = row['groupname']
        recordingname = row['recordingname']
        cid = row['cid']

        fig = plot_combined_psth_and_raster(
            groupname, recordingname, cid, time_window=time_window, smoothing_window=smoothing_window, show=show
        )

        if not show:
            save_path = os.path.join(save_dir, f'plot_{groupname}_{recordingname}_{cid}.png')
            fig.savefig(save_path)
        plt.close(fig)

iterate_and_plot(RS_SUA_df, whisker_df_manager.plot_combined_psth_and_raster, '/Volumes/MannySSD/figures', 'RSSUA_opto', time_window=None, smoothing_window=10, show=False)

def iterate_and_plot_normalizedwaveforms(df, output_dir, folder_name, show=False):
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Create the folder within the output directory
    save_dir = os.path.join(output_dir, folder_name)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    unique_combinations = df[['groupname', 'recordingname', 'cid']].drop_duplicates()

    for idx, row in unique_combinations.iterrows():
        groupname = row['groupname']
        recordingname = row['recordingname']
        cid = row['cid']

        condition = (df['groupname'] == groupname) & (df['recordingname'] == recordingname) & (df['cid'] == cid)
        waveforms = df.loc[condition, 'Normalized_Template_Waveform'].values

        if waveforms.size > 0:
            waveform = waveforms[0]

            fig, ax = plt.subplots(figsize=(10, 6))
            ax.plot(waveform)
            ax.set_title(f'Normalized Template Waveform for CID: {cid}, Group: {groupname}, Recording: {recordingname}')
            ax.set_xlabel('Time')
            ax.set_ylabel('Amplitude')

            if show:
                plt.show()
            else:
                save_path = os.path.join(save_dir, f'normalized_waveform_{groupname}_{recordingname}_{cid}.svg')
                fig.savefig(save_path)
            plt.close(fig)          
def iterate_and_plot_unnormalizedwaveforms(df, output_dir, folder_name, show=False):
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Create the folder within the output directory
    save_dir = os.path.join(output_dir, folder_name)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    unique_combinations = df[['groupname', 'recordingname', 'cid']].drop_duplicates()

    for idx, row in unique_combinations.iterrows():
        groupname = row['groupname']
        recordingname = row['recordingname']
        cid = row['cid']

        condition = (df['groupname'] == groupname) & (df['recordingname'] == recordingname) & (df['cid'] == cid)
        waveforms = df.loc[condition, 'UnNormalized_Template_Waveform'].values

        if waveforms.size > 0:
            waveform = waveforms[0]

            fig, ax = plt.subplots(figsize=(10, 6))
            ax.plot(waveform)
            ax.set_title(f'UnNormalized Template Waveform for CID: {cid}, Group: {groupname}, Recording: {recordingname}')
            ax.set_xlabel('Time')
            ax.set_ylabel('Amplitude')

            if show:
                plt.show()
            else:
                save_path = os.path.join(save_dir, f'unnormalized_waveform_{groupname}_{recordingname}_{cid}.svg')
                fig.savefig(save_path)
            plt.close(fig)

iterate_and_plot_normalizedwaveforms(RS_SUA_df, '/Volumes/MannySSD/figures', 'RSSUA_opto_early_normalized_waveforms', show=False)
iterate_and_plot_unnormalizedwaveforms(RS_SUA_df, '/Volumes/MannySSD/figures', 'RSSUA_opto_early_unnormalized_waveforms', show=False)

# distribuition fun 

Understanding the KS Test:
KS Statistic: Measures the maximum distance between the empirical cumulative distribution function (CDF) of the data and the CDF of the fitted distribution. A lower KS statistic means the distribution fits the data better.
KS p-value: The p-value in the KS test tells you the probability that the observed data could have come from the fitted distribution. A higher p-value indicates that the difference between the observed and expected distribution is small, meaning the fit is good.
Why Use the Highest KS p-value?
Higher p-value = Better Fit: A higher p-value suggests that the observed data is not significantly different from the expected distribution, implying that the fitted distribution is a good match for the data.
Lower p-value = Poorer Fit: A lower p-value suggests that there is a significant difference between the observed and expected distributions, meaning the fit is not good.
Example:
If the KS p-value is 0.8, it suggests that there is an 80% chance that the data could come from the fitted distribution (a good fit).
If the KS p-value is 0.05, it suggests that there is only a 5% chance that the data could come from the fitted distribution (a poor fit).
Summary:
Select the Distribution with the Highest KS p-value: This approach ensures that the distribution that best fits the data is chosen.
Avoid Selecting the Lowest KS p-value: A low p-value indicates that the distribution is not a good fit for the dat



https://github.com/scipy/scipy/issues/11806 

https://stackoverflow.com/questions/37487830/how-to-find-probability-distribution-and-parameters-for-real-data-python-3 

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as st

def evaluate_distributions(data):
    dist_names = ["norm", "exponweib", "weibull_max", "weibull_min", "pareto", "genextreme"]
    results = []
    
    num_bins = 20
    x_range = np.linspace(-1, 1, num_bins + 1)
    
    for dist_name in dist_names:
        dist = getattr(st, dist_name)
        params = dist.fit(data)
        
        bin_counts, bin_edges = np.histogram(data, bins=x_range, density=True)
        pdf_fitted = dist.pdf((bin_edges[:-1] + bin_edges[1:]) / 2, *params[:-2], loc=params[-2], scale=params[-1])
        
        sse = np.sum((pdf_fitted - bin_counts) ** 2)
        log_likelihood = np.sum(np.log(dist.pdf(data, *params)))
        aic = 2 * len(params) - 2 * log_likelihood
        bic = len(params) * np.log(len(data)) - 2 * log_likelihood
        
        ks_statistic, ks_pvalue = st.kstest(data, dist_name, args=params)
        
        results.append({
            "distribution": dist_name,
            "sumsquare_error": sse,
            "aic": aic,
            "bic": bic,
            "ks_statistic": ks_statistic,
            "ks_pvalue": ks_pvalue,
            "params": params
        })
    
    results_df = pd.DataFrame(results).sort_values(by="ks_pvalue", ascending=False)
    return results_df

# Load your data
df = pd.read_csv('/Volumes/MannySSD/figures/laminaranalysis/basic_metrics_for_plotting_baseline_andspikes.csv')


# Initialize a list to store the results
best_fits = []

# Iterate over each combination of groupname, Cell_Type, and IsSingleUnit
for (group, cell_type, is_sua), group_data in df.groupby(['groupname', 'Cell_Type', 'IsSingleUnit']):
    
    # Extract the Modulation Index data
    data = group_data['ModulationIndex_Numeric'].dropna().values
    
    # Skip if the data is too small for analysis
    if len(data) < 5:
        continue
    
    # Evaluate the best distribution
    result = evaluate_distributions(data)
    best_fit = result.iloc[0]  # Get the best fit (highest p-value)
    
    # Store the result with metadata
    best_fits.append({
        "groupname": group,
        "Cell_Type": cell_type,
        "IsSingleUnit": is_sua,
        "best_distribution": best_fit['distribution'],
        "ks_pvalue": best_fit['ks_pvalue'],
        "aic": best_fit['aic'],
        "bic": best_fit['bic'],
        "params": best_fit['params'],
        "ModulationIndex_Numeric": list(data)  # Store the data used for fitting
    })

# Convert the results to a DataFrame
best_fits_df = pd.DataFrame(best_fits)

# Save the best fits to a CSV file
best_fits_df.to_csv('/Volumes/MannySSD/figures/laminaranalysis/bestfit_output.csv', index=False)

# Load the CSV file (if you need to reload)
best_fits_df = pd.read_csv('/Volumes/MannySSD/figures/laminaranalysis/bestfit_output.csv')

# Determine the most frequent best distribution across all combinations
best_overall_distribution = best_fits_df['best_distribution'].mode()[0]
print(f"The best overall distribution is: {best_overall_distribution}")

# Aggregate all data and fit the best overall distribution
all_data = np.concatenate(best_fits_df['ModulationIndex_Numeric'].apply(lambda x: eval(x)).values)

# Fit the best overall distribution to the aggregated data
dist = getattr(st, best_overall_distribution)
params = dist.fit(all_data)

# Generate synthetic data using the best overall distribution
synthetic_data = dist.rvs(*params, size=1000)
print(synthetic_data[:10])




In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as st

# Load the best fits DataFrame (if not already loaded)
best_fits_df = pd.read_csv('/Volumes/MannySSD/figures/laminaranalysis/bestfit_output.csv')

# Determine the most frequent best distribution across all combinations
best_overall_distribution = best_fits_df['best_distribution'].mode()[0]
print(f"The best overall distribution is: {best_overall_distribution}")

# Aggregate all ModulationIndex_Numeric data
all_data = np.concatenate(best_fits_df['ModulationIndex_Numeric'].apply(lambda x: eval(x)).values)

# Fit the best overall distribution to the aggregated data
dist = getattr(st, best_overall_distribution)
params = dist.fit(all_data)

# Print the parameters of the best overall distribution
print(f"Parameters for {best_overall_distribution}: {params}")

# Generate synthetic data using the best overall distribution
synthetic_data = dist.rvs(*params, size=1000)

# Show the first few values of the synthetic data
print("Synthetic data (first 10 values):", synthetic_data[:10])

# Optional: Save the synthetic data and parameters for further use
pd.DataFrame(synthetic_data, columns=['synthetic_data']).to_csv('/Volumes/MannySSD/figures/laminaranalysis/synthetic_data.csv', index=False)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st

# Assuming best_overall_distribution = 'weibull_max' and params are already determined
dist = getattr(st, best_overall_distribution)
params = dist.fit(all_data)

# Print the parameters of the distribution
print(f"Parameters for {best_overall_distribution}: {params}")

# Define the range of values over which to calculate the PDF
x_values = np.linspace(-1, 1, 1000)

# Calculate the PDF using the fitted parameters
pdf_values = dist.pdf(x_values, *params[:-2], loc=params[-2], scale=params[-1])

# Plot only the histogram first to check the data distribution
plt.figure(figsize=(10, 6))
plt.hist(all_data, bins=20, density=True, alpha=0.5, label='Empirical Data', color='gray')
plt.title(f'Empirical Data - Modulation Index')
plt.xlabel('Modulation Index')
plt.ylabel('Density')
plt.legend()
plt.grid(True)
plt.show()

# Now plot the PDF over the histogram
plt.figure(figsize=(10, 6))
plt.hist(all_data, bins=20, density=True, alpha=0.5, label='Empirical Data', color='gray')
plt.plot(x_values, pdf_values, label=f'{best_overall_distribution} PDF', color='blue')
plt.title(f'Empirical Data and PDF - {best_overall_distribution}')
plt.xlabel('Modulation Index')
plt.ylabel('Density')
plt.legend()
plt.grid(True)
plt.show()

# Optional: Try fitting a different distribution for comparison
alternative_dist = 'norm'  # Example: change this to any other distribution you want to test
alt_dist = getattr(st, alternative_dist)
alt_params = alt_dist.fit(all_data)
alt_pdf_values = alt_dist.pdf(x_values, *alt_params[:-2], loc=alt_params[-2], scale=alt_params[-1])

# Plot the alternative PDF
plt.figure(figsize=(10, 6))
plt.hist(all_data, bins=20, density=True, alpha=0.5, label='Empirical Data', color='gray')
plt.plot(x_values, alt_pdf_values, label=f'{alternative_dist} PDF', color='red')
plt.title(f'Empirical Data and PDF - {alternative_dist}')
plt.xlabel('Modulation Index')
plt.ylabel('Density')
plt.legend()
plt.grid(True)
plt.show()


Key Distributions to Try:
Gamma Distribution: Often used to model skewed data, could capture some of the asymmetry.
Exponential Weibull (exponweib): A flexible distribution that can model different types of data with varying skewness.
Lognormal (lognorm): Might capture the multiplicative effects in data and handle skewness better.
Conclusion:
Iterative Testing: By testing multiple distributions, you can identify which one provides the best visual and statistical fit for your data.
Parameter Analysis: Ensure that the parameters make sense given the context of your data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as st

def evaluate_distributions(data):
    dist_names = ["norm", "exponweib", "weibull_max", "weibull_min", "pareto", "genextreme"]
    results = []
    
    num_bins = 20
    x_range = np.linspace(-1, 1, num_bins + 1)
    
    for dist_name in dist_names:
        dist = getattr(st, dist_name)
        params = dist.fit(data)
        
        bin_counts, bin_edges = np.histogram(data, bins=x_range, density=True)
        pdf_fitted = dist.pdf((bin_edges[:-1] + bin_edges[1:]) / 2, *params[:-2], loc=params[-2], scale=params[-1])
        
        sse = np.sum((pdf_fitted - bin_counts) ** 2)
        log_likelihood = np.sum(np.log(dist.pdf(data, *params)))
        aic = 2 * len(params) - 2 * log_likelihood
        bic = len(params) * np.log(len(data)) - 2 * log_likelihood
        
        ks_statistic, ks_pvalue = st.kstest(data, dist_name, args=params)
        
        results.append({
            "distribution": dist_name,
            "sumsquare_error": sse,
            "aic": aic,
            "bic": bic,
            "ks_statistic": ks_statistic,
            "ks_pvalue": ks_pvalue,
            "params": params
        })
    
    results_df = pd.DataFrame(results).sort_values(by="ks_pvalue", ascending=False)
    return results_df

# Load your data
df = pd.read_csv('/Volumes/MannySSD/figures/laminaranalysis/basic_metrics_for_plotting_baseline_andspikes.csv')


# Initialize a list to store the results
best_fits = []

# Iterate over each combination of groupname, Cell_Type, and IsSingleUnit
for (group, cell_type, is_sua), group_data in df.groupby(['groupname', 'Cell_Type', 'IsSingleUnit']):
    
    # Extract the Modulation Index data
    data = group_data['ModulationIndex_Numeric'].dropna().values
    
    # Skip if the data is too small for analysis
    if len(data) < 5:
        continue
    
    # Evaluate the best distribution
    result = evaluate_distributions(data)
    best_fit = result.iloc[0]  # Get the best fit (highest p-value)
    
    # Store the result with metadata
    best_fits.append({
        "groupname": group,
        "Cell_Type": cell_type,
        "IsSingleUnit": is_sua,
        "best_distribution": best_fit['distribution'],
        "ks_pvalue": best_fit['ks_pvalue'],
        "aic": best_fit['aic'],
        "bic": best_fit['bic'],
        "params": best_fit['params'],
        "ModulationIndex_Numeric": list(data)  # Store the data used for fitting
    })

# Convert the results to a DataFrame
best_fits_df = pd.DataFrame(best_fits)

# Save the best fits to a CSV file
best_fits_df.to_csv('/Volumes/MannySSD/figures/laminaranalysis/bestfit_output.csv', index=False)

# Load the CSV file (if you need to reload)
best_fits_df = pd.read_csv('/Volumes/MannySSD/figures/laminaranalysis/bestfit_output.csv')

# Determine the most frequent best distribution across all combinations
best_overall_distribution = best_fits_df['best_distribution'].mode()[0]
print(f"The best overall distribution is: {best_overall_distribution}")

# Aggregate all data and fit the best overall distribution
all_data = np.concatenate(best_fits_df['ModulationIndex_Numeric'].apply(lambda x: eval(x)).values)

# Fit the best overall distribution to the aggregated data
dist = getattr(st, best_overall_distribution)
params = dist.fit(all_data)

# Generate synthetic data using the best overall distribution
synthetic_data = dist.rvs(*params, size=1000)
print(synthetic_data[:10])




In [ ]:
# Function to test and plot additional distributions
def plot_distribution(dist_name, all_data, x_values):
    dist = getattr(st, dist_name)
    params = dist.fit(all_data)
    
    # Print the parameters of the distribution
    print(f"Parameters for {dist_name}: {params}")
    
    # Calculate the PDF using the fitted parameters
    pdf_values = dist.pdf(x_values, *params[:-2], loc=params[-2], scale=params[-1])
    
    # Plot the PDF over the histogram
    plt.figure(figsize=(10, 6))
    plt.hist(all_data, bins=20, density=True, alpha=0.5, label='Empirical Data', color='gray')
    plt.plot(x_values, pdf_values, label=f'{dist_name} PDF', color='blue')
    plt.title(f'Empirical Data and PDF - {dist_name}')
    plt.xlabel('Modulation Index')
    plt.ylabel('Density')
    plt.legend()
    plt.grid(True)
    plt.show()

# Example of trying different distributions
distributions_to_try = ['gamma', 'exponweib', 'lognorm']  # Add more if needed

for dist_name in distributions_to_try:
    plot_distribution(dist_name, all_data, x_values)


. Gamma Distribution:
The Gamma distribution provides a moderately better fit than the previous attempts (like Weibull Max), but it still fails to capture the peaks at the extremes (around -1 and 1).
The PDF has a smooth curve but doesn’t align well with the highly skewed nature of your data at the extremes.
2. Lognormal Distribution:
Similar to the Gamma distribution, the Lognormal distribution also fails to capture the extreme peaks.
The PDF is more spread out and has a flatter curve, which suggests that it’s not matching the data's density at the extreme ends (-1 and 1).
3. Exponential Weibull (exponweib) Distribution:
The exponweib distribution shows an extreme spike at -1, which suggests that the parameters might be leading to a very poor fit.
The density value is incredibly high (in the order of 10^10), which is not realistic and indicates that the distribution is overfitting to the extreme data points.
Key Observations:
Extremes Are Poorly Fitted: None of these distributions, including gamma, lognorm, and exponweib, are capturing the behavior of the data well, especially at the extremes of the modulation index (-1 and 1).
Highly Skewed Data: Your data is heavily skewed, with significant peaks at both ends, which makes it challenging to model with standard distributions.
Next Steps:
Consider a Mixture Model:
Since your data might have two distinct modes (bimodal distribution), a mixture model that combines two or more distributions (e.g., two normal distributions) might be more appropriate.
You can use Gaussian Mixture Models (GMMs) to fit this data.
Manual Binning/Analysis:
Consider breaking down the data into smaller bins and analyzing each segment separately. This might help in fitting different distributions to different sections of the data.
Try a Custom Distribution:
If none of the standard distributions work, you might need to create a custom distribution that can better capture the specific characteristics of your data

In [ ]:
!pip install -U scikit-learn
from sklearn.mixture import GaussianMixture

# Fit a Gaussian Mixture Model with 2 components
gmm = GaussianMixture(n_components=2)
gmm.fit(all_data.reshape(-1, 1))

# Generate the GMM PDF
x_values = np.linspace(-1, 1, 1000).reshape(-1, 1)
pdf_values = np.exp(gmm.score_samples(x_values))

# Plot the GMM PDF
plt.figure(figsize=(10, 6))
plt.hist(all_data, bins=20, density=True, alpha=0.5, label='Empirical Data', color='gray')
plt.plot(x_values, pdf_values, label='GMM PDF', color='green')
plt.title('Empirical Data and GMM PDF')
plt.xlabel('Modulation Index')
plt.ylabel('Density')
plt.legend()
plt.grid(True)
plt.show()


 seems that the Gaussian Mixture Model (GMM) is also struggling to fit the data well, especially around the extremes. The GMM PDF shows a very high density at -1, but it doesn't capture the rest of the distribution effectively.

Observations:
Overfitting to Extremes: The GMM appears to be overfitting to the extreme values (particularly -1), leading to a sharp spike that doesn’t reflect the true spread of the data.
Flat Regions: The GMM fails to capture the distribution in the middle range of the modulation index values, leading to a near-zero density across most of the plot.
Possible Reasons:
Highly Skewed Data: Your data's extreme skewness and the presence of strong peaks at both ends (-1 and 1) make it difficult for typical parametric models, including GMMs, to fit the data well.
Bimodal or Multimodal Nature: If your data is truly bimodal or even multimodal, the GMM might need more than two components, or it might require a more complex mixture model.
Next Steps:
Increase the Number of Components in GMM:
Try fitting the GMM with more components (e.g., 3 or 4) to see if it can better capture the multimodal nature of your data.
Non-Parametric Methods:
Consider using non-parametric methods like Kernel Density Estimation (KDE) to estimate the PDF. KDE does not assume any specific parametric form, making it more flexible for complex data distributions.
Manual Segmentation:
Manually segment the data into different regions and fit different distributions to each segment. This might help in better understanding and modeling the data.
Example: Fitting GMM with More Components
Here’s how you can try fitting a GMM with more components:

In [ ]:
from sklearn.mixture import GaussianMixture

# Fit a Gaussian Mixture Model with more components (e.g., 3 components)
gmm = GaussianMixture(n_components=3)
gmm.fit(all_data.reshape(-1, 1))

# Generate the GMM PDF
x_values = np.linspace(-1, 1, 1000).reshape(-1, 1)
pdf_values = np.exp(gmm.score_samples(x_values))

# Plot the GMM PDF
plt.figure(figsize=(10, 6))
plt.hist(all_data, bins=20, density=True, alpha=0.5, label='Empirical Data', color='gray')
plt.plot(x_values, pdf_values, label='GMM PDF', color='green')
plt.title('Empirical Data and GMM PDF (3 Components)')
plt.xlabel('Modulation Index')
plt.ylabel('Density')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
from sklearn.neighbors import KernelDensity

# Fit Kernel Density Estimation (KDE)
kde = KernelDensity(kernel='gaussian', bandwidth=0.1).fit(all_data.reshape(-1, 1))

# Generate the KDE PDF
pdf_values = np.exp(kde.score_samples(x_values))

# Plot the KDE PDF
plt.figure(figsize=(10, 6))
plt.hist(all_data, bins=20, density=True, alpha=0.5, label='Empirical Data', color='gray')
plt.plot(x_values, pdf_values, label='KDE PDF', color='purple')
plt.title('Empirical Data and KDE PDF')
plt.xlabel('Modulation Index')
plt.ylabel('Density')
plt.legend()
plt.grid(True)
plt.show()


Observations:
GMM with 3 Components:
The Gaussian Mixture Model (GMM) with 3 components still struggles to capture the distribution effectively. The density is extremely high at -1, but it fails to model the rest of the data properly.
This suggests that the data's structure might be too complex or too heavily skewed for a simple GMM to capture, even with additional components.
Kernel Density Estimation (KDE):
The KDE approach provides a much better fit to the data. The KDE curve captures the bimodal nature of the data, with peaks around -1 and 1, and it follows the distribution more closely across the entire range.
This method is more flexible as it does not assume a specific parametric form, which allows it to adapt to the data's shape more effectively.
Key Insights:
GMM Limitations: The GMM, even with additional components, is not flexible enough to capture the heavy skewness and extreme peaks present in your data.
KDE Strength: The KDE method shows clear advantages in this case, as it is non-parametric and can adapt to the data's unique characteristics without being constrained by a predefined distribution form.
Next Steps:
Consider Using KDE for Your Analysis:
Given the results, KDE might be the best method for estimating the Probability Density Function (PDF) of your data.
This approach is particularly useful when the data does not follow a common distribution or when it exhibits complex multimodal behavior.
Fine-Tune KDE:
You can adjust the bandwidth parameter in KDE to see how different levels of smoothing affect the PDF.
A smaller bandwidth might capture more detailed features, while a larger bandwidth will provide a smoother curve.
Apply KDE to Subsets:
If your data contains different subgroups (e.g., by treatment or cell type), you might apply KDE to each subgroup to understand the differences in distribution more clearly.
Use KDE for Further Modeling:
You can use the KDE-derived PDF in subsequent analyses or simulations, especially if you need to generate synthetic data or compute probabilities based on your observations

o proceed with the 2x2 comparison as before, we'll follow these steps:

Steps:
Apply KDE to Each Subgroup:
We'll apply Kernel Density Estimation (KDE) to each combination of Cell_Type (FS or RS) and Treatment (CTZ or No_CTZ).
Plot the Distributions:
We'll create a 2x2 grid of plots where each subplot corresponds to one combination of Cell_Type and Treatment.
The KDE curve for each subgroup will be plotted along with the histogram of the data.
Overlay the Distributions:
For easier comparison, we'll overlay the KDE distributions of CTZ vs. No_CTZ within the same plot for each Cell_Type.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KernelDensity

# Load your data
df = pd.read_csv('/Volumes/MannySSD/figures/laminaranalysis/basic_metrics_for_plotting_baseline_andspikes.csv')

# Define the groups
cell_types = ['FS', 'RS']
treatments = ['CTZ', 'No_CTZ']

# Prepare the 2x2 plot
fig, axs = plt.subplots(2, 2, figsize=(14, 10))

# Iterate over each combination of cell type and treatment
for i, cell_type in enumerate(cell_types):
    for j, treatment in enumerate(treatments):
        
        # Filter the data for the current combination
        data = df[(df['Cell_Type'] == cell_type) & (df['groupname'] == treatment)]['ModulationIndex_Numeric'].dropna().values
        
        # Fit KDE
        kde = KernelDensity(kernel='gaussian', bandwidth=0.1).fit(data.reshape(-1, 1))
        x_values = np.linspace(-1, 1, 1000).reshape(-1, 1)
        pdf_values = np.exp(kde.score_samples(x_values))
        
        # Plot the histogram and KDE for this group
        axs[i, j].hist(data, bins=20, density=True, alpha=0.5, label=f'{treatment} Data', color='gray')
        axs[i, j].plot(x_values, pdf_values, label=f'{treatment} KDE', color='purple')
        axs[i, j].set_title(f'{cell_type} Cells - {treatment}')
        axs[i, j].set_xlabel('Modulation Index')
        axs[i, j].set_ylabel('Density')
        axs[i, j].legend()
        axs[i, j].grid(True)

# Adjust the layout
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KernelDensity

# Load your data
df = pd.read_csv('/Volumes/MannySSD/figures/laminaranalysis/basic_metrics_for_plotting_baseline_andspikes.csv')

# Define the groups
cell_types = ['FS', 'RS']
is_single_unit_types = [1.0, 0.0]  # 1.0 for SUA, 0.0 for MUA
treatments = ['CTZ', 'No_CTZ']

# Define colors for each treatment
colors = {'CTZ': 'purple', 'No_CTZ': 'gray'}

# Prepare the 2x2 plot with shared y-axis
fig, axs = plt.subplots(2, 2, figsize=(14, 10), sharey=True)
plt.subplots_adjust(hspace=0.3, wspace=0.2)

# Iterate over each combination of cell type and unit type (SUA or MUA)
for i, cell_type in enumerate(cell_types):
    for j, is_sua in enumerate(is_single_unit_types):
        ax = axs[i, j]
        
        # Iterate over each treatment to overlay them on the same plot
        for treatment in treatments:
            # Filter the data for the current combination
            data = df[(df['Cell_Type'] == cell_type) & (df['IsSingleUnit'] == is_sua) & (df['groupname'] == treatment)]['ModulationIndex_Numeric'].dropna().values
            
            # Fit KDE
            kde = KernelDensity(kernel='gaussian', bandwidth=0.1).fit(data.reshape(-1, 1))
            x_values = np.linspace(-1, 1, 1000).reshape(-1, 1)
            pdf_values = np.exp(kde.score_samples(x_values))
            
            # Plot the histogram and KDE for this group
            ax.hist(data, bins=20, density=True, alpha=0.7, label=f'{treatment} (N={len(data)})', color=colors[treatment], edgecolor='black')
            ax.plot(x_values, pdf_values, color=colors[treatment], linewidth=2)
        
        # Set title and labels
        cell_label = 'SUA' if is_sua == 1.0 else 'MUA'
        ax.set_title(f'{cell_type} {cell_label} Cells - Normalized Distribution (PDF)', fontsize=12)
        ax.set_xlabel('Modulation Index', fontsize=10)
        ax.set_ylabel('Density', fontsize=10)
        ax.legend()
        ax.grid(True, linestyle='--', alpha=0.6)
        ax.set_ylim(0, 6)  # Adjust this value to fit your data range

# Adjust the layout and show the plot
plt.tight_layout()
plt.show()


In [ ]:
from scipy.stats import ks_2samp

# Iterate over each combination of cell type and unit type (SUA or MUA)
results = []
for cell_type in cell_types:
    for is_sua in is_single_unit_types:
        # Filter the data for the two treatments
        data_ctz = df[(df['Cell_Type'] == cell_type) & (df['IsSingleUnit'] == is_sua) & (df['groupname'] == 'CTZ')]['ModulationIndex_Numeric'].dropna().values
        data_no_ctz = df[(df['Cell_Type'] == cell_type) & (df['IsSingleUnit'] == is_sua) & (df['groupname'] == 'No_CTZ')]['ModulationIndex_Numeric'].dropna().values
        
        # Perform KS test
        ks_statistic, p_value = ks_2samp(data_ctz, data_no_ctz)
        
        # Store results
        cell_label = 'SUA' if is_sua == 1.0 else 'MUA'
        results.append({
            "Cell_Type": cell_type,
            "Unit_Type": cell_label,
            "KS_Statistic": ks_statistic,
            "P_Value": p_value
        })

# Convert results to a DataFrame for easier viewing
results_df = pd.DataFrame(results)
print(results_df)


nterpretation of the KS Test Results:
FS SUA (Fast-Spiking Single Unit Activity)
KS Statistic: 0.318182
P-Value: 0.428730
Interpretation: There is no statistically significant difference in the distribution of the Modulation Index between the CTZ and No_CTZ groups for FS SUA cells. The p-value is well above 0.05, indicating that any observed differences could be due to random chance.
FS MUA (Fast-Spiking Multi-Unit Activity)
KS Statistic: 0.223077
P-Value: 0.672240
Interpretation: Similar to FS SUA, there is no statistically significant difference in the Modulation Index distributions for FS MUA cells between the CTZ and No_CTZ groups. The high p-value suggests no meaningful difference between the two distributions.
RS SUA (Regular-Spiking Single Unit Activity)
KS Statistic: 0.244061
P-Value: 0.026209
Interpretation: There is a statistically significant difference in the Modulation Index distributions for RS SUA cells between the CTZ and No_CTZ groups. The p-value is below 0.05, indicating that the distributions are likely different, and this difference is unlikely to be due to random variation.
RS MUA (Regular-Spiking Multi-Unit Activity)
KS Statistic: 0.207071
P-Value: 0.486006
Interpretation: No statistically significant difference is observed between the CTZ and No_CTZ groups for RS MUA cells. The p-value is well above 0.05, suggesting no meaningful difference in the distributions

Explanation:
Filtering the Data:
The DataFrame filtered_df is created by filtering df to include only sensory-responsive cells (StimResponsivity == 1) and Modulation Index values greater than or equal to ±0.3 (ModulationIndex_Numeric.abs() >= 0.3).
Running the KS Test:
After filtering the data, the script performs the KS test for each combination of Cell_Type, IsSingleUnit, and groupname (CTZ vs. No_CTZ). The results are stored and then printed.
Plotting:
The histograms and KDE plots are generated for each subgroup, similar to the previous approach, but now only include the filtered data.
The histograms and KDE plots are generated for each subgroup, similar to the previous approach, but now only include the filtered data.

In [ ]:
## this assumes you aready have ran this 

df = whisker_df_manager.dataframes['basic_metrics']
#exposrt and save the dataframe but onluy for the columns that are needed for the plotting: groupname, recordingname, cid, Cell_Type, IsSingleUnit, StimResponsivity, ModulationIndex, ModulationIndex_Numeric, save the dataframe as a csv file in my directory/Volumes/MannySSD/figures/laminaranalysis/basic_metrics_for_plotting_baseline_andspikes.csv
df[['groupname', 'recordingname', 'cid', 'Cell_Type', 'IsSingleUnit', 'StimResponsivity', 'ModulationIndex', 'ModulationIndex_Numeric', 'MeanFR_baseline', 'SpikeTimes_all']].to_csv('/Volumes/MannySSD/figures/laminaranalysis/basic_metrics_for_plotting_baseline_andspikes.csv', index=False)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KernelDensity
from scipy.stats import ks_2samp

# Load your data
df = pd.read_csv('/Volumes/MannySSD/figures/laminaranalysis/basic_metrics_for_plotting_baseline_andspikes.csv')

# Define the groups
cell_types = ['FS', 'RS']
is_single_unit_types = [1.0, 0.0]  # 1.0 for SUA, 0.0 for MUA
treatments = ['CTZ', 'No_CTZ']

# Apply filter for sensory responsive cells and modulation index greater than or equal to ±0.3
filtered_df = df[(df['StimResponsivity'] == 1) & (df['ModulationIndex_Numeric'].abs() >= 0.3)]

# Prepare to store KS test results
results = []

# Prepare the 2x2 plot with shared y-axis
fig, axs = plt.subplots(2, 2, figsize=(14, 10), sharey=True)
plt.subplots_adjust(hspace=0.3, wspace=0.2)

# Iterate over each combination of cell type and unit type (SUA or MUA)
for i, cell_type in enumerate(cell_types):
    for j, is_sua in enumerate(is_single_unit_types):
        ax = axs[i, j]
        
        # Iterate over each treatment to overlay them on the same plot
        for treatment in treatments:
            # Filter the data for the current combination
            data = filtered_df[(filtered_df['Cell_Type'] == cell_type) & (filtered_df['IsSingleUnit'] == is_sua) & (filtered_df['groupname'] == treatment)]['ModulationIndex_Numeric'].dropna().values
            
            # Fit KDE
            kde = KernelDensity(kernel='gaussian', bandwidth=0.1).fit(data.reshape(-1, 1))
            x_values = np.linspace(-1, 1, 1000).reshape(-1, 1)
            pdf_values = np.exp(kde.score_samples(x_values))
            
            # Plot the histogram and KDE for this group
            ax.hist(data, bins=20, density=True, alpha=0.7, label=f'{treatment} (N={len(data)})', color=colors[treatment], edgecolor='black')
            ax.plot(x_values, pdf_values, color=colors[treatment], linewidth=2)
        
        # Set title and labels
        cell_label = 'SUA' if is_sua == 1.0 else 'MUA'
        ax.set_title(f'{cell_type} {cell_label} Cells - Normalized Distribution (PDF)', fontsize=12)
        ax.set_xlabel('Modulation Index', fontsize=10)
        ax.set_ylabel('Density', fontsize=10)
        ax.legend()
        ax.grid(True, linestyle='--', alpha=0.6)
        ax.set_ylim(0, 6)  # Adjust this value to fit your data range
        
        # Perform KS test between CTZ and No_CTZ for this group
        data_ctz = filtered_df[(filtered_df['Cell_Type'] == cell_type) & (filtered_df['IsSingleUnit'] == is_sua) & (filtered_df['groupname'] == 'CTZ')]['ModulationIndex_Numeric'].dropna().values
        data_no_ctz = filtered_df[(filtered_df['Cell_Type'] == cell_type) & (filtered_df['IsSingleUnit'] == is_sua) & (filtered_df['groupname'] == 'No_CTZ')]['ModulationIndex_Numeric'].dropna().values
        
        ks_statistic, p_value = ks_2samp(data_ctz, data_no_ctz)
        
        results.append({
            "Cell_Type": cell_type,
            "Unit_Type": cell_label,
            "KS_Statistic": ks_statistic,
            "P_Value": p_value
        })

# Convert results to a DataFrame for easier viewing
results_df = pd.DataFrame(results)
print(results_df)

# Adjust the layout and show the plot
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
from sklearn.neighbors import KernelDensity
from scipy.stats import ks_2samp

# Load your data
df = pd.read_csv('/Volumes/MannySSD/figures/laminaranalysis/basic_metrics_for_plotting_baseline_andspikes.csv')

# Define the groups and criteria
cell_types = ['FS', 'RS']
is_single_unit_types = [1.0, 0.0]  # 1.0 for SUA, 0.0 for MUA
treatments = ['CTZ', 'No_CTZ']

# Apply filter for sensory responsive cells and modulation index greater than or equal to ±0.3
filtered_df = df[(df['StimResponsivity'] == 1) & (df['ModulationIndex_Numeric'].abs() >= 0.3)]

# Initialize results storage
best_fits = []
ks_results = []

# Iterate over each combination of cell type and unit type (SUA or MUA)
for cell_type in cell_types:
    for is_sua in is_single_unit_types:
        
        # Filter the data for the current combination
        data_ctz = filtered_df[(filtered_df['Cell_Type'] == cell_type) & (filtered_df['IsSingleUnit'] == is_sua) & (filtered_df['groupname'] == 'CTZ')]['ModulationIndex_Numeric'].dropna().values
        data_no_ctz = filtered_df[(filtered_df['Cell_Type'] == cell_type) & (filtered_df['IsSingleUnit'] == is_sua) & (filtered_df['groupname'] == 'No_CTZ')]['ModulationIndex_Numeric'].dropna().values
        
        # Skip if data is too small for analysis
        if len(data_ctz) < 5 or len(data_no_ctz) < 5:
            continue
        
        # Perform distribution fitting and evaluation
        for data, treatment in zip([data_ctz, data_no_ctz], treatments):
            dist_names = ["norm", "exponweib", "weibull_max", "weibull_min", "pareto", "genextreme"]
            results = []

            for dist_name in dist_names:
                dist = getattr(st, dist_name)
                params = dist.fit(data)
                
                # Histogram bins
                num_bins = 10
                x_values = np.linspace(-1, 1, num_bins + 1)
                
                # Calculate PDF
                bin_counts, bin_edges = np.histogram(data, bins=x_values, density=True)
                pdf_fitted = dist.pdf((bin_edges[:-1] + bin_edges[1:]) / 2, *params[:-2], loc=params[-2], scale=params[-1])
                
                # Calculate goodness-of-fit metrics
                sse = np.sum((pdf_fitted - bin_counts) ** 2)
                log_likelihood = np.sum(np.log(dist.pdf(data, *params)))
                aic = 2 * len(params) - 2 * log_likelihood
                bic = len(params) * np.log(len(data)) - 2 * log_likelihood
                ks_statistic, ks_pvalue = st.kstest(data, dist_name, args=params)
                
                results.append({
                    "distribution": dist_name,
                    "sumsquare_error": sse,
                    "aic": aic,
                    "bic": bic,
                    "ks_statistic": ks_statistic,
                    "ks_pvalue": ks_pvalue,
                    "params": params
                })
            
            results_df = pd.DataFrame(results).sort_values(by="ks_pvalue", ascending=False)
            best_fit = results_df.iloc[0]  # Get the best fit (highest p-value)
            
            # Store best fit results
            cell_label = 'SUA' if is_sua == 1.0 else 'MUA'
            best_fits.append({
                "groupname": treatment,
                "Cell_Type": cell_type,
                "IsSingleUnit": cell_label,
                "best_distribution": best_fit['distribution'],
                "ks_pvalue": best_fit['ks_pvalue'],
                "aic": best_fit['aic'],
                "bic": best_fit['bic'],
                "params": best_fit['params']
            })
        
        # Perform KS test
        ks_statistic, p_value = ks_2samp(data_ctz, data_no_ctz)
        ks_results.append({
            "Cell_Type": cell_type,
            "Unit_Type": cell_label,
            "KS_Statistic": ks_statistic,
            "P_Value": p_value
        })

# Convert the results to DataFrames
best_fits_df = pd.DataFrame(best_fits)
ks_results_df = pd.DataFrame(ks_results)

# Save the best fits and KS results to CSV files
best_fits_df.to_csv('/Volumes/MannySSD/figures/laminaranalysis/bestfit_output_filtered.csv', index=False)
ks_results_df.to_csv('/Volumes/MannySSD/figures/laminaranalysis/ks_results_filtered.csv', index=False)

# Plot the 2x2 grid
colors = {'CTZ': 'purple', 'No_CTZ': 'gray'}
fig, axs = plt.subplots(2, 2, figsize=(14, 10), sharey=True)
plt.subplots_adjust(hspace=0.3, wspace=0.2)

# Iterate over each combination for plotting
for i, cell_type in enumerate(cell_types):
    for j, is_sua in enumerate(is_single_unit_types):
        ax = axs[i, j]
        
        for treatment in treatments:
            data = filtered_df[(filtered_df['Cell_Type'] == cell_type) & (filtered_df['IsSingleUnit'] == is_sua) & (filtered_df['groupname'] == treatment)]['ModulationIndex_Numeric'].dropna().values
            
            kde = KernelDensity(kernel='gaussian', bandwidth=0.1).fit(data.reshape(-1, 1))
            x_values = np.linspace(-1, 1, 1000).reshape(-1, 1)
            pdf_values = np.exp(kde.score_samples(x_values))
            
            ax.hist(data, bins=20, density=True, alpha=0.7, label=f'{treatment} (N={len(data)})', color=colors[treatment], edgecolor='black')
            ax.plot(x_values, pdf_values, color=colors[treatment], linewidth=2)
        
        cell_label = 'SUA' if is_sua == 1.0 else 'MUA'
        ax.set_title(f'{cell_type} {cell_label} Cells - Normalized Distribution (PDF)', fontsize=12)
        ax.set_xlabel('Modulation Index', fontsize=10)
        ax.set_ylabel('Density', fontsize=10)
        ax.legend()
        ax.grid(True, linestyle='--', alpha=0.6)
        ax.set_ylim(0, 6)  # Adjust to fit data range

# Final adjustments and show the plot
plt.tight_layout()
plt.show()


o run the statistical comparisons between the CTZ and No_CTZ groups for each combination of Cell_Type and IsSingleUnit under the current filtering criteria (sensory responsive cells with a Modulation Index of ±0.3 or greater), I will use the Mann-Whitney U test, which is a non-parametric test suitable for comparing two independent samples.

how the comparisons will be conducted:

FS SUA Cells: Compare CTZ vs No_CTZ.
FS MUA Cells: Compare CTZ vs No_CTZ.
RS SUA Cells: Compare CTZ vs No_CTZ.
RS MUA Cells: Compare CTZ vs No_CTZ.



1. FS SUA Cells: CTZ vs No_CTZ
Null Hypothesis: The distributions of Modulation Index values for FS SUA cells in the CTZ and No_CTZ groups are the same.
Alternative Hypothesis: The distributions are different.
2. FS MUA Cells: CTZ vs No_CTZ
Null Hypothesis: The distributions of Modulation Index values for FS MUA cells in the CTZ and No_CTZ groups are the same.
Alternative Hypothesis: The distributions are different.
3. RS SUA Cells: CTZ vs No_CTZ
Null Hypothesis: The distributions of Modulation Index values for RS SUA cells in the CTZ and No_CTZ groups are the same.
Alternative Hypothesis: The distributions are different.
4. RS MUA Cells: CTZ vs No_CTZ
Null Hypothesis: The distributions of Modulation Index values for RS MUA cells in the CTZ and No_CTZ groups are the same.
Alternative Hypothesis: The distributions are different.
I will execute the Mann-Whitney U tests for these comparisons now

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KernelDensity

# Load the filtered data
df_filtered = pd.read_csv('/Volumes/MannySSD/figures/laminaranalysis/basic_metrics_for_plotting_baseline_andspikes.csv')


# Apply the additional filter for StimResponsivity = 1.0 and ModulationIndex_Numeric >= 0.3
df_filtered = df_filtered[(df_filtered['StimResponsivity'] == 1.0) & 
                          (df_filtered['ModulationIndex_Numeric'].abs() >= 0.3)]

# Set a shared y-axis limit based on the maximum density value observed in the data
y_max = max([np.max(np.histogram(df_filtered[(df_filtered['Cell_Type'] == cell_type) & 
                                             (df_filtered['groupname'] == treatment)]
                                             ['ModulationIndex_Numeric'].dropna().values, bins=20, density=True)[0])
             for cell_type in cell_types for treatment in treatments])

# Define the groups
cell_types = ['FS', 'RS']
treatments = ['CTZ', 'No_CTZ']

# Re-plot with overlaid data and color-coded matching
fig, axs = plt.subplots(2, 2, figsize=(14, 10))

# Colors for the two groups
colors = {'CTZ': 'purple', 'No_CTZ': 'gray'}

# Iterate over each combination of cell type and treatment
for i, cell_type in enumerate(cell_types):
    for j, treatment in enumerate(treatments):
        
        # Filter the data for the current combination
        data_ctz = df_filtered[(df_filtered['Cell_Type'] == cell_type) & 
                               (df_filtered['groupname'] == 'CTZ')]['ModulationIndex_Numeric'].dropna().values
        
        data_no_ctz = df_filtered[(df_filtered['Cell_Type'] == cell_type) & 
                                  (df_filtered['groupname'] == 'No_CTZ')]['ModulationIndex_Numeric'].dropna().values
        
        # Fit KDE for both groups
        if len(data_ctz) > 0 and len(data_no_ctz) > 0:
            kde_ctz = KernelDensity(kernel='gaussian', bandwidth=0.1).fit(data_ctz.reshape(-1, 1))
            kde_no_ctz = KernelDensity(kernel='gaussian', bandwidth=0.1).fit(data_no_ctz.reshape(-1, 1))
            
            x_values = np.linspace(-1, 1, 1000).reshape(-1, 1)
            pdf_values_ctz = np.exp(kde_ctz.score_samples(x_values))
            pdf_values_no_ctz = np.exp(kde_no_ctz.score_samples(x_values))
            
            # Plot the histograms and KDEs for both groups
            axs[i, j].hist(data_ctz, bins=20, density=True, alpha=0.5, label=f'CTZ Data (N={len(data_ctz)})', color=colors['CTZ'])
            axs[i, j].hist(data_no_ctz, bins=20, density=True, alpha=0.5, label=f'No_CTZ Data (N={len(data_no_ctz)})', color=colors['No_CTZ'])
            axs[i, j].plot(x_values, pdf_values_ctz, label='CTZ KDE', color=colors['CTZ'])
            axs[i, j].plot(x_values, pdf_values_no_ctz, label='No_CTZ KDE', color=colors['No_CTZ'])
            axs[i, j].set_title(f'{cell_type} Cells')
            axs[i, j].set_xlabel('Modulation Index')
            axs[i, j].set_ylabel('Density')
            axs[i, j].set_ylim(0, y_max)  # Set the shared y-axis limit
            axs[i, j].legend()
            axs[i, j].grid(True)

# Adjust the layout
plt.tight_layout()
plt.show()


# Plot cell density and cell distribution on probe with stats 

### plot and runs stats on IHC data

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mannwhitneyu
from scipy.stats import sem, t

def calculate_confidence_interval(data, confidence=0.95):
    n = len(data)
    m = np.mean(data)
    se = sem(data)
    h = se * t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

def plot_and_save_all_combinations(data, save_dir):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    # Regions and cell types to iterate over
    regions = ['PFC', 'BC']
    cell_types = ['SST', 'PV']
    
    # Prepare to store statistics
    stats_data = []

    for region in regions:
        for cell_type in cell_types:
            # Filter the data
            filtered_data = data[(data['Region'] == region) & (data['Cell'] == cell_type)]
            
            if filtered_data.empty:
                continue

            # Separate CTZ and VEH data
            ctz_data = filtered_data[filtered_data['Group'] == 'CTZ']
            veh_data = filtered_data[filtered_data['Group'] == 'VEH']

            # Calculate statistics for both groups
            ctz_summary = ctz_data['Count'].agg(['mean', 'median', 'std', 'min', 'max', 'count'])
            veh_summary = veh_data['Count'].agg(['mean', 'median', 'std', 'min', 'max', 'count'])
            
            # Calculate confidence intervals
            ctz_ci = calculate_confidence_interval(ctz_data['Count'])
            veh_ci = calculate_confidence_interval(veh_data['Count'])
            
            # Perform a Mann-Whitney U test
            stat, p_value = mannwhitneyu(ctz_data['Count'], veh_data['Count'])
            
            # Save statistics
            stats_data.extend([
                {'Region': region, 'Cell_Type': cell_type, 'Group': 'CTZ', 'N': ctz_summary['count'], 'Mean': ctz_summary['mean'],
                 'Median': ctz_summary['median'], 'CI_Lower': ctz_ci[1], 'CI_Upper': ctz_ci[2], 'Min': ctz_summary['min'],
                 'Max': ctz_summary['max'], 'P_Value': p_value},
                
                {'Region': region, 'Cell_Type': cell_type, 'Group': 'VEH', 'N': veh_summary['count'], 'Mean': veh_summary['mean'],
                 'Median': veh_summary['median'], 'CI_Lower': veh_ci[1], 'CI_Upper': veh_ci[2], 'Min': veh_summary['min'],
                 'Max': veh_summary['max'], 'P_Value': p_value}
            ])
            
            # Plotting
            plt.figure(figsize=(6, 4))
            
            # Bar plot for CTZ
            plt.bar(x=0, height=ctz_summary['mean'], yerr=ctz_summary['std']/np.sqrt(ctz_summary['count']),
                    color='purple', alpha=0.7, capsize=5, width=0.4)
            
            # Scatter plot for CTZ
            plt.scatter(np.zeros(len(ctz_data)) - 0.2, ctz_data['Count'], color='purple', edgecolor='black', s=50, alpha=0.8)
            
            # Bar plot for VEH
            plt.bar(x=1, height=veh_summary['mean'], yerr=veh_summary['std']/np.sqrt(veh_summary['count']),
                    color='grey', alpha=0.7, capsize=5, width=0.4)
            
            # Scatter plot for VEH
            plt.scatter(np.ones(len(veh_data)) + 0.2, veh_data['Count'], color='grey', edgecolor='black', s=50, alpha=0.8)
            
            # Adding labels and title
            plt.xticks([0, 1], ['CTZ', 'VEH'])
            plt.ylabel('Cell Count')
            plt.title(f'{cell_type} Cells in {region}')
            
            # Setting y-axis limits and ticks
            plt.ylim(0, 1500)
            plt.yticks([0, 500, 1000, 1500])
            
            # Save plot
            plot_filename = f"{cell_type}_{region}_Cell_Count.svg"
            plot_path = os.path.join(save_dir, plot_filename)
            plt.savefig(plot_path, format='svg')
            plt.close()

    # Convert stats data to DataFrame and save as CSV
    stats_df = pd.DataFrame(stats_data)
    stats_filename = os.path.join(save_dir, 'cell_count_statistics.csv')
    stats_df.to_csv(stats_filename, index=False)

# Example usage
data_path = '/Volumes/MannySSD/figures/thesis_final_figs/CellCountData_Final.csv'
save_directory = '/Volumes/MannySSD/figures/thesis_final_figs/cell_count_analysis'
cell_count_data = pd.read_csv(data_path)

plot_and_save_all_combinations(cell_count_data, save_directory)


### plots and analyzes the cell counts but with ephys related stuff

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency, ttest_ind, mannwhitneyu, sem, t
import numpy as np
import os

def calculate_confidence_interval(data, confidence=0.95):
    """Calculate confidence interval for a data series."""
    n = len(data)
    mean = np.mean(data)
    std_err = sem(data)
    h = std_err * t.ppf((1 + confidence) / 2., n-1)
    return mean - h, mean + h

def analyze_and_plot_data(data_path, save_directory):
    # Load the data
    df = pd.read_csv(data_path)
    
    # Explore the basic structure and some key statistics
    print(df.info())
    print(df.describe())

    # Check unique values in important columns
    unique_values = {
        'groupname': df['groupname'].unique(),
        'Cell_Type': df['Cell_Type'].unique(),
        'IsSingleUnit': df['IsSingleUnit'].unique(),
        'StimResponsivity': df['StimResponsivity'].unique()
    }
    print(f"Unique values in important columns: {unique_values}")
    
    # Ensure the save directory exists
    os.makedirs(save_directory, exist_ok=True)
    
    # Plotting the proportion of FS vs RS in CTZ and No_CTZ groups
    plt.figure(figsize=(10, 6))
    sns.countplot(data=df, x='groupname', hue='Cell_Type')
    plt.title('Proportion of FS vs RS in CTZ and No_CTZ Groups')
    plt.ylabel('Cell Count')
    plt.xlabel('Group')
    plt.legend(title='Cell Type')
    plt.savefig(os.path.join(save_directory, 'fs_vs_rs_proportion.png'))
    plt.close()

    # Plotting the proportion of Single-Unit vs Multi-Unit in CTZ and No_CTZ groups
    plt.figure(figsize=(10, 6))
    sns.countplot(data=df, x='groupname', hue='IsSingleUnit')
    plt.title('Proportion of Single-Unit vs Multi-Unit in CTZ and No_CTZ Groups')
    plt.ylabel('Unit Count')
    plt.xlabel('Group')
    plt.legend(title='Is Single Unit', labels=['Multi-Unit', 'Single-Unit'])
    plt.savefig(os.path.join(save_directory, 'su_vs_mu_proportion.png'))
    plt.close()

    # Plotting the distribution of Modulation Index by Cell Type and Group
    plt.figure(figsize=(12, 8))
    sns.violinplot(data=df, x='groupname', y='ModulationIndex_Numeric', hue='Cell_Type', split=True)
    plt.title('Distribution of Modulation Index by Cell Type and Group')
    plt.ylabel('Modulation Index (Numeric)')
    plt.xlabel('Group')
    plt.legend(title='Cell Type')
    plt.savefig(os.path.join(save_directory, 'modulation_index_distribution.png'))
    plt.close()

    # Descriptive statistics and statistical analysis
    results = []

    for group in df['groupname'].unique():
        for cell_type in df['Cell_Type'].unique():
            subset = df[(df['groupname'] == group) & (df['Cell_Type'] == cell_type)]['ModulationIndex_Numeric'].dropna()
            if len(subset) > 1:
                mean = subset.mean()
                median = subset.median()
                std_dev = subset.std()
                min_val = subset.min()
                max_val = subset.max()
                range_val = max_val - min_val
                ci_low, ci_high = calculate_confidence_interval(subset)

                results.append({
                    'Group': group,
                    'Cell_Type': cell_type,
                    'N': len(subset),
                    'Mean': mean,
                    'Median': median,
                    'CI_Low': ci_low,
                    'CI_High': ci_high,
                    'Range': range_val,
                    'SD': std_dev
                })

    # Converting descriptive stats to DataFrame
    descriptive_stats_df = pd.DataFrame(results)

    # Chi-square tests
    fs_rs_contingency = pd.crosstab(df['groupname'], df['Cell_Type'])
    su_mu_contingency = pd.crosstab(df['groupname'], df['IsSingleUnit'])

    chi2_fs_rs, p_fs_rs, dof_fs_rs, ex_fs_rs = chi2_contingency(fs_rs_contingency)
    chi2_su_mu, p_su_mu, dof_su_mu, ex_su_mu = chi2_contingency(su_mu_contingency)

    # T-tests for Modulation Index (FS vs RS)
    fs_modulation_ctz = df[(df['groupname'] == 'CTZ') & (df['Cell_Type'] == 'FS')]['ModulationIndex_Numeric'].dropna()
    rs_modulation_ctz = df[(df['groupname'] == 'CTZ') & (df['Cell_Type'] == 'RS')]['ModulationIndex_Numeric'].dropna()

    fs_modulation_noctz = df[(df['groupname'] == 'No_CTZ') & (df['Cell_Type'] == 'FS')]['ModulationIndex_Numeric'].dropna()
    rs_modulation_noctz = df[(df['groupname'] == 'No_CTZ') & (df['Cell_Type'] == 'RS')]['ModulationIndex_Numeric'].dropna()

    t_test_ctz = ttest_ind(fs_modulation_ctz, rs_modulation_ctz, nan_policy='omit')
    t_test_noctz = ttest_ind(fs_modulation_noctz, rs_modulation_noctz, nan_policy='omit')

    # Mann-Whitney U tests (non-parametric alternative)
    mw_test_ctz = mannwhitneyu(fs_modulation_ctz, rs_modulation_ctz)
    mw_test_noctz = mannwhitneyu(fs_modulation_noctz, rs_modulation_noctz)

    # Adding statistical test results to descriptive statistics
    statistical_results = {
        "Chi2_FS_RS": {"statistic": chi2_fs_rs, "p_value": p_fs_rs},
        "Chi2_SingleUnit_MultiUnit": {"statistic": chi2_su_mu, "p_value": p_su_mu},
        "TTest_ModulationIndex_CTz": {"statistic": t_test_ctz.statistic, "p_value": t_test_ctz.pvalue},
        "TTest_ModulationIndex_NoCTZ": {"statistic": t_test_noctz.statistic, "p_value": t_test_noctz.pvalue},
        "MW_Test_ModulationIndex_CTz": {"statistic": mw_test_ctz.statistic, "p_value": mw_test_ctz.pvalue},
        "MW_Test_ModulationIndex_NoCTZ": {"statistic": mw_test_noctz.statistic, "p_value": mw_test_noctz.pvalue}
    }

    stats_df = pd.DataFrame(statistical_results).T

    # Save descriptive stats and statistical results
    descriptive_stats_df.to_csv(os.path.join(save_directory, 'descriptive_statistics.csv'), index=False)
    stats_df.to_csv(os.path.join(save_directory, 'statistical_results.csv'))

    # Print the results
    print("Descriptive Statistics:")
    print(descriptive_stats_df)
    print("\nStatistical Results:")
    print(stats_df)

# Call the function with your specific paths
data_path = '/Volumes/MannySSD/figures/laminaranalysis/basic_metrics_for_plotting_baseline.csv'
save_directory = '/Volumes/MannySSD/figures/laminaranalysis'

analyze_and_plot_data(data_path, save_directory)
